<a href="https://colab.research.google.com/github/ishivanshgoel/heart-link-ml/blob/main/outpipeline_clwearables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install wfdb
import wfdb
import os, subprocess
import datetime
import IPython
import IPython.display
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

import csv, requests
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 KB 5.0 MB/s eta 0:00:00
  Attempting uninstall: SoundFile
    Found existing installation: soundfile 0.12.1
    Uninstalling soundfile-0.12.1:
      Successfully uninstalled soundfile-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.


In [32]:
import numpy as np
import torch

In [ ]:
# import subprocess,paths

In [ ]:
url, label = "https://physionet.org/files/vfdb/1.0.0/", "vfdb"
print("fetching from ",url)
cmd = f"wget -r -N -c -np -q {url}"
subprocess.call(cmd, shell=True)

fetching from  https://physionet.org/files/vfdb/1.0.0/


KeyboardInterrupt: ignored

In [22]:
class DataShaper():
    @staticmethod
    def __split_signal_and_add_label(record, stamp_length=10):
      # checking the greater amplitude
      if(record[1]["n_sig"] > 1):
        ecg = 0 if max(record[0][:,0]) > max(record[0][:,1]) else 1
      else:
        ecg = 0
        
      # no.of datapoints per signal = no of cols = frequency*stamp
      cols = record[1]["fs"]*stamp_length 

      rows = record[1]["sig_len"]//cols
      sampled_data = np.reshape( record[0][:rows*cols,ecg], (rows, cols))

      return sampled_data

In [ ]:
record = wfdb.rdsamp("physionet.org/files/vfdb/1.0.0/418")

FileNotFoundError: ignored

In [ ]:
record

(array([[-0.64 , -0.07 ],
        [-0.91 , -0.065],
        [-0.935, -0.06 ],
        ...,
        [-0.495, -0.245],
        [-0.58 , -0.295],
        [-0.68 , -0.325]]),
 {'fs': 250,
  'sig_len': 525000,
  'n_sig': 2,
  'base_date': None,
  'base_time': None,
  'units': ['mV', 'mV'],
  'sig_name': ['ECG', 'ECG'],
  'comments': []})

In [ ]:
real_data = DataShaper._DataShaper__split_signal_and_add_label(record, stamp_length=10)

In [ ]:
real_data.shape

(210, 2500)

In [33]:

# Time-frequency analysis with superlets
# Based on 'Time-frequency super-resolution with superlets'
# by Moca et al., 2021 Nature Communications
#
# Implementation by Gregor Mönke: github.com/tensionhead
#

def superlet(
    data_arr,
    samplerate,
    scales,
    order_max,
    order_min=1,
    c_1=3,
    adaptive=False,
):

    """
    Performs Superlet Transform (SLT) according to Moca et al. [1]_
    Both multiplicative SLT and fractional adaptive SLT are available. 
    The former is recommended for a narrow frequency band of interest, 
    whereas the  is better suited for the analysis of a broad range 
    of frequencies.

    A superlet (SL) is a set of Morlet wavelets with increasing number
    of cycles within the Gaussian envelope. Hence the bandwith 
    is constrained more and more with more cycles yielding a sharper
    frequency resolution. Complementary the low cycle numbers will give a
    high time resolution. The SLT then is the geometric mean 
    of the set of individual wavelet transforms, combining both wide
    and narrow-bandwidth wavelets into a super-resolution estimate.

    Parameters
    ----------
    data_arr : nD :class:`numpy.ndarray`
        Uniformly sampled time-series data
        The 1st dimension is interpreted as the time axis
    samplerate : float
        Samplerate of the time-series in Hz
    scales : 1D :class:`numpy.ndarray`
        Set of scales to use in wavelet transform. 
        Note that for the SL Morlet the relationship
        between scale and frequency simply is s(f) = 1/(2*pi*f)
        Need to be ordered high to low for `adaptive=True`
    order_max : int
        Maximal order of the superlet set. Controls the maximum
        number of cycles within a SL together
        with the `c_1` parameter: c_max = c_1 * order_max
    order_min : Minimal order of the superlet set. Controls 
        the minimal number of cycles within a SL together
        with the `c_1` parameter: c_min = c_1 * order_min
        Note that for admissability reasons c_min should be at least 3!
    c_1 : int
        Number of cycles of the base Morlet wavelet. If set to lower
        than 3 increase `order_min` as to never have less than 3 cycles
        in a wavelet!
    adaptive : bool
        Wether to perform multiplicative SLT or fractional adaptive SLT.
        If set to True, the order of the wavelet set will increase
        linearly with the frequencies of interest from `order_min` 
        to `order_max`. If set to False the same SL will be used for
        all frequencies.
    
    Returns
    -------
    gmean_spec : :class:`numpy.ndarray`
        Complex time-frequency representation of the input data. 
        Shape is (len(scales), data_arr.shape[0], data_arr.shape[1]).

    Notes
    -----
    .. [1] Moca, Vasile V., et al. "Time-frequency super-resolution with superlets." 
       Nature communications 12.1 (2021): 1-18.
 
 
    """

    # adaptive SLT
    if adaptive:

        gmean_spec = FASLT(data_arr, samplerate, scales, order_max, order_min, c_1)

    # multiplicative SLT
    else:

        gmean_spec = multiplicativeSLT(
            data_arr, samplerate, scales, order_max, order_min, c_1
        )

    return gmean_spec


def multiplicativeSLT(data_arr, samplerate, scales, order_max, order_min=1, c_1=3):

    dt = 1 / samplerate
    # create the complete multiplicative set spanning
    # order_min - order_max
    cycles = c_1 * np.arange(order_min, order_max + 1)
    order_num = order_max + 1 - order_min # number of different orders
    SL = [MorletSL(c) for c in cycles]

    # lowest order
    gmean_spec = cwtSL(data_arr, SL[0], scales, dt)
    gmean_spec = np.power(gmean_spec, 1 / order_num)

    for wavelet in SL[1:]:

        spec = cwtSL(data_arr, wavelet, scales, dt)
        gmean_spec *= np.power(spec, 1 / order_num)

    return gmean_spec


def FASLT(data_arr, samplerate, scales, order_max, order_min=1, c_1=3):

    """ Fractional adaptive SL transform

    For non-integer orders fractional SLTs are
    calculated in the interval [order, order+1) via:
    
    R(o_f) = R_1 * R_2 * ... * R_i * R_i+1 ** alpha 
    with o_f = o_i + alpha
    """

    dt = 1 / samplerate
    # frequencies of interest
    # from the scales for the SL Morlet
    fois = 1 / (2 * np.pi * scales)
    orders = compute_adaptive_order(fois, order_min, order_max)

    # create the complete superlet set from
    # all enclosed integer orders
    orders_int = np.int32(np.floor(orders))
    cycles = c_1 * np.unique(orders_int)
    SL = [MorletSL(c) for c in cycles]

    # every scale needs a different exponent
    # for the geometric mean
    exponents = 1 / (orders - order_min + 1)

    # which frequencies/scales use the same integer orders SL
    order_jumps = np.where(np.diff(orders_int))[0]
    # each frequency/scale will have its own multiplicative SL
    # which overlap -> higher orders have all the lower orders

    # the fractions
    alphas = orders % orders_int

    # 1st order
    # lowest order is needed for all scales/frequencies
    gmean_spec = cwtSL(data_arr, SL[0], scales, dt)  # 1st order <-> order_min
    # Geometric normalization according to scale dependent order
    gmean_spec = np.power(gmean_spec.T, exponents).T

    # we go to the next scale and order in any case..
    # but for order_max == 1 for which order_jumps is empty
    last_jump = 1

    for i, jump in enumerate(order_jumps):

        # relevant scales for the next order
        scales_o = scales[last_jump:]
        # order + 1 spec
        next_spec = cwtSL(data_arr, SL[i + 1], scales_o, dt)

        # which fractions for the current next_spec
        # in the interval [order, order+1)
        scale_span = slice(last_jump, jump + 1)
        gmean_spec[scale_span, :] *= np.power(
            next_spec[: jump - last_jump + 1].T,
            alphas[scale_span] * exponents[scale_span],
        ).T

        # multiply non-fractional next_spec for
        # all remaining scales/frequencies
        gmean_spec[jump + 1 :] *= np.power(
            next_spec[jump - last_jump + 1 :].T, exponents[jump + 1 :]
        ).T

        # go to the next [order, order+1) interval
        last_jump = jump + 1

    return gmean_spec


class MorletSL:
    def __init__(self, c_i=3, k_sd=5):

        """ The Morlet formulation according to
        Moca et al. shifts the admissability criterion from
        the central frequency to the number of cycles c_i
        within the Gaussian envelope which has a constant 
        standard deviation of k_sd.
        """

        self.c_i = c_i
        self.k_sd = k_sd

    def __call__(self, *args, **kwargs):
        return self.time(*args, **kwargs)

    def time(self, t, s=1.0):

        """
        Complext Morlet wavelet in the SL formulation.
        
        Parameters
        ----------
        t : float
            Time. If s is not specified, this can be used as the
            non-dimensional time t/s.
        s : float
            Scaling factor. Default is 1.

        Returns
        -------
        out : complex
            Value of the Morlet wavelet at the given time
        
        """

        ts = t / s
        # scaled time spread parameter
        # also includes scale normalisation!
        B_c = self.k_sd / (s * self.c_i * (2 * np.pi) ** 1.5)

        output = B_c * np.exp(1j * ts)
        output *= np.exp(-0.5 * (self.k_sd * ts / (2 * np.pi * self.c_i)) ** 2)

        return output


def fourier_period(scale):

    """
    This is the approximate Morlet fourier period
    as used in the source publication of Moca et al. 2021

    Note that w0 (central frequency) is always 1 in this 
    Morlet formulation, hence the scales are not compatible
    to the standard Wavelet definitions!
    """

    return 2 * np.pi * scale


def scale_from_period(period):

    return period / (2 * np.pi)


def cwtSL(data, wavelet, scales, dt):

    """
    The continuous Wavelet transform specifically
    for Morlets with the Superlet formulation
    of Moca et al. 2021.

    - Morlet support gets adjusted by number of cycles
    - normalisation is with 1/(scale * 4pi)
    - this way the norm of the spectrum (modulus) 
      at the corresponding harmonic frequency is the 
      harmonic signal's amplitude

    Notes
    -----
    
    The time axis is expected to be along the 1st dimension.
    """

    # wavelets can be complex so output is complex
    output = np.zeros((len(scales),) + data.shape, dtype=np.complex64)

    # this checks if really a Superlet Wavelet is being used
    if not isinstance(wavelet, MorletSL):
        raise ValueError("Wavelet is not of MorletSL type!")

    # 1st axis is time
    slices = [None for _ in data.shape]
    slices[0] = slice(None)

    # compute in time
    for ind, scale in enumerate(scales):

        t = _get_superlet_support(scale, dt, wavelet.c_i)
        # sample wavelet and normalise
        norm = dt ** 0.5 / (4 * np.pi)
        wavelet_data = norm * wavelet(t, scale)  # this is an 1d array for sure!
        output[ind, :] = fftconvolve(data, wavelet_data[tuple(slices)], mode="same")

    return output


def _get_superlet_support(scale, dt, cycles):

    """
    Effective support for the convolution is here not only 
    scale but also cycle dependent.
    """

    # number of points needed to capture wavelet
    M = 10 * scale * cycles / dt
    # times to use, centred at zero
    t = np.arange((-M + 1) / 2.0, (M + 1) / 2.0) * dt

    return t


def compute_adaptive_order(freq, order_min, order_max):

    """
    Computes the superlet order for a given frequency of interest 
    for the fractional adaptive SLT (FASLT) according to 
    equation 7 of Moca et al. 2021.
    
    This is a simple linear mapping between the minimal
    and maximal order onto the respective minimal and maximal
    frequencies. 

    Note that `freq` should be ordered low to high.
    """

    f_min, f_max = freq[0], freq[-1]

    assert f_min < f_max

    order = (order_max - order_min) * (freq - f_min) / (f_max - f_min)

    # return np.int32(order_min + np.rint(order))
    return order_min + order


# ---------------------------------------------------------
# Some test data akin to figure 3 of the source publication
# ---------------------------------------------------------


def gen_superlet_testdata(freqs=[20, 40, 60], cycles=11, fs=1000, eps=0):

    """
    Harmonic superposition of multiple
    few-cycle oscillations akin to the
    example of Figure 3 in Moca et al. 2021 NatComm
    """

    signal = []
    for freq in freqs:

        # 10 cycles of f1
        tvec = np.arange(cycles / freq, step=1 / fs)

        harmonic = np.cos(2 * np.pi * freq * tvec)
        f_neighbor = np.cos(2 * np.pi * (freq + 10) * tvec)
        packet = harmonic + f_neighbor

        # 2 cycles time neighbor
        delta_t = np.zeros(int(2 / freq * fs))

        # 5 cycles break
        pad = np.zeros(int(5 / freq * fs))

        signal.extend([pad, packet, delta_t, harmonic])

    # stack the packets together with some padding
    signal.append(pad)
    signal = np.concatenate(signal)

    # additive white noise
    if eps > 0:
        signal = np.random.randn(len(signal)) * eps + signal

    return signal

In [ ]:
# real_data.shape

In [ ]:
# temp = [
#     322.0, 311.0, 302.0, 299.0, 301.0, 307.0, 317.0, 330.0, 341.0, 352.0, 359.0,
#     361.0, 359.0, 352.0, 342.0, 331.0, 320.0, 309.0, 318.0, 354.0, 337.0, 298.0,
#     317.0, 352.0, 326.0, 293.0, 323.0, 350.0, 317.0, 295.0, 329.0, 347.0, 314.0,
#     313.0, 357.0, 365.0, 326.0, 326.0, 363.0, 356.0, 312.0, 319.0, 351.0, 334.0,
#     301.0, 325.0, 360.0, 335.0, 303.0, 334.0, 361.0, 325.0, 294.0, 314.0, 301.0,
#     242.0, 220.0, 296.0, 390.0, 401.0, 393.0, 423.0, 416.0, 368.0, 367.0, 401.0,
#     377.0, 318.0, 317.0, 349.0, 330.0, 300.0, 329.0, 358.0, 324.0, 301.0, 339.0,
#     359.0, 320.0, 305.0, 348.0, 362.0, 322.0, 314.0, 354.0, 354.0, 315.0, 322.0,
#     364.0, 352.0, 311.0, 326.0, 365.0, 345.0, 312.0, 338.0, 365.0, 336.0, 316.0,
#     352.0, 378.0, 344.0, 326.0, 362.0, 376.0, 340.0, 330.0, 364.0, 359.0, 313.0,
#     313.0, 354.0, 347.0, 304.0, 316.0, 355.0, 335.0, 300.0, 324.0, 356.0, 323.0,
#     290.0, 324.0, 352.0, 322.0, 305.0, 345.0, 359.0, 320.0, 313.0, 356.0, 360.0,
#     317.0, 316.0, 356.0, 353.0, 312.0, 324.0, 363.0, 344.0, 307.0, 328.0, 360.0,
#     332.0, 304.0, 332.0, 355.0, 320.0, 296.0, 331.0, 348.0, 310.0, 300.0, 343.0,
#     349.0, 307.0, 304.0, 345.0, 343.0, 303.0, 313.0, 352.0, 343.0, 313.0, 339.0,
#     378.0, 355.0, 319.0, 341.0, 363.0, 329.0, 306.0, 343.0, 363.0, 324.0, 311.0,
#     350.0, 358.0, 318.0, 315.0, 357.0, 359.0, 304.0, 274.0, 290.0, 272.0, 262.0,
#     365.0, 471.0, 457.0, 408.0, 424.0, 445.0, 397.0, 357.0, 377.0, 377.0, 325.0,
#     303.0, 344.0, 353.0, 311.0, 304.0, 348.0, 349.0, 308.0, 314.0, 352.0, 341.0,
#     304.0, 325.0, 364.0, 342.0, 306.0, 335.0, 367.0, 335.0, 311.0, 346.0, 371.0,
#     339.0, 325.0, 366.0, 384.0, 351.0, 347.0, 394.0, 403.0, 364.0, 369.0, 416.0,
#     411.0, 372.0, 389.0, 435.0, 418.0, 378.0, 405.0, 438.0, 407.0, 378.0, 414.0,
#     442.0, 408.0, 385.0, 426.0, 447.0, 411.0, 400.0, 442.0, 449.0, 410.0, 413.0,
#     455.0, 449.0, 405.0, 419.0, 463.0, 442.0, 398.0, 422.0, 459.0, 433.0, 398.0,
#     420.0, 445.0, 415.0, 392.0, 426.0, 443.0, 399.0, 382.0, 422.0, 428.0, 383.0,
#     379.0, 418.0, 416.0, 374.0, 381.0, 420.0, 403.0, 364.0, 384.0, 420.0, 392.0,
#     358.0, 388.0, 419.0, 385.0, 358.0, 393.0, 412.0, 373.0, 359.0, 394.0, 406.0,
#     370.0, 367.0, 412.0, 412.0, 368.0, 369.0, 405.0, 392.0, 352.0, 368.0, 400.0,
#     373.0, 340.0, 364.0, 393.0, 364.0, 339.0, 373.0, 391.0, 344.0, 304.0, 311.0,
#     304.0, 273.0, 330.0, 459.0, 499.0, 455.0, 458.0, 492.0, 466.0, 412.0, 412.0,
#     430.0, 395.0, 354.0, 373.0, 405.0, 379.0, 356.0, 389.0, 411.0, 378.0, 362.0,
#     399.0, 411.0, 372.0, 364.0, 409.0, 416.0, 373.0, 373.0, 415.0, 408.0, 366.0,
#     375.0, 414.0, 397.0, 360.0, 383.0, 420.0, 396.0, 366.0, 396.0, 421.0, 387.0,
#     368.0, 409.0, 426.0, 382.0, 373.0, 416.0, 419.0, 376.0, 375.0, 412.0, 404.0,
#     360.0, 370.0, 405.0, 386.0, 351.0, 368.0, 397.0, 371.0, 341.0, 371.0, 400.0,
#     369.0, 346.0, 383.0, 399.0, 362.0, 347.0, 382.0, 389.0, 346.0, 340.0, 375.0,
#     366.0, 324.0, 332.0, 367.0, 347.0, 303.0, 319.0, 352.0, 324.0, 291.0, 317.0,
#     347.0, 312.0, 290.0, 323.0, 338.0, 298.0, 287.0, 329.0, 338.0, 298.0, 292.0,
#     329.0, 324.0, 283.0, 290.0, 326.0, 311.0, 274.0, 298.0, 332.0, 304.0, 268.0,
#     298.0, 327.0, 298.0, 276.0, 310.0, 336.0, 305.0, 298.0, 343.0, 357.0, 314.0,
#     305.0, 343.0, 339.0, 297.0, 301.0, 337.0, 323.0, 282.0, 301.0, 345.0, 326.0,
#     292.0, 316.0, 345.0, 304.0, 247.0, 250.0, 258.0, 236.0, 289.0, 410.0, 449.0,
#     395.0, 376.0, 404.0, 393.0, 339.0, 325.0, 346.0, 321.0, 275.0, 286.0, 326.0,
#     307.0, 270.0, 295.0, 326.0, 297.0, 270.0, 308.0, 333.0, 292.0, 267.0, 310.0,
#     330.0, 293.0, 281.0,
#   ]

In [ ]:
# image = np.array(temp)/max(temp)

In [ ]:
# import cv2

In [ ]:
# record

NameError: ignored

In [ ]:
# type(temp)

In [ ]:
# temp = np.column_stack((image,image))

In [ ]:
# sample = [temp, {
#     "fs": 5,
#     "sig_len":len(image),
#     "sig_name": ["ECG", "ECG"],
#     "n_sig":1
# }]

In [ ]:
# len(image)

487

In [ ]:
# ans= DataShaper._DataShaper__split_signal_and_add_label(real_data, stamp_length=10)

IndexError: ignored

In [42]:
# TAKE DATA AND SAVE EACH RECORD INTO A FOLDER.
fs = 256  # sampling frequency
A = 500 # amplitude
for i in range(len(ans)):
  # signal = A * gen_superlet_testdata(fs=fs, eps=0)  # 20Hz, 40Hz and 60Hz
  signal = ans[i]

  # frequencies of interest in Hz
  foi = np.linspace(1, 100, 50)
  scales = scale_from_period(1 / foi)

  spec = superlet(
      signal,
      samplerate=fs,
      scales=scales,
      order_max=30,
      order_min=1,
      c_1=5,
      adaptive=True,
  )

  # amplitude scalogram
  ampls = np.abs(spec)

  fig, (ax2) = plt.subplots(1, 1,
                                # sharex=True,
                                # gridspec_kw={"height_ratios": [1, 1]},
                                figsize=(6, 6))

  extent = [0, len(signal) / fs, foi[0], foi[-1]]


  im = ax2.imshow(ampls, cmap="Greys", aspect="auto", extent=extent, origin='lower')
  outpath = "/content/data/"
  label="afdb"
  plt.savefig(f"{outpath}{label}_{i}", dpi=300)
  print(f"saved to {outpath}{label}_{i}")
  plt.close()

  fig.tight_layout()

saved to /content/data/afdb_0
saved to /content/data/afdb_1
saved to /content/data/afdb_2
saved to /content/data/afdb_3
saved to /content/data/afdb_4
saved to /content/data/afdb_5
saved to /content/data/afdb_6
saved to /content/data/afdb_7
saved to /content/data/afdb_8
saved to /content/data/afdb_9
saved to /content/data/afdb_10
saved to /content/data/afdb_11
saved to /content/data/afdb_12
saved to /content/data/afdb_13
saved to /content/data/afdb_14
saved to /content/data/afdb_15
saved to /content/data/afdb_16
saved to /content/data/afdb_17
saved to /content/data/afdb_18
saved to /content/data/afdb_19
saved to /content/data/afdb_20
saved to /content/data/afdb_21
saved to /content/data/afdb_22
saved to /content/data/afdb_23
saved to /content/data/afdb_24
saved to /content/data/afdb_25
saved to /content/data/afdb_26
saved to /content/data/afdb_27
saved to /content/data/afdb_28


In [43]:
len(image)

57526

In [36]:
import torch

In [37]:
weights = torch.load("/content/model_20221114_083203_9", map_location=torch.device('cpu'))

FileNotFoundError: ignored

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
model.load_state_dict(weights)

<All keys matched successfully>

In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
path = "/content/ecg_data.csv"

In [8]:
import numpy as np
import pandas as pd

In [13]:
sample_data_new = [
    309, 307, 320, 321, 307, 309, 325, 324, 312, 318, 328, 321, 312, 319, 333, 327, 316, 327, 338, 330, 324, 337, 346, 335, 333, 348, 352, 338, 342, 358, 357, 345, 348, 362, 356, 340, 342, 350, 335, 318, 325, 330, 319, 311, 321, 331, 318, 308, 321, 326, 314, 311, 324, 326, 312, 313, 330, 329, 314, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 343, 329, 327, 342, 341, 328, 333, 348, 344, 334, 344, 358, 352, 342, 353, 367, 358, 347, 358, 364, 349, 337, 342, 341, 324, 318, 330, 330, 315, 316, 330, 329, 314, 317, 331, 328, 313, 321, 333, 322, 314, 324, 333, 323, 315, 328, 335, 323, 320, 335, 339, 321, 316, 329, 332, 323, 323, 334, 332, 316, 321, 339, 335, 321, 324, 336, 331, 325, 338, 349, 336, 330, 350, 361, 349, 348, 363, 362, 338, 332, 341, 336, 321, 324, 338, 332, 316, 320, 333, 326, 313, 316, 298, 260, 243, 283, 371, 423, 415, 403, 387, 358, 348, 363, 364, 337, 326, 333, 329, 314, 320, 332, 325, 312, 321, 334, 324, 312, 326, 340, 330, 321, 333, 342, 332, 326, 340, 345, 336, 336, 352, 353, 339, 340, 356, 357, 347, 353, 367, 360, 350, 358, 366, 351, 336, 344, 349, 333, 321, 328, 330, 317, 313, 324, 329, 317, 315, 325, 324, 310, 316, 334, 333, 321, 322, 330, 324, 314, 325, 338, 327, 317, 326, 335, 324, 319, 330, 333, 319, 316, 332, 333, 320, 321, 335, 329, 315, 324, 341, 338, 326, 332, 350, 350, 340, 350, 361, 348, 335, 340, 342, 325, 316, 328, 330, 316, 314, 327, 325, 309, 312, 324, 297, 252, 245, 269, 322, 403, 442, 428, 388, 354, 353, 359, 348, 332, 330, 327, 311, 305, 321, 322, 311, 315, 330, 326, 309, 314, 330, 327, 317, 324, 336, 330, 320, 332, 344, 334, 325, 335, 344, 333, 330, 346, 354, 342, 340, 356, 363, 352, 352, 363, 356, 339, 340, 349, 338, 320, 322, 333, 321, 310, 321, 327, 314, 308, 320, 327, 314, 308, 321, 325, 314, 314, 326, 324, 312, 314, 330, 328, 317, 326, 337, 331, 318, 326, 340, 333, 322, 329, 339, 329, 321, 333, 341, 329, 323, 335, 338, 326, 326, 341, 342, 330, 333, 349, 352, 344, 350, 362, 356, 341, 342, 345, 330, 320, 332, 339, 325, 316, 326, 329, 317, 313, 326, 318, 280, 255, 257, 276, 343, 443, 467, 428, 382, 368, 373, 363, 341, 334, 331, 315, 307, 317, 324, 313, 308, 320, 323, 312, 314, 328, 327, 315, 318, 333, 330, 320, 326, 338, 330, 320, 331, 345, 336, 328, 339, 348, 338, 336, 349, 355, 345, 346, 359, 360, 345, 341, 351, 345, 326, 326, 335, 325, 310, 315, 326, 320, 308, 316, 324, 314, 307, 319, 325, 315, 309, 323, 329, 313, 315, 329, 330, 316, 317, 331, 329, 317, 323, 334, 328, 317, 324, 336, 328, 317, 327, 337, 326, 319, 331, 339, 327, 320, 332, 336, 322, 321, 336, 339, 328, 333, 347, 343, 335, 348, 361, 353, 342, 349, 352, 338, 325, 333, 339, 326, 319, 332, 335, 320, 316, 326, 324, 306, 288, 273, 256, 256, 327, 443, 469, 427, 403, 390, 366, 353, 357, 350, 322, 307, 318, 325, 313, 307, 320, 322, 311, 311, 324, 323, 310, 311, 327, 325, 312, 321, 336, 331, 319, 327, 339, 333, 325, 337, 345, 335, 331, 345, 352, 342, 341, 353, 358, 346, 345, 356, 349, 330, 330, 339, 331, 313, 319, 330, 323, 314, 318, 326, 318, 310, 321, 329, 318, 313, 323, 328, 317, 317, 332, 335, 320, 317, 332, 331, 316, 321, 334, 330, 321, 330, 337, 324, 316, 327, 339, 328, 320, 331, 336, 323, 320, 334, 337, 324, 324, 338, 337, 321, 323, 337, 333, 323, 334, 350, 343, 329, 346, 361, 350, 344, 358, 360, 339, 327, 337, 337, 322, 321, 337, 336, 316, 316, 330, 327, 314, 308, 288, 258, 240, 286, 390, 452, 435, 409, 388, 357, 342, 358, 359, 331, 314, 321, 325, 313, 310, 326, 328, 315, 316, 330, 324, 312, 319, 333, 326, 316, 326, 338, 331, 322, 331, 338, 328, 324, 339, 345, 332, 329, 345, 352, 341, 343, 359, 360, 350, 350, 357, 348, 334, 338, 344, 331, 316, 322, 333, 319, 306, 317, 326, 313, 306, 318, 325, 313, 311, 325, 325, 313, 316, 330, 328, 315, 318, 331, 325, 317, 327, 337, 326, 317, 327, 337, 327, 318, 328, 335, 323, 321, 334, 337, 321, 319, 333, 331, 318, 321, 335, 332, 320, 323, 338, 336, 332, 341, 353, 353, 346, 359, 369, 355, 343, 349, 351, 333, 324, 336, 337, 323, 323, 338, 337, 320, 322, 333, 312, 269, 260, 276, 306, 375, 431, 433, 397, 364, 361, 360, 349, 341, 346, 340, 322, 318, 330, 334, 319, 322, 335, 332, 319, 321, 335, 332, 321, 330, 345, 337, 326, 337, 346, 337, 331, 345, 352, 341, 335, 349, 354, 345, 349, 365, 363, 350, 352, 366, 364, 350, 351, 359, 347, 330, 334, 342, 330, 317, 323, 329, 317, 310, 324, 331, 318, 314, 325, 327, 315, 315, 329, 329, 315, 317, 332, 328, 315, 322, 338, 331, 319, 328, 340, 329, 322, 334, 340, 326, 320, 334, 341, 328, 321, 335, 336, 326, 332, 349, 347, 334, 342, 363, 359, 345, 357, 368, 353, 335, 338, 341, 328, 320, 333, 338, 326, 319, 328, 332, 318, 313, 312, 282, 250, 253, 310, 397, 439, 432, 412, 375, 347, 351, 366, 351, 325, 325, 330, 318, 311, 324, 329, 317, 317, 328, 328, 316, 317, 333, 332, 319, 321, 338, 336, 324, 330, 344, 338, 329, 339, 349, 338, 332, 346, 357, 348, 345, 357, 362, 349, 349, 360, 358, 343, 338, 348, 341, 325, 326, 334, 325, 311, 320, 330, 320, 309, 321, 330, 318, 310, 324, 329, 316, 314, 329, 332, 317, 319, 334, 334, 319, 322, 336, 332, 318, 324, 337, 328, 316, 325, 337, 327, 318, 326, 334, 325, 319, 333, 341, 333, 334, 345, 351, 344, 347, 363, 360, 342, 339, 347, 340, 323, 326, 335, 326, 314, 321, 331, 320, 312, 321, 318, 280, 250, 253, 278, 344, 439, 468, 432, 384, 366, 370, 368, 345, 333, 332, 319, 306, 315, 326, 318, 307, 318, 328, 318, 310, 321, 328, 317, 314, 328, 334, 321, 320, 334, 333, 320, 327, 344, 339, 329, 335, 345, 341, 335, 346, 359, 352, 344, 358, 365, 350, 338, 346, 346, 328, 320, 332, 331, 313, 312, 327, 323, 307, 313, 324, 318, 307, 315, 327, 318, 308, 318, 329, 320, 313, 326, 334, 321, 317, 330, 334, 321, 318, 331, 332, 320, 321, 335, 331, 318, 324, 338, 329, 316, 323, 336, 331, 322, 337, 345, 337, 337, 355, 361, 351, 346, 353, 348, 330, 325, 335, 331, 316, 320, 332, 325, 312, 316, 328, 320, 299, 280, 266, 248, 271, 371, 464, 451, 413, 394, 377, 356, 355, 358, 340, 314, 313, 326, 321, 307, 313, 328, 323, 313, 319, 328, 320, 312, 324, 334, 323, 315, 329, 335, 325, 322, 337, 342, 330, 332, 346, 343, 330, 340, 359, 356, 344, 353, 367, 358, 345, 352, 360, 345, 331, 338, 340, 323, 318, 332, 333, 316, 310, 324, 325, 312, 313, 326, 322, 310, 318, 329, 322, 309, 320, 337, 325, 312, 323, 335, 326, 317, 330, 338, 324, 318, 332, 336, 322, 321, 337, 334, 318, 322, 336, 333, 321, 329, 346, 342, 329, 342, 359, 353, 345, 357, 359, 340, 326, 333, 337, 324, 319, 329, 327, 314, 316, 332, 331, 311, 316, 320, 282, 243, 247, 298, 376, 428, 433, 410, 372, 347, 351, 360, 342, 325, 328, 329, 315, 311, 324, 326, 312, 315, 331, 326, 312, 318, 332, 325, 316, 324, 338, 331, 320, 330, 341, 335, 328, 342, 349, 334, 332, 351, 358, 343, 341, 356, 356, 345, 351, 362, 354, 338, 338, 347, 337, 322, 328, 333, 320, 311, 322, 327, 315, 309, 321, 325, 313, 311, 325, 328, 314, 317, 332, 330, 315, 318, 333, 329, 316, 321, 334, 326, 315, 326, 336, 326, 317, 331, 342, 330, 321, 336, 342, 331, 328, 344, 351, 344, 350, 367, 362, 344, 340, 348, 340, 323, 327, 337, 326, 314, 321, 333, 323, 316, 328, 329, 298, 261, 260, 271, 305, 392, 442, 420, 379, 362, 361, 349, 339, 341, 343, 330, 313, 320, 327, 317, 309, 324, 332, 319, 313, 327, 333, 321, 321, 338, 339, 325, 325, 339, 342, 331, 334, 349, 343, 330, 341, 355, 349, 342, 353, 362, 353, 350, 362, 371, 356, 346, 354, 353, 333, 326, 336, 335, 320, 321, 331, 325, 311, 315, 327, 322, 314, 322, 332, 323, 312, 325, 337, 327, 320, 331, 337, 325, 320, 333, 335, 323, 322, 335, 335, 321, 325, 335, 329, 318, 325, 338, 331, 321, 335, 347, 341, 337, 351, 357, 347, 344, 351, 347, 327, 321, 332, 332, 316, 314, 326, 323, 309, 312, 325, 321, 303, 283, 268, 248, 264, 356, 452, 449, 405, 387, 374, 354, 353, 360, 344, 315, 308, 321, 322, 310, 312, 325, 323, 310, 315, 330, 324, 312, 321, 333, 324, 314, 328, 339, 326, 319, 334, 338, 329, 329, 342, 343, 331, 335, 353, 353, 341, 345, 361, 358, 345, 350, 358, 346, 329, 333, 338, 323, 312, 322, 328, 316, 310, 322, 324, 311, 313, 325, 324, 311, 316, 332, 330, 315, 317, 332, 326, 314, 324, 336, 326, 317, 328, 337, 325, 315, 328, 335, 325, 321, 333, 336, 325, 328, 344, 344, 333, 343, 361, 357, 347, 355, 360, 344, 327, 332, 341, 329, 318, 327, 333, 320, 316, 327, 332, 319, 306, 288, 262, 241, 277, 381, 462, 445, 416, 398, 373, 354, 364, 364, 337, 313, 319, 330, 319, 312, 324, 332, 322, 316, 327, 331, 319, 317, 333, 334, 322, 329, 343, 338, 325, 332, 346, 339, 330, 342, 355, 345, 336, 347, 356, 348, 345, 360, 367, 353, 349, 360, 359, 343, 335, 343, 339, 322, 323, 335, 327, 316, 322, 331, 323, 313, 324, 333, 321, 313, 328, 337, 323, 317, 331, 336, 322, 322, 336, 333, 320, 323, 338, 335, 322, 326, 339, 334, 322, 328, 339, 331, 321, 334, 348, 339, 329, 343, 355, 348, 343, 360, 366, 347, 337, 346, 343, 325, 324, 337, 330, 315, 319, 330, 327, 315, 322, 327, 294, 252, 250, 276, 337, 428, 464, 440, 394, 365, 368, 371, 350, 331, 332, 325, 308, 313, 328, 323, 310, 317, 330, 322, 311, 319, 330, 324, 317, 329, 336, 324, 318, 334, 343, 332, 329, 340, 343, 332, 336, 353, 353, 338, 346, 361, 351, 341, 351, 364, 354, 340, 345, 349, 330, 318, 328, 335, 322, 317, 327, 326, 311, 311, 326, 324, 312, 314, 329, 322, 307, 315, 329, 324, 316, 325, 337, 327, 320, 332, 337, 325, 321, 333, 335, 321, 318, 334, 336, 323, 323, 336, 332, 317, 326, 343, 343, 333, 342, 359, 356, 345, 358, 370, 356, 340, 345, 344, 327, 321, 336, 340, 326, 323, 334, 334, 320, 320, 330, 303, 259, 256, 286, 348, 414, 434, 419, 386, 356, 357, 367, 355, 337, 339, 337, 320, 317, 331, 332, 318, 317, 333, 334, 321, 324, 336, 331, 318, 328, 345, 338, 325, 334, 345, 333, 325, 338, 349, 341, 335, 346, 354, 346, 344, 359, 361, 349, 351, 365, 359, 341, 342, 351, 340, 326, 328, 338, 329, 315, 324, 333, 320, 312, 326, 333, 318, 312, 327, 332, 319, 318, 333, 334, 319, 323, 338, 335, 321, 326, 339, 332, 320, 329, 340, 332, 324, 335, 339, 324, 318, 335, 345, 334, 331, 345, 352, 345, 347, 362, 365, 350, 345, 350, 341, 325, 328, 338, 328, 317, 326, 336, 322, 312, 323, 332, 308, 272, 265, 266, 283, 363, 435, 428, 386, 366, 362, 349, 342, 349, 350, 331, 311, 319, 331, 320, 308, 318, 331, 322, 315, 329, 335, 322, 316, 330, 333, 323, 324, 340, 339, 328, 332, 343, 339, 329, 337, 353, 344, 334, 347, 361, 353, 342, 355, 363, 352, 341, 348, 350, 330, 324, 335, 333, 317, 319, 333, 328, 311, 315, 327, 321, 308, 316, 330, 321, 311, 321, 331, 323, 315, 327, 335, 320, 314, 328, 331, 319, 318, 331, 332, 317, 321, 335, 331, 317, 326, 340, 337, 328, 336, 354, 354, 345, 356, 364, 347, 334, 340, 343, 326, 316, 328, 331, 314, 311, 324, 323, 310, 314, 320, 291, 251, 243, 275, 344, 425, 453, 430, 388, 355, 359, 370, 352, 331, 330, 326, 312, 310, 324, 326, 312, 313, 328, 324, 308, 314, 329, 324, 312, 322, 337, 330, 320, 333, 343, 333, 325, 337, 345, 336, 333, 348, 352, 341, 342, 356, 357, 345, 350, 364, 359, 339, 339, 343, 332, 318, 325, 332, 322, 312, 322, 329, 316, 308, 321, 326, 314, 313, 327, 328, 317, 317, 330, 329, 316, 320, 333, 329, 316, 324, 337, 329, 319, 332, 342, 330, 322, 334, 338, 327, 323, 339, 346, 333, 332, 348, 353, 348, 352, 367, 364, 345, 341, 347, 338, 323, 328, 337, 327, 315, 323, 332, 321, 315, 325, 327, 294, 259, 256, 269, 320, 424, 478, 445, 395, 373, 374, 370, 353, 342, 337, 324, 311, 320, 333, 322, 312, 324, 332, 319, 310, 324, 331, 322, 320, 333, 334, 319, 322, 338, 337, 323, 329, 343, 339, 329, 338, 350, 344, 333, 346, 359, 350, 345, 358, 361, 346, 338, 347, 345, 329, 323, 334, 330, 315, 318, 329, 321, 306, 315, 326, 319, 308, 319, 330, 320, 310, 322, 332, 320, 315, 326, 332, 321, 318, 331, 334, 320, 320, 333, 333, 320, 324, 337, 332, 318, 326, 338, 332, 321, 333, 344, 338, 333, 348, 361, 347, 341, 359, 362, 341, 333, 343, 340, 321, 318, 329, 327, 312, 319, 332, 326, 312, 316, 320, 282, 246, 249, 290, 368, 439, 454, 429, 382, 359, 369, 368, 343, 325, 327, 322, 309, 313, 325, 321, 308, 314, 326, 320, 309, 320, 331, 324, 314, 327, 336, 326, 320, 332, 337, 325, 324, 340, 342, 331, 336, 352, 351, 339, 344, 358, 357, 346, 354, 366, 353, 337, 341, 347, 330, 318, 330, 335, 319, 310, 323, 324, 310, 310, 321, 321, 312, 316, 327, 321, 308, 320, 337, 330, 315, 323, 334, 324, 314, 324, 335, 324, 320, 332, 335, 322, 318, 330, 334, 321, 322, 336, 334, 321, 325, 344, 339, 326, 339, 361, 354, 342, 356, 367, 347, 330, 341, 345, 326, 320, 333, 333, 317, 316, 330, 329, 314, 316, 323, 297, 259, 253, 282, 345, 407, 421, 407, 376, 351, 350, 357, 347, 332, 331, 328, 312, 311, 325, 328, 315, 312, 328, 331, 317, 322, 336, 329, 316, 324, 338, 334, 322, 331, 345, 338, 330, 342, 350, 338, 334, 351, 359, 346, 345, 359, 363, 351, 352, 366, 361, 342, 341, 350, 338, 321, 328, 336, 327, 314, 322, 331, 319, 309, 322, 328, 317, 314, 328, 335, 319, 315, 331, 329, 316, 318, 335, 334, 321, 323, 336, 331, 320, 328, 340, 331, 319, 328, 339, 328, 321, 333, 340, 331, 330, 348, 353, 346, 349, 362, 365, 351, 346, 350, 342, 324, 325, 335, 327, 317, 324, 332, 321, 313, 325, 331, 308, 276, 266, 262, 276, 358, 440, 443, 398, 370, 367, 356, 348, 348, 350, 333, 315, 320, 330, 321, 311, 320, 332, 321, 311, 326, 335, 323, 318, 332, 335, 321, 322, 337, 340, 329, 332, 346, 345, 333, 337, 348, 342, 335, 348, 360, 348, 340, 354, 364, 350, 338, 348, 349, 333, 327, 335, 334, 316, 315, 328, 324, 311, 317, 330, 323, 308, 317, 330, 323, 313, 324, 336, 326, 316, 325, 333, 323, 318, 331, 335, 323, 320, 333, 333, 319, 320, 336, 334, 320, 325, 336, 330, 320, 329, 343, 334, 324, 337, 354, 350, 344, 355, 362, 348, 340, 345, 341, 320, 317, 331, 329, 313, 317, 328, 321, 307, 316, 327, 302, 257, 249, 265, 309, 397, 455, 442, 398, 365, 363, 366, 352, 336, 332, 325, 310, 311, 326, 324, 311, 313, 326, 318, 309, 323, 334, 319, 309, 320, 330, 323, 316, 329, 337, 326, 320, 335, 341, 329, 328, 344, 347, 336, 341, 357, 353, 342, 351, 360, 351, 337, 342, 349, 332, 318, 325, 330, 317, 311, 325, 328, 314, 310, 323, 324, 311, 313, 328, 325, 311, 316, 328, 322, 311, 319, 332, 325, 317, 326, 334, 326, 316, 327, 334, 323, 319, 335, 340, 323, 319, 331, 336, 323, 323, 338, 336, 326, 330, 346, 349, 345, 354, 362, 352, 339, 339, 342, 328, 316, 326, 331, 319, 315, 329, 330, 315, 314, 329, 321, 280, 252, 254, 274, 343, 442, 467, 427, 385, 372, 371, 361, 344, 338, 334, 316, 307, 323, 327, 314, 313, 325, 326, 314, 315, 327, 324, 312, 317, 331, 327, 317, 325, 336, 329, 321, 332, 341, 331, 326, 342, 353, 339, 335, 350, 354, 343, 346, 358, 356, 339, 341, 353, 342, 323, 326, 338, 327, 313, 321, 332, 319, 308, 321, 330, 318, 310, 323, 328, 317, 315, 329, 331, 316, 316, 333, 334, 323, 321, 334, 331, 321, 330, 339, 328, 316, 328, 341, 333, 321, 328, 337, 326, 320, 334, 339, 330, 330, 348, 348, 337, 347, 359, 356, 348, 354, 360, 346, 330, 333, 339, 328, 317, 327, 336, 322, 312, 326, 333, 317, 298, 280, 263, 249, 291, 395, 453, 429, 396, 380, 361, 345, 354, 358, 332, 310, 318, 327, 316, 310, 323, 331, 317, 310, 325, 332, 324, 320, 330, 331, 321, 324, 339, 337, 323, 327, 343, 339, 327, 337, 350, 345, 339, 350, 358, 351, 345, 357, 365, 353, 349, 360, 357, 339, 331, 342, 341, 324, 321, 333, 324, 312, 320, 333, 324, 312, 322, 329, 318, 311, 323, 333, 324, 317, 327, 332, 321, 320, 333, 334, 322, 323, 338, 336, 321, 326, 338, 333, 320, 328, 341, 332, 320, 330, 346, 338, 329, 341, 357, 351, 345, 362, 367, 346, 337, 347, 344, 323, 320, 335, 334, 319, 320, 333, 325, 310, 319, 326, 294, 253, 255, 282, 338, 409, 433, 414, 378, 358, 362, 362, 348, 336, 338, 330, 314, 318, 333, 329, 317, 323, 336, 328, 317, 330, 340, 331, 323, 334, 340, 329, 323, 339, 342, 327, 325, 341, 346, 335, 336, 351, 352, 343, 351, 365, 359, 346, 355, 367, 356, 341, 347, 351, 334, 323, 334, 340, 323, 315, 327, 331, 317, 313, 325, 325, 314, 320, 332, 328, 317, 323, 337, 327, 312, 324, 338, 327, 315, 324, 336, 327, 321, 333, 338, 326, 322, 334, 334, 323, 324, 338, 335, 327, 335, 349, 344, 336, 355, 372, 358, 346, 357, 361, 343, 330, 337, 340, 325, 321, 335, 335, 317, 314, 328, 329, 315, 302, 286, 262, 244, 295, 405, 461, 434, 410, 392, 365, 351, 362, 362, 334, 318, 325, 329, 318, 314, 328, 329, 316, 317, 333, 330, 317, 324, 337, 333, 322, 331, 344, 333, 323, 334, 346, 336, 328, 342, 353, 343, 339, 351, 358, 347, 347, 364, 366, 349, 347, 357, 352, 334, 333, 343, 333, 317, 325, 335, 324, 311, 320, 330, 322, 311, 320, 326, 316, 313, 328, 330, 316, 319, 336, 333, 317, 319, 334, 331, 318, 324, 338, 331, 319, 325, 336, 328, 317, 328, 335, 324, 317, 331, 339, 331, 327, 340, 346, 340, 345, 357, 360, 345, 340, 347, 337, 322, 327, 334, 323, 312, 322, 331, 317, 308, 323, 329, 305, 268, 256, 255, 281, 371, 461, 458, 411, 380, 374, 366, 350, 342, 338, 322, 306, 313, 324, 313, 303, 317, 328, 316, 307, 321, 326, 317, 317, 330, 333, 318, 320, 334, 334, 320, 327, 345, 340, 325, 333, 347, 342, 334, 346, 356, 347, 340, 355, 360, 346, 339, 348, 346, 329, 323, 336, 330, 312, 312, 326, 323, 310, 316, 325, 317, 306, 316, 329, 320, 308, 318, 328, 320, 315, 327, 335, 322, 314, 329, 331, 319, 321, 338, 335, 320, 324, 337, 333, 321, 326, 337, 331, 320, 329, 342, 335, 327, 342, 352, 343, 345, 361, 361, 350, 349, 358, 349, 330, 329, 340, 334, 318, 323, 335, 328, 315, 321, 331, 320, 298, 279, 262, 245, 270, 368, 460, 452, 413, 395, 379, 355, 354, 360, 340, 310, 308, 322, 320, 307, 315, 329, 321, 309, 318, 330, 319, 311, 322, 332, 322, 315, 328, 332, 319, 320, 336, 338, 322, 324, 341, 343, 330, 334, 348, 345, 336, 343, 357, 351, 338, 345, 352, 336, 324, 335, 338, 317, 310, 322, 324, 310, 308, 320, 323, 309, 311, 325, 321, 310, 318, 328, 319, 308, 320, 333, 324, 317, 325, 335, 324, 318, 332, 335, 320, 316, 328, 332, 320, 319, 332, 334, 323, 323, 329, 324, 316, 323, 335, 331, 324, 332, 344, 340, 340, 354, 360, 351, 343, 347, 346, 329, 320, 331, 333, 318, 320, 331, 326, 314, 320, 331, 322, 298, 277, 269, 255, 276, 368, 438, 421, 383, 372, 366, 348, 344, 354, 344, 320, 315, 324, 323, 311, 317, 329, 322, 311, 322, 338, 329, 313, 321, 335, 326, 318, 331, 338, 327, 324, 339, 344, 332, 331, 347, 352, 338, 341, 360, 360, 344, 349, 366, 361, 349, 356, 364, 346, 330, 336, 342, 330, 318, 330, 336, 322, 315, 326, 328, 315, 315, 329, 327, 313, 319, 336, 330, 315, 324, 338, 327, 314, 325, 338, 328, 318, 329, 336, 324, 318, 333, 338, 324, 318, 333, 335, 320, 321, 334, 334, 322, 328, 344, 340, 335, 349, 359, 351, 343, 351, 357, 340, 325, 330, 334, 320, 311, 325, 331, 316, 312, 324, 326, 310, 296, 280, 259, 250, 305, 404, 437, 407, 390, 379, 354, 338, 352, 355, 331, 315, 323, 325, 314, 313, 329, 329, 312, 315, 331, 329, 317, 321, 338, 334, 321, 329, 343, 337, 329, 338, 347, 337, 330, 343, 352, 343, 338, 352, 360, 346, 346, 363, 365, 350, 352, 364, 353, 335, 337, 346, 332, 316, 325, 334, 323, 312, 323, 330, 316, 310, 324, 331, 319, 313, 325, 328, 315, 317, 332, 332, 320, 323, 336, 331, 316, 323, 337, 331, 318, 329, 339, 328, 319, 331, 339, 324, 319, 332, 338, 326, 323, 338, 341, 331, 334, 347, 352, 347, 351, 366, 359, 341, 341, 345, 333, 319, 327, 336, 324, 315, 327, 331, 315, 312, 325, 319, 277, 251, 253, 276, 341, 437, 466, 426, 380, 365, 372, 366, 343, 335, 334, 318, 307, 320, 328, 315, 309, 323, 328, 314, 313, 331, 333, 319, 320, 332, 330, 320, 329, 347, 340, 324, 333, 346, 340, 331, 341, 352, 343, 338, 355, 365, 353, 348, 361, 363, 346, 340, 350, 345, 329, 328, 338, 328, 311, 317, 331, 324, 310, 319, 329, 320, 311, 322, 330, 316, 308, 325, 332, 317, 314, 326, 328, 315, 317, 332, 331, 318, 321, 334, 329, 319, 327, 338, 327, 314, 325, 337, 327, 317, 328, 337, 329, 322, 333, 335, 325, 328, 345, 344, 334, 344, 361, 357, 346, 352, 359, 347, 331, 331, 338, 327, 316, 325, 332, 319, 311, 322, 330, 318, 302, 286, 263, 241, 273, 375, 462, 454, 420, 399, 372, 352, 359, 360, 333, 310, 317, 329, 317, 308, 320, 328, 320, 313, 324, 328, 317, 318, 329, 330, 316, 318, 337, 333, 321, 327, 339, 334, 325, 335, 346, 340, 332, 343, 352, 343, 340, 355, 361, 348, 344, 357, 355, 336, 332, 340, 333, 318, 321, 330, 320, 307, 316, 327, 320, 307, 317, 328, 316, 306, 320, 330, 318, 310, 323, 328, 316, 314, 327, 329, 316, 317, 330, 327, 317, 319, 331, 326, 317, 327, 337, 324, 311, 324, 335, 326, 319, 331, 334, 324, 321, 334, 339, 328, 331, 345, 345, 335, 342, 357, 357, 347, 351, 355, 341, 326, 330, 335, 323, 311, 323, 332, 318, 311, 326, 330, 311, 284, 268, 257, 261, 328, 428, 446, 400, 374, 370, 357, 343, 351, 351, 328, 310, 317, 327, 319, 311, 324, 327, 311, 308, 323, 327, 315, 316, 334, 333, 320, 323, 336, 332, 321, 329, 344, 336, 326, 338, 349, 343, 339, 349, 357, 347, 343, 361, 368, 352, 348, 360, 356, 336, 334, 344, 337, 319, 321, 331, 324, 313, 321, 333, 322, 307, 318, 328, 318, 313, 324, 329, 318, 313, 328, 331, 320, 319, 332, 335, 321, 321, 336, 330, 317, 325, 338, 332, 318, 326, 338, 331, 323, 332, 338, 328, 325, 339, 349, 338, 336, 356, 363, 347, 349, 366, 361, 338, 338, 346, 337, 321, 326, 338, 328, 316, 324, 332, 321, 315, 323, 311, 269, 246, 268, 328, 403, 432, 421, 395, 359, 351, 363, 358, 334, 326, 335, 325, 312, 320, 331, 321, 311, 325, 334, 324, 317, 332, 341, 326, 322, 338, 342, 328, 330, 347, 345, 329, 334, 349, 347, 336, 342, 357, 352, 342, 353, 369, 361, 351, 361, 365, 348, 337, 347, 349, 331, 321, 333, 331, 317, 317, 329, 324, 313, 320, 328, 321, 312, 320, 330, 320, 311, 323, 331, 319, 311, 327, 333, 320, 319, 333, 332, 319, 318, 333, 332, 317, 320, 334, 331, 320, 326, 338, 334, 321, 330, 344, 337, 331, 341, 351, 347, 343, 359, 368, 353, 341, 343, 340, 323, 322, 335, 329, 314, 320, 334, 328, 313, 318, 328, 303, 261, 251, 262, 298, 385, 458, 449, 401, 365, 364, 367, 354, 342, 338, 325, 306, 312, 325, 321, 310, 316, 327, 322, 312, 323, 331, 321, 315, 327, 336, 323, 317, 332, 339, 325, 323, 338, 339, 327, 331, 348, 348, 336, 342, 361, 356, 344, 353, 366, 354, 339, 345, 348, 333, 321, 330, 333, 318, 311, 326, 328, 311, 309, 322, 320, 306, 311, 326, 322, 307, 312, 326, 322, 309, 320, 334, 323, 310, 324, 334, 320, 316, 330, 335, 319, 318, 334, 338, 325, 324, 336, 333, 320, 326, 341, 338, 329, 337, 352, 350, 343, 353, 365, 355, 340, 345, 344, 327, 319, 331, 334, 319, 317, 330, 328, 312, 318, 333, 311, 265, 248, 258, 290, 369, 450, 456, 416, 376, 369, 371, 359, 339, 335, 334, 318, 310, 323, 327, 313, 311, 324, 325, 314, 317, 330, 324, 314, 324, 337, 328, 318, 331, 344, 334, 325, 338, 345, 334, 332, 346, 353, 342, 342, 357, 359, 345, 350, 365, 359, 343, 345, 352, 339, 320, 327, 333, 324, 311, 320, 327, 313, 306, 321, 328, 312, 308, 325, 323, 307, 311, 326, 327, 312, 315, 330, 326, 314, 327, 338, 328, 316, 328, 338, 325, 316, 331, 338, 323, 317, 332, 336, 323, 321, 335, 338, 327, 332, 348, 347, 341, 353, 366, 360, 347, 348, 352, 341, 329, 332, 335, 323, 318, 326, 331, 318, 317, 331, 330, 310, 283, 268, 252, 255, 333, 442, 464, 418, 393, 384, 364, 353, 356, 347, 323, 309, 315, 323, 315, 314, 325, 329, 315, 313, 329, 330, 316, 319, 336, 338, 322, 326, 339, 333, 324, 333, 345, 337, 330, 345, 358, 342, 332, 347, 356, 349, 349, 362, 365, 352, 344, 352, 348, 331, 330, 338, 330, 315, 321, 336, 325, 309, 317, 325, 318, 312, 325, 330, 317, 315, 327, 329, 317, 315, 333, 338, 325, 325, 344, 346, 332, 329, 339, 331, 318, 323, 331, 325, 322, 332, 335, 324, 318, 332, 337, 323, 320, 333, 331, 316, 325, 343, 340, 324, 329, 347, 349, 341, 348, 361, 353, 343, 345, 346, 330, 319, 329, 333, 320, 313, 327, 329, 315, 314, 324, 322, 288, 266, 268, 274, 327, 414, 445, 411, 371, 359, 357, 349, 345, 345, 340, 322, 312, 322, 330, 318, 311, 322, 328, 318, 317, 330, 331, 319, 322, 338, 336, 324, 329, 336, 331, 324, 335, 344, 337, 334, 346, 352, 344, 343, 357, 365, 354, 352, 367, 368, 351, 349, 357, 343, 325, 328, 335, 325, 313, 321, 330, 322, 306, 311, 324, 319, 311, 319, 325, 313, 311, 326, 328, 315, 316, 329, 329, 318, 325, 338, 328, 313, 321, 335, 329, 317, 324, 336, 328, 320, 328, 332, 322, 318, 331, 336, 323, 321, 339, 342, 328, 335, 358, 357, 344, 350, 361, 348, 329, 333, 340, 328, 317, 325, 333, 320, 310, 325, 331, 318, 312, 314, 290, 259, 258, 304, 378, 418, 408, 397, 373, 343, 340, 351, 345, 328, 327, 334, 321, 309, 320, 328, 316, 311, 325, 332, 322, 320, 335, 336, 324, 326, 339, 339, 328, 335, 347, 344, 334, 345, 359, 352, 344, 357, 366, 358, 352, 367, 375, 364, 355, 365, 365, 346, 338, 344, 339, 323, 321, 332, 328, 313, 318, 332, 324, 311, 321, 332, 323, 314, 322, 332, 321, 314, 327, 333, 320, 319, 334, 334, 320, 322, 336, 335, 322, 327, 340, 334, 321, 329, 341, 336, 325, 332, 341, 331, 324, 335, 345, 336, 333, 344, 350, 344, 345, 361, 363, 349, 345, 352, 345, 326, 327, 338, 331, 317, 324, 335, 325, 316, 327, 336, 318, 288, 277, 269, 271, 335, 433, 455, 411, 383, 373, 358, 347, 348, 348, 331, 311, 314, 327, 319, 306, 321, 334, 324, 316, 324, 331, 320, 320, 336, 339, 325, 320, 333, 334, 323, 328, 343, 342, 329, 333, 350, 348, 339, 346, 358, 351, 346, 358, 366, 355, 342, 350, 354, 337, 324, 332, 334, 316, 313, 326, 327, 313, 317, 327, 318, 303, 310, 324, 318, 309, 320, 330, 318, 308, 319, 328, 317, 312, 327, 331, 313, 310, 329, 330, 318, 317, 329, 327, 316, 318, 332, 327, 314, 323, 338, 325, 314, 324, 335, 326, 322, 338, 350, 341, 340, 357, 356, 334, 326, 335, 328, 312, 315, 330, 328, 313, 313, 324, 320, 310, 317, 320, 285, 251, 252, 293, 371, 437, 442, 410, 368, 352, 364, 366, 342, 329, 331, 324, 309, 314, 329, 326, 314, 319, 330, 322, 318, 333, 341, 330, 320, 332, 341, 333, 330, 341, 343, 329, 327, 347, 356, 340, 360, 380, 363, 336, 342, 364, 363, 355, 372, 385, 362, 335, 342, 350, 337, 323, 336, 341, 319, 314, 335, 342, 333, 336, 339, 323, 302, 319, 351, 327, 302, 312, 336, 338, 324, 334, 336, 320, 329, 348, 349, 338, 325, 319, 315, 312, 329, 343, 333, 315, 325, 339, 332, 320, 328, 332, 329, 334, 363, 360, 311, 296, 336, 367, 342, 334, 350, 363, 336, 328, 348, 351, 322, 319, 325, 321, 314, 309, 328, 323, 291, 303, 315, 280, 254, 261, 286, 353, 419, 423, 403, 370, 344, 360, 369, 338, 323, 318, 309, 296, 304, 322, 315, 305, 319, 333, 309, 291, 312, 341, 326, 296, 310, 323, 321, 323, 341, 336, 325, 333, 335, 316, 310, 338, 371, 358, 329, 331, 354, 364, 371, 372, 357, 352, 341, 327, 332, 317, 305, 328, 334, 322, 310, 314, 317, 318, 312, 321, 334, 318, 304, 322, 328, 313, 312, 317, 313, 314, 315, 322, 329, 325, 322, 330, 320, 316, 331, 347, 338, 311, 324, 332, 321, 310, 325, 349, 325, 304, 319, 328, 336, 355, 374, 352, 324, 349, 368, 341, 318, 335, 348, 343, 337, 332, 337, 330, 314, 318, 328, 329, 336, 334, 313, 303, 295, 281, 257, 257, 329, 422, 422, 390, 383, 379, 351, 337, 353, 356, 331, 322, 331, 337, 311, 305, 325, 325, 310, 308, 328, 333, 326, 326, 324, 322, 326, 340, 335, 320, 323, 341, 343, 333, 331, 349, 350, 339, 338, 352, 356, 353, 360, 369, 361, 343, 366, 368, 348, 333, 336, 338, 324, 327, 344, 340, 312, 296, 304, 314, 311, 325, 336, 335, 311, 309, 329, 329, 321, 320, 324, 328, 323, 324, 332, 334, 328, 320, 336, 336, 322, 326, 344, 338, 316, 314, 319, 330, 330, 338, 330, 316, 328, 347, 367, 343, 329, 347, 356, 346, 356, 370, 344, 326, 329, 335, 328, 326, 339, 338, 313, 305, 339, 347, 311, 308, 320, 284, 251, 259, 323, 409, 432, 416, 388, 369, 359, 350, 348, 327, 317, 321, 334, 329, 311, 315, 318, 316, 317, 339, 318, 298, 307, 328, 330, 316, 321, 340, 345, 331, 327, 332, 324, 328, 351, 350, 332, 334, 363, 366, 347, 337, 357, 373, 357, 353, 362, 361, 341, 330, 338, 325, 313, 331, 340, 333, 320, 306, 318, 323, 313, 315, 324, 315, 304, 335, 332, 300, 304, 329, 327, 315, 317, 343, 348, 311, 315, 327, 323, 320, 321, 330, 320, 320, 323, 325, 317, 326, 338, 342, 332, 327, 352, 362, 330, 310, 334, 356, 346, 328, 334, 347, 340, 333, 331, 312, 297, 333, 360, 334, 306, 319, 331, 309, 282, 273, 265, 253, 265, 346, 438, 432, 389, 374, 360, 348, 347, 344, 331, 312, 311, 317, 314, 315, 327, 326, 312, 299, 312, 328, 318, 315, 326, 324, 318, 342, 360, 328, 301, 311, 336, 342, 322, 325, 350, 350, 344, 365, 374, 356, 335, 361, 378, 353, 349, 367, 369, 326, 303, 325, 342, 325, 326, 342, 328, 324, 329, 316, 293, 292, 321, 341, 332, 308, 317, 327, 319, 314, 310, 311, 327, 330, 334, 315, 301, 333, 374, 340, 304, 308, 324, 326, 324, 342, 339, 329, 335, 332, 331, 316, 326, 343, 352, 365, 375, 352, 334, 328, 327, 330, 317, 306, 333, 368, 349, 328, 301, 300, 306, 307, 314, 330, 298, 257, 258, 276, 323, 401, 436, 406, 364, 341, 351, 363, 338, 329, 334, 325, 314, 305, 312, 311, 308, 317, 328, 320, 308, 320, 334, 326, 316, 326, 331, 318, 318, 336, 340, 328, 326, 339, 341, 330, 334, 349, 352, 341, 346, 365, 363, 350, 356, 364, 355, 340, 345, 348, 333, 320, 327, 330, 317, 309, 323, 326, 313, 312, 322, 323, 309, 312, 325, 323, 313, 321, 332, 321, 311, 321, 332, 323, 315, 326, 335, 323, 318, 331, 340, 326, 321, 334, 335, 325, 330, 347, 350, 344, 350, 362, 361, 351, 350, 354, 339, 324, 331, 338, 329, 320, 333, 337, 321, 318, 333, 333, 314, 296, 283, 268, 265, 325, 425, 451, 410, 385, 377, 360, 348, 359, 362, 338, 320, 325, 330, 321, 321, 335, 339, 325, 322, 335, 337, 324, 327, 342, 341, 326, 329, 342, 338, 331, 338, 353, 349, 338, 346, 360, 355, 349, 363, 372, 363, 358, 373, 378, 362, 354, 363, 357, 338, 335, 345, 338, 322, 321, 335, 332, 318, 322, 331, 321, 311, 323, 330, 318, 314, 327, 330, 319, 317, 335, 336, 321, 322, 338, 336, 323, 323, 335, 332, 321, 330, 345, 339, 330, 342, 359, 350, 345, 360, 363, 348, 340, 345, 342, 327, 323, 334, 330, 317, 319, 332, 329, 316, 317, 315, 283, 252, 265, 328, 406, 434, 419, 396, 362, 342, 351, 360, 339, 323, 333, 332, 314, 312, 329, 329, 316, 318, 330, 324, 315, 323, 332, 326, 320, 331, 345, 333, 323, 335, 346, 338, 338, 351, 354, 341, 338, 351, 358, 353, 356, 369, 363, 351, 356, 368, 356, 337, 343, 348, 328, 313, 320, 332, 324, 317, 324, 325, 312, 312, 328, 332, 316, 313, 328, 329, 317, 320, 333, 332, 320, 327, 336, 326, 315, 328, 338, 326, 319, 331, 340, 328, 325, 343, 357, 353, 348, 360, 359, 336, 330, 343, 338, 317, 318, 332, 328, 319, 320, 325, 316, 312, 329, 332, 291, 252, 257, 286, 344, 419, 443, 410, 368, 350, 359, 361, 344, 335, 335, 325, 311, 317, 330, 323, 311, 324, 337, 322, 312, 321, 325, 314, 313, 330, 337, 326, 326, 344, 345, 328, 334, 348, 338, 325, 338, 356, 351, 340, 352, 369, 364, 354, 362, 368, 350, 336, 345, 347, 329, 321, 334, 333, 318, 314, 324, 323, 307, 311, 325, 323, 313, 318, 328, 320, 308, 318, 331, 324, 315, 323, 331, 320, 316, 330, 335, 324, 322, 336, 338, 328, 331, 342, 341, 334, 344, 357, 353, 343, 347, 352, 338, 322, 328, 335, 323, 313, 322, 330, 320, 315, 326, 326, 310, 300, 290, 269, 249, 287, 390, 449, 426, 393, 374, 354, 343, 355, 356, 326, 308, 316, 326, 316, 309, 323, 329, 316, 313, 327, 331, 317, 320, 335, 331, 319, 326, 340, 336, 322, 331, 343, 336, 329, 341, 352, 344, 338, 348, 359, 351, 346, 360, 365, 353, 350, 359, 352, 330, 328, 338, 333, 318, 318, 329, 320, 306, 317, 328, 318, 308, 318, 327, 316, 312, 327, 332, 319, 315, 329, 329, 317, 319, 336, 333, 320, 325, 337, 329, 317, 330, 346, 341, 330, 339, 354, 350, 345, 362, 368, 346, 331, 340, 344, 326, 320, 332, 332, 315, 316, 331, 325, 311, 317, 328, 306, 267, 263, 284, 330, 406, 445, 426, 380, 350, 353, 358, 346, 330, 332, 326, 311, 312, 327, 325, 313, 313, 325, 324, 313, 320, 329, 323, 317, 329, 341, 330, 324, 336, 345, 331, 324, 338, 343, 334, 337, 354, 355, 345, 350, 362, 361, 347, 349, 361, 354, 340, 341, 343, 332, 320, 328, 333, 319, 312, 322, 325, 311, 310, 326, 326, 310, 311, 326, 323, 309, 312, 325, 321, 308, 317, 332, 323, 312, 322, 330, 320, 313, 325, 333, 323, 320, 337, 343, 331, 332, 354, 356, 342, 350, 363, 354, 332, 334, 341, 328, 313, 324, 335, 322, 311, 321, 330, 317, 310, 311, 288, 258, 257, 315, 401, 431, 411, 393, 364, 338, 341, 358, 348, 324, 322, 328, 321, 312, 319, 328, 318, 313, 326, 331, 320, 317, 332, 335, 326, 327, 339, 338, 323, 329, 348, 346, 331, 336, 350, 347, 339, 352, 365, 354, 345, 356, 368, 359, 352, 363, 363, 342, 332, 342, 340, 325, 323, 336, 332, 315, 319, 332, 326, 313, 321, 334, 325, 312, 321, 334, 325, 318, 331, 338, 323, 319, 333, 337, 324, 322, 336, 334, 324, 328, 344, 340, 328, 334, 352, 353, 344, 353, 366, 360, 344, 346, 347, 328, 319, 331, 334, 317, 313, 329, 331, 316, 315, 327, 317, 279, 260, 268, 295, 367, 447, 453, 403, 358, 354, 362, 356, 341, 337, 332, 314, 311, 326, 330, 316, 313, 327, 328, 316, 318, 331, 326, 315, 323, 337, 334, 324, 331, 343, 336, 328, 340, 347, 334, 329, 342, 353, 345, 343, 358, 362, 350, 349, 362, 358, 340, 337, 343, 335, 321, 326, 331, 322, 316, 323, 326, 313, 306, 319, 327, 318, 313, 324, 328, 318, 317, 330, 328, 314, 316, 331, 330, 320, 322, 331, 326, 315, 322, 336, 327, 320, 336, 344, 330, 327, 349, 360, 346, 345, 358, 353, 331, 328, 341, 333, 313, 314, 327, 324, 308, 314, 328, 321, 310, 304, 283, 252, 239, 288, 389, 445, 428, 406, 382, 350, 344, 358, 352, 323, 314, 322, 318, 308, 316, 325, 318, 309, 316, 325, 320, 315, 326, 334, 322, 319, 333, 338, 323, 322, 339, 341, 331, 330, 342, 346, 338, 343, 354, 348, 339, 346, 359, 355, 345, 352, 357, 343, 328, 334, 339, 321, 317, 326, 323, 306, 307, 326, 327, 314, 313, 323, 319, 312, 321, 334, 325, 307, 317, 333, 327, 317, 325, 334, 324, 322, 334, 333, 320, 318, 331, 331, 322, 326, 339, 336, 329, 336, 349, 349, 343, 349, 360, 352, 339, 344, 348, 336, 325, 327, 329, 320, 320, 327, 326, 317, 313, 322, 315, 280, 254, 256, 277, 348, 435, 458, 418, 373, 361, 363, 360, 349, 342, 329, 311, 307, 321, 328, 315, 312, 325, 330, 320, 318, 332, 330, 318, 321, 335, 327, 317, 328, 344, 336, 326, 336, 350, 336, 325, 338, 350, 343, 343, 359, 364, 353, 354, 369, 368, 350, 342, 348, 342, 328, 330, 336, 329, 315, 318, 329, 325, 317, 322, 329, 320, 306, 320, 331, 322, 312, 331, 343, 324, 313, 342, 356, 337, 331, 356, 364, 343, 340, 368, 370, 344, 348, 374, 370, 350, 362, 389, 378, 356, 373, 400, 393, 377, 397, 411, 387, 370, 391, 398, 365, 352, 377, 381, 358, 356, 380, 380, 357, 359, 373, 341, 292, 292, 328, 384, 446, 479, 467, 417, 377, 387, 404, 382, 359, 369, 366, 340, 335, 359, 360, 335, 334, 358, 354, 333, 341, 367, 360, 337, 349, 372, 362, 345, 363, 384, 365, 350, 371, 385, 366, 359, 383, 391, 373, 371, 397, 398, 368, 373, 398, 389, 361, 361, 375, 357, 332, 345, 363, 346, 325, 338, 351, 331, 320, 344, 351, 328, 325, 348, 345, 319, 322, 348, 348, 325, 328, 353, 343, 322, 333, 354, 341, 321, 339, 356, 340, 327, 350, 359, 341, 334, 355, 366, 352, 353, 376, 377, 348, 347, 365, 353, 327, 335, 353, 334, 311, 326, 345, 330, 315, 332, 335, 289, 255, 273, 303, 347, 408, 437, 415, 365, 347, 358, 353, 335, 332, 342, 327, 304, 317, 336, 320, 302, 320, 338, 319, 305, 328, 338, 314, 308, 332, 338, 317, 316, 340, 340, 318, 323, 345, 339, 320, 332, 355, 342, 324, 341, 362, 349, 334, 351, 363, 337, 321, 338, 342, 315, 308, 329, 331, 305, 304, 326, 321, 296, 301, 326, 320, 296, 303, 322, 308, 293, 311, 329, 313, 300, 320, 329, 305, 298, 323, 327, 306, 302, 323, 323, 305, 312, 336, 331, 314, 329, 354, 343, 329, 343, 351, 325, 307, 321, 329, 306, 295, 316, 324, 300, 295, 318, 319, 291, 272, 265, 253, 245, 314, 420, 431, 382, 367, 363, 336, 318, 337, 349, 320, 296, 309, 321, 302, 295, 316, 323, 300, 300, 320, 321, 300, 305, 331, 326, 304, 314, 338, 327, 308, 324, 344, 328, 313, 332, 347, 329, 319, 344, 356, 338, 336, 357, 356, 333, 333, 352, 346, 320, 320, 337, 326, 304, 315, 334, 315, 297, 313, 328, 312, 298, 317, 328, 307, 300, 323, 330, 310, 308, 330, 329, 308, 311, 333, 328, 306, 313, 334, 325, 307, 324, 345, 330, 317, 341, 361, 345, 336, 356, 358, 328, 317, 337, 336, 310, 306, 327, 323, 302, 310, 329, 320, 295, 291, 280, 248, 232, 300, 413, 451, 415, 397, 379, 345, 337, 360, 352, 312, 299, 319, 321, 300, 302, 323, 317, 297, 307, 328, 318, 302, 317, 334, 321, 308, 326, 337, 318, 310, 332, 340, 321, 318, 343, 348, 328, 329, 352, 350, 331, 340, 363, 355, 334, 344, 358, 339, 318, 329, 340, 318, 303, 321, 330, 310, 300, 320, 325, 305, 304, 325, 324, 303, 306, 328, 323, 305, 315, 335, 325, 305, 318, 336, 324, 309, 326, 339, 324, 316, 342, 354, 332, 329, 360, 364, 344, 346, 364, 352, 323, 326, 345, 332, 311, 323, 341, 326, 306, 321, 337, 321, 297, 288, 271, 242, 264, 372, 468, 452, 411, 399, 378, 351, 352, 362, 340, 308, 313, 330, 320, 301, 315, 333, 316, 302, 322, 337, 320, 308, 329, 342, 321, 315, 339, 341, 320, 322, 346, 346, 325, 329, 349, 343, 328, 343, 363, 351, 335, 351, 367, 352, 338, 353, 360, 332, 321, 338, 341, 316, 312, 332, 331, 310, 314, 335, 330, 308, 315, 333, 324, 307, 320, 340, 326, 309, 325, 339, 323, 313, 333, 342, 322, 317, 337, 340, 322, 325, 348, 347, 327, 337, 367, 362, 342, 355, 370, 350, 325, 334, 344, 327, 312, 329, 338, 318, 311, 333, 336, 313, 302, 301, 273, 237, 258, 356, 446, 446, 420, 405, 375, 348, 360, 368, 336, 308, 322, 334, 314, 300, 320, 330, 310, 304, 326, 332, 310, 311, 333, 334, 314, 321, 345, 340, 320, 329, 347, 336, 322, 336, 353, 343, 332, 352, 364, 344, 337, 360, 369, 348, 341, 359, 354, 330, 325, 340, 332, 310, 316, 333, 320, 301, 316, 336, 320, 302, 320, 337, 319, 307, 327, 337, 318, 314, 333, 336, 314, 315, 338, 334, 311, 315, 335, 329, 312, 327, 345, 336, 324, 345, 365, 353, 339, 360, 373, 345, 326, 343, 345, 320, 320, 338, 334, 309, 307, 330, 328, 307, 310, 314, 277, 244, 263, 331, 402, 430, 430, 409, 366, 342, 354, 366, 344, 326, 336, 336, 311, 312, 335, 336, 313, 318, 339, 334, 316, 327, 346, 334, 320, 335, 352, 335, 320, 342, 355, 336, 325, 347, 358, 339, 337, 362, 365, 348, 352, 373, 369, 348, 355, 373, 359, 336, 343, 358, 345, 325, 332, 343, 327, 316, 332, 339, 318, 312, 334, 340, 322, 319, 340, 339, 319, 325, 346, 340, 316, 324, 343, 333, 319, 334, 349, 333, 322, 342, 354, 339, 335, 358, 369, 355, 355, 367, 357, 329, 329, 347, 338, 314, 320, 341, 332, 314, 324, 340, 325, 297, 285, 282, 268, 294, 397, 456, 421, 384, 380, 370, 338, 340, 361, 351, 320, 318, 336, 329, 310, 321, 338, 322, 306, 322, 341, 328, 314, 330, 344, 327, 319, 340, 348, 332, 328, 347, 349, 328, 333, 356, 354, 338, 347, 367, 360, 344, 356, 373, 358, 339, 350, 357, 336, 321, 338, 342, 317, 311, 330, 332, 310, 309, 330, 329, 309, 312, 332, 326, 308, 319, 338, 324, 306, 321, 341, 326, 313, 330, 340, 321, 312, 332, 341, 325, 324, 345, 349, 335, 342, 366, 363, 343, 345, 358, 344, 320, 326, 340, 328, 312, 325, 337, 321, 309, 326, 330, 291, 256, 261, 277, 317, 405, 458, 427, 367, 345, 358, 359, 341, 338, 341, 323, 306, 318, 332, 316, 304, 319, 330, 315, 307, 328, 335, 318, 316, 339, 342, 324, 325, 342, 341, 323, 330, 352, 345, 329, 342, 362, 351, 336, 352, 368, 356, 345, 359, 367, 345, 334, 351, 352, 328, 323, 339, 336, 316, 318, 335, 328, 309, 320, 337, 325, 306, 318, 337, 325, 311, 329, 343, 327, 314, 333, 340, 320, 316, 337, 339, 318, 321, 347, 349, 328, 334, 356, 355, 342, 354, 373, 354, 331, 340, 350, 330, 315, 332, 341, 318, 309, 330, 336, 311, 306, 322, 296, 249, 244, 290, 364, 431, 446, 430, 383, 342, 352, 374, 352, 320, 322, 326, 308, 300, 319, 326, 305, 297, 320, 328, 309, 310, 331, 326, 307, 317, 340, 332, 315, 328, 346, 334, 319, 338, 355, 338, 326, 345, 356, 339, 333, 355, 362, 341, 339, 358, 354, 325, 326, 344, 335, 312, 316, 335, 326, 307, 318, 333, 316, 303, 322, 334, 317, 309, 328, 333, 315, 311, 331, 333, 312, 314, 337, 336, 317, 322, 340, 332, 314, 325, 345, 337, 324, 340, 357, 350, 338, 354, 364, 348, 336, 348, 349, 322, 315, 333, 333, 311, 312, 330, 324, 305, 314, 331, 305, 256, 250, 270, 307, 391, 461, 453, 400, 359, 365, 376, 352, 327, 331, 328, 306, 305, 326, 323, 305, 313, 333, 326, 306, 318, 334, 324, 314, 327, 338, 326, 318, 335, 344, 325, 322, 343, 349, 331, 332, 355, 357, 338, 345, 365, 360, 340, 347, 366, 354, 331, 338, 348, 328, 311, 327, 336, 318, 307, 326, 330, 309, 305, 325, 324, 302, 302, 327, 326, 307, 314, 335, 327, 309, 321, 342, 327, 309, 322, 337, 324, 315, 332, 339, 320, 313, 337, 346, 333, 329, 344, 348, 336, 343, 368, 364, 339, 337, 348, 337, 317, 328, 338, 322, 308, 323, 337, 318, 305, 321, 320, 274, 246, 262, 301, 365, 421, 435, 402, 357, 349, 363, 360, 334, 334, 342, 326, 308, 321, 332, 317, 306, 327, 339, 321, 314, 333, 336, 317, 319, 340, 339, 319, 323, 346, 345, 328, 333, 353, 346, 331, 345, 365, 352, 337, 353, 370, 355, 344, 362, 369, 344, 334, 352, 351, 323, 318, 337, 333, 314, 318, 334, 327, 309, 318, 337, 323, 305, 322, 341, 328, 312, 327, 338, 322, 316, 339, 340, 316, 316, 336, 339, 320, 320, 338, 335, 316, 324, 346, 341, 324, 336, 357, 351, 338, 355, 373, 354, 336, 348, 351, 326, 318, 337, 339, 316, 313, 334, 334, 313, 316, 327, 292, 251, 254, 297, 361, 416, 428, 414, 374, 344, 352, 365, 345, 325, 334, 335, 311, 309, 328, 328, 311, 314, 334, 328, 309, 315, 338, 333, 316, 328, 344, 330, 318, 336, 349, 334, 323, 340, 352, 335, 331, 353, 357, 336, 338, 361, 361, 342, 347, 365, 356, 333, 337, 350, 336, 315, 327, 343, 324, 306, 322, 333, 316, 308, 326, 331, 311, 308, 329, 333, 312, 312, 333, 330, 311, 317, 338, 329, 310, 322, 341, 328, 313, 330, 343, 325, 314, 334, 345, 331, 328, 350, 357, 344, 344, 365, 367, 345, 340, 353, 341, 316, 322, 342, 330, 310, 323, 336, 322, 310, 323, 322, 274, 241, 256, 299, 366, 435, 450, 417, 361, 346, 364, 363, 334, 326, 336, 322, 305, 316, 331, 316, 304, 320, 332, 317, 308, 323, 330, 312, 310, 332, 338, 318, 315, 335, 337, 317, 321, 343, 339, 321, 330, 350, 346, 332, 345, 364, 352, 337, 351, 358, 338, 326, 340, 342, 315, 309, 327, 330, 308, 306, 324, 322, 302, 307, 324, 316, 300, 314, 330, 319, 304, 315, 330, 315, 307, 324, 333, 315, 308, 328, 335, 317, 313, 330, 329, 310, 314, 335, 329, 310, 319, 340, 334, 320, 333, 351, 347, 338, 352, 365, 348, 330, 340, 345, 324, 317, 334, 334, 311, 309, 330, 328, 309, 313, 328, 302, 252, 245, 273, 321, 403, 455, 445, 398, 361, 364, 374, 350, 326, 332, 331, 310, 307, 329, 328, 309, 312, 332, 326, 306, 316, 336, 327, 310, 321, 342, 330, 318, 335, 348, 332, 322, 341, 348, 331, 330, 352, 353, 336, 341, 364, 362, 343, 348, 366, 357, 334, 339, 352, 335, 312, 323, 338, 321, 308, 324, 335, 313, 303, 324, 329, 310, 308, 327, 327, 308, 313, 333, 329, 312, 320, 339, 326, 308, 322, 340, 326, 312, 326, 336, 321, 314, 330, 338, 320, 316, 336, 336, 316, 319, 338, 339, 321, 330, 352, 346, 331, 345, 366, 358, 343, 352, 358, 337, 322, 336, 342, 322, 311, 330, 336, 315, 312, 334, 334, 298, 268, 265, 258, 282, 385, 473, 451, 398, 380, 380, 362, 346, 348, 343, 316, 303, 322, 330, 312, 304, 320, 325, 307, 306, 328, 329, 312, 317, 337, 330, 312, 326, 348, 335, 317, 331, 348, 332, 317, 338, 352, 337, 331, 352, 359, 339, 339, 362, 360, 338, 339, 354, 346, 321, 323, 338, 328, 308, 315, 328, 313, 299, 315, 329, 312, 302, 321, 329, 312, 307, 327, 330, 312, 309, 330, 333, 312, 313, 331, 328, 311, 320, 339, 328, 311, 322, 337, 322, 310, 328, 339, 322, 318, 335, 335, 314, 312, 333, 337, 320, 324, 348, 342, 326, 341, 366, 360, 344, 353, 362, 343, 326, 333, 340, 320, 310, 331, 338, 317, 311, 331, 333, 306, 282, 275, 265, 260, 331, 433, 442, 394, 376, 375, 354, 338, 352, 357, 330, 312, 322, 334, 319, 312, 328, 334, 317, 315, 336, 337, 318, 318, 339, 338, 321, 330, 348, 340, 320, 332, 354, 344, 326, 341, 360, 349, 339, 356, 368, 353, 350, 370, 374, 351, 347, 364, 358, 336, 336, 350, 338, 316, 323, 340, 329, 314, 325, 339, 323, 312, 330, 341, 322, 312, 333, 341, 322, 317, 338, 341, 320, 321, 342, 338, 318, 325, 343, 334, 317, 328, 344, 330, 315, 329, 343, 329, 319, 338, 349, 332, 326, 350, 364, 349, 348, 370, 364, 337, 334, 350, 339, 317, 324, 340, 328, 309, 323, 336, 320, 309, 318, 308, 262, 239, 271, 341, 406, 420, 417, 394, 348, 336, 357, 359, 330, 322, 334, 321, 300, 312, 333, 323, 308, 323, 336, 319, 307, 329, 341, 323, 315, 334, 339, 323, 322, 341, 345, 327, 331, 354, 348, 329, 337, 358, 354, 339, 348, 364, 354, 343, 358, 367, 345, 329, 341, 347, 325, 319, 337, 336, 314, 311, 331, 328, 309, 316, 337, 329, 308, 319, 338, 328, 313, 324, 338, 323, 311, 327, 337, 321, 315, 334, 339, 320, 319, 338, 337, 318, 322, 339, 333, 314, 326, 351, 344, 325, 335, 354, 350, 341, 358, 370, 350, 330, 343, 348, 323, 313, 331, 333, 313, 313, 333, 329, 309, 312, 318, 281, 239, 242, 294, 371, 432, 443, 423, 377, 347, 358, 372, 348, 324, 331, 330, 308, 307, 328, 328, 307, 311, 332, 328, 311, 321, 338, 327, 310, 322, 341, 329, 314, 329, 344, 331, 321, 340, 349, 333, 330, 349, 355, 339, 343, 362, 357, 340, 347, 365, 355, 332, 334, 345, 330, 312, 324, 338, 323, 312, 324, 330, 316, 310, 327, 334, 315, 312, 331, 331, 312, 316, 334, 333, 316, 323, 339, 330, 314, 326, 341, 330, 313, 325, 338, 324, 314, 332, 340, 323, 316, 335, 338, 318, 321, 345, 346, 324, 332, 359, 359, 339, 349, 366, 348, 324, 333, 345, 325, 311, 323, 332, 315, 307, 325, 332, 313, 305, 305, 274, 238, 246, 322, 416, 449, 430, 415, 379, 346, 356, 369, 341, 312, 320, 332, 313, 300, 318, 327, 311, 308, 324, 328, 311, 313, 332, 329, 314, 319, 339, 334, 315, 323, 344, 335, 318, 332, 349, 337, 326, 343, 357, 342, 335, 355, 365, 350, 347, 363, 362, 337, 334, 348, 338, 317, 322, 338, 327, 308, 316, 334, 321, 307, 319, 332, 319, 308, 322, 332, 318, 315, 332, 336, 318, 316, 336, 336, 317, 324, 344, 338, 319, 327, 342, 332, 317, 330, 342, 328, 314, 326, 339, 324, 315, 332, 338, 319, 318, 344, 348, 328, 328, 350, 354, 341, 347, 367, 359, 336, 338, 351, 337, 316, 327, 340, 326, 315, 329, 336, 316, 310, 324, 309, 256, 236, 262, 324, 405, 456, 452, 415, 370, 366, 379, 362, 329, 325, 334, 318, 307, 321, 331, 317, 308, 326, 331, 311, 307, 328, 334, 317, 317, 336, 335, 314, 324, 346, 341, 322, 332, 350, 338, 323, 337, 354, 344, 336, 352, 362, 344, 342, 362, 365, 341, 336, 350, 346, 325, 327, 341, 331, 311, 321, 342, 330, 308, 319, 339, 330, 315, 330, 343, 322, 311, 331, 342, 327, 318, 335, 341, 325, 325, 343, 340, 321, 328, 346, 338, 321, 332, 347, 331, 313, 330, 348, 331, 318, 338, 347, 325, 316, 338, 348, 329, 325, 344, 341, 320, 328, 352, 350, 330, 340, 366, 360, 342, 360, 378, 359, 335, 342, 353, 332, 317, 334, 342, 320, 312, 330, 332, 312, 308, 311, 277, 239, 245, 315, 406, 447, 442, 425, 379, 345, 355, 372, 346, 317, 324, 334, 313, 306, 327, 332, 311, 307, 328, 331, 313, 314, 332, 332, 318, 328, 342, 330, 312, 324, 342, 331, 320, 336, 347, 332, 324, 346, 354, 335, 333, 356, 361, 341, 341, 364, 363, 341, 340, 353, 342, 317, 320, 337, 327, 309, 318, 331, 319, 309, 324, 331, 314, 306, 324, 333, 314, 309, 328, 330, 311, 312, 332, 332, 317, 324, 341, 332, 313, 323, 341, 328, 313, 326, 340, 325, 315, 333, 342, 324, 315, 332, 336, 318, 317, 340, 340, 320, 323, 346, 340, 324, 340, 363, 353, 338, 351, 362, 342, 323, 331, 339, 321, 313, 329, 331, 311, 312, 334, 330, 304, 293, 285, 264, 245, 290, 398, 448, 418, 395, 386, 360, 334, 350, 360, 335, 315, 324, 329, 313, 309, 330, 334, 314, 312, 333, 332, 311, 317, 339, 336, 316, 324, 343, 336, 320, 335, 350, 336, 323, 338, 351, 336, 333, 354, 359, 342, 342, 364, 369, 346, 343, 362, 353, 326, 329, 347, 339, 318, 322, 334, 322, 307, 320, 333, 317, 306, 324, 331, 314, 308, 329, 336, 319, 315, 335, 336, 315, 316, 337, 333, 313, 321, 341, 332, 313, 324, 341, 327, 314, 328, 338, 323, 315, 333, 340, 324, 320, 339, 342, 319, 318, 339, 339, 321, 329, 350, 343, 328, 345, 366, 354, 342, 355, 365, 342, 327, 338, 342, 322, 317, 336, 337, 316, 312, 331, 333, 311, 295, 277, 257, 245, 307, 415, 457, 422, 399, 385, 356, 341, 356, 355, 324, 306, 322, 326, 308, 303, 322, 328, 310, 307, 327, 330, 311, 314, 332, 328, 311, 319, 338, 331, 318, 332, 345, 334, 325, 339, 350, 333, 325, 345, 354, 337, 336, 357, 363, 348, 347, 360, 351, 329, 331, 345, 331, 311, 318, 333, 323, 307, 320, 331, 314, 303, 324, 334, 314, 307, 328, 333, 314, 311, 330, 329, 311, 316, 336, 331, 312, 322, 339, 327, 311, 324, 338, 326, 314, 329, 336, 321, 316, 335, 338, 318, 316, 335, 337, 319, 320, 338, 337, 322, 331, 350, 346, 335, 347, 364, 355, 342, 348, 350, 330, 319, 333, 338, 318, 310, 329, 332, 313, 312, 331, 326, 286, 257, 259, 263, 308, 416, 478, 442, 391, 377, 377, 364, 345, 343, 337, 316, 305, 321, 327, 310, 307, 328, 331, 310, 311, 331, 328, 311, 318, 336, 330, 316, 327, 345, 335, 322, 335, 347, 335, 326, 342, 353, 341, 335, 354, 361, 344, 344, 365, 368, 347, 345, 357, 346, 322, 326, 341, 332, 314, 322, 337, 323, 307, 323, 335, 320, 313, 328, 333, 314, 312, 331, 334, 316, 318, 336, 331, 312, 318, 339, 335, 317, 323, 339, 329, 314, 327, 342, 330, 317, 331, 339, 323, 318, 336, 339, 321, 319, 337, 337, 319, 319, 339, 337, 321, 331, 350, 345, 329, 341, 363, 357, 342, 355, 368, 352, 333, 344, 346, 325, 318, 336, 340, 318, 314, 331, 329, 312, 316, 320, 282, 242, 248, 310, 393, 450, 452, 432, 390, 362, 372, 377, 346, 320, 328, 330, 310, 308, 328, 330, 311, 313, 330, 325, 306, 316, 337, 334, 318, 324, 339, 331, 320, 336, 350, 337, 328, 345, 353, 335, 331, 354, 360, 341, 339, 358, 360, 347, 353, 369, 361, 336, 341, 354, 338, 320, 330, 342, 324, 313, 330, 334, 316, 306, 323, 328, 313, 312, 332, 331, 310, 313, 335, 331, 310, 315, 334, 328, 314, 325, 338, 324, 312, 327, 339, 326, 317, 336, 342, 324, 319, 333, 332, 316, 320, 341, 342, 320, 320, 337, 333, 317, 324, 341, 332, 315, 327, 339, 326, 317, 336, 349, 336, 325, 344, 358, 344, 344, 369, 371, 345, 334, 345, 337, 317, 324, 343, 333, 313, 318, 335, 324, 309, 314, 304, 265, 243, 272, 345, 414, 432, 422, 397, 359, 350, 366, 362, 333, 325, 337, 329, 312, 316, 332, 323, 310, 322, 339, 323, 310, 324, 335, 322, 316, 333, 340, 322, 319, 340, 345, 324, 326, 347, 348, 330, 337, 357, 351, 334, 344, 361, 356, 347, 359, 368, 353, 338, 344, 345, 330, 322, 337, 334, 311, 309, 329, 329, 311, 312, 329, 324, 310, 316, 334, 325, 308, 317, 334, 324, 311, 325, 338, 325, 318, 337, 342, 324, 317, 335, 338, 322, 323, 341, 339, 322, 326, 340, 335, 319, 326, 340, 330, 316, 329, 343, 332, 320, 334, 344, 331, 325, 344, 351, 339, 340, 359, 361, 349, 349, 361, 351, 327, 324, 340, 333, 315, 326, 339, 326, 310, 324, 335, 313, 282, 272, 269, 262, 317, 424, 455, 411, 379, 374, 360, 340, 349, 362, 343, 315, 317, 334, 325, 307, 316, 329, 320, 307, 325, 335, 318, 316, 337, 344, 324, 321, 340, 344, 326, 328, 345, 347, 332, 337, 351, 345, 334, 348, 366, 358, 344, 359, 370, 355, 343, 355, 360, 339, 324, 335, 337, 316, 312, 330, 326, 303, 305, 327, 318, 296, 304, 321, 312, 293, 307, 323, 310, 300, 314, 322, 308, 303, 320, 325, 308, 310, 331, 326, 302, 304, 327, 324, 305, 312, 330, 315, 297, 309, 328, 315, 297, 309, 323, 307, 298, 318, 325, 305, 297, 324, 336, 315, 316, 336, 325, 293, 293, 310, 297, 270, 275, 290, 279, 257, 269, 289, 271, 246, 249, 231, 196, 182, 230, 316, 360, 351, 328, 284, 253, 254, 264, 247, 220, 215, 219, 208, 202, 208, 213, 199, 201, 219, 221, 206, 213, 230, 228, 216, 221, 241, 241, 228, 236, 250, 247, 245, 257, 269, 263, 260, 274, 287, 281, 276, 297, 309, 299, 298, 311, 306, 289, 289, 299, 292, 275, 276, 297, 297, 280, 282, 308, 304, 279, 293, 326, 323, 303, 313, 338, 330, 316, 339, 355, 340, 331, 355, 364, 346, 347, 368, 368, 348, 348, 360, 350, 336, 342, 349, 339, 333, 341, 342, 328, 323, 336, 336, 320, 317, 335, 340, 329, 331, 343, 346, 344, 350, 363, 359, 346, 350, 352, 338, 327, 335, 340, 330, 323, 330, 336, 329, 329, 339, 335, 300, 272, 273, 281, 331, 430, 475, 442, 404, 391, 391, 378, 359, 353, 351, 340, 332, 342, 347, 339, 335, 344, 351, 343, 335, 345, 348, 344, 347, 360, 359, 348, 351, 365, 360, 349, 363, 374, 364, 357, 371, 382, 373, 371, 386, 393, 384, 379, 391, 394, 381, 374, 381, 371, 354, 356, 368, 356, 340, 345, 356, 350, 341, 350, 359, 345, 336, 346, 353, 343, 342, 353, 357, 346, 345, 358, 357, 344, 347, 361, 357, 347, 347, 359, 356, 346, 352, 361, 354, 344, 351, 359, 346, 338, 350, 357, 348, 346, 359, 362, 348, 348, 363, 364, 358, 364, 377, 377, 363, 365, 370, 358, 343, 350, 354, 341, 333, 344, 350, 341, 332, 343, 350, 336, 311, 289, 275, 277, 345, 445, 465, 422, 396, 389, 373, 362, 372, 374, 344, 324, 329, 339, 330, 322, 335, 342, 332, 331, 342, 341, 327, 327, 343, 344, 333, 337, 346, 345, 335, 344, 361, 356, 343, 351, 362, 354, 347, 361, 371, 364, 361, 373, 378, 365, 356, 362, 362, 343, 336, 343, 336, 320, 325, 338, 327, 313, 322, 334, 322, 310, 318, 329, 323, 315, 321, 328, 320, 316, 328, 332, 322, 323, 332, 332, 320, 325, 337, 334, 318, 322, 337, 330, 322, 329, 339, 330, 320, 329, 339, 327, 320, 329, 337, 331, 332, 345, 353, 343, 344, 359, 364, 348, 341, 346, 337, 322, 326, 339, 331, 316, 322, 331, 324, 318, 325, 323, 294, 263, 264, 274, 312, 390, 435, 417, 383, 364, 360, 349, 339, 344, 344, 326, 310, 318, 328, 320, 310, 319, 327, 321, 318, 330, 334, 320, 316, 332, 340, 326, 323, 336, 339, 326, 329, 344, 341, 331, 339, 354, 353, 346, 352, 362, 354, 349, 362, 371, 356, 342, 350, 350, 330, 322, 334, 335, 318, 317, 327, 325, 313, 313, 325, 319, 307, 314, 326, 320, 309, 317, 327, 320, 313, 326, 333, 320, 314, 327, 332, 319, 319, 333, 333, 321, 321, 333, 331, 321, 325, 336, 329, 321, 329, 333, 325, 320, 335, 346, 335, 333, 353, 358, 346, 346, 358, 350, 329, 322, 331, 327, 311, 314, 329, 324, 309, 313, 325, 320, 304, 291, 275, 251, 258, 334, 430, 440, 403, 387, 370, 345, 346, 359, 345, 320, 313, 322, 319, 305, 311, 325, 322, 310, 316, 325, 317, 310, 323, 332, 319, 314, 332, 340, 328, 326, 336, 338, 327, 329, 347, 347, 337, 342, 359, 358, 347, 354, 368, 361, 347, 350, 358, 343, 327, 335, 337, 322, 318, 329, 328, 313, 313, 324, 320, 306, 309, 324, 322, 310, 314, 330, 326, 314, 320, 326, 317, 311, 325, 334, 321, 313, 326, 334, 322, 319, 330, 331, 318, 319, 334, 333, 320, 324, 335, 330, 316, 324, 338, 330, 321, 328, 334, 326, 321, 337, 350, 338, 334, 351, 362, 350, 348, 359, 354, 334, 329, 339, 336, 320, 323, 338, 333, 318, 318, 326, 319, 310, 314, 298, 261, 243, 287, 382, 448, 442, 423, 396, 361, 354, 371, 359, 323, 310, 321, 321, 310, 316, 327, 318, 307, 317, 330, 322, 315, 321, 330, 320, 314, 330, 335, 324, 326, 338, 338, 330, 332, 345, 344, 335, 342, 357, 356, 347, 356, 368, 362, 355, 363, 367, 350, 337, 346, 347, 329, 319, 332, 335, 320, 317, 326, 326, 315, 318, 328, 323, 312, 319, 330, 324, 314, 322, 333, 324, 315, 328, 337, 322, 317, 332, 339, 327, 323, 333, 339, 327, 326, 337, 334, 321, 327, 339, 334, 322, 328, 342, 335, 320, 326, 336, 329, 323, 336, 341, 325, 319, 338, 350, 342, 338, 348, 351, 345, 348, 362, 362, 344, 343, 352, 342, 326, 331, 338, 330, 322, 332, 337, 326, 320, 330, 327, 291, 261, 262, 284, 341, 428, 463, 430, 386, 371, 374, 366, 349, 342, 343, 327, 314, 325, 336, 324, 313, 324, 333, 321, 320, 331, 333, 322, 323, 335, 338, 328, 328, 338, 337, 325, 332, 344, 339, 330, 342, 357, 350, 342, 356, 367, 358, 353, 367, 371, 354, 347, 353, 349, 330, 327, 340, 336, 318, 316, 329, 325, 312, 320, 329, 318, 306, 317, 330, 322, 310, 326, 336, 321, 312, 325, 335, 323, 318, 332, 334, 318, 322, 339, 338, 326, 329, 339, 330, 319, 328, 336, 330, 323, 330, 337, 328, 320, 334, 342, 331, 321, 330, 334, 323, 321, 335, 338, 328, 334, 348, 347, 339, 350, 363, 356, 345, 347, 352, 340, 326, 333, 341, 329, 319, 328, 333, 320, 314, 328, 329, 308, 283, 275, 268, 268, 338, 429, 436, 395, 377, 372, 353, 338, 348, 351, 327, 310, 320, 330, 319, 313, 326, 331, 316, 315, 329, 330, 314, 320, 338, 337, 323, 327, 340, 333, 322, 332, 345, 341, 333, 342, 350, 342, 340, 356, 364, 351, 348, 364, 372, 361, 356, 358, 353, 334, 329, 337, 333, 318, 321, 330, 324, 313, 324, 333, 321, 311, 320, 328, 313, 304, 317, 325, 314, 313, 330, 332, 320, 319, 328, 327, 317, 320, 331, 326, 316, 325, 335, 327, 316, 330, 344, 332, 317, 325, 335, 325, 319, 332, 334, 322, 322, 337, 338, 327, 333, 349, 351, 345, 351, 364, 359, 344, 343, 349, 338, 322, 327, 336, 327, 315, 323, 329, 315, 312, 328, 323, 287, 257, 257, 269, 320, 414, 456, 422, 381, 366, 366, 355, 341, 335, 331, 317, 307, 316, 319, 306, 303, 319, 325, 312, 308, 320, 322, 314, 318, 331, 326, 314, 318, 332, 327, 318, 327, 339, 332, 324, 337, 346, 336, 334, 348, 358, 350, 347, 361, 364, 349, 346, 356, 349, 327, 325, 335, 329, 318, 320, 331, 322, 308, 317, 327, 319, 308, 320, 328, 317, 309, 322, 328, 316, 313, 329, 329, 316, 316, 329, 327, 316, 322, 337, 331, 319, 326, 337, 330, 318, 326, 336, 321, 314, 326, 335, 324, 318, 331, 333, 321, 320, 333, 334, 322, 327, 342, 339, 332, 344, 358, 351, 344, 350, 355, 339, 325, 332, 339, 325, 318, 327, 330, 315, 315, 331, 330, 315, 302, 285, 257, 240, 289, 399, 462, 434, 410, 395, 372, 356, 362, 358, 332, 314, 319, 324, 313, 309, 322, 324, 313, 311, 324, 327, 316, 320, 333, 331, 317, 324, 340, 335, 321, 329, 342, 333, 323, 339, 349, 338, 333, 348, 356, 348, 346, 361, 367, 355, 355, 361, 352, 338, 336, 342, 334, 319, 325, 334, 323, 310, 319, 329, 317, 306, 318, 328, 316, 310, 322, 329, 316, 312, 322, 323, 309, 314, 330, 327, 315, 319, 332, 324, 311, 321, 334, 325, 314, 326, 336, 324, 317, 329, 333, 322, 321, 334, 335, 322, 323, 339, 338, 325, 332, 348, 340, 329, 348, 366, 355, 343, 349, 354, 340, 330, 337, 338, 326, 318, 329, 334, 321, 318, 330, 331, 315, 302, 287, 264, 246, 288, 389, 446, 427, 406, 391, 366, 350, 359, 359, 335, 319, 325, 328, 314, 313, 324, 325, 312, 314, 330, 332, 318, 320, 334, 331, 320, 327, 341, 337, 327, 336, 345, 335, 326, 339, 352, 344, 339, 355, 361, 351, 353, 370, 371, 359, 356, 367, 360, 341, 340, 351, 336, 318, 325, 335, 326, 314, 322, 332, 320, 310, 323, 329, 317, 312, 326, 329, 318, 318, 332, 333, 321, 325, 335, 329, 318, 324, 336, 330, 316, 326, 339, 331, 319, 329, 338, 329, 322, 337, 343, 327, 321, 334, 338, 327, 329, 344, 344, 336, 347, 364, 363, 351, 353, 359, 343, 327, 334, 342, 330, 319, 330, 339, 326, 318, 331, 334, 319, 300, 282, 270, 262, 316, 417, 444, 408, 391, 385, 365, 345, 357, 362, 340, 319, 322, 333, 327, 319, 329, 334, 321, 316, 330, 334, 323, 325, 342, 343, 324, 328, 340, 337, 327, 336, 349, 341, 332, 344, 358, 348, 340, 355, 365, 355, 350, 366, 375, 360, 353, 362, 359, 339, 335, 343, 334, 316, 320, 334, 326, 311, 318, 330, 321, 311, 320, 329, 319, 312, 325, 332, 319, 316, 329, 332, 320, 319, 333, 335, 321, 324, 337, 331, 318, 326, 337, 327, 315, 325, 336, 327, 319, 328, 337, 328, 323, 337, 345, 334, 334, 353, 356, 342, 346, 356, 345, 327, 329, 339, 328, 311, 316, 329, 322, 311, 319, 328, 316, 304, 297, 274, 246, 246, 320, 420, 447, 416, 395, 374, 349, 352, 364, 346, 315, 312, 323, 315, 304, 314, 325, 315, 308, 320, 328, 316, 311, 325, 333, 323, 319, 332, 336, 324, 326, 338, 337, 328, 333, 346, 342, 334, 341, 353, 344, 338, 353, 365, 356, 347, 355, 358, 342, 330, 340, 340, 322, 316, 331, 329, 314, 313, 321, 319, 310, 315, 328, 322, 310, 316, 328, 322, 314, 327, 335, 320, 313, 326, 335, 324, 319, 329, 331, 319, 321, 334, 331, 321, 326, 339, 331, 316, 323, 337, 331, 321, 332, 342, 330, 322, 338, 349, 346, 345, 357, 362, 350, 342, 351, 345, 324, 321, 332, 331, 316, 321, 335, 327, 311, 317, 328, 315, 284, 266, 260, 257, 309, 421, 469, 429, 391, 381, 372, 360, 348, 342, 328, 308, 308, 324, 321, 308, 312, 324, 319, 309, 318, 332, 326, 315, 325, 335, 324, 319, 332, 336, 325, 323, 340, 346, 331, 331, 345, 345, 335, 343, 360, 358, 346, 351, 364, 358, 346, 356, 361, 342, 327, 337, 340, 325, 315, 325, 331, 321, 316, 327, 325, 314, 316, 329, 328, 313, 315, 329, 324, 313, 321, 331, 324, 316, 327, 334, 323, 320, 332, 334, 320, 318, 331, 333, 320, 320, 335, 333, 320, 324, 338, 332, 321, 329, 338, 332, 326, 337, 346, 338, 335, 354, 361, 350, 347, 357, 353, 335, 328, 338, 335, 319, 320, 336, 330, 316, 319, 331, 326, 311, 297, 276, 249, 246, 317, 425, 456, 424, 408, 392, 362, 355, 365, 350, 321, 310, 321, 321, 309, 314, 328, 323, 311, 320, 334, 326, 315, 324, 336, 329, 321, 330, 338, 329, 324, 338, 343, 331, 327, 339, 342, 333, 338, 355, 355, 343, 345, 359, 359, 349, 356, 365, 351, 333, 336, 345, 331, 317, 326, 332, 316, 307, 318, 324, 312, 312, 328, 328, 312, 314, 329, 325, 314, 322, 333, 326, 313, 322, 334, 326, 317, 325, 333, 324, 319, 331, 335, 321, 318, 332, 335, 322, 327, 342, 336, 325, 330, 342, 338, 328, 342, 361, 353, 345, 357, 363, 345, 332, 339, 341, 327, 319, 331, 333, 318, 314, 328, 329, 317, 315, 309, 275, 246, 257, 327, 410, 425, 411, 397, 368, 343, 349, 361, 341, 322, 325, 329, 318, 312, 326, 329, 317, 317, 328, 329, 315, 320, 336, 333, 321, 331, 344, 334, 323, 334, 346, 337, 327, 340, 352, 342, 337, 351, 360, 348, 348, 365, 368, 355, 355, 366, 363, 344, 343, 351, 340, 322, 329, 337, 325, 314, 325, 333, 319, 310, 325, 335, 322, 317, 328, 331, 318, 318, 334, 334, 319, 321, 336, 331, 320, 326, 338, 333, 322, 330, 341, 330, 319, 328, 333, 323, 321, 337, 344, 335, 331, 348, 360, 352, 352, 366, 362, 342, 338, 344, 334, 320, 326, 336, 324, 314, 323, 332, 322, 311, 322, 319, 279, 251, 258, 292, 358, 423, 439, 410, 368, 354, 362, 364, 341, 332, 335, 324, 311, 317, 327, 318, 309, 320, 330, 320, 315, 329, 335, 323, 317, 332, 337, 328, 331, 343, 337, 324, 330, 345, 340, 330, 339, 354, 348, 338, 349, 361, 353, 346, 361, 369, 352, 340, 347, 347, 328, 321, 332, 329, 317, 318, 329, 326, 313, 319, 332, 326, 312, 316, 328, 322, 315, 327, 331, 319, 311, 326, 334, 320, 316, 331, 335, 318, 315, 331, 331, 317, 322, 334, 330, 318, 325, 340, 339, 328, 337, 353, 349, 342, 356, 364, 345, 329, 335, 336, 320, 316, 330, 330, 314, 311, 323, 322, 311, 316, 323, 288, 243, 240, 275, 345, 427, 451, 428, 388, 360, 364, 374, 353, 326, 325, 324, 313, 311, 326, 326, 314, 318, 329, 325, 314, 321, 335, 327, 314, 324, 336, 329, 321, 333, 341, 333, 328, 338, 344, 331, 330, 347, 350, 338, 342, 358, 357, 346, 351, 361, 352, 336, 341, 349, 334, 317, 324, 333, 321, 312, 323, 329, 314, 306, 318, 325, 314, 310, 323, 324, 314, 317, 329, 328, 311, 315, 331, 324, 312, 323, 333, 324, 317, 329, 337, 326, 316, 325, 332, 322, 320, 335, 336, 326, 331, 350, 356, 341, 345, 360, 351, 332, 333, 343, 330, 316, 323, 331, 321, 313, 325, 332, 320, 311, 317, 301, 260, 237, 263, 336, 419, 447, 434, 402, 366, 361, 376, 361, 330, 322, 325, 318, 309, 318, 328, 318, 312, 320, 327, 317, 315, 327, 330, 318, 318, 334, 335, 322, 325, 338, 337, 327, 332, 344, 336, 327, 339, 354, 347, 341, 353, 362, 351, 346, 357, 360, 345, 336, 343, 340, 325, 321, 329, 328, 314, 315, 327, 322, 312, 321, 329, 318, 309, 322, 333, 323, 315, 327, 335, 324, 319, 330, 335, 324, 321, 333, 332, 320, 326, 340, 335, 320, 324, 340, 336, 324, 333, 342, 336, 330, 344, 357, 349, 344, 362, 370, 352, 338, 349, 349, 328, 324, 335, 333, 320, 324, 336, 330, 318, 324, 332, 304, 265, 257, 270, 315, 398, 448, 436, 391, 359, 360, 365, 356, 345, 343, 336, 320, 317, 329, 328, 316, 322, 334, 327, 315, 324, 335, 327, 318, 330, 338, 330, 326, 338, 345, 333, 329, 343, 348, 339, 337, 353, 353, 342, 348, 363, 361, 352, 359, 373, 363, 349, 351, 356, 337, 325, 334, 337, 321, 314, 328, 334, 318, 317, 331, 332, 316, 317, 330, 326, 313, 318, 332, 330, 320, 328, 337, 327, 319, 330, 341, 329, 320, 334, 344, 329, 320, 333, 339, 328, 327, 340, 344, 334, 340, 358, 356, 345, 355, 369, 358, 337, 342, 348, 332, 317, 327, 335, 323, 313, 326, 331, 317, 312, 320, 299, 260, 253, 286, 349, 410, 419, 407, 382, 355, 350, 361, 356, 333, 329, 332, 320, 310, 323, 331, 319, 312, 323, 331, 320, 322, 338, 335, 322, 325, 336, 335, 326, 335, 347, 339, 328, 338, 351, 343, 335, 349, 360, 348, 342, 357, 366, 356, 352, 363, 363, 344, 339, 348, 341, 324, 326, 336, 324, 308, 319, 334, 323, 308, 316, 328, 323, 316, 324, 328, 318, 314, 328, 333, 319, 319, 332, 331, 320, 323, 335, 331, 317, 322, 338, 331, 318, 323, 336, 330, 321, 334, 345, 337, 337, 358, 364, 348, 344, 356, 351, 329, 324, 332, 327, 313, 316, 328, 325, 313, 320, 327, 317, 299, 287, 270, 245, 253, 339, 440, 446, 406, 388, 373, 351, 349, 356, 342, 317, 312, 322, 317, 305, 314, 329, 322, 307, 316, 326, 318, 310, 321, 331, 321, 317, 330, 337, 325, 320, 334, 340, 330, 329, 344, 344, 333, 339, 357, 354, 339, 346, 361, 355, 342, 347, 352, 337, 324, 330, 336, 322, 312, 322, 321, 306, 305, 322, 322, 308, 307, 321, 320, 306, 313, 327, 321, 309, 315, 327, 319, 310, 322, 336, 323, 314, 321, 331, 322, 316, 324, 330, 321, 322, 336, 334, 319, 324, 342, 342, 327, 338, 360, 353, 341, 352, 360, 341, 321, 331, 340, 326, 315, 323, 328, 318, 314, 324, 325, 312, 308, 305, 275, 244, 253, 323, 417, 448, 433, 416, 382, 356, 367, 370, 341, 316, 320, 325, 314, 310, 325, 328, 312, 309, 322, 326, 314, 316, 331, 328, 313, 320, 334, 330, 319, 328, 343, 333, 322, 334, 347, 342, 335, 345, 353, 344, 343, 359, 360, 344, 345, 360, 356, 335, 332, 340, 333, 316, 319, 332, 323, 310, 318, 328, 317, 307, 317, 326, 314, 306, 319, 325, 315, 314, 328, 329, 317, 319, 332, 331, 320, 326, 338, 328, 314, 326, 341, 329, 317, 326, 337, 327, 318, 329, 337, 328, 323, 339, 341, 329, 330, 346, 354, 344, 346, 362, 354, 337, 337, 346, 339, 323, 329, 339, 330, 314, 322, 333, 322, 315, 324, 321, 282, 251, 254, 287, 360, 429, 444, 415, 374, 358, 361, 357, 341, 334, 332, 320, 312, 322, 330, 320, 311, 322, 331, 318, 311, 329, 335, 322, 320, 332, 333, 321, 327, 342, 338, 323, 330, 347, 341, 330, 337, 350, 348, 343, 357, 366, 355, 348, 360, 363, 344, 335, 346, 345, 327, 324, 336, 333, 315, 316, 329, 324, 312, 315, 327, 320, 310, 320, 330, 321, 313, 325, 331, 316, 313, 328, 334, 323, 324, 336, 336, 321, 324, 336, 334, 322, 328, 341, 333, 320, 329, 342, 333, 322, 332, 342, 334, 328, 343, 349, 337, 339, 356, 362, 349, 350, 359, 350, 334, 333, 339, 331, 317, 323, 336, 328, 316, 321, 329, 320, 306, 294, 277, 256, 268, 351, 434, 426, 395, 386, 372, 350, 352, 362, 347, 323, 323, 338, 328, 313, 321, 334, 327, 316, 328, 338, 330, 323, 333, 341, 331, 324, 333, 338, 326, 326, 344, 348, 336, 337, 351, 349, 340, 346, 359, 356, 348, 356, 368, 359, 349, 359, 362, 345, 335, 345, 345, 323, 318, 331, 333, 317, 317, 333, 330, 313, 316, 329, 324, 312, 321, 333, 326, 317, 327, 336, 324, 316, 329, 336, 327, 323, 334, 337, 323, 324, 338, 339, 324, 326, 340, 337, 323, 328, 339, 331, 320, 331, 342, 332, 326, 342, 350, 342, 342, 358, 365, 353, 347, 355, 347, 328, 324, 334, 330, 318, 322, 332, 326, 314, 323, 334, 318, 285, 266, 259, 259, 313, 417, 462, 424, 386, 377, 369, 357, 349, 345, 332, 316, 314, 325, 320, 308, 315, 330, 323, 308, 316, 329, 322, 314, 327, 338, 325, 319, 333, 337, 325, 322, 337, 340, 327, 330, 347, 349, 337, 340, 357, 356, 345, 354, 368, 359, 343, 350, 356, 342, 327, 334, 338, 323, 312, 322, 326, 313, 313, 325, 325, 312, 315, 328, 326, 312, 317, 332, 324, 312, 320, 331, 325, 318, 327, 337, 325, 319, 330, 335, 324, 319, 331, 334, 320, 316, 330, 333, 323, 326, 335, 331, 320, 324, 338, 332, 319, 331, 343, 332, 325, 339, 351, 345, 341, 357, 363, 349, 341, 347, 344, 327, 325, 337, 334, 319, 319, 333, 329, 317, 323, 332, 306, 263, 251, 266, 315, 406, 463, 447, 407, 375, 370, 371, 353, 338, 336, 326, 311, 313, 326, 323, 309, 311, 323, 318, 309, 322, 335, 326, 314, 325, 333, 323, 320, 337, 343, 329, 326, 343, 347, 333, 332, 348, 351, 341, 347, 363, 359, 351, 359, 369, 357, 344, 348, 353, 337, 324, 333, 340, 323, 315, 330, 335, 322, 316, 327, 325, 312, 315, 331, 326, 312, 320, 334, 326, 316, 326, 337, 326, 314, 325, 335, 324, 317, 331, 338, 323, 316, 331, 336, 324, 325, 338, 336, 319, 321, 338, 333, 318, 323, 337, 331, 320, 332, 345, 340, 331, 340, 353, 347, 340, 354, 362, 346, 337, 345, 343, 327, 325, 335, 330, 316, 321, 333, 325, 313, 321, 332, 302, 261, 254, 274, 330, 408, 445, 424, 381, 357, 361, 361, 345, 336, 340, 334, 315, 312, 323, 318, 309, 317, 329, 323, 311, 319, 331, 322, 316, 326, 333, 323, 317, 332, 342, 333, 330, 341, 340, 332, 337, 353, 351, 340, 347, 361, 359, 350, 358, 369, 357, 342, 346, 351, 335, 325, 334, 338, 321, 312, 323, 324, 309, 309, 325, 328, 312, 311, 324, 321, 309, 318, 330, 323, 311, 319, 333, 326, 316, 327, 335, 326, 323, 335, 336, 321, 317, 332, 335, 326, 325, 337, 335, 320, 325, 341, 338, 328, 338, 351, 345, 341, 355, 365, 358, 352, 357, 355, 339, 329, 337, 340, 325, 323, 336, 333, 318, 318, 333, 330, 308, 290, 277, 260, 264, 342, 433, 436, 397, 382, 374, 355, 350, 363, 353, 325, 315, 327, 331, 317, 315, 327, 327, 316, 320, 333, 329, 318, 327, 341, 334, 324, 333, 345, 337, 331, 344, 350, 335, 329, 346, 354, 344, 347, 363, 365, 351, 355, 372, 371, 353, 352, 363, 355, 339, 339, 342, 330, 318, 328, 335, 322, 311, 323, 332, 321, 317, 329, 332, 321, 319, 330, 329, 320, 322, 334, 330, 320, 324, 334, 327, 317, 328, 339, 328, 317, 329, 336, 325, 318, 329, 335, 323, 325, 341, 343, 334, 334, 347, 352, 346, 352, 367, 363, 348, 343, 347, 338, 326, 332, 340, 324, 315, 329, 337, 321, 313, 325, 320, 284, 254, 257, 276, 333, 421, 454, 421, 376, 365, 370, 365, 348, 341, 338, 322, 311, 323, 333, 320, 310, 319, 328, 316, 317, 331, 332, 321, 326, 338, 334, 325, 331, 343, 339, 325, 332, 347, 342, 335, 345, 357, 349, 345, 361, 365, 351, 347, 365, 372, 356, 346, 350, 345, 326, 323, 336, 330, 315, 321, 330, 322, 311, 319, 329, 323, 314, 324, 329, 316, 312, 329, 336, 324, 320, 332, 332, 317, 319, 333, 335, 323, 324, 333, 329, 318, 326, 336, 328, 319, 329, 335, 325, 320, 336, 346, 331, 330, 350, 357, 344, 346, 358, 352, 329, 326, 336, 328, 315, 321, 332, 325, 312, 318, 327, 317, 305, 300, 279, 248, 240, 304, 406, 453, 427, 405, 383, 359, 359, 369, 349, 317, 311, 324, 318, 304, 312, 327, 318, 306, 317, 328, 320, 311, 322, 330, 321, 320, 334, 336, 323, 322, 337, 338, 325, 330, 345, 342, 330, 336, 352, 347, 337, 349, 365, 357, 344, 350, 357, 342, 327, 335, 339, 321, 314, 323, 325, 312, 312, 324, 325, 310, 311, 326, 323, 313, 323, 335, 325, 312, 323, 336, 325, 315, 327, 337, 331, 323, 332, 331, 320, 319, 332, 333, 321, 323, 336, 335, 319, 324, 341, 337, 327, 334, 353, 353, 345, 354, 364, 349, 333, 337, 343, 328, 319, 331, 333, 315, 313, 329, 330, 315, 317, 327, 308, 261, 247, 262, 308, 392, 451, 445, 402, 363, 358, 365, 353, 336, 331, 326, 311, 307, 320, 321, 309, 312, 328, 326, 311, 313, 325, 319, 311, 322, 336, 328, 318, 326, 336, 328, 323, 336, 345, 333, 329, 342, 347, 338, 340, 357, 360, 346, 348, 365, 361, 345, 343, 344, 333, 321, 327, 335, 325, 315, 320, 322, 314, 310, 320, 323, 309, 307, 322, 324, 311, 312, 329, 331, 320, 321, 332, 325, 316, 324, 333, 325, 315, 325, 335, 321, 314, 328, 335, 325, 320, 335, 344, 331, 331, 351, 358, 347, 350, 363, 357, 337, 337, 344, 331, 320, 325, 335, 327, 315, 325, 332, 320, 314, 320, 300, 262, 251, 279, 344, 409, 422, 414, 392, 358, 348, 361, 364, 341, 333, 338, 327, 317, 326, 335, 326, 317, 330, 336, 324, 320, 331, 337, 328, 328, 340, 339, 330, 333, 345, 339, 328, 336, 351, 345, 335, 345, 358, 352, 342, 355, 368, 359, 350, 359, 365, 349, 339, 346, 341, 325, 325, 337, 330, 314, 318, 333, 326, 310, 319, 331, 323, 315, 326, 337, 326, 316, 327, 339, 327, 320, 334, 341, 328, 325, 338, 336, 324, 325, 342, 338, 324, 329, 342, 340, 333, 341, 354, 349, 344, 357, 368, 358, 349, 354, 351, 331, 322, 333, 336, 323, 320, 334, 331, 316, 319, 334, 326, 292, 270, 268, 270, 318, 416, 454, 417, 378, 367, 366, 355, 347, 345, 336, 317, 312, 323, 324, 310, 312, 328, 330, 318, 320, 329, 323, 314, 326, 338, 330, 321, 331, 339, 332, 325, 339, 348, 335, 330, 343, 348, 341, 341, 354, 354, 343, 349, 363, 359, 344, 344, 349, 336, 320, 324, 332, 321, 312, 323, 326, 312, 304, 317, 324, 313, 313, 326, 325, 310, 313, 329, 327, 313, 319, 332, 327, 319, 328, 333, 324, 318, 330, 339, 329, 323, 333, 340, 328, 325, 342, 347, 338, 342, 358, 358, 348, 349, 357, 344, 326, 327, 337, 331, 315, 321, 332, 324, 313, 323, 330, 316, 292, 274, 260, 249, 298, 408, 455, 418, 385, 377, 363, 349, 353, 353, 335, 314, 316, 324, 314, 304, 318, 333, 322, 311, 321, 329, 320, 314, 327, 331, 321, 320, 335, 337, 324, 327, 341, 337, 324, 333, 348, 343, 331, 338, 351, 348, 344, 352, 358, 349, 341, 352, 354, 334, 322, 328, 327, 314, 316, 328, 324, 309, 308, 321, 318, 306, 314, 326, 319, 307, 319, 331, 321, 315, 329, 335, 321, 315, 330, 334, 324, 324, 336, 334, 318, 320, 334, 334, 326, 334, 347, 341, 336, 351, 361, 354, 343, 350, 351, 334, 325, 333, 336, 320, 313, 324, 328, 315, 316, 330, 325, 296, 268, 262, 257, 279, 374, 458, 442, 392, 373, 371, 358, 352, 356, 344, 318, 308, 322, 334, 320, 314, 328, 330, 317, 318, 333, 332, 320, 324, 338, 335, 324, 330, 339, 333, 324, 336, 349, 337, 329, 343, 354, 345, 345, 356, 358, 346, 348, 364, 364, 348, 345, 352, 344, 326, 329, 336, 326, 313, 322, 332, 321, 310, 320, 330, 318, 311, 324, 332, 320, 317, 330, 328, 316, 316, 330, 327, 314, 322, 338, 333, 319, 326, 338, 331, 323, 337, 349, 336, 331, 351, 361, 349, 347, 360, 355, 335, 328, 337, 334, 318, 319, 331, 324, 310, 318, 331, 325, 306, 294, 273, 248, 246, 318, 430, 464, 426, 398, 378, 359, 356, 361, 345, 318, 313, 323, 323, 311, 312, 326, 323, 311, 319, 332, 320, 313, 325, 335, 323, 313, 327, 337, 328, 324, 338, 342, 328, 329, 346, 348, 335, 337, 353, 351, 340, 348, 363, 358, 345, 348, 355, 338, 325, 334, 338, 323, 315, 325, 328, 315, 309, 322, 325, 308, 306, 324, 326, 313, 316, 330, 324, 312, 320, 335, 326, 315, 323, 335, 329, 321, 333, 341, 330, 325, 340, 347, 336, 337, 357, 361, 351, 354, 364, 354, 334, 331, 338, 330, 318, 325, 334, 324, 315, 325, 333, 319, 306, 291, 266, 241, 260, 351, 449, 455, 420, 401, 378, 355, 361, 367, 341, 314, 314, 324, 317, 308, 318, 328, 318, 310, 322, 331, 320, 312, 327, 335, 326, 320, 334, 336, 325, 324, 337, 337, 325, 333, 349, 344, 331, 341, 357, 351, 340, 346, 356, 350, 342, 349, 352, 334, 324, 334, 332, 318, 320, 332, 325, 310, 314, 329, 325, 310, 314, 329, 325, 313, 320, 332, 324, 317, 330, 337, 325, 320, 334, 340, 329, 325, 333, 333, 320, 322, 342, 343, 329, 334, 347, 347, 344, 352, 366, 360, 345, 348, 352, 337, 325, 334, 337, 324, 318, 330, 333, 319, 315, 328, 325, 292, 262, 258, 263, 310, 415, 470, 441, 395, 373, 372, 364, 349, 344, 335, 315, 307, 324, 330, 316, 313, 326, 327, 313, 317, 333, 327, 311, 317, 333, 330, 321, 326, 337, 332, 324, 334, 344, 335, 328, 341, 350, 340, 337, 353, 360, 347, 345, 357, 355, 343, 342, 347, 338, 322, 324, 334, 325, 313, 321, 326, 313, 307, 319, 327, 318, 311, 322, 326, 312, 310, 326, 331, 319, 318, 331, 328, 315, 320, 335, 333, 317, 323, 334, 326, 319, 330, 337, 328, 319, 330, 340, 330, 326, 340, 352, 347, 343, 356, 363, 348, 339, 345, 339, 321, 321, 334, 330, 319, 324, 332, 322, 312, 323, 323, 285, 250, 256, 282, 339, 410, 437, 415, 373, 351, 357, 358, 348, 340, 340, 328, 312, 317, 329, 323, 312, 324, 335, 324, 317, 328, 335, 324, 320, 333, 340, 327, 325, 340, 343, 331, 331, 343, 343, 333, 341, 357, 353, 344, 351, 362, 355, 345, 357, 366, 350, 340, 349, 350, 335, 325, 332, 329, 315, 314, 328, 325, 313, 314, 328, 325, 315, 320, 331, 325, 315, 322, 333, 323, 314, 324, 334, 327, 324, 334, 334, 323, 319, 333, 333, 316, 317, 333, 332, 323, 330, 343, 337, 329, 344, 363, 360, 351, 358, 360, 343, 328, 336, 339, 326, 321, 331, 333, 319, 319, 334, 335, 312, 298, 283, 261, 253, 309, 408, 435, 403, 387, 379, 357, 341, 356, 363, 340, 322, 330, 333, 321, 317, 330, 331, 315, 316, 334, 336, 324, 329, 344, 339, 325, 331, 344, 338, 329, 340, 351, 342, 333, 347, 353, 343, 340, 354, 360, 348, 345, 359, 359, 347, 348, 358, 350, 331, 332, 340, 333, 319, 323, 329, 318, 309, 324, 330, 320, 309, 321, 327, 319, 316, 328, 329, 316, 319, 333, 330, 319, 321, 336, 331, 319, 324, 336, 329, 317, 325, 334, 328, 321, 329, 339, 330, 324, 339, 344, 333, 340, 359, 361, 351, 351, 357, 345, 325, 326, 339, 330, 316, 324, 334, 324, 313, 322, 332, 315, 286, 267, 256, 256, 322, 430, 460, 415, 380, 374, 364, 353, 353, 352, 332, 311, 315, 327, 320, 309, 319, 332, 320, 308, 319, 329, 318, 313, 328, 335, 323, 320, 336, 339, 326, 326, 343, 342, 329, 336, 353, 346, 333, 343, 357, 352, 343, 354, 363, 353, 345, 351, 350, 333, 325, 337, 340, 324, 320, 332, 327, 311, 316, 332, 326, 311, 318, 331, 323, 313, 323, 334, 324, 315, 329, 337, 325, 318, 330, 335, 323, 319, 333, 336, 323, 321, 335, 334, 321, 325, 337, 334, 325, 336, 351, 343, 334, 350, 364, 355, 348, 357, 358, 339, 329, 337, 336, 321, 321, 333, 331, 320, 320, 333, 330, 310, 293, 273, 249, 249, 323, 430, 455, 420, 403, 386, 362, 357, 363, 353, 323, 310, 323, 326, 313, 313, 328, 323, 305, 309, 327, 324, 309, 316, 330, 322, 314, 324, 336, 328, 323, 334, 341, 329, 322, 333, 339, 332, 330, 346, 351, 340, 342, 357, 354, 340, 344, 355, 345, 329, 330, 339, 329, 318, 323, 326, 312, 306, 319, 327, 314, 311, 326, 331, 320, 320, 335, 332, 315, 316, 330, 328, 320, 328, 340, 331, 318, 329, 343, 333, 322, 329, 336, 327, 325, 334, 334, 320, 322, 340, 347, 333, 332, 353, 358, 343, 347, 364, 356, 338, 341, 348, 331, 318, 325, 332, 321, 315, 330, 336, 322, 316, 318, 296, 260, 250, 292, 376, 425, 426, 411, 380, 349, 348, 363, 348, 324, 322, 329, 317, 310, 320, 329, 318, 309, 321, 326, 318, 320, 332, 329, 316, 323, 339, 338, 323, 326, 343, 340, 329, 334, 345, 336, 327, 341, 356, 347, 338, 349, 360, 352, 346, 358, 359, 340, 333, 340, 333, 316, 317, 330, 325, 310, 313, 326, 317, 303, 312, 327, 318, 308, 318, 326, 314, 309, 323, 330, 318, 316, 333, 334, 319, 321, 335, 328, 313, 320, 336, 331, 318, 323, 337, 331, 321, 332, 344, 336, 335, 352, 359, 346, 343, 357, 356, 333, 324, 334, 332, 316, 317, 330, 326, 310, 316, 330, 319, 301, 285, 273, 257, 269, 351, 431, 421, 387, 378, 365, 341, 338, 354, 349, 323, 316, 328, 327, 315, 317, 329, 324, 313, 319, 330, 322, 313, 324, 336, 327, 320, 332, 340, 328, 323, 340, 349, 335, 330, 342, 343, 333, 339, 356, 355, 343, 351, 367, 361, 346, 353, 361, 344, 329, 335, 339, 326, 314, 326, 334, 321, 314, 324, 326, 315, 315, 331, 332, 318, 318, 330, 327, 316, 324, 339, 332, 319, 329, 342, 331, 321, 329, 338, 327, 320, 333, 337, 324, 321, 338, 344, 334, 335, 351, 353, 345, 353, 368, 359, 340, 339, 347, 335, 321, 332, 343, 326, 311, 323, 332, 319, 312, 316, 292, 252, 240, 285, 368, 430, 435, 417, 385, 354, 352, 370, 359, 331, 327, 333, 322, 311, 319, 326, 316, 311, 325, 330, 317, 315, 327, 330, 319, 326, 340, 338, 323, 326, 342, 339, 327, 335, 348, 342, 334, 345, 355, 346, 340, 356, 367, 351, 344, 359, 361, 342, 334, 343, 340, 326, 323, 332, 324, 310, 318, 329, 321, 311, 322, 333, 319, 307, 323, 335, 321, 312, 326, 334, 321, 315, 328, 331, 321, 321, 334, 332, 317, 320, 332, 326, 314, 322, 336, 329, 322, 335, 346, 334, 333, 354, 360, 348, 344, 352, 350, 332, 326, 337, 335, 319, 320, 332, 330, 317, 317, 328, 321, 295, 271, 264, 252, 285, 394, 465, 438, 397, 383, 378, 365, 352, 349, 335, 313, 307, 323, 325, 311, 312, 327, 325, 311, 317, 330, 324, 313, 323, 334, 326, 318, 332, 343, 335, 327, 339, 344, 333, 331, 345, 346, 337, 343, 358, 357, 345, 346, 360, 355, 343, 349, 354, 340, 325, 330, 337, 325, 316, 325, 332, 318, 312, 323, 326, 315, 314, 329, 331, 314, 314, 331, 328, 316, 322, 334, 328, 318, 329, 339, 329, 321, 333, 341, 328, 322, 333, 335, 322, 321, 337, 342, 332, 334, 346, 346, 341, 353, 367, 363, 355, 356, 358, 340, 325, 333, 341, 329, 318, 328, 335, 319, 313, 328, 330, 305, 270, 262, 258, 283, 374, 458, 450, 404, 381, 379, 368, 354, 350, 344, 324, 310, 320, 330, 320, 311, 322, 328, 317, 313, 326, 327, 315, 315, 330, 329, 317, 324, 338, 331, 318, 325, 340, 334, 324, 332, 345, 341, 333, 343, 355, 346, 343, 361, 366, 345, 337, 347, 345, 327, 323, 333, 327, 313, 317, 327, 315, 302, 311, 322, 314, 309, 319, 327, 314, 307, 324, 331, 317, 313, 327, 329, 317, 320, 334, 330, 316, 320, 335, 332, 322, 329, 342, 330, 314, 323, 340, 330, 322, 336, 346, 340, 341, 355, 363, 349, 342, 350, 343, 323, 320, 330, 328, 312, 316, 330, 322, 310, 317, 329, 312, 275, 258, 262, 281, 353, 426, 431, 393, 360, 360, 357, 346, 343, 345, 333, 312, 311, 326, 324, 312, 317, 332, 324, 310, 318, 330, 322, 314, 327, 336, 322, 316, 331, 338, 328, 326, 340, 343, 331, 331, 346, 348, 335, 341, 356, 354, 343, 351, 361, 350, 336, 341, 348, 332, 320, 331, 337, 320, 309, 321, 326, 311, 306, 324, 327, 314, 315, 329, 322, 311, 319, 333, 324, 312, 321, 337, 328, 315, 323, 332, 321, 314, 327, 335, 322, 316, 332, 334, 324, 328, 347, 347, 332, 338, 360, 359, 347, 354, 365, 349, 331, 338, 344, 330, 319, 330, 336, 324, 316, 328, 334, 318, 305, 292, 272, 250, 280, 383, 446, 424, 395, 385, 365, 343, 352, 362, 341, 318, 323, 333, 322, 312, 324, 332, 319, 313, 331, 338, 324, 320, 336, 338, 328, 330, 348, 343, 326, 333, 347, 343, 334, 342, 355, 350, 337, 350, 365, 356, 349, 364, 367, 354, 348, 359, 360, 344, 337, 344, 336, 321, 324, 335, 328, 316, 322, 333, 323, 312, 322, 332, 320, 313, 328, 336, 325, 319, 332, 335, 321, 321, 336, 336, 322, 322, 337, 336, 320, 321, 336, 333, 322, 333, 348, 340, 330, 345, 362, 352, 346, 361, 364, 345, 333, 342, 340, 324, 321, 333, 332, 317, 320, 334, 328, 310, 307, 295, 262, 239, 271, 361, 439, 441, 418, 396, 365, 351, 367, 362, 332, 316, 327, 328, 313, 313, 329, 326, 312, 317, 331, 327, 316, 324, 337, 329, 318, 331, 342, 333, 324, 337, 347, 336, 332, 348, 351, 336, 335, 351, 353, 344, 350, 365, 360, 347, 355, 365, 346, 326, 335, 344, 328, 315, 326, 333, 322, 314, 326, 330, 318, 313, 328, 328, 315, 316, 334, 330, 314, 320, 336, 330, 316, 322, 335, 327, 319, 332, 340, 328, 320, 333, 340, 327, 322, 335, 339, 331, 334, 348, 345, 338, 348, 363, 356, 344, 348, 354, 339, 324, 330, 338, 329, 318, 327, 334, 320, 310, 325, 334, 309, 275, 263, 254, 270, 356, 456, 459, 414, 389, 385, 371, 353, 348, 341, 323, 310, 318, 327, 314, 306, 322, 327, 314, 308, 324, 330, 317, 315, 331, 334, 319, 318, 336, 337, 327, 331, 343, 336, 326, 335, 350, 343, 333, 348, 361, 350, 342, 357, 365, 349, 339, 351, 351, 330, 325, 338, 334, 317, 317, 331, 325, 312, 319, 331, 324, 315, 326, 337, 325, 317, 331, 338, 325, 322, 332, 333, 321, 320, 337, 339, 324, 325, 340, 336, 325, 330, 341, 330, 319, 332, 344, 332, 320, 332, 345, 339, 332, 348, 364, 353, 347, 363, 366, 344, 334, 348, 342, 323, 322, 337, 331, 318, 326, 339, 329, 314, 316, 301, 263, 239, 273, 367, 442, 444, 426, 403, 370, 356, 372, 365, 332, 318, 329, 325, 313, 313, 326, 318, 305, 316, 329, 319, 307, 319, 332, 323, 312, 323, 333, 324, 320, 334, 338, 324, 322, 342, 347, 335, 334, 348, 346, 336, 347, 364, 355, 341, 351, 361, 344, 327, 335, 340, 324, 314, 327, 330, 314, 308, 322, 324, 310, 310, 327, 323, 306, 309, 327, 325, 311, 317, 331, 326, 312, 320, 333, 323, 315, 328, 336, 323, 318, 332, 335, 322, 319, 331, 331, 317, 319, 337, 338, 327, 333, 354, 353, 340, 350, 361, 348, 331, 338, 346, 330, 318, 330, 334, 319, 311, 327, 329, 313, 309, 314, 284, 251, 252, 299, 380, 421, 421, 409, 378, 350, 348, 359, 350, 331, 333, 334, 319, 312, 325, 330, 320, 319, 332, 334, 318, 319, 338, 336, 323, 330, 344, 340, 327, 332, 347, 340, 330, 341, 348, 337, 332, 349, 360, 350, 346, 364, 366, 351, 348, 364, 360, 339, 337, 351, 341, 321, 324, 337, 327, 312, 321, 335, 322, 306, 317, 332, 322, 316, 331, 333, 317, 313, 332, 337, 319, 317, 332, 331, 318, 322, 336, 333, 319, 327, 342, 331, 315, 326, 341, 335, 330, 341, 347, 340, 342, 362, 367, 352, 347, 358, 351, 330, 325, 337, 333, 321, 328, 340, 332, 314, 320, 334, 318, 280, 270, 272, 289, 359, 435, 437, 399, 372, 369, 362, 348, 348, 354, 339, 316, 317, 331, 323, 307, 317, 334, 326, 313, 322, 337, 330, 319, 331, 341, 334, 327, 338, 344, 331, 329, 346, 349, 335, 338, 356, 352, 337, 347, 365, 358, 346, 354, 369, 359, 342, 344, 351, 338, 323, 332, 337, 325, 318, 329, 331, 318, 317, 331, 329, 316, 317, 331, 327, 315, 325, 340, 331, 318, 327, 339, 332, 325, 337, 342, 329, 318, 333, 339, 325, 324, 341, 346, 334, 334, 352, 356, 349, 353, 371, 366, 346, 343, 351, 339, 323, 330, 340, 330, 318, 325, 333, 322, 317, 330, 323, 281, 255, 261, 288, 356, 434, 453, 416, 372, 361, 370, 365, 345, 339, 338, 322, 309, 324, 333, 320, 309, 322, 329, 315, 311, 329, 332, 319, 319, 335, 331, 320, 325, 344, 338, 323, 330, 345, 336, 323, 338, 354, 347, 337, 351, 362, 353, 348, 361, 359, 342, 334, 346, 342, 323, 322, 335, 328, 309, 311, 325, 321, 308, 318, 330, 319, 306, 318, 328, 316, 306, 322, 331, 319, 311, 329, 332, 316, 314, 333, 333, 319, 322, 334, 332, 320, 326, 340, 332, 326, 337, 348, 343, 340, 353, 364, 353, 342, 347, 344, 326, 320, 333, 332, 316, 313, 328, 326, 311, 317, 328, 307, 263, 248, 259, 295, 377, 455, 453, 406, 367, 364, 371, 357, 337, 336, 329, 310, 307, 326, 325, 306, 307, 327, 324, 309, 313, 329, 323, 313, 323, 337, 328, 317, 330, 339, 328, 322, 338, 345, 334, 330, 346, 350, 339, 340, 354, 348, 336, 344, 357, 349, 333, 338, 348, 335, 320, 327, 335, 320, 309, 323, 328, 314, 307, 317, 321, 308, 309, 327, 323, 305, 310, 332, 327, 311, 318, 334, 329, 314, 322, 334, 328, 320, 332, 338, 323, 316, 333, 338, 325, 324, 345, 349, 334, 335, 356, 356, 342, 349, 361, 346, 321, 326, 336, 328, 316, 324, 333, 321, 312, 324, 331, 313, 296, 283, 264, 240, 271, 378, 458, 442, 409, 396, 374, 352, 358, 362, 336, 310, 313, 325, 316, 307, 320, 331, 317, 312, 326, 331, 316, 312, 329, 335, 320, 319, 337, 338, 327, 330, 345, 341, 328, 335, 352, 349, 338, 348, 359, 352, 345, 358, 368, 356, 347, 359, 357, 335, 328, 340, 339, 320, 317, 330, 326, 311, 317, 331, 322, 308, 317, 330, 321, 311, 326, 334, 322, 314, 329, 337, 322, 318, 336, 340, 330, 327, 340, 340, 328, 328, 340, 335, 318, 325, 343, 338, 330, 343, 357, 353, 346, 358, 367, 353, 343, 352, 351, 334, 326, 338, 336, 320, 319, 335, 331, 317, 323, 337, 316, 271, 257, 268, 290, 359, 431, 436, 398, 366, 364, 362, 352, 346, 346, 338, 320, 318, 334, 327, 311, 315, 335, 330, 316, 325, 339, 329, 315, 327, 339, 332, 323, 336, 345, 335, 329, 343, 343, 330, 331, 352, 355, 341, 341, 361, 361, 346, 352, 368, 360, 341, 344, 354, 340, 327, 333, 339, 325, 315, 326, 329, 316, 312, 328, 329, 314, 316, 334, 331, 316, 322, 339, 336, 320, 324, 337, 330, 318, 328, 339, 328, 317, 329, 339, 329, 321, 334, 337, 327, 326, 348, 352, 339, 344, 367, 365, 350, 356, 372, 358, 333, 336, 345, 334, 320, 330, 340, 328, 315, 328, 336, 321, 305, 299, 278, 252, 265, 351, 430, 423, 399, 394, 372, 348, 352, 367, 348, 319, 321, 334, 328, 316, 324, 331, 322, 317, 331, 338, 324, 321, 339, 342, 327, 329, 345, 342, 328, 332, 345, 339, 327, 338, 356, 347, 337, 350, 361, 353, 345, 361, 371, 357, 345, 358, 359, 342, 334, 343, 337, 320, 322, 334, 326, 311, 317, 333, 326, 312, 319, 331, 322, 310, 323, 335, 325, 316, 326, 331, 319, 316, 334, 335, 319, 316, 333, 333, 318, 321, 334, 330, 318, 327, 344, 337, 323, 337, 356, 348, 342, 360, 364, 342, 325, 337, 340, 323, 317, 332, 329, 313, 314, 329, 324, 307, 308, 303, 266, 238, 256, 329, 412, 435, 419, 398, 361, 340, 357, 362, 334, 317, 329, 323, 305, 303, 321, 321, 305, 308, 324, 318, 306, 314, 330, 322, 308, 322, 337, 328, 318, 330, 338, 327, 321, 338, 344, 334, 334, 352, 355, 341, 343, 363, 360, 343, 345, 358, 350, 333, 333, 339, 326, 312, 322, 330, 317, 307, 321, 326, 309, 303, 321, 326, 312, 311, 328, 328, 312, 314, 331, 329, 319, 321, 332, 326, 313, 325, 337, 326, 319, 332, 340, 328, 317, 330, 336, 326, 330, 348, 345, 334, 340, 358, 358, 346, 346, 354, 338, 322, 328, 339, 328, 312, 323, 334, 323, 313, 325, 330, 311, 280, 264, 251, 253, 326, 440, 464, 414, 385, 380, 366, 355, 356, 353, 328, 309, 318, 329, 318, 308, 321, 331, 320, 314, 325, 328, 316, 314, 330, 331, 321, 325, 339, 334, 319, 324, 341, 338, 325, 334, 349, 345, 332, 343, 355, 343, 337, 352, 358, 344, 337, 349, 347, 328, 319, 335, 331, 313, 313, 330, 323, 304, 313, 327, 318, 305, 315, 327, 317, 310, 323, 334, 321, 311, 326, 333, 318, 313, 329, 333, 319, 318, 335, 330, 316, 322, 338, 332, 316, 322, 337, 332, 322, 335, 346, 336, 330, 349, 361, 347, 342, 359, 357, 335, 329, 341, 336, 321, 322, 338, 329, 311, 318, 334, 324, 309, 304, 282, 248, 233, 292, 403, 456, 432, 415, 392, 362, 356, 371, 358, 324, 310, 322, 322, 306, 310, 328, 322, 308, 317, 330, 321, 309, 319, 330, 321, 316, 332, 340, 325, 319, 335, 339, 327, 326, 342, 340, 327, 332, 352, 351, 335, 337, 356, 352, 338, 345, 356, 342, 326, 336, 339, 319, 308, 323, 327, 312, 307, 326, 327, 311, 310, 326, 324, 310, 315, 333, 331, 318, 321, 333, 326, 313, 326, 336, 322, 313, 329, 336, 323, 318, 333, 335, 321, 321, 340, 336, 317, 319, 338, 338, 324, 331, 346, 339, 334, 351, 365, 355, 344, 353, 354, 336, 326, 337, 337, 321, 315, 329, 331, 315, 315, 332, 327, 299, 274, 265, 254, 268, 360, 445, 438, 393, 374, 368, 351, 345, 353, 344, 318, 308, 324, 326, 311, 310, 329, 332, 312, 310, 327, 328, 315, 318, 333, 328, 318, 326, 337, 329, 321, 337, 346, 331, 320, 339, 350, 338, 333, 348, 354, 344, 343, 360, 358, 346, 348, 357, 348, 329, 332, 339, 328, 314, 324, 332, 317, 307, 321, 329, 318, 312, 326, 328, 314, 313, 330, 330, 315, 315, 333, 332, 320, 325, 341, 334, 320, 328, 339, 328, 315, 329, 340, 326, 317, 334, 345, 333, 329, 346, 350, 340, 344, 363, 365, 346, 344, 354, 346, 327, 328, 338, 327, 313, 325, 335, 320, 308, 324, 333, 303, 262, 261, 269, 292, 374, 436, 419, 377, 360, 366, 353, 341, 344, 349, 328, 309, 318, 331, 321, 307, 322, 333, 319, 311, 327, 336, 322, 318, 336, 341, 326, 327, 346, 344, 326, 331, 350, 346, 331, 341, 355, 348, 336, 347, 361, 353, 344, 357, 365, 354, 342, 350, 347, 328, 324, 337, 336, 315, 315, 332, 328, 312, 315, 331, 324, 311, 317, 330, 322, 310, 324, 334, 323, 316, 330, 335, 320, 314, 332, 334, 319, 322, 340, 338, 319, 321, 340, 338, 326, 335, 351, 347, 338, 354, 368, 359, 344, 349, 352, 337, 325, 336, 335, 319, 314, 332, 332, 317, 316, 330, 321, 284, 261, 265, 270, 320, 420, 460, 424, 381, 369, 366, 355, 345, 349, 341, 321, 314, 329, 331, 315, 315, 332, 328, 313, 318, 334, 329, 318, 326, 343, 337, 323, 333, 347, 338, 324, 336, 349, 340, 335, 352, 355, 338, 334, 356, 361, 346, 349, 367, 360, 339, 339, 351, 341, 324, 329, 341, 328, 312, 320, 333, 322, 312, 324, 329, 315, 311, 327, 331, 319, 320, 337, 334, 319, 319, 336, 334, 321, 325, 339, 333, 320, 328, 339, 329, 322, 336, 348, 336, 332, 354, 366, 349, 344, 361, 359, 336, 330, 344, 340, 318, 320, 334, 325, 309, 320, 333, 321, 304, 302, 284, 250, 236, 289, 391, 449, 435, 416, 392, 357, 349, 368, 355, 321, 312, 326, 322, 306, 312, 327, 321, 308, 316, 331, 323, 312, 327, 337, 322, 316, 331, 337, 325, 322, 340, 343, 329, 331, 349, 348, 335, 339, 358, 355, 338, 347, 364, 355, 339, 347, 353, 339, 326, 336, 341, 324, 315, 331, 335, 317, 310, 324, 325, 311, 314, 332, 326, 311, 318, 332, 325, 312, 324, 338, 327, 314, 327, 336, 325, 317, 331, 336, 320, 317, 337, 344, 328, 326, 343, 345, 339, 346, 365, 360, 341, 341, 349, 337, 322, 331, 338, 323, 309, 324, 333, 318, 313, 327, 327, 286, 251, 255, 270, 322, 423, 474, 440, 390, 374, 380, 375, 348, 339, 335, 320, 307, 321, 332, 320, 309, 324, 334, 319, 314, 333, 337, 320, 318, 339, 339, 322, 324, 342, 336, 324, 335, 353, 347, 332, 342, 355, 345, 336, 353, 364, 351, 341, 357, 359, 338, 333, 348, 344, 322, 321, 337, 332, 314, 318, 332, 324, 308, 316, 329, 320, 310, 324, 335, 321, 312, 328, 336, 323, 318, 335, 338, 324, 325, 338, 334, 319, 325, 342, 335, 318, 328, 346, 343, 332, 342, 359, 353, 342, 356, 367, 351, 333, 341, 343, 323, 318, 337, 335, 314, 315, 333, 329, 313, 316, 327, 296, 252, 250, 285, 352, 423, 448, 428, 387, 357, 360, 369, 349, 331, 337, 334, 313, 311, 332, 331, 309, 311, 333, 329, 315, 326, 342, 333, 317, 328, 344, 334, 326, 341, 348, 330, 325, 347, 355, 337, 331, 349, 355, 341, 345, 365, 364, 345, 349, 366, 354, 332, 338, 349, 335, 319, 328, 337, 322, 311, 325, 331, 315, 310, 325, 330, 316, 314, 329, 328, 313, 316, 336, 332, 316, 322, 337, 328, 315, 327, 341, 330, 314, 325, 337, 329, 324, 342, 346, 331, 335, 360, 361, 343, 346, 360, 347, 325, 328, 342, 328, 308, 317, 329, 319, 307, 320, 329, 314, 298, 288, 269, 248, 264, 354, 428, 414, 386, 386, 366, 338, 341, 361, 345, 313, 311, 328, 322, 306, 317, 331, 318, 308, 328, 338, 318, 311, 329, 333, 319, 316, 332, 337, 322, 326, 345, 342, 327, 333, 349, 344, 333, 345, 361, 353, 341, 353, 365, 356, 341, 351, 355, 336, 328, 341, 338, 322, 320, 331, 324, 307, 313, 331, 326, 309, 317, 335, 328, 313, 320, 331, 323, 314, 328, 340, 322, 313, 330, 336, 322, 317, 337, 338, 320, 322, 344, 340, 325, 332, 352, 347, 336, 346, 362, 358, 341, 350, 357, 340, 323, 332, 339, 324, 315, 332, 335, 315, 313, 333, 331, 290, 259, 262, 272, 321, 419, 460, 425, 381, 371, 372, 360, 348, 349, 344, 323, 311, 328, 333, 317, 312, 330, 332, 318, 319, 335, 336, 319, 322, 337, 333, 322, 332, 344, 334, 323, 336, 349, 338, 330, 345, 353, 342, 338, 358, 361, 346, 344, 361, 359, 339, 337, 353, 344, 321, 324, 340, 330, 312, 317, 329, 319, 309, 323, 333, 318, 307, 324, 328, 311, 310, 333, 337, 320, 316, 333, 331, 316, 324, 344, 336, 317, 323, 338, 331, 318, 332, 345, 335, 324, 342, 359, 350, 343, 366, 370, 344, 332, 345, 337, 313, 315, 333, 329, 313, 318, 331, 320, 306, 313, 307, 264, 232, 253, 331, 419, 445, 429, 404, 364, 347, 368, 366, 329, 312, 326, 323, 305, 311, 329, 321, 305, 315, 329, 321, 309, 323, 336, 324, 313, 330, 340, 326, 320, 339, 344, 330, 326, 344, 347, 333, 338, 358, 354, 338, 346, 362, 355, 343, 352, 362, 346, 326, 336, 345, 328, 316, 331, 334, 317, 310, 329, 332, 313, 314, 334, 329, 308, 314, 335, 332, 315, 325, 342, 331, 312, 324, 340, 329, 318, 333, 341, 325, 318, 338, 345, 329, 323, 340, 342, 330, 335, 353, 352, 345, 352, 368, 362, 345, 349, 353, 338, 322, 332, 338, 324, 315, 331, 337, 319, 313, 330, 327, 286, 258, 262, 276, 332, 432, 474, 440, 391, 374, 379, 370, 347, 340, 338, 318, 306, 323, 328, 310, 304, 325, 332, 314, 312, 331, 333, 316, 317, 336, 335, 321, 328, 342, 335, 321, 332, 349, 341, 332, 346, 358, 345, 340, 360, 367, 350, 345, 364, 366, 345, 339, 350, 345, 324, 325, 340, 332, 313, 320, 332, 325, 313, 319, 330, 317, 306, 325, 334, 317, 309, 329, 336, 318, 315, 335, 338, 321, 323, 341, 336, 318, 324, 342, 331, 313, 325, 341, 330, 316, 331, 341, 329, 322, 340, 346, 330, 330, 357, 365, 347, 346, 361, 354, 333, 335, 349, 340, 319, 322, 336, 325, 310, 320, 334, 323, 304, 294, 274, 246, 259, 355, 439, 435, 399, 390, 375, 348, 349, 366, 350, 319, 321, 333, 321, 306, 319, 334, 322, 309, 322, 336, 323, 315, 330, 337, 323, 322, 342, 344, 324, 325, 343, 344, 328, 328, 350, 347, 333, 344, 361, 354, 342, 356, 373, 360, 344, 356, 360, 337, 324, 336, 340, 322, 315, 328, 330, 311, 311, 333, 329, 311, 318, 334, 326, 310, 319, 334, 323, 311, 325, 338, 327, 315, 329, 339, 326, 320, 336, 337, 321, 323, 341, 333, 315, 323, 342, 335, 320, 330, 348, 342, 326, 343, 360, 348, 339, 356, 363, 341, 327, 336, 333, 318, 318, 335, 333, 313, 315, 334, 332, 311, 297, 285, 261, 248, 305, 404, 426, 396, 392, 383, 352, 339, 357, 358, 328, 311, 324, 326, 310, 312, 326, 324, 307, 314, 337, 331, 311, 319, 338, 331, 317, 327, 340, 331, 321, 338, 348, 335, 330, 345, 347, 333, 334, 355, 359, 343, 345, 365, 361, 342, 345, 357, 347, 323, 330, 343, 329, 313, 323, 334, 320, 311, 324, 329, 312, 306, 326, 330, 314, 313, 331, 330, 312, 320, 340, 334, 315, 321, 341, 332, 317, 331, 342, 324, 312, 332, 346, 328, 316, 330, 338, 328, 330, 347, 347, 338, 346, 363, 357, 343, 346, 356, 343, 324, 332, 345, 329, 312, 326, 338, 320, 309, 324, 329, 301, 265, 261, 258, 283, 377, 459, 447, 395, 369, 370, 360, 345, 340, 340, 322, 305, 317, 329, 318, 307, 324, 333, 316, 305, 324, 330, 313, 313, 332, 333, 318, 320, 337, 335, 320, 329, 347, 340, 325, 336, 353, 345, 332, 348, 361, 351, 342, 360, 366, 346, 336, 348, 348, 329, 325, 338, 335, 316, 317, 331, 324, 308, 317, 334, 323, 308, 322, 335, 321, 308, 321, 332, 319, 312, 332, 337, 320, 318, 334, 335, 318, 320, 339, 338, 318, 321, 338, 332, 317, 325, 340, 331, 321, 334, 347, 336, 328, 352, 365, 350, 345, 364, 359, 333, 327, 341, 335, 313, 316, 334, 327, 310, 318, 332, 322, 304, 298, 276, 245, 242, 319, 428, 454, 420, 407, 388, 357, 352, 364, 347, 311, 308, 326, 324, 306, 309, 327, 324, 308, 316, 330, 323, 312, 328, 341, 324, 315, 332, 337, 325, 322, 339, 341, 325, 324, 346, 351, 336, 338, 355, 352, 338, 349, 365, 355, 341, 350, 354, 338, 323, 334, 341, 325, 316, 329, 332, 314, 309, 327, 328, 311, 313, 334, 332, 313, 317, 336, 327, 311, 321, 337, 327, 313, 329, 344, 330, 318, 334, 340, 324, 317, 338, 344, 326, 318, 334, 334, 317, 321, 339, 333, 318, 330, 350, 345, 329, 342, 365, 355, 342, 361, 370, 346, 328, 338, 341, 324, 322, 338, 336, 315, 315, 334, 329, 309, 306, 296, 265, 239, 283, 387, 449, 435, 414, 395, 366, 350, 368, 367, 334, 318, 328, 330, 313, 311, 331, 331, 313, 314, 331, 330, 313, 320, 336, 328, 316, 328, 340, 328, 318, 332, 342, 328, 322, 342, 349, 334, 334, 355, 355, 340, 346, 367, 364, 345, 346, 360, 351, 330, 334, 344, 330, 314, 325, 335, 321, 308, 322, 329, 313, 310, 329, 331, 313, 310, 328, 330, 316, 316, 331, 329, 314, 322, 339, 330, 315, 327, 339, 326, 315, 332, 342, 327, 316, 330, 333, 317, 318, 341, 342, 326, 328, 351, 354, 338, 345, 366, 360, 338, 342, 354, 336, 312, 324, 338, 324, 315, 329, 335, 317, 310, 324, 309, 263, 249, 275, 324, 388, 422, 422, 392, 355, 349, 362, 354, 334, 334, 337, 320, 307, 324, 333, 316, 311, 332, 334, 314, 314, 336, 339, 321, 323, 340, 335, 320, 329, 347, 338, 322, 333, 352, 343, 331, 344, 359, 347, 338, 359, 366, 349, 345, 365, 365, 343, 338, 348, 339, 319, 322, 340, 335, 316, 318, 334, 324, 305, 317, 333, 321, 307, 324, 337, 319, 311, 329, 335, 319, 316, 336, 334, 317, 320, 336, 331, 313, 318, 337, 330, 315, 324, 338, 327, 314, 332, 348, 335, 328, 349, 360, 346, 345, 364, 358, 329, 321, 338, 332, 312, 319, 337, 328, 309, 316, 332, 323, 302, 291, 275, 246, 244, 326, 431, 447, 413, 400, 380, 350, 348, 366, 355, 324, 316, 331, 327, 309, 314, 333, 326, 312, 323, 335, 322, 313, 329, 339, 326, 318, 334, 343, 329, 325, 341, 343, 327, 328, 348, 347, 331, 337, 359, 355, 339, 350, 368, 359, 342, 349, 360, 342, 326, 339, 343, 324, 312, 328, 330, 311, 308, 328, 328, 310, 310, 326, 323, 307, 314, 332, 326, 313, 326, 339, 328, 313, 325, 337, 325, 317, 334, 341, 325, 316, 333, 337, 320, 321, 342, 343, 329, 331, 350, 352, 343, 350, 368, 357, 339, 345, 350, 330, 314, 329, 338, 319, 310, 326, 330, 313, 312, 328, 309, 256, 238, 262, 318, 399, 451, 444, 404, 363, 361, 376, 360, 327, 324, 329, 313, 304, 318, 323, 308, 306, 325, 326, 308, 309, 330, 330, 312, 316, 335, 332, 318, 329, 347, 337, 321, 332, 344, 333, 324, 344, 356, 343, 335, 355, 361, 344, 344, 364, 363, 339, 333, 345, 337, 318, 325, 338, 324, 303, 315, 331, 319, 304, 319, 331, 318, 306, 324, 333, 318, 312, 330, 334, 317, 317, 334, 332, 315, 320, 337, 332, 317, 327, 345, 333, 317, 329, 341, 329, 320, 340, 353, 337, 329, 353, 367, 351, 349, 364, 359, 336, 334, 347, 338, 318, 325, 339, 330, 315, 326, 339, 325, 308, 306, 286, 252, 242, 303, 408, 454, 433, 422, 398, 361, 355, 371, 354, 319, 315, 332, 326, 307, 317, 336, 321, 304, 320, 338, 326, 314, 325, 332, 320, 315, 336, 342, 323, 320, 339, 340, 326, 331, 351, 349, 335, 343, 359, 350, 336, 348, 366, 357, 345, 356, 362, 343, 330, 343, 346, 323, 315, 331, 333, 315, 316, 334, 331, 314, 318, 335, 329, 314, 324, 339, 329, 314, 326, 343, 330, 319, 336, 346, 329, 320, 337, 345, 329, 325, 339, 338, 320, 327, 351, 346, 325, 334, 357, 353, 339, 351, 371, 358, 339, 346, 352, 331, 316, 331, 336, 320, 315, 333, 336, 314, 313, 326, 299, 253, 247, 280, 336, 398, 426, 419, 384, 350, 354, 364, 352, 334, 336, 336, 315, 305, 324, 328, 312, 313, 330, 328, 310, 313, 334, 329, 312, 323, 343, 333, 317, 329, 346, 335, 320, 334, 348, 336, 330, 348, 355, 339, 338, 361, 365, 346, 345, 366, 360, 335, 335, 352, 342, 320, 325, 341, 332, 314, 322, 335, 322, 312, 325, 334, 321, 317, 337, 338, 320, 320, 338, 336, 321, 324, 340, 337, 321, 328, 345, 334, 320, 332, 342, 328, 318, 338, 352, 341, 332, 350, 365, 349, 343, 365, 364, 338, 333, 349, 343, 321, 324, 340, 331, 313, 323, 338, 326, 311, 315, 298, 261, 245, 290, 376, 430, 424, 412, 389, 354, 346, 370, 365, 331, 320, 335, 330, 313, 322, 337, 325, 311, 323, 335, 322, 312, 329, 338, 324, 317, 336, 342, 327, 325, 342, 344, 327, 327, 347, 346, 325, 333, 357, 350, 334, 342, 361, 352, 340, 350, 357, 338, 326, 339, 343, 320, 308, 327, 331, 311, 308, 327, 324, 306, 310, 330, 325, 307, 313, 332, 324, 310, 323, 336, 321, 309, 328, 337, 321, 313, 330, 338, 321, 317, 335, 339, 324, 328, 349, 345, 332, 341, 359, 356, 344, 350, 355, 336, 320, 330, 340, 324, 311, 326, 335, 320, 313, 331, 331, 302, 269, 262, 253, 261, 351, 454, 456, 404, 378, 374, 360, 349, 349, 342, 320, 305, 319, 331, 315, 305, 321, 327, 311, 308, 325, 327, 311, 315, 334, 331, 315, 321, 338, 331, 316, 328, 345, 336, 325, 337, 347, 335, 329, 348, 354, 339, 334, 353, 359, 342, 338, 350, 343, 320, 323, 338, 327, 306, 312, 330, 321, 304, 316, 331, 316, 305, 324, 332, 315, 307, 327, 333, 314, 313, 333, 336, 319, 318, 337, 333, 316, 322, 336, 329, 317, 327, 341, 332, 322, 337, 350, 342, 340, 357, 367, 352, 342, 354, 351, 328, 322, 337, 334, 313, 317, 336, 330, 312, 321, 335, 315, 274, 259, 260, 268, 335, 443, 471, 422, 381, 379, 378, 362, 348, 345, 333, 311, 313, 332, 329, 310, 315, 331, 323, 308, 320, 334, 323, 308, 324, 338, 325, 317, 336, 343, 326, 321, 342, 347, 329, 330, 348, 345, 329, 335, 356, 354, 337, 347, 363, 354, 335, 341, 350, 336, 321, 332, 337, 317, 310, 325, 328, 312, 310, 326, 328, 312, 314, 332, 328, 314, 321, 336, 328, 313, 328, 342, 326, 313, 332, 343, 330, 320, 336, 341, 326, 320, 337, 343, 324, 322, 345, 345, 327, 333, 355, 356, 342, 351, 369, 358, 337, 342, 350, 330, 317, 332, 343, 327, 318, 333, 335, 320, 316, 328, 309, 260, 246, 274, 325, 401, 449, 446, 404, 360, 357, 373, 360, 333, 330, 331, 316, 308, 325, 329, 314, 310, 330, 332, 312, 311, 333, 333, 314, 322, 342, 336, 320, 332, 348, 337, 322, 335, 349, 338, 329, 347, 355, 341, 339, 358, 363, 345, 343, 361, 360, 340, 340, 349, 338, 314, 317, 334, 327, 312, 318, 332, 321, 307, 321, 333, 319, 309, 325, 334, 316, 311, 332, 335, 318, 319, 339, 336, 317, 324, 343, 336, 318, 325, 339, 328, 318, 330, 344, 334, 325, 343, 352, 341, 343, 363, 366, 349, 345, 360, 356, 331, 325, 337, 332, 315, 322, 341, 331, 311, 320, 333, 317, 287, 274, 272, 262, 306, 407, 443, 407, 375, 370, 360, 338, 343, 356, 343, 313, 313, 332, 327, 308, 316, 334, 324, 311, 325, 337, 321, 311, 331, 344, 327, 319, 335, 341, 325, 326, 346, 348, 329, 332, 355, 353, 337, 344, 364, 355, 340, 351, 368, 357, 344, 351, 352, 330, 320, 336, 339, 318, 312, 329, 329, 309, 310, 327, 326, 307, 313, 332, 324, 307, 320, 336, 324, 310, 324, 337, 321, 313, 332, 341, 322, 313, 331, 337, 320, 318, 335, 334, 320, 329, 354, 346, 327, 339, 365, 357, 344, 357, 368, 348, 329, 342, 347, 327, 316, 333, 337, 320, 315, 330, 331, 312, 303, 296, 269, 240, 266, 364, 438, 428, 406, 391, 361, 340, 356, 365, 338, 316, 326, 332, 311, 305, 326, 333, 313, 311, 331, 333, 315, 316, 335, 334, 319, 327, 346, 338, 321, 332, 346, 337, 327, 342, 354, 341, 335, 353, 358, 342, 342, 363, 363, 342, 341, 362, 354, 330, 330, 343, 333, 311, 317, 333, 322, 308, 320, 330, 314, 306, 325, 333, 314, 310, 331, 336, 315, 314, 331, 330, 314, 320, 339, 333, 314, 322, 337, 328, 314, 329, 343, 331, 319, 334, 343, 329, 326, 343, 350, 341, 344, 364, 365, 341, 334, 347, 340, 320, 323, 336, 324, 310, 321, 334, 323, 311, 322, 323, 282, 246, 251, 276, 333, 423, 461, 435, 389, 365, 372, 372, 343, 330, 337, 325, 305, 316, 332, 321, 306, 319, 333, 316, 307, 324, 336, 320, 311, 329, 336, 321, 322, 342, 342, 325, 330, 349, 345, 325, 334, 354, 347, 335, 346, 359, 350, 340, 355, 364, 344, 330, 344, 346, 325, 321, 333, 330, 312, 311, 328, 325, 309, 314, 331, 324, 307, 317, 333, 324, 309, 320, 334, 322, 316, 333, 339, 320, 316, 336, 340, 320, 318, 337, 338, 321, 324, 342, 338, 320, 327, 345, 338, 326, 334, 348, 341, 336, 353, 367, 353, 339, 348, 348, 327, 317, 336, 338, 316, 313, 330, 324, 306, 316, 334, 314, 262, 243, 253, 276, 353, 444, 458, 409, 366, 365, 369, 353, 338, 339, 331, 308, 306, 325, 324, 304, 310, 325, 324, 308, 314, 334, 325, 307, 319, 336, 325, 314, 333, 346, 331, 321, 338, 348, 335, 334, 354, 360, 341, 335, 352, 353, 339, 347, 367, 358, 336, 342, 356, 339, 320, 329, 336, 317, 308, 325, 332, 313, 304, 323, 326, 307, 309, 328, 327, 309, 313, 333, 331, 314, 321, 334, 325, 315, 328, 340, 327, 318, 330, 338, 322, 313, 332, 340, 321, 314, 333, 337, 322, 328, 350, 345, 326, 340, 365, 358, 342, 351, 362, 339, 320, 329, 338, 322, 317, 333, 334, 314, 310, 332, 333, 309, 294, 285, 265, 246, 294, 400, 436, 403, 385, 383, 362, 340, 353, 361, 336, 313, 326, 336, 318, 311, 328, 332, 314, 312, 333, 337, 319, 320, 335, 333, 317, 325, 343, 332, 316, 329, 346, 336, 326, 340, 352, 336, 329, 348, 358, 344, 343, 360, 364, 346, 341, 357, 351, 328, 328, 340, 330, 313, 323, 337, 321, 304, 318, 334, 320, 305, 321, 330, 314, 309, 330, 334, 314, 311, 334, 337, 319, 319, 336, 332, 315, 324, 341, 329, 315, 327, 345, 333, 316, 330, 340, 322, 316, 339, 352, 334, 333, 358, 362, 345, 350, 367, 356, 329, 331, 345, 335, 312, 320, 336, 328, 313, 325, 336, 320, 299, 288, 275, 248, 266, 363, 447, 431, 393, 387, 374, 346, 349, 364, 346, 318, 319, 334, 320, 303, 318, 336, 322, 310, 323, 332, 317, 313, 330, 336, 320, 319, 338, 337, 319, 322, 343, 342, 325, 332, 352, 345, 328, 342, 362, 351, 336, 350, 365, 351, 342, 355, 357, 332, 322, 339, 339, 317, 314, 330, 326, 305, 308, 327, 322, 305, 315, 333, 326, 310, 322, 335, 322, 311, 327, 339, 323, 315, 331, 336, 317, 316, 336, 337, 318, 318, 340, 335, 312, 318, 339, 333, 317, 329, 347, 335, 324, 349, 366, 348, 341, 359, 362, 335, 325, 337, 334, 314, 315, 336, 331, 311, 315, 334, 328, 308, 295, 278, 250, 238, 305, 422, 462, 423, 402, 387, 356, 349, 365, 356, 322, 314, 330, 325, 304, 310, 331, 326, 306, 310, 330, 325, 312, 324, 340, 329, 314, 328, 343, 330, 323, 342, 348, 329, 324, 344, 350, 335, 337, 357, 355, 335, 339, 360, 354, 337, 344, 357, 343, 325, 334, 341, 323, 311, 324, 332, 315, 307, 325, 332, 312, 310, 327, 326, 312, 317, 335, 329, 310, 316, 336, 330, 315, 325, 338, 328, 314, 329, 341, 327, 319, 335, 341, 324, 319, 337, 341, 319, 321, 346, 348, 331, 335, 360, 361, 345, 353, 370, 354, 331, 339, 350, 331, 317, 331, 339, 323, 316, 329, 331, 315, 313, 320, 288, 244, 238, 293, 380, 443, 448, 433, 394, 358, 364, 377, 351, 318, 324, 333, 317, 310, 326, 331, 310, 306, 326, 332, 316, 314, 332, 330, 314, 322, 342, 334, 320, 331, 346, 336, 322, 336, 351, 340, 331, 352, 361, 342, 335, 354, 361, 346, 346, 365, 360, 338, 338, 349, 337, 318, 323, 339, 330, 313, 323, 339, 325, 313, 326, 337, 320, 315, 336, 342, 325, 322, 340, 338, 322, 328, 347, 343, 320, 326, 346, 341, 322, 330, 348, 338, 321, 334, 348, 336, 327, 341, 345, 331, 331, 351, 358, 348, 349, 367, 369, 349, 349, 361, 347, 320, 327, 343, 332, 317, 326, 338, 324, 312, 329, 333, 292, 256, 259, 280, 330, 412, 454, 430, 376, 356, 364, 363, 343, 338, 343, 326, 305, 316, 331, 320, 308, 324, 335, 317, 310, 329, 336, 318, 316, 339, 343, 324, 320, 336, 338, 325, 332, 351, 342, 329, 342, 359, 348, 334, 347, 363, 354, 344, 357, 363, 342, 332, 348, 351, 328, 318, 332, 328, 310, 314, 330, 323, 303, 314, 336, 326, 308, 315, 330, 320, 310, 327, 336, 320, 313, 327, 334, 319, 316, 334, 334, 313, 317, 338, 336, 318, 323, 338, 332, 317, 329, 346, 337, 323, 341, 365, 352, 342, 359, 366, 341, 327, 339, 339, 320, 317, 331, 327, 309, 311, 328, 321, 302, 304, 301, 269, 243, 272, 355, 416, 412, 398, 390, 359, 343, 357, 360, 330, 313, 324, 325, 310, 313, 329, 325, 309, 316, 336, 328, 310, 323, 342, 334, 320, 332, 342, 328, 322, 340, 350, 332, 326, 349, 356, 337, 336, 354, 357, 345, 351, 370, 367, 347, 351, 364, 349, 329, 336, 344, 329, 315, 328, 337, 317, 308, 324, 332, 315, 311, 330, 330, 309, 311, 329, 328, 310, 317, 338, 330, 311, 320, 337, 330, 320, 329, 339, 324, 315, 329, 336, 320, 321, 341, 345, 329, 333, 357, 361, 342, 349, 369, 358, 334, 336, 347, 332, 313, 324, 337, 320, 309, 327, 338, 318, 305, 305, 282, 246, 246, 319, 419, 448, 423, 407, 380, 353, 361, 371, 347, 318, 322, 333, 320, 308, 323, 334, 317, 308, 325, 332, 316, 315, 334, 335, 316, 319, 342, 339, 320, 327, 345, 339, 323, 334, 354, 346, 331, 345, 362, 350, 338, 356, 369, 353, 345, 359, 360, 333, 326, 342, 340, 318, 317, 333, 330, 313, 319, 335, 325, 309, 322, 335, 321, 311, 326, 335, 319, 314, 336, 341, 320, 318, 339, 342, 324, 322, 339, 336, 318, 325, 344, 337, 320, 331, 349, 340, 324, 340, 362, 354, 344, 361, 371, 348, 336, 347, 347, 324, 320, 336, 334, 316, 319, 335, 329, 312, 322, 330, 290, 245, 244, 283, 354, 431, 457, 436, 385, 357, 370, 379, 351, 327, 331, 327, 311, 310, 327, 322, 305, 312, 330, 325, 307, 318, 335, 322, 310, 331, 339, 320, 312, 333, 344, 329, 325, 343, 348, 331, 330, 348, 350, 336, 345, 362, 355, 339, 347, 363, 349, 331, 342, 349, 330, 317, 330, 337, 318, 310, 326, 332, 314, 311, 328, 328, 314, 318, 335, 329, 313, 319, 337, 328, 313, 322, 338, 332, 317, 327, 338, 325, 316, 333, 341, 321, 316, 336, 339, 320, 320, 343, 346, 330, 330, 352, 357, 344, 352, 369, 354, 328, 335, 347, 329, 316, 329, 339, 323, 315, 329, 333, 318, 315, 323, 295, 248, 237, 280, 361, 431, 446, 434, 397, 361, 367, 381, 352, 320, 325, 335, 316, 304, 322, 332, 315, 308, 324, 328, 313, 314, 332, 332, 318, 321, 338, 330, 316, 327, 347, 339, 324, 336, 348, 337, 331, 348, 354, 337, 334, 354, 360, 341, 339, 357, 355, 331, 329, 347, 336, 313, 317, 330, 318, 302, 316, 327, 312, 301, 319, 330, 315, 308, 325, 331, 315, 309, 329, 333, 315, 313, 330, 330, 315, 320, 336, 325, 309, 323, 341, 331, 317, 331, 343, 324, 312, 332, 347, 334, 328, 346, 361, 349, 350, 370, 365, 336, 334, 352, 347, 329, 336, 351, 338, 320, 332, 347, 334, 324, 336, 325, 280, 259, 291, 365, 434, 449, 441, 415, 376, 367, 384, 381, 350, 341, 350, 341, 324, 335, 352, 338, 325, 340, 352, 334, 324, 340, 350, 334, 327, 345, 350, 330, 331, 354, 351, 334, 339, 357, 352, 338, 348, 366, 355, 341, 357, 372, 359, 347, 362, 368, 346, 330, 341, 345, 326, 323, 341, 338, 313, 311, 331, 330, 313, 316, 334, 326, 309, 320, 336, 324, 312, 328, 340, 323, 312, 330, 341, 319, 314, 334, 340, 320, 318, 338, 335, 317, 323, 342, 336, 323, 338, 352, 341, 332, 346, 361, 354, 345, 357, 355, 330, 318, 333, 337, 318, 316, 336, 331, 310, 313, 333, 322, 285, 266, 272, 271, 310, 402, 442, 407, 372, 370, 368, 345, 340, 351, 344, 317, 312, 330, 332, 312, 311, 331, 327, 309, 316, 334, 328, 313, 324, 342, 331, 316, 327, 343, 331, 324, 344, 357, 334, 326, 348, 357, 344, 345, 365, 365, 345, 349, 368, 361, 341, 344, 354, 341, 324, 334, 346, 328, 311, 324, 335, 320, 314, 329, 334, 315, 315, 336, 338, 320, 320, 336, 335, 318, 323, 342, 337, 321, 332, 348, 335, 323, 335, 346, 332, 324, 339, 345, 326, 323, 345, 356, 337, 339, 364, 368, 353, 362, 379, 363, 337, 341, 355, 339, 324, 333, 346, 333, 318, 331, 340, 321, 309, 306, 282, 248, 256, 338, 430, 444, 418, 409, 384, 353, 357, 371, 350, 321, 323, 335, 322, 307, 323, 337, 320, 311, 327, 334, 320, 317, 337, 340, 318, 318, 340, 341, 322, 326, 346, 342, 328, 341, 357, 347, 333, 349, 367, 351, 340, 357, 369, 354, 348, 361, 361, 336, 331, 348, 343, 320, 316, 335, 335, 319, 322, 338, 329, 312, 321, 338, 325, 313, 330, 342, 329, 318, 335, 341, 324, 325, 345, 344, 321, 321, 341, 341, 324, 331, 348, 337, 318, 332, 348, 336, 324, 341, 356, 343, 335, 356, 370, 355, 351, 368, 365, 338, 333, 350, 342, 320, 324, 342, 337, 322, 328, 340, 329, 313, 306, 290, 260, 251, 321, 431, 465, 437, 424, 405, 369, 361, 376, 362, 329, 323, 342, 337, 319, 325, 338, 330, 316, 328, 340, 329, 318, 332, 345, 333, 328, 344, 349, 333, 331, 348, 352, 335, 334, 353, 353, 338, 345, 365, 360, 343, 354, 371, 363, 347, 355, 362, 344, 328, 339, 345, 329, 318, 332, 334, 316, 312, 329, 333, 315, 316, 334, 330, 312, 321, 337, 327, 311, 323, 339, 327, 315, 330, 341, 327, 317, 333, 339, 322, 316, 334, 336, 319, 321, 343, 341, 322, 325, 342, 335, 318, 326, 343, 337, 323, 334, 347, 342, 340, 357, 367, 355, 345, 354, 351, 330, 325, 342, 339, 313, 313, 333, 329, 311, 317, 332, 312, 268, 254, 258, 281, 364, 452, 457, 411, 377, 379, 379, 359, 340, 339, 331, 311, 312, 328, 324, 306, 314, 331, 320, 305, 313, 328, 320, 309, 325, 340, 328, 315, 332, 340, 324, 319, 340, 348, 335, 336, 352, 348, 331, 337, 357, 355, 339, 349, 367, 355, 337, 348, 359, 341, 325, 336, 342, 324, 311, 326, 331, 310, 307, 328, 331, 313, 315, 335, 333, 316, 323, 339, 332, 314, 326, 340, 326, 316, 331, 341, 323, 311, 330, 342, 324, 318, 335, 339, 320, 316, 332, 332, 315, 321, 339, 332, 313, 321, 341, 337, 327, 339, 351, 341, 336, 355, 366, 354, 345, 353, 351, 331, 325, 337, 337, 319, 320, 337, 329, 313, 318, 336, 324, 284, 260, 259, 263, 325, 429, 463, 414, 375, 371, 369, 354, 348, 349, 335, 313, 315, 334, 327, 308, 315, 333, 328, 314, 324, 342, 334, 317, 327, 341, 326, 318, 334, 342, 329, 325, 342, 347, 332, 332, 350, 349, 334, 339, 361, 359, 338, 345, 366, 357, 341, 352, 360, 341, 323, 332, 340, 323, 313, 327, 333, 315, 312, 329, 331, 313, 313, 332, 329, 310, 319, 334, 324, 309, 321, 338, 329, 318, 332, 340, 324, 315, 334, 344, 324, 316, 335, 339, 321, 319, 338, 339, 323, 325, 341, 337, 319, 329, 346, 341, 326, 340, 359, 352, 340, 360, 372, 351, 339, 347, 342, 320, 319, 341, 340, 318, 316, 335, 334, 319, 319, 320, 280, 245, 260, 319, 390, 423, 424, 410, 373, 348, 357, 368, 348, 327, 332, 327, 311, 312, 332, 329, 309, 312, 333, 331, 317, 327, 341, 332, 320, 333, 349, 336, 325, 338, 346, 334, 331, 351, 355, 335, 334, 357, 363, 346, 345, 361, 359, 345, 351, 366, 356, 337, 343, 354, 340, 325, 334, 337, 320, 309, 326, 332, 316, 313, 330, 330, 313, 317, 340, 336, 313, 319, 335, 331, 317, 324, 343, 336, 322, 333, 344, 330, 317, 330, 341, 328, 320, 331, 336, 322, 319, 334, 335, 321, 329, 347, 340, 326, 336, 355, 354, 343, 354, 368, 352, 334, 342, 348, 328, 317, 332, 339, 319, 314, 328, 328, 311, 312, 315, 278, 240, 242, 294, 374, 429, 436, 417, 375, 347, 352, 366, 345, 320, 323, 323, 306, 303, 321, 324, 306, 308, 327, 325, 305, 312, 333, 329, 312, 319, 337, 327, 311, 324, 341, 332, 322, 338, 347, 332, 327, 345, 352, 340, 340, 357, 360, 341, 341, 359, 352, 331, 334, 348, 337, 318, 326, 337, 323, 312, 323, 331, 317, 307, 324, 335, 319, 315, 333, 336, 315, 317, 337, 331, 312, 318, 338, 332, 316, 327, 344, 331, 315, 328, 340, 328, 319, 334, 339, 322, 318, 337, 341, 322, 317, 336, 341, 326, 331, 347, 349, 340, 349, 363, 357, 341, 344, 350, 334, 319, 330, 338, 322, 314, 330, 334, 314, 312, 329, 320, 273, 247, 255, 273, 336, 434, 464, 427, 381, 369, 376, 364, 340, 334, 333, 312, 302, 318, 330, 317, 307, 319, 324, 312, 309, 327, 327, 308, 313, 332, 327, 312, 322, 341, 333, 317, 328, 344, 336, 327, 338, 346, 335, 333, 353, 360, 342, 338, 356, 358, 337, 338, 352, 342, 318, 321, 335, 327, 310, 316, 329, 315, 301, 317, 332, 319, 305, 320, 328, 315, 311, 330, 335, 317, 317, 338, 337, 316, 320, 338, 333, 316, 328, 347, 334, 320, 334, 344, 326, 316, 331, 343, 329, 323, 343, 349, 333, 331, 354, 361, 348, 353, 372, 365, 340, 340, 355, 343, 321, 327, 342, 331, 320, 333, 340, 325, 319, 325, 300, 257, 239, 283, 373, 440, 447, 434, 405, 365, 363, 384, 367, 329, 324, 332, 320, 308, 320, 335, 321, 309, 324, 334, 318, 311, 331, 334, 318, 320, 341, 342, 325, 332, 348, 342, 324, 335, 352, 346, 333, 345, 360, 349, 339, 355, 366, 349, 344, 364, 366, 340, 332, 345, 339, 319, 319, 334, 328, 310, 315, 331, 325, 311, 321, 335, 324, 307, 321, 334, 320, 312, 328, 337, 321, 316, 336, 341, 324, 321, 339, 339, 320, 321, 337, 334, 318, 325, 340, 332, 321, 334, 347, 343, 340, 356, 369, 357, 347, 353, 350, 331, 323, 336, 337, 318, 319, 336, 332, 317, 322, 336, 318, 273, 259, 267, 279, 340, 425, 441, 403, 372, 370, 362, 343, 341, 351, 346, 324, 320, 332, 329, 311, 319, 338, 332, 315, 322, 338, 329, 319, 335, 350, 332, 320, 336, 346, 329, 324, 345, 352, 334, 332, 351, 355, 342, 345, 364, 361, 345, 355, 373, 363, 340, 345, 354, 339, 324, 336, 343, 324, 315, 328, 332, 314, 310, 329, 331, 312, 313, 331, 330, 313, 316, 334, 328, 316, 328, 340, 328, 315, 332, 346, 331, 317, 333, 342, 324, 322, 341, 345, 331, 338, 366, 367, 349, 354, 368, 356, 331, 331, 344, 333, 318, 325, 335, 320, 309, 324, 335, 318, 303, 291, 269, 246, 268, 360, 436, 420, 390, 382, 362, 338, 344, 360, 339, 313, 318, 330, 317, 304, 320, 332, 320, 310, 322, 329, 314, 312, 332, 337, 320, 320, 335, 333, 319, 327, 344, 336, 320, 334, 351, 343, 331, 343, 355, 342, 333, 349, 358, 344, 334, 345, 346, 324, 321, 335, 328, 305, 309, 326, 321, 305, 312, 325, 314, 299, 311, 329, 318, 306, 318, 331, 317, 309, 324, 331, 314, 307, 327, 331, 314, 314, 332, 334, 318, 323, 343, 337, 324, 336, 355, 347, 338, 354, 363, 340, 324, 336, 340, 320, 310, 325, 328, 310, 311, 332, 328, 306, 301, 293, 261, 232, 257, 353, 437, 438, 409, 392, 363, 344, 359, 363, 333, 311, 322, 326, 311, 308, 325, 327, 311, 311, 331, 330, 310, 313, 332, 329, 318, 328, 343, 334, 322, 332, 344, 333, 325, 344, 351, 337, 335, 354, 357, 344, 346, 362, 358, 340, 345, 361, 348, 325, 329, 342, 332, 314, 322, 336, 322, 309, 321, 333, 318, 308, 322, 330, 316, 315, 332, 331, 316, 319, 339, 335, 315, 323, 341, 334, 320, 331, 345, 336, 327, 339, 349, 338, 333, 356, 364, 351, 349, 363, 359, 337, 334, 347, 341, 321, 325, 338, 331, 318, 326, 339, 329, 308, 294, 276, 250, 256, 341, 446, 454, 418, 406, 387, 360, 360, 370, 350, 318, 318, 333, 324, 308, 316, 331, 322, 307, 320, 335, 325, 315, 330, 341, 327, 320, 338, 350, 333, 326, 343, 347, 331, 333, 350, 351, 337, 345, 361, 357, 344, 358, 375, 364, 347, 351, 356, 340, 328, 337, 342, 326, 321, 336, 336, 315, 315, 334, 330, 311, 316, 338, 333, 317, 326, 341, 327, 313, 329, 343, 331, 318, 331, 343, 331, 320, 334, 342, 329, 328, 346, 347, 330, 337, 359, 361, 348, 348, 358, 345, 328, 335, 339, 324, 311, 325, 334, 321, 312, 327, 331, 314, 294, 280, 263, 246, 291, 402, 455, 424, 394, 387, 366, 342, 352, 360, 335, 310, 320, 330, 317, 309, 324, 331, 315, 312, 335, 337, 315, 313, 331, 333, 318, 322, 341, 336, 318, 326, 343, 339, 327, 338, 351, 338, 331, 349, 361, 347, 341, 359, 365, 348, 344, 353, 345, 324, 322, 333, 327, 311, 316, 332, 324, 311, 321, 333, 320, 305, 319, 332, 318, 309, 327, 336, 320, 313, 330, 337, 324, 324, 340, 336, 318, 324, 345, 343, 327, 334, 356, 356, 343, 355, 371, 355, 337, 339, 344, 326, 318, 331, 334, 318, 314, 332, 329, 316, 316, 318, 284, 248, 252, 296, 368, 414, 422, 406, 370, 340, 346, 361, 345, 323, 325, 328, 314, 308, 324, 327, 309, 308, 326, 330, 312, 313, 331, 329, 315, 323, 341, 336, 322, 327, 342, 331, 319, 336, 348, 336, 332, 353, 363, 345, 341, 356, 357, 342, 345, 361, 352, 327, 330, 341, 330, 315, 327, 334, 318, 304, 320, 331, 315, 307, 322, 330, 313, 311, 330, 331, 310, 314, 334, 332, 313, 316, 334, 331, 316, 327, 340, 329, 318, 335, 352, 347, 343, 358, 366, 354, 348, 355, 349, 327, 320, 333, 336, 317, 316, 331, 324, 308, 316, 332, 311, 266, 257, 267, 294, 374, 453, 452, 402, 366, 365, 365, 351, 340, 344, 333, 311, 311, 330, 325, 307, 314, 333, 326, 309, 320, 335, 324, 314, 330, 339, 326, 319, 335, 342, 328, 327, 344, 348, 331, 332, 351, 355, 341, 343, 362, 361, 343, 352, 366, 354, 336, 342, 350, 337, 326, 334, 335, 316, 311, 328, 334, 316, 309, 327, 331, 314, 313, 331, 329, 313, 323, 341, 330, 313, 324, 339, 330, 318, 330, 339, 325, 321, 341, 352, 340, 335, 357, 365, 347, 348, 365, 358, 334, 331, 343, 335, 319, 324, 336, 324, 312, 323, 335, 321, 309, 299, 277, 246, 252, 336, 442, 454, 418, 403, 380, 353, 357, 367, 346, 314, 314, 330, 319, 302, 313, 328, 318, 306, 321, 332, 318, 310, 325, 335, 323, 321, 335, 335, 320, 321, 342, 343, 326, 329, 349, 344, 329, 341, 360, 353, 340, 351, 364, 349, 336, 347, 349, 328, 317, 327, 330, 314, 309, 324, 323, 305, 310, 329, 324, 308, 315, 328, 318, 303, 314, 328, 318, 308, 326, 334, 320, 314, 331, 338, 321, 316, 335, 334, 315, 317, 339, 340, 329, 336, 353, 350, 341, 351, 365, 358, 342, 346, 351, 334, 322, 330, 335, 320, 316, 330, 330, 311, 312, 330, 320, 271, 248, 261, 290, 369, 451, 462, 421, 376, 370, 378, 360, 335, 332, 332, 313, 307, 324, 331, 313, 306, 320, 321, 310, 313, 328, 325, 309, 316, 333, 327, 317, 329, 339, 328, 317, 330, 344, 335, 328, 341, 351, 339, 339, 356, 359, 343, 341, 356, 353, 333, 333, 348, 338, 315, 320, 332, 322, 307, 316, 328, 316, 307, 318, 327, 311, 307, 324, 328, 309, 311, 333, 334, 314, 315, 336, 336, 319, 321, 335, 329, 317, 327, 338, 326, 314, 329, 338, 324, 319, 336, 340, 327, 325, 347, 357, 343, 342, 362, 355, 330, 328, 339, 330, 316, 326, 341, 328, 312, 320, 332, 319, 308, 312, 290, 253, 244, 295, 389, 437, 426, 412, 386, 354, 354, 370, 356, 323, 318, 332, 324, 313, 322, 333, 319, 308, 322, 330, 313, 305, 325, 335, 320, 319, 337, 336, 320, 323, 343, 342, 323, 328, 346, 341, 327, 339, 357, 352, 343, 356, 367, 351, 341, 354, 358, 338, 328, 342, 339, 315, 312, 330, 330, 311, 314, 331, 323, 304, 316, 335, 324, 306, 316, 329, 321, 311, 325, 338, 327, 320, 333, 336, 322, 321, 339, 339, 319, 318, 335, 331, 316, 324, 340, 332, 315, 328, 343, 333, 323, 343, 362, 350, 340, 360, 364, 342, 332, 344, 338, 316, 316, 333, 333, 315, 316, 330, 325, 311, 310, 301, 271, 248, 275, 355, 413, 407, 397, 386, 355, 340, 356, 363, 337, 321, 333, 332, 317, 318, 332, 327, 310, 317, 333, 326, 315, 326, 341, 333, 323, 335, 344, 330, 324, 344, 350, 336, 335, 351, 352, 337, 340, 360, 360, 347, 353, 370, 363, 345, 351, 365, 350, 332, 336, 343, 330, 323, 333, 336, 318, 309, 327, 331, 315, 312, 329, 330, 314, 318, 336, 332, 317, 322, 336, 326, 313, 325, 341, 329, 317, 334, 344, 326, 319, 332, 336, 320, 320, 336, 336, 319, 325, 348, 345, 332, 345, 367, 357, 343, 354, 366, 348, 327, 334, 341, 325, 315, 331, 336, 318, 313, 326, 328, 309, 294, 281, 261, 245, 303, 411, 446, 409, 391, 385, 363, 350, 358, 357, 328, 310, 322, 331, 314, 308, 327, 333, 317, 312, 325, 327, 313, 315, 332, 333, 321, 326, 340, 334, 324, 335, 349, 336, 327, 342, 355, 343, 337, 352, 359, 348, 348, 368, 366, 342, 338, 353, 351, 326, 323, 339, 332, 309, 315, 330, 322, 309, 319, 330, 317, 307, 320, 324, 308, 305, 326, 332, 315, 315, 333, 332, 315, 318, 335, 332, 319, 324, 335, 327, 315, 325, 338, 327, 318, 331, 339, 327, 317, 330, 337, 325, 327, 345, 348, 339, 345, 362, 364, 348, 344, 351, 339, 321, 327, 337, 324, 314, 328, 336, 321, 312, 325, 329, 290, 256, 254, 272, 332, 427, 463, 427, 382, 368, 376, 370, 344, 335, 335, 321, 304, 312, 326, 317, 307, 322, 332, 316, 309, 328, 332, 315, 314, 336, 339, 318, 320, 342, 339, 321, 328, 348, 342, 328, 338, 355, 352, 342, 353, 363, 348, 343, 358, 365, 347, 340, 352, 347, 322, 320, 339, 338, 319, 318, 334, 329, 313, 321, 333, 323, 310, 323, 338, 326, 315, 331, 341, 327, 322, 336, 339, 323, 324, 341, 340, 324, 327, 346, 338, 323, 332, 346, 336, 321, 332, 345, 332, 322, 336, 344, 327, 321, 339, 348, 333, 330, 349, 354, 344, 349, 367, 369, 350, 346, 354, 342, 325, 330, 339, 325, 315, 331, 338, 320, 310, 328, 327, 286, 250, 252, 273, 333, 426, 465, 435, 390, 374, 381, 376, 349, 337, 335, 317, 302, 313, 327, 317, 304, 315, 324, 311, 306, 322, 330, 315, 309, 326, 329, 319, 323, 337, 330, 314, 322, 340, 333, 320, 330, 347, 341, 331, 344, 354, 342, 337, 349, 351, 337, 334, 343, 341, 321, 320, 333, 327, 308, 310, 323, 318, 304, 312, 327, 319, 306, 318, 330, 318, 309, 324, 334, 320, 316, 333, 333, 314, 312, 329, 331, 318, 321, 337, 329, 310, 317, 336, 330, 316, 328, 339, 324, 315, 330, 339, 328, 319, 331, 341, 328, 325, 349, 358, 341, 339, 359, 358, 336, 332, 343, 335, 320, 330, 339, 324, 306, 320, 335, 320, 310, 316, 295, 252, 237, 280, 361, 425, 436, 419, 389, 350, 342, 363, 358, 329, 321, 328, 320, 307, 318, 332, 320, 311, 325, 331, 314, 308, 326, 335, 322, 318, 334, 339, 324, 328, 348, 343, 322, 328, 347, 342, 331, 343, 362, 357, 342, 353, 366, 357, 345, 355, 361, 342, 332, 346, 344, 322, 321, 337, 332, 312, 316, 329, 321, 301, 310, 331, 325, 314, 324, 335, 322, 313, 330, 341, 326, 315, 332, 340, 324, 322, 338, 340, 323, 321, 336, 333, 324, 333, 345, 330, 315, 332, 349, 334, 322, 334, 352, 340, 333, 354, 368, 352, 349, 364, 361, 337, 335, 348, 341, 320, 321, 336, 330, 315, 324, 338, 327, 310, 309, 291, 258, 250, 304, 398, 428, 406, 400, 387, 356, 344, 362, 359, 331, 324, 338, 332, 314, 322, 336, 324, 311, 323, 338, 331, 321, 335, 343, 327, 323, 342, 351, 334, 328, 346, 351, 336, 337, 353, 353, 340, 345, 362, 354, 342, 352, 367, 359, 345, 354, 358, 341, 330, 343, 343, 323, 315, 330, 332, 314, 313, 332, 333, 314, 311, 329, 328, 314, 321, 335, 325, 312, 325, 342, 333, 319, 328, 336, 322, 318, 335, 338, 318, 317, 336, 339, 322, 321, 337, 334, 315, 320, 337, 334, 325, 337, 349, 339, 333, 351, 364, 354, 346, 355, 355, 333, 323, 334, 337, 322, 318, 330, 325, 309, 316, 335, 327, 296, 270, 261, 246, 272, 375, 459, 441, 397, 382, 372, 353, 352, 355, 342, 317, 311, 326, 325, 307, 312, 330, 322, 307, 314, 329, 321, 309, 322, 336, 325, 317, 335, 347, 330, 320, 338, 348, 331, 328, 349, 352, 339, 342, 359, 356, 344, 351, 366, 359, 337, 342, 356, 342, 323, 332, 340, 325, 314, 327, 332, 318, 313, 325, 328, 311, 309, 331, 332, 313, 313, 330, 327, 313, 320, 340, 329, 312, 320, 335, 325, 317, 331, 337, 320, 314, 330, 334, 317, 316, 333, 332, 314, 317, 331, 330, 319, 328, 344, 336, 328, 346, 361, 352, 342, 352, 356, 336, 322, 330, 334, 318, 312, 326, 330, 314, 314, 329, 328, 302, 276, 264, 242, 253, 345, 450, 450, 405, 389, 384, 365, 352, 357, 349, 319, 307, 317, 320, 306, 306, 322, 321, 304, 308, 329, 327, 312, 320, 336, 325, 309, 323, 338, 331, 323, 338, 349, 338, 328, 345, 353, 338, 334, 353, 358, 342, 342, 362, 361, 345, 348, 358, 346, 327, 333, 344, 329, 309, 319, 330, 317, 307, 322, 332, 317, 311, 327, 333, 318, 316, 333, 330, 315, 318, 335, 329, 313, 320, 338, 333, 319, 328, 339, 326, 315, 328, 338, 326, 321, 331, 335, 320, 321, 338, 338, 323, 328, 347, 343, 329, 339, 358, 353, 345, 353, 357, 337, 322, 335, 344, 324, 309, 321, 330, 315, 312, 327, 327, 303, 283, 272, 252, 246, 322, 435, 457, 409, 386, 384, 368, 350, 353, 351, 329, 312, 322, 330, 315, 309, 328, 329, 310, 308, 329, 330, 314, 316, 334, 330, 315, 325, 342, 336, 322, 331, 343, 332, 322, 339, 348, 336, 329, 347, 362, 349, 343, 358, 360, 343, 342, 357, 348, 326, 328, 339, 331, 312, 317, 333, 322, 309, 318, 328, 316, 308, 325, 330, 312, 308, 329, 335, 315, 313, 332, 333, 316, 319, 335, 333, 318, 326, 340, 329, 312, 323, 340, 327, 312, 326, 339, 327, 319, 336, 343, 327, 324, 347, 352, 339, 341, 358, 352, 331, 330, 343, 329, 312, 322, 337, 323, 309, 321, 333, 322, 309, 313, 299, 256, 238, 270, 342, 403, 417, 412, 385, 348, 344, 362, 358, 330, 327, 333, 319, 304, 314, 330, 319, 310, 322, 331, 319, 312, 333, 341, 329, 328, 344, 340, 323, 325, 342, 340, 325, 334, 352, 345, 332, 342, 360, 356, 346, 358, 366, 353, 347, 363, 368, 348, 340, 351, 346, 322, 322, 339, 334, 314, 318, 331, 322, 306, 317, 333, 321, 309, 321, 333, 321, 314, 331, 340, 325, 318, 335, 340, 321, 319, 338, 337, 316, 320, 338, 333, 314, 323, 341, 333, 322, 337, 351, 341, 332, 351, 365, 350, 345, 359, 358, 335, 329, 344, 341, 319, 319, 336, 330, 310, 316, 332, 325, 305, 288, 275, 254, 258, 338, 431, 431, 394, 382, 372, 347, 344, 365, 354, 321, 313, 330, 331, 311, 316, 333, 328, 312, 321, 337, 330, 320, 330, 344, 332, 324, 339, 348, 335, 329, 347, 355, 341, 336, 353, 355, 341, 345, 366, 360, 343, 351, 369, 362, 346, 355, 363, 348, 332, 338, 345, 329, 317, 327, 334, 324, 318, 328, 328, 312, 313, 334, 333, 315, 317, 333, 328, 315, 325, 339, 331, 320, 335, 342, 325, 315, 333, 341, 322, 318, 336, 339, 323, 324, 346, 349, 332, 340, 364, 365, 354, 357, 361, 343, 325, 333, 342, 326, 311, 323, 333, 319, 312, 329, 334, 313, 296, 281, 258, 237, 282, 393, 452, 423, 395, 387, 367, 348, 359, 361, 332, 309, 318, 326, 314, 307, 325, 334, 315, 311, 328, 332, 312, 314, 335, 333, 316, 323, 340, 334, 322, 332, 347, 338, 323, 335, 352, 345, 337, 352, 360, 347, 342, 360, 366, 351, 346, 356, 351, 330, 324, 340, 334, 314, 320, 335, 322, 307, 316, 329, 317, 307, 320, 331, 321, 316, 330, 331, 315, 313, 330, 333, 318, 322, 338, 333, 315, 321, 340, 334, 319, 326, 338, 328, 320, 338, 351, 337, 329, 353, 365, 353, 349, 366, 363, 334, 328, 341, 336, 320, 321, 334, 326, 312, 318, 330, 323, 304, 293, 274, 246, 247, 329, 434, 448, 414, 402, 388, 361, 358, 364, 349, 318, 310, 325, 323, 307, 317, 333, 322, 306, 316, 330, 321, 313, 328, 337, 323, 316, 333, 341, 329, 328, 341, 342, 326, 326, 343, 343, 331, 338, 358, 351, 335, 342, 360, 351, 339, 346, 354, 337, 322, 330, 336, 319, 308, 322, 328, 311, 307, 325, 326, 309, 311, 332, 326, 308, 315, 334, 328, 313, 324, 336, 324, 311, 325, 340, 327, 319, 329, 335, 323, 320, 334, 336, 324, 326, 345, 346, 332, 336, 360, 356, 338, 347, 361, 348, 331, 339, 345, 324, 313, 330, 336, 317, 309, 325, 333, 318, 311, 304, 269, 236, 252, 338, 426, 438, 422, 413, 382, 355, 364, 372, 342, 316, 322, 330, 318, 310, 323, 329, 315, 311, 327, 328, 315, 319, 337, 333, 320, 325, 340, 336, 324, 335, 346, 335, 327, 343, 355, 341, 334, 356, 366, 345, 340, 360, 366, 352, 351, 361, 351, 333, 336, 349, 338, 316, 321, 337, 325, 309, 319, 329, 316, 308, 328, 338, 321, 314, 327, 337, 324, 320, 328, 328, 317, 323, 340, 335, 314, 322, 339, 329, 317, 326, 336, 327, 317, 328, 332, 319, 316, 337, 343, 326, 324, 348, 352, 338, 342, 361, 356, 332, 328, 340, 332, 318, 326, 336, 323, 309, 319, 330, 317, 307, 309, 283, 245, 238, 297, 390, 427, 417, 410, 383, 349, 348, 369, 353, 320, 317, 334, 325, 310, 317, 327, 317, 309, 323, 335, 321, 314, 328, 334, 319, 319, 338, 341, 328, 329, 345, 344, 330, 333, 349, 346, 336, 347, 361, 354, 344, 356, 368, 356, 348, 363, 366, 342, 331, 342, 342, 326, 324, 333, 323, 305, 314, 334, 327, 309, 316, 333, 327, 313, 323, 334, 319, 312, 327, 335, 320, 315, 332, 338, 322, 320, 336, 337, 320, 324, 336, 331, 318, 322, 336, 331, 318, 330, 346, 338, 330, 348, 361, 348, 343, 360, 361, 336, 329, 341, 340, 318, 319, 334, 327, 307, 313, 332, 325, 308, 301, 286, 257, 246, 309, 407, 428, 399, 393, 381, 354, 340, 359, 360, 332, 320, 332, 328, 310, 313, 333, 328, 312, 320, 340, 331, 316, 327, 341, 329, 324, 338, 342, 328, 324, 341, 347, 330, 328, 349, 351, 338, 342, 359, 359, 344, 351, 365, 355, 341, 348, 358, 344, 329, 341, 341, 320, 310, 324, 331, 313, 309, 326, 327, 312, 314, 335, 332, 311, 314, 331, 326, 312, 320, 337, 329, 319, 333, 347, 328, 313, 334, 346, 318, 310, 342, 351, 323, 318, 345, 347, 322, 328, 354, 346, 324, 345, 374, 356, 333, 353, 373, 342, 318, 337, 347, 318, 301, 333, 343, 309, 303, 333, 336, 302, 283, 283, 259, 249, 337, 451, 454, 396, 383, 391, 362, 342, 362, 369, 332, 310, 334, 341, 314, 310, 337, 340, 313, 308, 334, 337, 311, 317, 345, 342, 319, 331, 353, 339, 318, 337, 360, 341, 326, 357, 376, 352, 339, 367, 377, 352, 353, 383, 379, 342, 344, 372, 361, 328, 336, 358, 341, 314, 327, 346, 322, 305, 334, 350, 324, 306, 329, 338, 315, 313, 343, 342, 313, 321, 348, 339, 312, 320, 348, 336, 307, 320, 352, 339, 314, 329, 348, 326, 311, 335, 348, 325, 323, 357, 359, 324, 326, 362, 369, 339, 349, 381, 368, 330, 335, 356, 334, 309, 328, 352, 326, 301, 321, 340, 318, 306, 327, 311, 258, 238, 294, 377, 420, 430, 434, 401, 351, 352, 381, 359, 320, 325, 342, 322, 302, 326, 346, 317, 300, 330, 345, 313, 304, 339, 345, 318, 322, 353, 347, 315, 322, 348, 339, 320, 340, 362, 340, 322, 349, 369, 341, 330, 360, 374, 347, 335, 363, 371, 336, 322, 342, 336, 307, 311, 339, 331, 303, 315, 342, 325, 295, 312, 336, 318, 297, 325, 343, 312, 299, 327, 341, 316, 308, 337, 346, 317, 312, 340, 341, 315, 317, 344, 337, 314, 327, 346, 329, 310, 331, 348, 329, 317, 342, 360, 339, 332, 363, 373, 338, 327, 349, 349, 320, 317, 339, 333, 311, 324, 346, 329, 305, 326, 342, 307, 260, 262, 279, 304, 375, 438, 434, 384, 358, 371, 367, 340, 336, 350, 332, 302, 308, 331, 319, 295, 316, 339, 320, 301, 319, 337, 318, 307, 332, 343, 315, 311, 345, 350, 324, 323, 353, 355, 332, 343, 368, 354, 333, 348, 375, 364, 346, 365, 375, 347, 329, 349, 359, 332, 314, 334, 339, 310, 306, 336, 340, 308, 304, 330, 327, 304, 312, 338, 326, 297, 312, 341, 328, 307, 327, 342, 315, 306, 335, 343, 313, 307, 336, 340, 313, 312, 339, 337, 312, 320, 347, 335, 310, 326, 360, 349, 327, 349, 368, 346, 325, 347, 355, 320, 305, 332, 337, 308, 308, 336, 331, 299, 307, 338, 311, 252, 244, 278, 327, 380, 415, 420, 380, 339, 350, 362, 340, 326, 343, 340, 304, 300, 332, 330, 300, 302, 337, 336, 309, 316, 343, 330, 308, 325, 348, 333, 313, 331, 351, 331, 320, 348, 361, 334, 329, 363, 372, 340, 339, 374, 379, 350, 353, 380, 365, 336, 342, 356, 333, 307, 327, 346, 320, 298, 324, 342, 315, 300, 331, 336, 303, 299, 337, 341, 303, 303, 334, 328, 301, 315, 344, 331, 302, 319, 341, 323, 306, 334, 347, 315, 302, 330, 345, 319, 313, 345, 347, 318, 327, 363, 359, 330, 335, 356, 339, 309, 320, 344, 325, 297, 312, 331, 313, 296, 315, 326, 299, 278, 276, 262, 235, 273, 387, 429, 380, 361, 375, 349, 311, 324, 342, 310, 277, 285, 303, 277, 250, 268, 282, 259, 248, 277, 288, 264, 261, 296, 303, 282, 290, 322, 320, 292, 303, 327, 315, 297, 324, 349, 336, 325, 351, 380, 370, 361, 386, 399, 378, 376, 404, 408, 379, 370, 398, 398, 368, 372, 399, 387, 356, 361, 382, 368, 346, 363, 380, 360, 345, 367, 380, 359, 349, 370, 381, 360, 363, 393, 398, 373, 377, 407, 396, 368, 378, 404, 389, 365, 387, 412, 391, 369, 394, 415, 388, 373, 398, 400, 358, 337, 355, 355, 325, 317, 333, 325, 295, 301, 336, 333, 310, 315, 310, 270, 245, 308, 416, 446, 411, 404, 395, 358, 346, 375, 373, 326, 306, 327, 324, 295, 299, 328, 319, 290, 300, 327, 316, 293, 307, 328, 312, 296, 320, 335, 314, 300, 322, 332, 305, 297, 320, 320, 297, 302, 326, 317, 288, 296, 322, 308, 282, 293, 304, 276, 250, 268, 288, 269, 256, 284, 300, 281, 281, 315, 321, 296, 296, 321, 318, 287, 288, 307, 294, 266, 275, 308, 308, 301, 328, 352, 336, 324, 349, 365, 341, 331, 359, 364, 337, 334, 359, 361, 336, 338, 372, 375, 353, 366, 397, 387, 358, 361, 375, 355, 328, 335, 342, 315, 297, 322, 333, 305, 298, 327, 317, 262, 250, 287, 341, 409, 455, 456, 412, 366, 371, 398, 380, 344, 349, 363, 342, 333, 355, 360, 334, 325, 349, 352, 324, 320, 345, 345, 317, 322, 345, 335, 310, 320, 346, 331, 307, 330, 354, 335, 316, 341, 362, 343, 334, 362, 373, 347, 345, 370, 364, 326, 319, 340, 330, 297, 305, 330, 311, 281, 297, 321, 299, 274, 294, 316, 298, 285, 312, 324, 298, 291, 318, 320, 298, 306, 329, 321, 292, 302, 331, 325, 301, 313, 336, 324, 304, 320, 335, 315, 303, 333, 348, 322, 319, 352, 361, 336, 334, 359, 353, 319, 320, 344, 336, 304, 313, 343, 335, 314, 324, 342, 328, 312, 318, 309, 273, 266, 349, 446, 450, 415, 409, 391, 353, 353, 380, 367, 324, 324, 344, 331, 309, 329, 354, 330, 304, 326, 348, 330, 314, 335, 352, 326, 315, 344, 348, 323, 325, 356, 357, 327, 332, 360, 358, 340, 353, 375, 359, 336, 358, 382, 358, 339, 353, 356, 324, 307, 333, 337, 306, 299, 325, 324, 292, 293, 328, 324, 294, 304, 335, 322, 298, 314, 336, 320, 301, 320, 339, 316, 303, 327, 336, 310, 305, 332, 337, 307, 307, 335, 337, 317, 330, 359, 350, 330, 350, 374, 359, 341, 355, 365, 337, 318, 332, 341, 315, 305, 331, 337, 309, 303, 334, 339, 308, 293, 294, 279, 259, 318, 426, 440, 393, 389, 397, 367, 333, 352, 367, 333, 311, 332, 340, 318, 313, 340, 336, 307, 311, 342, 339, 313, 319, 346, 340, 317, 333, 361, 346, 323, 343, 361, 338, 326, 353, 370, 344, 335, 363, 371, 347, 346, 375, 370, 337, 336, 355, 342, 315, 322, 342, 322, 299, 317, 338, 313, 293, 310, 326, 305, 297, 324, 332, 305, 303, 331, 332, 309, 312, 337, 329, 298, 306, 335, 326, 305, 322, 346, 326, 311, 335, 353, 332, 321, 344, 359, 342, 336, 362, 368, 341, 335, 353, 346, 316, 318, 343, 331, 307, 321, 343, 330, 309, 323, 342, 312, 269, 271, 279, 286, 358, 457, 461, 399, 369, 382, 373, 344, 342, 355, 337, 302, 308, 335, 322, 301, 317, 339, 325, 309, 326, 338, 315, 303, 331, 340, 315, 313, 341, 341, 315, 319, 349, 347, 322, 333, 358, 348, 325, 343, 373, 356, 334, 353, 372, 349, 327, 345, 357, 328, 318, 334, 330, 302, 300, 330, 329, 299, 303, 329, 320, 297, 311, 334, 318, 296, 314, 335, 314, 297, 321, 335, 311, 301, 323, 330, 308, 305, 332, 333, 308, 310, 344, 344, 319, 330, 366, 359, 330, 343, 367, 348, 323, 338, 351, 326, 311, 335, 340, 315, 309, 339, 345, 319, 316, 331, 304, 255, 252, 311, 390, 442, 452, 442, 397, 358, 372, 392, 359, 325, 334, 343, 320, 313, 335, 336, 310, 309, 339, 343, 317, 318, 346, 341, 313, 323, 349, 340, 319, 334, 356, 340, 322, 346, 369, 349, 332, 358, 369, 341, 336, 367, 372, 344, 341, 366, 359, 327, 330, 354, 338, 308, 320, 338, 319, 301, 321, 338, 318, 302, 324, 337, 314, 305, 330, 337, 309, 303, 331, 331, 304, 312, 342, 332, 302, 315, 344, 331, 308, 326, 348, 329, 314, 342, 361, 335, 328, 356, 367, 346, 337, 354, 351, 322, 318, 342, 335, 306, 314, 338, 327, 305, 321, 342, 320, 288, 280, 278, 249, 274, 396, 478, 438, 393, 394, 389, 361, 354, 365, 348, 313, 315, 339, 329, 302, 317, 344, 326, 300, 319, 341, 321, 306, 328, 341, 321, 315, 341, 348, 326, 324, 348, 347, 325, 334, 365, 365, 344, 353, 379, 369, 348, 367, 383, 353, 330, 347, 361, 335, 320, 342, 346, 314, 308, 338, 341, 309, 306, 330, 325, 300, 311, 337, 323, 298, 311, 334, 322, 305, 321, 336, 314, 300, 327, 342, 317, 305, 333, 341, 315, 316, 345, 343, 318, 326, 357, 353, 329, 341, 370, 356, 330, 341, 357, 336, 314, 332, 348, 325, 313, 337, 343, 316, 312, 337, 319, 260, 242, 273, 319, 387, 444, 451, 406, 355, 358, 379, 357, 326, 333, 341, 314, 300, 325, 333, 311, 303, 329, 338, 313, 310, 333, 332, 310, 318, 349, 340, 315, 326, 351, 339, 320, 336, 350, 333, 324, 348, 363, 339, 332, 358, 364, 341, 335, 361, 362, 333, 329, 350, 343, 312, 321, 346, 329, 305, 326, 348, 330, 310, 336, 353, 332, 325, 351, 358, 335, 331, 358, 363, 335, 330, 357, 356, 329, 334, 360, 348, 323, 336, 361, 351, 329, 338, 348, 331, 321, 344, 358, 334, 321, 339, 339, 308, 301, 327, 327, 296, 298, 322, 313, 288, 296, 315, 292, 246, 238, 250, 248, 297, 399, 434, 384, 343, 351, 350, 324, 322, 349, 343, 307, 308, 338, 329, 304, 319, 346, 331, 305, 327, 354, 338, 318, 333, 348, 322, 308, 335, 351, 324, 310, 336, 343, 317, 320, 348, 344, 317, 327, 359, 351, 325, 339, 362, 343, 313, 327, 346, 323, 302, 321, 332, 310, 296, 318, 331, 306, 302, 331, 330, 300, 302, 334, 334, 307, 315, 339, 323, 299, 318, 341, 324, 308, 330, 343, 318, 309, 339, 346, 316, 313, 346, 353, 332, 339, 366, 360, 337, 342, 360, 344, 313, 323, 343, 325, 303, 320, 342, 323, 306, 330, 341, 312, 286, 282, 270, 244, 295, 415, 456, 408, 386, 395, 367, 335, 355, 378, 346, 308, 321, 342, 324, 306, 326, 341, 317, 307, 334, 345, 322, 318, 340, 344, 322, 327, 352, 344, 316, 329, 357, 345, 322, 338, 363, 348, 331, 359, 376, 350, 336, 365, 378, 351, 338, 360, 362, 332, 324, 348, 343, 316, 320, 343, 329, 302, 318, 342, 326, 302, 324, 341, 317, 303, 330, 348, 322, 307, 331, 336, 307, 307, 341, 342, 309, 311, 342, 339, 313, 324, 352, 337, 314, 336, 369, 358, 337, 356, 370, 340, 322, 341, 346, 319, 313, 337, 331, 300, 305, 338, 335, 307, 314, 327, 287, 239, 244, 303, 375, 428, 441, 424, 371, 342, 367, 379, 341, 314, 334, 343, 314, 306, 334, 332, 305, 312, 342, 335, 311, 326, 353, 340, 318, 330, 351, 332, 311, 328, 341, 316, 306, 334, 346, 324, 327, 362, 367, 342, 345, 375, 373, 346, 352, 373, 352, 321, 333, 351, 328, 308, 326, 344, 323, 309, 330, 341, 317, 310, 339, 347, 314, 310, 339, 337, 314, 321, 339, 324, 300, 317, 343, 324, 298, 316, 337, 314, 296, 320, 336, 316, 313, 345, 358, 336, 336, 366, 362, 332, 330, 347, 329, 297, 307, 328, 307, 274, 285, 310, 294, 277, 297, 310, 281, 253, 246, 231, 192, 225, 354, 426, 384, 359, 372, 361, 337, 350, 368, 343, 308, 323, 348, 332, 313, 337, 356, 330, 314, 337, 350, 328, 323, 350, 353, 327, 328, 354, 355, 329, 335, 364, 353, 328, 345, 378, 364, 338, 358, 383, 361, 339, 360, 376, 353, 342, 367, 372, 339, 330, 356, 353, 322, 322, 348, 339, 315, 321, 345, 332, 309, 327, 346, 325, 308, 334, 352, 326, 312, 337, 346, 320, 313, 343, 348, 319, 321, 349, 347, 320, 327, 355, 349, 330, 342, 360, 346, 332, 353, 377, 360, 341, 355, 363, 338, 328, 344, 346, 320, 317, 344, 342, 311, 313, 340, 334, 293, 271, 279, 271, 298, 404, 474, 437, 385, 386, 389, 359, 348, 363, 354, 319, 314, 342, 340, 308, 311, 337, 332, 307, 316, 343, 331, 305, 321, 349, 334, 310, 330, 352, 336, 321, 342, 357, 333, 330, 361, 366, 337, 341, 373, 375, 346, 349, 376, 365, 335, 341, 360, 344, 319, 332, 351, 327, 311, 333, 345, 319, 310, 338, 346, 320, 319, 346, 346, 320, 320, 345, 341, 314, 323, 344, 333, 308, 322, 346, 330, 311, 335, 354, 330, 316, 346, 358, 330, 325, 361, 371, 344, 346, 376, 367, 330, 334, 355, 337, 307, 320, 347, 334, 310, 323, 343, 322, 307, 323, 312, 258, 238, 283, 357, 408, 417, 420, 393, 347, 344, 371, 369, 334, 330, 346, 324, 303, 320, 340, 321, 302, 324, 341, 320, 313, 339, 345, 318, 317, 347, 350, 320, 322, 353, 350, 324, 336, 362, 349, 324, 342, 370, 358, 340, 359, 374, 353, 344, 365, 366, 333, 321, 344, 346, 314, 316, 344, 340, 308, 313, 340, 332, 306, 318, 343, 328, 304, 322, 344, 324, 302, 322, 335, 312, 304, 334, 343, 310, 306, 339, 343, 316, 322, 351, 341, 316, 329, 361, 353, 335, 352, 376, 358, 335, 352, 367, 337, 316, 339, 350, 320, 308, 338, 341, 311, 315, 342, 323, 264, 253, 278, 299, 359, 436, 456, 409, 357, 361, 377, 358, 335, 344, 348, 321, 312, 341, 343, 309, 304, 334, 334, 307, 312, 342, 337, 310, 323, 350, 337, 318, 335, 354, 337, 322, 344, 356, 335, 329, 359, 367, 341, 339, 369, 372, 343, 342, 369, 366, 335, 335, 356, 339, 310, 326, 347, 326, 299, 317, 335, 315, 303, 328, 337, 311, 304, 332, 336, 307, 309, 340, 336, 309, 314, 341, 333, 308, 322, 346, 332, 311, 331, 352, 336, 322, 345, 360, 344, 338, 361, 373, 348, 338, 356, 352, 324, 323, 351, 340, 309, 319, 344, 330, 308, 322, 340, 310, 260, 260, 270, 289, 368, 457, 458, 397, 361, 375, 379, 349, 336, 346, 330, 297, 305, 334, 327, 301, 312, 337, 321, 301, 321, 342, 323, 307, 335, 351, 325, 319, 348, 352, 323, 325, 353, 349, 329, 340, 367, 357, 331, 345, 375, 364, 340, 354, 372, 349, 324, 338, 351, 325, 312, 334, 336, 309, 306, 333, 334, 307, 307, 337, 333, 305, 314, 345, 331, 303, 319, 344, 324, 305, 328, 347, 326, 312, 334, 343, 320, 315, 344, 349, 319, 318, 351, 349, 326, 340, 371, 359, 338, 350, 367, 340, 312, 330, 345, 322, 301, 325, 343, 320, 305, 327, 336, 309, 291, 288, 268, 238, 285, 409, 461, 416, 396, 398, 370, 340, 355, 364, 330, 303, 320, 333, 311, 302, 331, 341, 311, 304, 334, 339, 308, 307, 333, 333, 309, 316, 347, 337, 314, 330, 356, 340, 321, 343, 365, 344, 328, 353, 365, 339, 335, 364, 370, 340, 335, 360, 355, 323, 323, 348, 337, 308, 318, 340, 325, 303, 321, 340, 318, 300, 322, 340, 320, 310, 338, 341, 316, 317, 347, 348, 313, 310, 341, 338, 311, 317, 345, 335, 312, 328, 348, 334, 319, 341, 359, 338, 330, 354, 364, 345, 338, 356, 350, 319, 320, 346, 341, 312, 316, 340, 330, 307, 321, 342, 319, 273, 262, 268, 256, 302, 422, 471, 422, 380, 383, 379, 348, 337, 350, 336, 300, 300, 327, 320, 294, 306, 331, 317, 296, 313, 333, 315, 301, 324, 340, 317, 308, 334, 344, 319, 313, 343, 350, 323, 325, 356, 355, 329, 333, 361, 355, 334, 350, 370, 349, 324, 338, 351, 326, 308, 329, 334, 306, 300, 328, 330, 298, 296, 327, 329, 304, 310, 333, 322, 297, 309, 336, 321, 298, 315, 337, 321, 306, 330, 345, 321, 311, 338, 346, 317, 311, 340, 344, 319, 325, 353, 349, 327, 342, 369, 356, 330, 340, 355, 336, 311, 325, 343, 319, 301, 319, 331, 307, 303, 328, 320, 268, 240, 260, 273, 312, 400, 449, 421, 367, 358, 374, 359, 341, 353, 356, 320, 300, 325, 339, 312, 299, 328, 338, 311, 308, 339, 340, 311, 315, 345, 340, 316, 326, 354, 341, 315, 331, 357, 344, 328, 349, 364, 341, 333, 359, 370, 347, 342, 364, 360, 327, 324, 347, 341, 311, 317, 342, 326, 297, 310, 337, 320, 298, 318, 336, 314, 301, 328, 340, 313, 304, 334, 339, 312, 313, 340, 338, 312, 316, 340, 330, 302, 315, 345, 333, 315, 332, 353, 335, 319, 345, 363, 345, 335, 361, 366, 333, 324, 346, 338, 306, 311, 342, 338, 307, 310, 336, 327, 304, 318, 323, 273, 232, 263, 344, 407, 417, 421, 405, 357, 341, 369, 375, 338, 325, 346, 336, 301, 307, 337, 328, 302, 319, 344, 325, 305, 326, 347, 329, 317, 341, 352, 327, 318, 345, 352, 327, 327, 355, 357, 335, 341, 369, 363, 335, 345, 372, 361, 338, 353, 371, 344, 320, 337, 348, 318, 306, 333, 337, 308, 303, 333, 335, 303, 305, 334, 331, 305, 316, 343, 330, 305, 318, 343, 327, 310, 330, 347, 323, 308, 333, 347, 318, 306, 336, 343, 318, 319, 349, 353, 333, 344, 371, 366, 340, 349, 367, 340, 310, 326, 348, 325, 306, 329, 342, 315, 300, 324, 329, 287, 251, 263, 265, 282, 381, 465, 444, 380, 363, 378, 366, 340, 341, 350, 325, 301, 321, 339, 315, 301, 328, 338, 308, 302, 333, 339, 312, 311, 339, 338, 310, 317, 347, 342, 317, 331, 357, 340, 319, 343, 366, 345, 327, 352, 370, 344, 332, 356, 360, 333, 324, 346, 342, 311, 313, 341, 331, 301, 309, 334, 321, 299, 313, 331, 310, 291, 319, 339, 314, 299, 325, 336, 310, 305, 335, 336, 311, 310, 333, 331, 308, 317, 341, 329, 306, 323, 351, 338, 319, 338, 357, 343, 332, 353, 365, 341, 330, 347, 346, 320, 318, 347, 344, 311, 314, 342, 338, 311, 316, 336, 313, 263, 255, 267, 271, 336, 442, 466, 410, 369, 379, 384, 355, 335, 339, 330, 303, 307, 334, 327, 302, 313, 340, 326, 300, 318, 342, 324, 307, 329, 346, 321, 307, 337, 349, 322, 314, 343, 350, 325, 330, 359, 357, 331, 342, 373, 362, 333, 349, 373, 356, 329, 340, 352, 329, 310, 332, 344, 318, 306, 333, 339, 308, 304, 331, 328, 302, 308, 337, 332, 306, 315, 340, 327, 305, 322, 346, 333, 317, 335, 346, 324, 312, 341, 353, 324, 316, 343, 343, 313, 316, 347, 347, 321, 330, 356, 344, 323, 346, 370, 355, 337, 354, 366, 338, 321, 340, 346, 321, 312, 337, 339, 308, 307, 337, 332, 297, 277, 282, 261, 254, 346, 460, 457, 403, 392, 396, 370, 349, 362, 356, 312, 296, 329, 338, 307, 302, 331, 334, 311, 313, 341, 333, 308, 318, 345, 334, 312, 326, 350, 337, 316, 335, 357, 336, 322, 348, 363, 339, 331, 358, 366, 340, 341, 370, 365, 333, 335, 357, 344, 317, 326, 347, 328, 304, 320, 336, 312, 300, 326, 338, 311, 301, 330, 339, 312, 308, 338, 338, 308, 311, 340, 339, 313, 319, 343, 331, 312, 328, 346, 327, 310, 334, 348, 322, 309, 335, 345, 318, 311, 339, 341, 313, 319, 353, 352, 324, 336, 369, 360, 334, 350, 372, 351, 325, 338, 350, 328, 308, 328, 339, 315, 308, 331, 332, 304, 298, 305, 277, 235, 246, 334, 412, 415, 405, 402, 366, 336, 355, 375, 342, 310, 323, 338, 313, 302, 326, 335, 309, 303, 331, 333, 307, 312, 340, 335, 310, 320, 341, 330, 311, 330, 353, 336, 315, 336, 355, 340, 329, 353, 362, 339, 336, 363, 366, 340, 342, 373, 360, 319, 320, 348, 334, 298, 314, 342, 327, 299, 316, 335, 316, 297, 320, 336, 313, 303, 328, 334, 309, 306, 335, 335, 308, 312, 338, 330, 306, 320, 345, 329, 303, 321, 344, 324, 306, 329, 343, 317, 303, 333, 345, 320, 318, 345, 347, 321, 327, 362, 363, 335, 346, 376, 360, 329, 338, 355, 334, 309, 327, 344, 323, 307, 327, 338, 310, 297, 310, 282, 227, 215, 275, 358, 384, 390, 397, 366, 331, 344, 381, 363, 329, 343, 367, 351, 333, 352, 356, 320, 301, 322, 332, 318, 322, 341, 332, 294, 293, 322, 322, 299, 303, 322, 314, 298, 327, 367, 359, 335, 339, 345, 334, 347, 387, 389, 349, 331, 346, 342, 311, 317, 351, 355, 334, 355, 399, 406, 376, 373, 390, 380, 370, 387, 390, 356, 331, 347, 359, 336, 321, 343, 347, 309, 291, 312, 318, 306, 329, 361, 348, 322, 334, 363, 354, 325, 330, 365, 393, 434, 504, 548, 533, 521, 534, 527, 485, 473, 498, 496, 462, 455, 481, 471, 436, 424, 422, 384, 336, 327, 329, 291, 253, 265, 275, 244, 223, 237, 226, 173, 143, 191, 292, 352, 364, 379, 363, 323, 317, 337, 315, 272, 277, 307, 300, 289, 322, 391, 436, 406, 382, 361, 317, 300, 336, 349, 307, 284, 299, 286, 245, 248, 284, 285, 254, 259, 283, 266, 239, 261, 295, 294, 286, 319, 347, 320, 304, 341, 364, 337, 330, 367, 378, 343, 341, 374, 356, 294, 268, 273, 248, 203, 218, 230, 165, 111, 112, 120, 91, 70, 87, 122, 135, 125, 151, 200, 196, 179, 218, 280, 289, 280, 321, 368, 349, 318, 334, 344, 325, 329, 363, 386, 376, 382, 381, 371, 356, 388, 405, 384, 393, 390, 376, 334, 295, 310, 316, 281, 304, 413, 474, 415, 412, 497, 506, 377, 208, 163, 212, 299, 368, 397, 371, 331, 339, 387, 386, 339, 325, 327, 300, 283, 304, 329, 324, 329, 359, 367, 338, 330, 355, 358, 335, 336, 363, 361, 332, 337, 368, 369, 347, 363, 402, 393, 367, 388, 412, 390, 372, 391, 395, 361, 333, 343, 338, 291, 269, 286, 276, 236, 228, 250, 239, 221, 243, 276, 273, 268, 310, 346, 333, 307, 325, 344, 318, 303, 336, 359, 338, 336, 367, 380, 359, 361, 397, 396, 366, 370, 400, 390, 355, 364, 388, 369, 341, 360, 387, 364, 345, 378, 397, 366, 352, 376, 384, 349, 345, 371, 359, 322, 328, 365, 365, 337, 346, 374, 359, 327, 336, 346, 307, 270, 300, 385, 432, 433, 445, 433, 377, 358, 390, 403, 370, 354, 371, 362, 331, 342, 368, 356, 336, 357, 387, 369, 338, 354, 383, 360, 341, 363, 368, 337, 319, 340, 351, 327, 328, 356, 353, 327, 337, 367, 345, 312, 323, 352, 335, 297, 292, 289, 256, 231, 258, 274, 240, 221, 242, 259, 249, 267, 326, 350, 338, 349, 382, 383, 353, 348, 376, 380, 365, 379, 399, 369, 342, 363, 375, 348, 334, 358, 364, 328, 317, 349, 360, 327, 323, 351, 352, 326, 337, 372, 368, 342, 358, 383, 364, 334, 347, 367, 342, 322, 343, 353, 322, 309, 335, 352, 337, 334, 352, 326, 265, 255, 299, 360, 402, 419, 424, 387, 340, 344, 360, 348, 329, 337, 336, 303, 290, 322, 335, 302, 292, 321, 323, 291, 286, 313, 308, 284, 298, 334, 330, 312, 334, 358, 340, 311, 328, 355, 343, 331, 356, 374, 354, 341, 365, 377, 360, 356, 376, 367, 330, 330, 351, 340, 316, 321, 340, 326, 305, 325, 349, 334, 315, 330, 342, 320, 315, 339, 341, 314, 310, 342, 345, 313, 316, 349, 346, 320, 327, 354, 346, 328, 347, 370, 350, 326, 347, 363, 344, 336, 360, 375, 350, 339, 361, 359, 322, 307, 325, 326, 305, 308, 326, 311, 286, 304, 333, 319, 288, 275, 270, 241, 256, 349, 388, 337, 301, 309, 301, 271, 275, 308, 309, 277, 283, 326, 329, 297, 291, 318, 315, 299, 319, 342, 322, 304, 327, 346, 330, 321, 347, 358, 337, 335, 364, 367, 341, 344, 373, 371, 343, 355, 387, 380, 355, 372, 401, 381, 351, 360, 372, 348, 332, 352, 357, 326, 315, 342, 349, 319, 312, 342, 344, 313, 315, 341, 335, 313, 332, 363, 352, 326, 343, 368, 349, 331, 352, 367, 340, 318, 344, 355, 324, 313, 335, 330, 301, 315, 356, 354, 333, 348, 371, 357, 332, 347, 362, 335, 313, 331, 345, 315, 285, 298, 307, 280, 262, 265, 245, 204, 221, 323, 399, 379, 349, 343, 315, 278, 290, 320, 300, 269, 285, 307, 294, 284, 312, 333, 317, 315, 350, 365, 341, 333, 333, 307, 264, 270, 310, 308, 283, 295, 327, 324, 314, 342, 371, 363, 352, 376, 389, 363, 348, 366, 373, 340, 324, 340, 340, 307, 301, 332, 332, 302, 307, 335, 325, 297, 312, 339, 324, 301, 321, 345, 328, 315, 341, 350, 324, 321, 353, 361, 334, 331, 362, 359, 331, 336, 362, 350, 321, 336, 367, 353, 339, 367, 392, 369, 351, 377, 392, 357, 336, 358, 363, 331, 327, 359, 355, 321, 328, 360, 354, 320, 322, 325, 285, 247, 287, 388, 451, 437, 422, 410, 362, 335, 364, 370, 325, 299, 320, 322, 295, 297, 325, 327, 305, 312, 340, 336, 314, 328, 352, 336, 312, 330, 343, 317, 300, 317, 324, 299, 292, 316, 330, 314, 319, 354, 366, 350, 360, 384, 379, 350, 355, 367, 348, 323, 331, 347, 331, 317, 338, 350, 330, 318, 338, 346, 329, 327, 354, 361, 341, 344, 364, 358, 334, 343, 367, 361, 343, 354, 369, 349, 328, 340, 347, 320, 300, 314, 326, 309, 307, 331, 337, 318, 323, 346, 339, 315, 337, 362, 344, 328, 351, 373, 351, 329, 346, 365, 334, 303, 317, 334, 314, 296, 289, 274, 258, 315, 443, 500, 448, 409, 414, 401, 374, 378, 387, 364, 338, 356, 379, 357, 326, 332, 337, 308, 286, 301, 304, 283, 293, 334, 351, 335, 337, 362, 361, 339, 344, 366, 356, 331, 341, 358, 345, 325, 340, 357, 341, 329, 359, 378, 355, 344, 368, 374, 350, 345, 371, 373, 344, 345, 369, 366, 343, 355, 378, 363, 340, 360, 381, 358, 339, 360, 374, 344, 326, 343, 349, 322, 316, 341, 343, 317, 318, 341, 338, 313, 319, 340, 328, 300, 314, 339, 326, 308, 332, 358, 341, 328, 353, 362, 331, 315, 334, 332, 296, 288, 313, 311, 285, 294, 324, 316, 289, 302, 318, 278, 233, 238, 294, 364, 412, 423, 404, 361, 338, 354, 359, 324, 308, 327, 325, 297, 297, 325, 323, 300, 310, 336, 326, 306, 321, 344, 329, 311, 332, 350, 332, 321, 345, 355, 329, 325, 354, 359, 335, 339, 370, 373, 347, 352, 378, 366, 343, 354, 372, 350, 325, 337, 352, 330, 309, 324, 334, 312, 306, 329, 330, 300, 297, 323, 325, 302, 303, 327, 322, 301, 312, 335, 323, 303, 318, 339, 322, 301, 320, 335, 315, 305, 327, 335, 312, 309, 334, 335, 314, 318, 347, 343, 319, 332, 363, 356, 335, 351, 371, 351, 325, 333, 344, 322, 308, 327, 336, 310, 306, 333, 336, 306, 301, 322, 302, 256, 249, 287, 342, 392, 416, 416, 381, 341, 347, 366, 348, 322, 330, 339, 314, 304, 327, 336, 312, 305, 332, 334, 307, 308, 337, 337, 316, 322, 346, 337, 317, 333, 354, 335, 316, 335, 355, 340, 330, 352, 366, 345, 341, 367, 371, 347, 346, 370, 364, 334, 331, 350, 335, 306, 315, 335, 320, 300, 316, 333, 310, 294, 316, 331, 309, 301, 324, 331, 308, 304, 330, 331, 308, 314, 338, 334, 310, 318, 343, 333, 309, 325, 345, 328, 310, 332, 348, 327, 318, 345, 356, 336, 337, 364, 366, 343, 343, 364, 352, 320, 322, 344, 331, 305, 316, 340, 326, 304, 316, 331, 310, 292, 291, 282, 250, 266, 369, 440, 413, 380, 381, 365, 332, 340, 363, 342, 303, 309, 330, 317, 298, 317, 336, 319, 302, 322, 337, 318, 308, 331, 341, 321, 320, 346, 344, 319, 323, 351, 346, 324, 335, 366, 354, 331, 347, 371, 359, 342, 362, 379, 359, 343, 360, 363, 331, 319, 341, 341, 310, 308, 334, 333, 306, 310, 336, 328, 302, 313, 335, 322, 302, 321, 342, 321, 307, 330, 344, 319, 310, 337, 341, 317, 315, 337, 336, 311, 315, 345, 340, 313, 324, 349, 339, 324, 348, 368, 348, 334, 353, 364, 333, 317, 336, 341, 314, 308, 332, 333, 306, 306, 332, 327, 299, 301, 305, 273, 238, 264, 355, 425, 422, 407, 394, 354, 329, 354, 363, 326, 305, 324, 328, 304, 302, 326, 325, 302, 308, 333, 329, 308, 317, 340, 327, 306, 323, 348, 333, 315, 337, 352, 332, 326, 353, 363, 338, 336, 364, 367, 343, 348, 376, 371, 346, 350, 367, 350, 324, 336, 352, 330, 305, 322, 339, 318, 306, 328, 339, 313, 304, 326, 332, 308, 305, 334, 335, 308, 312, 337, 331, 309, 323, 346, 331, 309, 327, 345, 328, 313, 334, 345, 319, 311, 339, 348, 328, 325, 354, 362, 339, 340, 369, 362, 330, 331, 350, 336, 309, 322, 342, 326, 306, 323, 338, 317, 304, 324, 326, 278, 245, 260, 291, 348, 427, 458, 417, 359, 351, 374, 369, 335, 329, 340, 322, 299, 316, 335, 315, 301, 325, 338, 316, 306, 332, 337, 313, 314, 343, 344, 317, 321, 349, 344, 319, 329, 356, 347, 328, 347, 369, 353, 337, 358, 375, 353, 342, 366, 373, 342, 332, 352, 348, 316, 315, 339, 333, 307, 312, 337, 327, 304, 318, 339, 325, 305, 321, 338, 321, 310, 333, 342, 319, 311, 338, 344, 318, 317, 339, 338, 315, 321, 344, 335, 312, 323, 349, 337, 319, 337, 358, 345, 332, 350, 367, 350, 334, 353, 354, 326, 317, 337, 336, 311, 314, 339, 333, 309, 316, 337, 324, 288, 273, 274, 256, 277, 385, 462, 427, 380, 378, 377, 350, 342, 356, 344, 310, 306, 332, 329, 302, 308, 333, 325, 304, 317, 340, 327, 309, 328, 345, 327, 314, 336, 349, 329, 324, 349, 355, 334, 331, 358, 360, 338, 345, 370, 364, 342, 352, 372, 358, 335, 348, 365, 343, 319, 332, 341, 319, 305, 325, 331, 309, 308, 336, 337, 308, 309, 336, 335, 310, 318, 341, 331, 310, 324, 345, 329, 315, 333, 346, 324, 309, 332, 344, 322, 317, 341, 345, 319, 323, 352, 349, 322, 333, 365, 363, 343, 355, 370, 348, 322, 336, 350, 328, 310, 329, 340, 319, 310, 332, 336, 310, 304, 312, 285, 244, 251, 327, 406, 418, 410, 405, 366, 330, 344, 371, 347, 316, 325, 336, 313, 304, 330, 340, 315, 306, 331, 334, 310, 313, 341, 338, 312, 322, 350, 340, 315, 330, 354, 341, 322, 343, 367, 350, 334, 357, 372, 353, 344, 368, 375, 351, 348, 367, 359, 329, 330, 352, 342, 308, 313, 336, 324, 303, 318, 338, 317, 301, 323, 340, 318, 307, 330, 340, 315, 310, 337, 341, 314, 314, 339, 333, 309, 320, 347, 333, 309, 326, 348, 331, 313, 338, 357, 339, 332, 359, 371, 350, 338, 359, 357, 322, 312, 335, 333, 309, 313, 336, 329, 304, 314, 334, 319, 285, 274, 278, 258, 283, 392, 448, 411, 372, 373, 370, 342, 339, 361, 347, 311, 310, 334, 331, 308, 317, 339, 326, 303, 319, 341, 328, 312, 328, 341, 325, 320, 343, 352, 324, 316, 345, 352, 330, 334, 361, 362, 338, 342, 366, 362, 341, 352, 376, 360, 334, 345, 358, 335, 315, 330, 341, 315, 304, 330, 334, 309, 308, 333, 330, 309, 316, 339, 331, 306, 316, 339, 327, 306, 320, 344, 325, 306, 325, 341, 321, 307, 328, 341, 318, 315, 342, 345, 323, 333, 366, 361, 336, 348, 369, 350, 318, 328, 344, 324, 302, 321, 338, 317, 302, 325, 336, 310, 299, 311, 287, 244, 246, 325, 409, 424, 409, 406, 373, 336, 347, 373, 347, 313, 323, 338, 318, 301, 320, 336, 317, 306, 327, 335, 312, 307, 334, 339, 315, 320, 346, 340, 317, 329, 353, 344, 324, 340, 362, 347, 333, 356, 374, 353, 342, 366, 376, 352, 340, 359, 356, 329, 325, 345, 336, 306, 312, 336, 325, 299, 311, 331, 316, 297, 315, 332, 315, 299, 320, 332, 308, 300, 326, 336, 315, 311, 333, 333, 309, 314, 340, 333, 311, 324, 350, 340, 317, 332, 359, 349, 334, 357, 372, 344, 323, 339, 346, 320, 313, 338, 340, 312, 309, 333, 328, 302, 308, 328, 300, 253, 250, 280, 330, 399, 447, 436, 380, 342, 358, 374, 346, 323, 335, 332, 305, 306, 334, 333, 308, 312, 335, 327, 304, 317, 339, 328, 308, 329, 349, 332, 319, 339, 352, 333, 326, 349, 357, 335, 338, 366, 366, 342, 347, 372, 367, 342, 351, 372, 360, 332, 338, 353, 334, 313, 330, 345, 323, 308, 327, 338, 315, 308, 334, 337, 311, 314, 343, 339, 313, 321, 347, 335, 310, 323, 348, 333, 312, 328, 345, 329, 314, 336, 349, 326, 320, 346, 353, 330, 333, 367, 367, 341, 344, 367, 354, 323, 329, 347, 333, 312, 327, 346, 324, 303, 323, 338, 320, 301, 304, 283, 241, 241, 328, 424, 437, 409, 405, 379, 341, 352, 374, 348, 309, 317, 336, 320, 302, 320, 337, 318, 300, 324, 337, 315, 309, 334, 337, 312, 314, 340, 341, 319, 325, 349, 344, 322, 334, 360, 349, 331, 348, 367, 351, 337, 359, 372, 347, 334, 354, 357, 327, 315, 338, 338, 312, 315, 336, 323, 300, 311, 336, 324, 299, 312, 333, 320, 304, 325, 340, 318, 305, 332, 341, 314, 308, 338, 341, 313, 311, 339, 336, 310, 317, 341, 332, 309, 323, 346, 334, 317, 336, 359, 346, 329, 350, 364, 339, 327, 343, 344, 315, 311, 335, 331, 304, 308, 335, 327, 300, 310, 334, 305, 255, 253, 272, 299, 373, 446, 442, 389, 354, 367, 374, 350, 334, 341, 330, 300, 306, 333, 329, 304, 310, 334, 324, 304, 317, 334, 321, 306, 329, 347, 328, 312, 333, 348, 327, 323, 352, 356, 332, 333, 362, 361, 337, 345, 372, 364, 342, 352, 371, 352, 329, 342, 352, 329, 311, 330, 339, 314, 303, 328, 333, 304, 300, 326, 331, 310, 313, 337, 331, 304, 315, 340, 326, 303, 322, 345, 325, 305, 326, 343, 326, 313, 335, 343, 317, 310, 337, 341, 314, 314, 345, 348, 324, 327, 353, 353, 339, 355, 375, 355, 335, 342, 350, 323, 308, 336, 344, 316, 308, 335, 340, 312, 309, 336, 326, 272, 254, 272, 292, 347, 418, 442, 402, 352, 352, 365, 352, 333, 338, 340, 314, 302, 324, 332, 307, 304, 331, 334, 309, 313, 340, 335, 311, 321, 348, 337, 312, 327, 349, 333, 316, 338, 355, 337, 329, 355, 365, 340, 335, 361, 368, 346, 344, 368, 365, 334, 334, 353, 334, 304, 317, 340, 326, 302, 316, 337, 318, 301, 321, 332, 309, 302, 331, 339, 311, 308, 336, 338, 311, 317, 346, 336, 308, 318, 344, 333, 309, 323, 342, 323, 304, 323, 340, 321, 314, 338, 347, 326, 322, 348, 356, 336, 335, 366, 366, 335, 336, 354, 339, 311, 322, 341, 323, 302, 323, 343, 322, 305, 327, 329, 280, 247, 263, 286, 331, 402, 441, 416, 363, 348, 364, 358, 337, 340, 349, 325, 300, 317, 337, 323, 307, 325, 337, 313, 305, 331, 338, 314, 314, 340, 341, 319, 323, 351, 346, 322, 331, 357, 349, 328, 344, 366, 352, 336, 359, 379, 359, 345, 365, 373, 344, 333, 354, 353, 321, 318, 340, 334, 307, 313, 338, 327, 302, 315, 336, 320, 300, 318, 337, 320, 308, 329, 342, 320, 313, 336, 341, 318, 316, 340, 340, 313, 318, 344, 337, 313, 323, 348, 333, 311, 328, 352, 336, 323, 345, 356, 339, 336, 360, 366, 344, 338, 357, 353, 323, 323, 345, 338, 313, 321, 345, 335, 310, 322, 338, 317, 280, 273, 275, 260, 306, 418, 462, 411, 371, 372, 366, 342, 340, 353, 338, 303, 306, 331, 325, 301, 315, 338, 321, 299, 318, 338, 320, 307, 331, 343, 319, 312, 338, 346, 324, 324, 352, 351, 325, 332, 359, 355, 334, 345, 368, 360, 342, 356, 374, 354, 334, 351, 365, 336, 317, 336, 342, 315, 310, 334, 336, 309, 308, 334, 330, 307, 315, 338, 326, 301, 315, 337, 322, 305, 326, 346, 327, 312, 333, 345, 325, 318, 342, 349, 323, 318, 343, 342, 318, 323, 348, 339, 317, 327, 347, 336, 316, 336, 357, 339, 328, 354, 369, 349, 338, 358, 360, 333, 322, 340, 339, 315, 318, 343, 338, 308, 315, 339, 330, 297, 281, 279, 253, 270, 379, 462, 432, 384, 383, 381, 354, 347, 359, 342, 305, 306, 335, 332, 303, 310, 335, 326, 304, 317, 339, 323, 304, 322, 341, 324, 313, 338, 351, 330, 321, 346, 355, 331, 327, 355, 359, 338, 343, 370, 365, 341, 352, 377, 362, 334, 346, 361, 338, 317, 339, 350, 319, 301, 323, 333, 310, 306, 330, 333, 309, 309, 334, 331, 308, 318, 346, 336, 310, 323, 346, 332, 314, 330, 345, 326, 313, 336, 346, 321, 314, 340, 344, 317, 314, 337, 338, 315, 319, 343, 336, 312, 323, 343, 331, 313, 333, 356, 337, 324, 350, 364, 345, 340, 359, 361, 330, 321, 344, 340, 312, 315, 342, 336, 308, 312, 337, 326, 301, 300, 292, 256, 241, 317, 424, 444, 407, 401, 390, 351, 340, 360, 349, 313, 310, 335, 328, 301, 307, 335, 325, 297, 311, 338, 324, 299, 317, 340, 324, 309, 334, 349, 327, 316, 340, 348, 324, 323, 353, 352, 328, 334, 363, 360, 340, 356, 378, 362, 338, 351, 368, 347, 323, 335, 340, 313, 303, 327, 333, 308, 303, 331, 332, 303, 306, 331, 328, 304, 312, 335, 326, 306, 323, 346, 328, 307, 323, 343, 328, 315, 335, 343, 318, 314, 342, 343, 316, 315, 341, 339, 311, 319, 346, 336, 311, 321, 347, 336, 318, 337, 357, 338, 326, 352, 366, 348, 340, 363, 365, 332, 325, 345, 341, 314, 319, 343, 338, 312, 317, 339, 326, 304, 308, 299, 261, 240, 304, 408, 427, 404, 403, 385, 343, 331, 361, 363, 329, 322, 340, 331, 303, 314, 343, 336, 308, 320, 342, 328, 309, 327, 350, 332, 315, 334, 349, 329, 321, 346, 352, 328, 330, 359, 359, 335, 339, 368, 365, 342, 354, 377, 363, 342, 357, 371, 343, 322, 339, 350, 326, 313, 332, 337, 313, 310, 336, 335, 308, 309, 334, 330, 307, 315, 341, 329, 306, 320, 342, 327, 309, 328, 346, 326, 312, 337, 348, 320, 308, 334, 340, 319, 320, 346, 344, 316, 322, 346, 337, 313, 327, 353, 341, 325, 343, 362, 351, 341, 362, 379, 356, 337, 351, 349, 322, 320, 345, 340, 313, 317, 342, 335, 309, 320, 342, 318, 272, 262, 269, 276, 343, 440, 463, 405, 360, 368, 371, 347, 336, 348, 339, 310, 309, 334, 332, 308, 315, 339, 329, 307, 323, 346, 330, 309, 328, 349, 330, 316, 340, 352, 328, 322, 348, 355, 333, 335, 362, 362, 341, 348, 373, 366, 345, 359, 382, 365, 336, 346, 361, 338, 318, 334, 344, 317, 304, 327, 334, 309, 306, 331, 331, 304, 307, 334, 330, 302, 313, 339, 326, 303, 320, 342, 324, 306, 326, 340, 320, 308, 330, 341, 319, 312, 336, 337, 313, 318, 343, 340, 316, 321, 340, 331, 311, 324, 347, 333, 315, 333, 350, 331, 322, 353, 368, 345, 341, 363, 360, 329, 323, 346, 340, 311, 316, 343, 336, 311, 319, 337, 324, 305, 313, 301, 259, 233, 291, 396, 442, 425, 414, 394, 353, 346, 371, 360, 318, 314, 334, 325, 301, 309, 333, 322, 302, 318, 340, 322, 305, 327, 345, 323, 309, 334, 345, 324, 320, 345, 349, 323, 325, 353, 352, 331, 338, 364, 360, 340, 351, 374, 362, 342, 352, 364, 342, 321, 338, 349, 324, 311, 331, 337, 312, 308, 332, 330, 302, 304, 330, 325, 302, 313, 338, 325, 304, 322, 341, 322, 307, 331, 345, 321, 308, 331, 339, 315, 310, 334, 333, 310, 316, 341, 336, 311, 318, 344, 333, 311, 326, 347, 329, 308, 329, 346, 326, 314, 338, 351, 330, 325, 354, 365, 344, 340, 368, 364, 333, 332, 350, 339, 315, 323, 346, 332, 311, 326, 343, 324, 308, 326, 329, 280, 245, 261, 305, 364, 430, 452, 419, 371, 359, 375, 367, 333, 328, 343, 325, 302, 320, 338, 321, 303, 319, 335, 316, 303, 329, 341, 319, 312, 338, 344, 318, 320, 351, 351, 322, 327, 353, 347, 327, 343, 367, 352, 334, 355, 376, 355, 341, 365, 374, 346, 331, 348, 346, 316, 313, 339, 334, 304, 306, 332, 324, 298, 310, 336, 320, 294, 308, 331, 313, 299, 323, 337, 316, 306, 331, 337, 312, 308, 336, 338, 311, 312, 340, 336, 311, 320, 344, 330, 306, 323, 346, 328, 309, 328, 342, 320, 307, 332, 342, 319, 312, 336, 339, 317, 318, 343, 343, 319, 329, 361, 358, 336, 349, 368, 350, 325, 335, 347, 325, 310, 330, 342, 316, 302, 328, 336, 310, 304, 321, 296, 249, 245, 301, 377, 408, 419, 417, 374, 336, 344, 366, 346, 314, 323, 333, 309, 297, 320, 329, 306, 302, 327, 328, 305, 310, 334, 329, 308, 317, 341, 332, 312, 325, 345, 331, 315, 336, 354, 336, 327, 350, 363, 345, 339, 364, 368, 341, 342, 372, 368, 335, 334, 352, 338, 312, 322, 339, 322, 303, 323, 340, 318, 301, 322, 335, 314, 306, 328, 335, 310, 307, 334, 333, 311, 315, 343, 336, 309, 320, 347, 337, 314, 324, 343, 329, 315, 332, 344, 322, 312, 336, 344, 319, 315, 341, 346, 320, 317, 343, 343, 324, 331, 354, 348, 333, 351, 376, 361, 337, 348, 358, 332, 315, 335, 345, 320, 309, 330, 335, 313, 313, 339, 334, 288, 262, 271, 268, 293, 393, 459, 428, 372, 362, 369, 348, 336, 348, 347, 314, 299, 321, 330, 311, 310, 332, 331, 305, 309, 336, 332, 308, 318, 342, 335, 316, 328, 347, 331, 314, 337, 354, 335, 324, 347, 361, 341, 338, 366, 374, 349, 350, 375, 370, 341, 342, 362, 349, 320, 327, 346, 329, 305, 320, 336, 316, 302, 323, 333, 307, 300, 327, 331, 306, 305, 332, 333, 309, 311, 336, 331, 308, 320, 342, 329, 310, 327, 344, 327, 310, 329, 345, 322, 311, 334, 342, 318, 315, 340, 343, 317, 317, 344, 339, 316, 327, 353, 341, 322, 346, 372, 355, 337, 353, 365, 337, 320, 337, 344, 317, 308, 333, 337, 309, 308, 337, 336, 308, 298, 293, 265, 238, 292, 407, 451, 415, 396, 389, 356, 337, 361, 365, 326, 305, 324, 331, 310, 307, 335, 335, 309, 311, 337, 334, 310, 322, 347, 333, 311, 327, 348, 334, 320, 341, 358, 339, 329, 352, 360, 340, 338, 364, 369, 349, 352, 377, 372, 344, 351, 372, 358, 330, 338, 352, 332, 313, 328, 340, 320, 303, 323, 337, 314, 304, 327, 331, 311, 310, 335, 334, 309, 314, 342, 336, 311, 322, 347, 335, 315, 330, 345, 325, 308, 334, 351, 328, 315, 340, 348, 323, 319, 344, 346, 319, 320, 347, 343, 317, 322, 346, 339, 319, 336, 358, 341, 325, 352, 371, 347, 339, 362, 367, 336, 324, 344, 345, 319, 318, 339, 333, 310, 317, 341, 330, 305, 309, 304, 264, 233, 280, 380, 440, 430, 416, 404, 364, 346, 369, 364, 323, 311, 331, 328, 303, 307, 332, 325, 300, 311, 335, 323, 304, 319, 340, 324, 306, 328, 345, 324, 317, 342, 351, 324, 318, 346, 352, 329, 331, 360, 359, 336, 345, 371, 362, 340, 354, 371, 349, 323, 338, 351, 326, 307, 325, 334, 310, 305, 331, 336, 306, 301, 327, 326, 302, 307, 333, 329, 308, 320, 340, 327, 306, 322, 343, 327, 310, 329, 340, 317, 310, 334, 340, 317, 315, 339, 336, 312, 315, 340, 334, 311, 321, 346, 334, 312, 327, 346, 326, 311, 328, 340, 322, 317, 342, 350, 324, 326, 360, 360, 333, 339, 364, 355, 321, 322, 346, 334, 309, 321, 343, 326, 303, 322, 339, 320, 302, 314, 297, 250, 238, 302, 389, 421, 416, 415, 387, 343, 337, 364, 351, 317, 318, 334, 320, 301, 318, 337, 318, 303, 324, 336, 314, 305, 328, 338, 316, 313, 338, 342, 320, 323, 345, 340, 318, 333, 358, 348, 326, 344, 367, 354, 340, 361, 375, 354, 343, 366, 371, 339, 326, 346, 344, 316, 315, 339, 334, 306, 309, 334, 327, 303, 314, 337, 323, 302, 318, 336, 316, 304, 328, 340, 316, 309, 337, 340, 313, 311, 338, 340, 317, 319, 340, 330, 309, 323, 349, 336, 311, 325, 344, 328, 312, 331, 346, 324, 312, 336, 344, 320, 315, 339, 338, 317, 327, 352, 348, 333, 344, 367, 361, 338, 347, 359, 337, 311, 323, 338, 320, 308, 332, 342, 319, 310, 333, 336, 302, 275, 279, 272, 273, 356, 443, 436, 384, 371, 379, 356, 334, 351, 358, 327, 303, 323, 335, 311, 304, 328, 332, 308, 308, 336, 337, 312, 314, 342, 338, 314, 325, 349, 338, 318, 334, 355, 342, 326, 343, 358, 338, 330, 358, 370, 348, 343, 369, 375, 350, 350, 372, 358, 324, 326, 345, 333, 309, 323, 345, 326, 303, 322, 339, 319, 306, 327, 336, 311, 304, 333, 342, 316, 313, 338, 339, 312, 318, 342, 336, 313, 322, 344, 333, 311, 328, 349, 328, 309, 331, 346, 326, 315, 337, 343, 318, 315, 341, 342, 317, 320, 346, 342, 318, 329, 353, 341, 327, 350, 372, 352, 335, 350, 360, 335, 319, 340, 346, 320, 314, 339, 340, 310, 308, 333, 334, 309, 298, 289, 261, 240, 306, 419, 452, 412, 397, 392, 356, 337, 359, 362, 322, 304, 327, 334, 309, 308, 333, 329, 306, 314, 337, 330, 308, 318, 343, 335, 315, 330, 351, 335, 319, 343, 357, 337, 328, 353, 360, 336, 335, 364, 371, 348, 349, 374, 373, 348, 354, 373, 354, 323, 334, 351, 333, 310, 325, 341, 321, 308, 328, 337, 315, 309, 334, 333, 305, 306, 332, 333, 311, 318, 343, 332, 311, 322, 342, 328, 308, 326, 346, 326, 310, 333, 344, 323, 315, 339, 347, 322, 317, 343, 346, 318, 318, 344, 337, 313, 324, 348, 335, 312, 324, 345, 329, 314, 338, 356, 336, 323, 352, 369, 343, 337, 367, 372, 340, 333, 351, 344, 317, 320, 344, 335, 314, 327, 348, 331, 310, 325, 330, 280, 245, 266, 320, 385, 435, 449, 426, 375, 357, 381, 377, 334, 320, 337, 329, 308, 318, 342, 330, 306, 319, 337, 323, 309, 328, 342, 320, 310, 337, 347, 324, 324, 349, 345, 320, 327, 356, 354, 331, 340, 366, 357, 337, 353, 375, 361, 344, 363, 376, 351, 334, 352, 358, 329, 320, 339, 338, 313, 314, 337, 331, 306, 313, 339, 332, 307, 316, 338, 325, 307, 326, 343, 323, 310, 335, 347, 325, 316, 339, 343, 319, 317, 340, 338, 314, 321, 346, 337, 315, 326, 347, 337, 314, 329, 348, 331, 313, 331, 346, 327, 318, 340, 348, 322, 315, 344, 347, 320, 320, 344, 341, 318, 327, 355, 346, 323, 343, 369, 353, 336, 356, 368, 341, 321, 340, 348, 326, 316, 338, 346, 319, 313, 336, 335, 310, 313, 327, 291, 248, 254, 314, 385, 416, 427, 417, 374, 342, 353, 366, 342, 323, 334, 334, 309, 305, 333, 336, 310, 315, 339, 330, 306, 320, 342, 328, 309, 324, 346, 330, 313, 335, 349, 328, 318, 342, 354, 334, 332, 359, 363, 341, 345, 373, 371, 344, 351, 378, 364, 334, 344, 360, 340, 314, 327, 342, 319, 303, 328, 339, 313, 301, 325, 333, 308, 307, 332, 332, 304, 309, 337, 332, 313, 323, 340, 326, 304, 325, 348, 330, 310, 332, 346, 324, 314, 337, 345, 319, 315, 340, 341, 317, 320, 344, 338, 313, 322, 349, 338, 312, 327, 349, 334, 316, 337, 354, 338, 328, 350, 364, 345, 338, 365, 376, 349, 340, 357, 347, 313, 317, 346, 340, 316, 324, 343, 325, 304, 324, 340, 307, 263, 267, 278, 298, 376, 449, 446, 387, 354, 366, 362, 340, 340, 353, 335, 306, 314, 338, 326, 303, 319, 338, 317, 302, 326, 343, 321, 311, 334, 341, 315, 312, 341, 345, 320, 322, 349, 345, 322, 335, 364, 356, 333, 349, 373, 358, 342, 365, 381, 356, 339, 357, 362, 333, 320, 340, 341, 314, 312, 336, 334, 307, 312, 336, 324, 302, 315, 338, 325, 305, 321, 341, 324, 306, 326, 342, 321, 311, 335, 339, 314, 313, 338, 338, 314, 320, 346, 340, 314, 320, 346, 335, 313, 326, 347, 329, 311, 330, 344, 324, 313, 336, 345, 321, 317, 344, 348, 321, 326, 359, 360, 334, 342, 368, 355, 328, 337, 352, 334, 311, 325, 342, 322, 307, 329, 340, 317, 311, 330, 318, 262, 241, 271, 322, 388, 440, 447, 409, 357, 354, 375, 360, 327, 329, 339, 317, 301, 320, 335, 314, 303, 327, 334, 313, 310, 337, 339, 312, 316, 344, 340, 315, 322, 346, 337, 317, 333, 358, 343, 324, 343, 364, 348, 338, 362, 375, 349, 342, 369, 369, 339, 333, 354, 349, 323, 325, 340, 326, 302, 316, 336, 320, 299, 311, 326, 310, 298, 318, 331, 310, 300, 325, 332, 309, 306, 333, 333, 306, 307, 334, 329, 306, 316, 341, 329, 308, 323, 344, 325, 306, 327, 344, 325, 314, 333, 339, 315, 310, 334, 338, 313, 315, 340, 337, 310, 320, 346, 337, 317, 330, 358, 351, 333, 349, 368, 346, 326, 344, 354, 325, 311, 335, 341, 314, 314, 340, 334, 307, 315, 340, 320, 266, 256, 273, 294, 361, 448, 460, 408, 365, 370, 376, 353, 335, 341, 336, 308, 307, 331, 330, 306, 310, 333, 327, 303, 313, 335, 324, 305, 321, 344, 331, 315, 333, 345, 323, 314, 341, 354, 330, 322, 349, 357, 334, 338, 367, 362, 339, 348, 375, 362, 332, 339, 358, 339, 313, 327, 341, 318, 302, 324, 337, 312, 301, 325, 331, 305, 304, 332, 332, 305, 311, 339, 333, 309, 318, 342, 333, 313, 327, 342, 324, 310, 330, 349, 327, 312, 334, 344, 320, 314, 338, 340, 316, 320, 347, 339, 311, 322, 348, 336, 315, 329, 346, 327, 309, 330, 349, 328, 314, 336, 348, 328, 323, 350, 361, 338, 339, 367, 365, 335, 332, 350, 338, 311, 324, 346, 330, 309, 324, 340, 321, 304, 326, 335, 294, 257, 268, 282, 317, 397, 456, 434, 375, 353, 368, 362, 337, 337, 348, 328, 305, 317, 335, 320, 301, 320, 337, 318, 307, 331, 342, 321, 316, 342, 345, 322, 320, 344, 341, 320, 331, 353, 346, 328, 342, 365, 352, 334, 353, 373, 358, 348, 368, 376, 350, 340, 359, 358, 326, 324, 350, 342, 307, 308, 334, 327, 301, 312, 335, 320, 302, 322, 341, 320, 303, 326, 341, 317, 305, 332, 340, 317, 313, 338, 342, 318, 321, 346, 339, 315, 321, 343, 332, 311, 324, 340, 323, 307, 333, 351, 324, 307, 333, 345, 321, 315, 341, 347, 326, 326, 350, 352, 332, 342, 370, 362, 334, 340, 357, 338, 310, 326, 342, 321, 306, 328, 340, 312, 304, 332, 339, 304, 272, 277, 274, 277, 360, 439, 432, 384, 369, 373, 350, 333, 349, 356, 326, 306, 326, 337, 313, 303, 329, 338, 313, 309, 332, 335, 314, 320, 347, 345, 318, 325, 349, 338, 317, 333, 351, 337, 322, 345, 362, 343, 334, 360, 375, 352, 344, 368, 374, 350, 350, 368, 353, 318, 323, 348, 339, 312, 317, 336, 324, 305, 321, 337, 317, 300, 320, 335, 311, 300, 326, 335, 313, 313, 337, 334, 310, 316, 340, 331, 307, 318, 341, 330, 309, 324, 344, 325, 305, 325, 343, 324, 310, 333, 342, 315, 308, 334, 341, 320, 322, 348, 350, 332, 341, 365, 358, 336, 345, 358, 332, 308, 326, 344, 323, 306, 329, 338, 313, 305, 328, 333, 305, 290, 284, 262, 244, 311, 430, 457, 406, 383, 380, 354, 335, 355, 363, 328, 304, 325, 337, 312, 301, 329, 338, 312, 308, 335, 338, 313, 316, 343, 340, 317, 328, 353, 341, 320, 335, 358, 341, 323, 345, 364, 345, 336, 365, 377, 354, 351, 376, 379, 350, 347, 370, 361, 329, 329, 348, 335, 312, 325, 346, 330, 308, 324, 338, 315, 303, 325, 337, 313, 303, 330, 339, 314, 309, 336, 338, 313, 317, 342, 334, 308, 320, 345, 332, 309, 326, 346, 330, 313, 334, 349, 330, 320, 342, 347, 319, 315, 342, 347, 322, 328, 357, 355, 333, 347, 372, 360, 343, 359, 375, 347, 322, 337, 352, 333, 316, 337, 347, 323, 313, 336, 342, 318, 310, 318, 287, 248, 266, 354, 437, 444, 426, 415, 378, 349, 372, 383, 345, 317, 333, 341, 320, 313, 337, 341, 317, 314, 340, 340, 315, 320, 345, 341, 321, 336, 358, 342, 320, 337, 359, 344, 329, 349, 366, 348, 342, 366, 376, 352, 347, 375, 379, 355, 354, 376, 368, 340, 343, 360, 342, 316, 328, 346, 329, 308, 321, 338, 319, 306, 328, 339, 317, 309, 336, 339, 312, 310, 335, 335, 312, 318, 343, 335, 310, 320, 345, 333, 314, 330, 347, 327, 314, 336, 348, 326, 315, 338, 345, 320, 316, 344, 346, 319, 321, 344, 339, 319, 331, 356, 349, 326, 339, 366, 354, 338, 359, 374, 347, 324, 343, 352, 325, 316, 339, 342, 315, 315, 341, 337, 310, 316, 339, 315, 263, 256, 273, 303, 377, 452, 453, 399, 360, 369, 382, 353, 325, 335, 334, 307, 305, 331, 330, 306, 311, 337, 331, 307, 319, 345, 334, 312, 328, 349, 329, 312, 336, 352, 331, 320, 342, 352, 330, 328, 357, 360, 336, 340, 367, 363, 341, 351, 373, 362, 337, 347, 362, 341, 318, 332, 345, 325, 311, 332, 336, 308, 301, 328, 335, 311, 309, 334, 330, 306, 312, 338, 328, 304, 321, 345, 330, 308, 325, 343, 322, 307, 328, 340, 319, 312, 337, 345, 318, 312, 335, 335, 314, 319, 344, 335, 310, 319, 341, 331, 312, 329, 348, 329, 313, 330, 341, 320, 312, 339, 346, 319, 318, 347, 355, 336, 339, 367, 365, 336, 338, 357, 343, 312, 324, 346, 329, 308, 327, 340, 317, 305, 328, 339, 313, 283, 278, 268, 254, 319, 431, 455, 399, 373, 377, 358, 333, 344, 355, 328, 300, 313, 331, 315, 300, 322, 335, 311, 302, 330, 337, 314, 312, 336, 337, 312, 317, 342, 336, 316, 329, 351, 339, 319, 337, 360, 346, 333, 353, 372, 355, 343, 366, 377, 355, 347, 367, 365, 335, 328, 349, 341, 311, 316, 339, 327, 303, 317, 338, 320, 302, 318, 335, 318, 307, 329, 338, 313, 303, 329, 338, 316, 314, 339, 336, 309, 313, 342, 336, 311, 320, 347, 333, 308, 325, 348, 330, 313, 331, 345, 323, 312, 337, 347, 320, 317, 341, 336, 311, 319, 350, 345, 321, 330, 356, 350, 335, 353, 377, 359, 334, 345, 355, 329, 311, 331, 342, 316, 312, 336, 337, 310, 309, 337, 332, 291, 272, 278, 268, 288, 393, 463, 433, 376, 369, 375, 350, 337, 349, 345, 318, 306, 330, 336, 309, 304, 334, 336, 309, 313, 338, 331, 307, 321, 345, 334, 312, 329, 352, 338, 319, 335, 350, 331, 324, 351, 362, 340, 337, 364, 366, 342, 348, 378, 374, 341, 342, 363, 350, 323, 333, 346, 324, 303, 321, 336, 316, 303, 325, 339, 314, 304, 326, 330, 308, 308, 334, 333, 307, 311, 336, 328, 305, 321, 344, 326, 306, 324, 344, 324, 311, 332, 343, 318, 310, 337, 345, 320, 317, 339, 339, 314, 317, 341, 334, 310, 321, 346, 334, 312, 325, 343, 329, 314, 336, 353, 332, 327, 360, 371, 344, 339, 364, 362, 328, 324, 345, 341, 315, 319, 343, 332, 309, 322, 344, 329, 306, 318, 314, 266, 237, 273, 356, 420, 436, 431, 403, 354, 344, 374, 369, 328, 322, 342, 332, 306, 315, 340, 325, 304, 320, 339, 324, 311, 333, 347, 324, 315, 342, 350, 324, 322, 349, 352, 325, 326, 355, 353, 331, 342, 367, 356, 337, 355, 382, 367, 344, 358, 371, 351, 337, 353, 355, 326, 320, 343, 341, 311, 310, 338, 338, 310, 313, 339, 333, 307, 319, 342, 328, 309, 329, 343, 323, 310, 335, 347, 325, 317, 339, 345, 322, 320, 345, 346, 322, 326, 348, 343, 321, 327, 348, 335, 314, 331, 349, 330, 313, 332, 348, 327, 316, 338, 344, 320, 319, 348, 346, 316, 317, 343, 342, 322, 331, 357, 355, 336, 348, 371, 355, 332, 345, 354, 329, 314, 333, 342, 319, 312, 335, 338, 313, 310, 334, 330, 292, 267, 270, 261, 289, 396, 471, 435, 380, 376, 382, 361, 343, 349, 344, 312, 299, 325, 333, 308, 303, 325, 330, 307, 309, 334, 327, 306, 317, 342, 328, 305, 322, 348, 337, 318, 332, 346, 327, 317, 343, 353, 331, 330, 360, 365, 339, 339, 364, 363, 337, 342, 364, 349, 319, 329, 346, 327, 305, 321, 336, 315, 300, 322, 333, 307, 297, 322, 327, 306, 307, 331, 328, 305, 310, 333, 328, 309, 321, 343, 326, 305, 322, 342, 325, 308, 329, 343, 321, 309, 333, 339, 314, 309, 334, 337, 313, 316, 342, 334, 308, 318, 345, 334, 309, 324, 345, 327, 307, 330, 349, 331, 318, 346, 364, 341, 334, 364, 371, 341, 329, 347, 342, 316, 320, 342, 330, 307, 319, 340, 325, 305, 324, 337, 295, 250, 256, 281, 324, 408, 466, 443, 383, 353, 365, 369, 342, 334, 343, 323, 294, 306, 331, 319, 301, 318, 338, 321, 303, 324, 338, 317, 311, 336, 342, 319, 313, 339, 342, 319, 321, 347, 346, 324, 334, 360, 351, 333, 351, 371, 354, 338, 361, 378, 354, 337, 354, 356, 327, 316, 335, 337, 314, 313, 332, 324, 298, 306, 333, 325, 299, 308, 329, 318, 300, 319, 341, 320, 303, 325, 342, 319, 307, 329, 337, 316, 315, 340, 337, 311, 316, 343, 337, 310, 318, 340, 328, 309, 323, 340, 324, 308, 329, 344, 325, 312, 333, 340, 317, 313, 341, 348, 325, 327, 358, 361, 338, 349, 371, 355, 321, 329, 347, 329, 309, 326, 341, 320, 306, 327, 340, 316, 309, 326, 308, 257, 241, 280, 346, 401, 422, 428, 388, 339, 339, 363, 354, 327, 329, 338, 315, 299, 321, 338, 318, 305, 327, 335, 312, 310, 337, 339, 315, 318, 345, 338, 315, 325, 350, 341, 316, 331, 355, 343, 327, 347, 365, 346, 335, 360, 374, 351, 344, 370, 373, 342, 335, 356, 350, 320, 322, 342, 332, 308, 320, 342, 326, 302, 319, 338, 320, 304, 327, 339, 316, 307, 329, 337, 313, 313, 342, 340, 312, 312, 337, 336, 313, 320, 342, 333, 311, 323, 342, 326, 312, 332, 346, 324, 314, 335, 340, 320, 318, 343, 348, 324, 327, 359, 361, 336, 343, 369, 355, 329, 338, 355, 334, 311, 330, 344, 320, 307, 328, 338, 317, 311, 334, 329, 285, 258, 271, 278, 315, 407, 462, 432, 372, 356, 367, 355, 338, 346, 350, 322, 304, 326, 337, 313, 305, 331, 335, 308, 305, 333, 336, 311, 316, 339, 337, 315, 323, 346, 334, 312, 328, 353, 339, 323, 342, 357, 338, 329, 357, 371, 351, 348, 371, 369, 341, 339, 360, 348, 317, 321, 343, 331, 307, 319, 339, 323, 303, 320, 335, 312, 296, 317, 332, 311, 303, 328, 332, 307, 308, 336, 334, 307, 312, 340, 336, 312, 320, 341, 327, 305, 321, 342, 325, 308, 330, 344, 325, 315, 337, 343, 321, 322, 349, 350, 326, 330, 362, 360, 338, 351, 375, 356, 323, 333, 353, 337, 316, 330, 341, 317, 305, 332, 340, 314, 306, 326, 317, 265, 246, 276, 328, 399, 446, 443, 398, 349, 347, 370, 356, 322, 326, 339, 317, 299, 321, 334, 311, 302, 324, 331, 307, 304, 331, 333, 307, 311, 338, 333, 312, 325, 348, 336, 318, 336, 356, 340, 327, 349, 366, 347, 337, 361, 371, 349, 347, 373, 369, 338, 335, 353, 345, 317, 322, 343, 332, 307, 316, 338, 324, 304, 317, 332, 315, 304, 324, 332, 311, 306, 333, 337, 311, 309, 337, 335, 311, 316, 339, 333, 312, 322, 342, 330, 312, 329, 346, 330, 316, 332, 343, 321, 314, 338, 347, 325, 322, 350, 352, 329, 338, 364, 359, 342, 351, 366, 341, 315, 329, 344, 324, 308, 329, 341, 316, 305, 331, 340, 317, 309, 313, 284, 240, 246, 324, 415, 440, 422, 416, 383, 347, 357, 373, 344, 310, 318, 329, 308, 296, 321, 336, 315, 307, 330, 336, 312, 309, 335, 333, 310, 316, 342, 334, 312, 325, 350, 338, 317, 334, 356, 339, 325, 350, 368, 349, 339, 361, 370, 349, 344, 366, 364, 334, 333, 352, 343, 316, 320, 339, 327, 305, 320, 340, 322, 304, 326, 342, 320, 308, 332, 343, 318, 312, 338, 342, 315, 316, 342, 338, 312, 319, 346, 339, 316, 326, 348, 330, 311, 331, 350, 330, 315, 337, 347, 324, 315, 339, 342, 317, 320, 349, 352, 329, 336, 365, 359, 336, 349, 370, 350, 323, 335, 347, 325, 309, 329, 342, 318, 305, 327, 336, 314, 309, 328, 308, 254, 241, 284, 350, 410, 437, 432, 393, 349, 351, 371, 350, 316, 323, 334, 311, 295, 317, 328, 309, 303, 327, 331, 306, 304, 330, 332, 309, 314, 338, 333, 312, 325, 348, 336, 318, 334, 354, 338, 326, 350, 365, 346, 342, 365, 370, 347, 349, 373, 367, 336, 335, 357, 348, 318, 323, 341, 324, 302, 316, 337, 319, 302, 318, 328, 307, 299, 325, 336, 313, 306, 329, 329, 307, 310, 337, 333, 310, 317, 342, 333, 307, 317, 341, 329, 310, 325, 338, 321, 310, 333, 342, 316, 308, 334, 342, 319, 318, 343, 341, 318, 327, 353, 353, 337, 345, 370, 358, 334, 344, 355, 332, 309, 328, 344, 318, 300, 324, 336, 313, 308, 332, 331, 298, 278, 281, 269, 260, 335, 430, 435, 393, 377, 374, 341, 322, 347, 358, 328, 305, 322, 329, 309, 309, 333, 334, 310, 311, 339, 334, 306, 315, 343, 336, 313, 325, 350, 336, 315, 333, 353, 337, 324, 345, 357, 335, 330, 361, 375, 352, 346, 369, 368, 342, 343, 366, 352, 321, 324, 342, 331, 310, 321, 335, 315, 298, 319, 336, 314, 303, 325, 333, 310, 306, 333, 336, 308, 309, 336, 333, 310, 320, 344, 335, 312, 324, 344, 328, 310, 329, 344, 322, 308, 332, 346, 322, 312, 336, 339, 314, 313, 339, 339, 314, 321, 348, 342, 325, 342, 365, 352, 332, 344, 355, 334, 315, 333, 344, 318, 306, 328, 336, 311, 306, 332, 335, 307, 300, 301, 268, 232, 263, 366, 433, 421, 404, 395, 357, 328, 351, 365, 333, 309, 326, 332, 311, 306, 330, 332, 307, 309, 336, 334, 308, 313, 340, 335, 316, 331, 349, 332, 316, 338, 357, 338, 324, 345, 358, 339, 338, 365, 370, 347, 344, 368, 371, 349, 354, 374, 360, 330, 334, 352, 338, 313, 327, 342, 320, 303, 324, 337, 315, 306, 331, 339, 315, 310, 334, 333, 309, 315, 341, 336, 308, 314, 337, 329, 309, 323, 346, 329, 310, 330, 346, 325, 313, 337, 344, 321, 316, 339, 340, 316, 318, 343, 341, 318, 324, 349, 341, 318, 332, 354, 340, 327, 352, 369, 349, 338, 356, 362, 335, 322, 343, 347, 321, 315, 340, 339, 312, 314, 339, 332, 307, 312, 314, 274, 238, 269, 357, 430, 437, 426, 406, 362, 340, 365, 370, 334, 313, 331, 330, 305, 309, 337, 332, 306, 314, 335, 325, 307, 324, 343, 328, 310, 325, 341, 324, 313, 339, 350, 329, 322, 348, 356, 333, 333, 359, 361, 338, 346, 372, 366, 343, 353, 375, 359, 332, 339, 351, 332, 317, 334, 340, 316, 305, 329, 336, 310, 305, 331, 333, 308, 310, 334, 329, 303, 315, 342, 333, 309, 322, 345, 329, 311, 331, 345, 322, 311, 331, 339, 320, 314, 338, 341, 317, 317, 339, 337, 314, 324, 350, 337, 311, 323, 344, 327, 308, 326, 345, 326, 312, 334, 349, 330, 321, 343, 357, 342, 340, 365, 365, 338, 338, 357, 346, 317, 324, 343, 330, 309, 326, 343, 323, 307, 327, 338, 309, 276, 272, 270, 268, 340, 447, 463, 404, 378, 386, 372, 346, 346, 354, 331, 304, 316, 333, 317, 299, 322, 338, 316, 305, 327, 339, 318, 311, 336, 342, 318, 317, 339, 340, 318, 325, 348, 340, 319, 335, 360, 349, 333, 350, 368, 353, 342, 363, 375, 354, 343, 360, 358, 328, 322, 343, 341, 315, 317, 340, 333, 306, 313, 338, 326, 304, 322, 338, 318, 302, 324, 339, 317, 309, 334, 341, 317, 315, 338, 337, 313, 317, 344, 338, 312, 321, 345, 336, 316, 327, 346, 332, 311, 328, 346, 328, 316, 336, 348, 324, 316, 338, 343, 320, 322, 347, 344, 317, 320, 346, 340, 317, 330, 356, 344, 332, 354, 371, 354, 341, 359, 365, 336, 317, 337, 344, 318, 313, 336, 336, 309, 313, 336, 330, 306, 306, 302, 266, 234, 281, 386, 431, 412, 400, 389, 348, 328, 357, 362, 325, 304, 322, 324, 300, 304, 329, 326, 301, 307, 331, 325, 302, 318, 342, 327, 307, 323, 341, 323, 312, 339, 351, 327, 318, 343, 352, 330, 332, 362, 362, 338, 344, 369, 364, 341, 352, 376, 357, 328, 336, 353, 334, 313, 328, 337, 314, 301, 325, 332, 308, 304, 330, 330, 301, 303, 331, 329, 305, 312, 337, 331, 309, 320, 340, 324, 310, 331, 344, 323, 311, 333, 344, 322, 318, 342, 342, 314, 314, 340, 338, 314, 320, 343, 335, 314, 325, 344, 329, 309, 328, 350, 331, 315, 340, 354, 332, 324, 351, 366, 351, 348, 371, 373, 342, 340, 357, 345, 315, 324, 348, 336, 313, 327, 345, 325, 305, 326, 342, 307, 267, 272, 276, 282, 360, 445, 447, 395, 369, 373, 359, 335, 341, 353, 332, 300, 311, 335, 324, 304, 320, 337, 320, 307, 329, 343, 320, 313, 341, 348, 323, 320, 347, 349, 321, 323, 350, 344, 320, 333, 360, 349, 333, 352, 374, 356, 342, 364, 377, 353, 338, 358, 360, 332, 324, 341, 339, 312, 312, 335, 327, 303, 315, 338, 326, 303, 316, 338, 325, 307, 329, 343, 321, 309, 330, 339, 316, 313, 336, 336, 314, 316, 342, 340, 314, 317, 341, 333, 312, 325, 345, 329, 313, 331, 345, 325, 313, 336, 345, 322, 313, 336, 340, 318, 321, 350, 348, 320, 327, 362, 363, 338, 348, 369, 349, 320, 330, 348, 326, 309, 330, 341, 318, 307, 330, 337, 314, 312, 333, 317, 264, 245, 269, 311, 378, 441, 452, 408, 355, 350, 370, 354, 326, 329, 338, 315, 299, 320, 334, 310, 303, 327, 331, 309, 311, 335, 332, 310, 320, 344, 334, 310, 324, 349, 337, 319, 338, 356, 336, 322, 345, 359, 339, 331, 358, 369, 348, 346, 371, 371, 340, 339, 362, 352, 318, 323, 342, 329, 309, 322, 337, 319, 303, 323, 338, 316, 303, 326, 337, 314, 308, 332, 335, 309, 312, 339, 336, 312, 317, 342, 334, 310, 321, 342, 328, 309, 327, 343, 324, 312, 334, 343, 319, 311, 332, 335, 312, 311, 336, 333, 311, 315, 339, 330, 309, 320, 342, 330, 312, 328, 347, 328, 314, 343, 364, 342, 333, 357, 361, 333, 322, 343, 339, 311, 313, 338, 332, 307, 317, 339, 327, 304, 316, 326, 288, 244, 250, 284, 335, 407, 453, 434, 379, 352, 369, 375, 340, 320, 332, 321, 296, 305, 331, 322, 298, 309, 331, 320, 300, 314, 331, 314, 302, 327, 341, 321, 313, 336, 344, 320, 317, 344, 349, 326, 329, 357, 354, 332, 344, 369, 359, 337, 352, 372, 355, 333, 347, 359, 334, 318, 336, 339, 312, 308, 331, 331, 305, 307, 331, 325, 302, 310, 336, 327, 304, 315, 338, 324, 305, 322, 337, 319, 305, 326, 338, 316, 308, 335, 340, 315, 314, 337, 336, 311, 319, 343, 333, 308, 320, 342, 332, 312, 328, 346, 325, 310, 332, 342, 316, 310, 338, 344, 319, 318, 343, 342, 315, 321, 350, 346, 323, 334, 361, 353, 333, 350, 369, 349, 326, 341, 352, 329, 318, 342, 349, 322, 310, 330, 333, 310, 312, 333, 312, 256, 243, 275, 322, 390, 447, 451, 407, 360, 362, 378, 355, 327, 332, 337, 311, 303, 329, 335, 313, 313, 338, 334, 307, 314, 341, 336, 313, 327, 351, 338, 316, 333, 349, 332, 320, 343, 357, 336, 329, 352, 362, 342, 338, 363, 366, 343, 347, 373, 368, 343, 348, 366, 349, 320, 327, 341, 324, 307, 326, 342, 320, 306, 327, 335, 311, 304, 330, 333, 309, 312, 340, 337, 307, 313, 336, 329, 308, 321, 343, 330, 308, 323, 342, 324, 308, 329, 344, 320, 307, 330, 339, 320, 315, 338, 338, 312, 317, 343, 333, 307, 317, 341, 329, 308, 324, 344, 324, 311, 334, 348, 329, 320, 341, 348, 330, 333, 361, 364, 342, 343, 364, 352, 323, 329, 347, 337, 316, 326, 343, 327, 305, 321, 341, 320, 302, 305, 290, 251, 251, 336, 422, 423, 401, 396, 370, 332, 336, 362, 345, 308, 314, 334, 320, 303, 322, 339, 318, 300, 323, 337, 316, 307, 330, 335, 314, 315, 340, 341, 319, 324, 347, 341, 318, 327, 351, 344, 325, 343, 367, 350, 334, 356, 373, 354, 344, 365, 373, 343, 330, 348, 348, 320, 314, 334, 330, 304, 309, 332, 322, 298, 312, 334, 316, 297, 316, 335, 317, 305, 326, 336, 312, 304, 329, 335, 310, 308, 334, 335, 311, 314, 337, 332, 310, 323, 341, 327, 307, 326, 346, 329, 312, 331, 344, 326, 317, 341, 348, 326, 322, 347, 353, 334, 338, 364, 366, 340, 338, 353, 340, 313, 322, 342, 327, 305, 322, 337, 317, 303, 323, 333, 309, 288, 282, 269, 247, 297, 412, 451, 408, 381, 376, 355, 327, 344, 361, 336, 306, 320, 336, 316, 303, 324, 336, 315, 309, 331, 336, 315, 315, 338, 337, 315, 318, 345, 340, 318, 327, 351, 340, 319, 338, 360, 345, 329, 355, 375, 355, 340, 367, 381, 358, 351, 370, 364, 331, 329, 350, 341, 311, 314, 337, 328, 306, 319, 337, 319, 298, 315, 333, 317, 304, 323, 330, 308, 303, 328, 331, 306, 308, 333, 329, 305, 311, 336, 330, 309, 321, 343, 328, 304, 317, 337, 318, 303, 324, 336, 314, 306, 333, 341, 318, 317, 345, 347, 328, 337, 361, 356, 333, 340, 357, 337, 310, 322, 339, 321, 303, 322, 337, 316, 303, 326, 333, 309, 293, 293, 273, 239, 261, 366, 444, 425, 390, 384, 359, 332, 347, 363, 332, 301, 314, 329, 309, 295, 318, 330, 308, 299, 325, 332, 309, 309, 336, 337, 311, 314, 341, 334, 312, 323, 349, 337, 314, 329, 353, 341, 328, 350, 366, 348, 339, 364, 372, 347, 346, 370, 363, 329, 326, 351, 341, 311, 315, 341, 331, 306, 317, 335, 323, 306, 322, 337, 321, 308, 330, 340, 317, 304, 332, 341, 316, 315, 341, 339, 315, 321, 345, 338, 315, 328, 352, 339, 316, 329, 346, 328, 310, 333, 349, 326, 314, 338, 347, 323, 321, 350, 353, 327, 333, 366, 362, 337, 351, 375, 355, 325, 334, 354, 336, 316, 335, 349, 324, 310, 333, 342, 318, 312, 332, 322, 269, 249, 273, 314, 381, 445, 453, 412, 363, 362, 379, 360, 330, 332, 339, 317, 304, 326, 334, 314, 310, 333, 336, 310, 309, 337, 338, 314, 317, 343, 337, 316, 330, 353, 339, 321, 337, 355, 338, 325, 346, 362, 342, 336, 361, 370, 349, 345, 372, 373, 344, 342, 361, 352, 325, 326, 344, 332, 311, 322, 339, 321, 305, 323, 337, 316, 303, 325, 335, 313, 310, 335, 334, 306, 308, 336, 333, 307, 315, 343, 335, 310, 321, 343, 327, 308, 326, 344, 328, 312, 332, 345, 324, 318, 340, 345, 318, 314, 341, 339, 315, 322, 349, 343, 322, 334, 362, 358, 337, 352, 376, 356, 334, 344, 354, 332, 320, 340, 346, 320, 313, 341, 344, 317, 317, 342, 332, 290, 271, 278, 272, 306, 411, 471, 431, 382, 378, 380, 359, 340, 348, 344, 317, 308, 329, 333, 309, 309, 334, 335, 312, 315, 340, 335, 312, 326, 347, 333, 313, 330, 350, 335, 321, 341, 354, 334, 328, 354, 367, 344, 338, 366, 371, 350, 354, 379, 368, 342, 352, 371, 352, 322, 331, 348, 329, 312, 329, 341, 319, 307, 330, 338, 316, 310, 332, 333, 309, 311, 338, 336, 308, 316, 343, 335, 312, 326, 351, 339, 317, 332, 349, 328, 314, 338, 350, 327, 317, 340, 346, 320, 317, 344, 344, 321, 325, 350, 342, 318, 326, 347, 336, 321, 340, 359, 341, 333, 357, 372, 354, 343, 360, 359, 331, 326, 347, 344, 315, 314, 340, 338, 314, 320, 341, 326, 305, 314, 308, 267, 237, 282, 381, 423, 412, 409, 392, 346, 329, 357, 365, 330, 319, 340, 334, 306, 312, 337, 329, 305, 316, 340, 329, 306, 321, 345, 331, 317, 335, 347, 325, 321, 347, 349, 323, 321, 350, 355, 332, 336, 364, 362, 341, 352, 376, 365, 345, 359, 374, 352, 328, 343, 352, 325, 311, 333, 344, 317, 309, 332, 332, 304, 306, 334, 328, 303, 312, 338, 329, 302, 309, 332, 321, 306, 326, 343, 325, 311, 331, 340, 316, 310, 333, 339, 315, 313, 338, 336, 312, 319, 341, 330, 306, 321, 345, 331, 310, 325, 343, 325, 311, 335, 351, 331, 321, 351, 366, 339, 335, 363, 361, 329, 328, 352, 343, 312, 318, 342, 330, 306, 321, 338, 318, 300, 318, 325, 279, 245, 263, 306, 365, 420, 443, 418, 365, 346, 358, 357, 332, 325, 337, 324, 301, 315, 337, 319, 302, 320, 338, 319, 303, 325, 337, 314, 304, 329, 340, 320, 317, 338, 338, 318, 326, 352, 346, 324, 335, 361, 350, 330, 348, 369, 353, 340, 360, 372, 348, 332, 350, 356, 330, 320, 338, 334, 305, 304, 330, 329, 307, 312, 334, 326, 303, 313, 331, 318, 301, 318, 335, 318, 306, 329, 340, 315, 306, 331, 338, 314, 314, 338, 336, 311, 315, 340, 334, 311, 323, 344, 330, 312, 327, 343, 322, 308, 330, 343, 322, 312, 336, 345, 323, 318, 344, 348, 326, 328, 358, 360, 336, 343, 371, 361, 335, 341, 355, 334, 312, 331, 343, 320, 309, 335, 342, 315, 308, 334, 336, 297, 267, 272, 270, 299, 397, 466, 436, 381, 367, 373, 360, 342, 346, 345, 317, 301, 321, 334, 313, 308, 334, 338, 313, 311, 336, 335, 310, 317, 345, 340, 316, 329, 354, 339, 316, 333, 353, 337, 324, 345, 362, 345, 339, 365, 373, 349, 349, 379, 381, 349, 348, 370, 358, 325, 329, 350, 337, 310, 321, 340, 327, 309, 327, 343, 321, 306, 327, 335, 313, 308, 332, 336, 313, 313, 337, 334, 313, 320, 342, 333, 312, 324, 343, 326, 307, 327, 345, 328, 314, 334, 343, 319, 312, 335, 339, 314, 311, 336, 335, 312, 316, 341, 332, 307, 316, 341, 331, 310, 326, 344, 327, 312, 330, 343, 325, 317, 344, 352, 328, 329, 360, 363, 340, 340, 359, 351, 326, 330, 348, 334, 311, 323, 343, 329, 308, 324, 339, 320, 307, 319, 300, 254, 238, 296, 389, 433, 429, 425, 400, 353, 349, 374, 357, 317, 317, 338, 324, 304, 324, 340, 318, 303, 324, 337, 316, 309, 329, 335, 314, 313, 338, 341, 320, 323, 347, 338, 315, 326, 354, 346, 327, 343, 365, 352, 337, 356, 370, 350, 340, 363, 373, 345, 330, 349, 350, 321, 316, 337, 332, 305, 311, 334, 324, 302, 315, 337, 323, 303, 323, 339, 317, 302, 323, 335, 313, 305, 330, 337, 315, 312, 338, 337, 312, 317, 340, 335, 311, 319, 340, 329, 310, 325, 341, 324, 308, 330, 346, 326, 315, 335, 343, 319, 315, 338, 338, 315, 321, 345, 338, 313, 321, 342, 331, 309, 322, 345, 331, 314, 329, 343, 323, 311, 337, 351, 329, 319, 345, 358, 340, 340, 366, 364, 336, 336, 355, 342, 315, 326, 346, 330, 308, 321, 338, 321, 307, 329, 336, 302, 265, 267, 266, 275, 356, 447, 448, 391, 363, 370, 358, 336, 337, 343, 321, 299, 313, 332, 315, 299, 317, 330, 311, 298, 320, 333, 311, 307, 332, 334, 308, 311, 339, 338, 312, 318, 345, 338, 315, 329, 352, 342, 327, 348, 366, 349, 338, 363, 376, 352, 342, 365, 364, 331, 323, 345, 342, 315, 313, 332, 325, 304, 316, 337, 320, 299, 318, 335, 314, 297, 318, 334, 314, 300, 318, 327, 308, 307, 332, 334, 307, 311, 341, 337, 309, 313, 338, 330, 308, 322, 344, 329, 308, 321, 338, 322, 308, 331, 342, 318, 308, 332, 338, 313, 313, 340, 338, 313, 319, 343, 333, 310, 321, 343, 331, 312, 327, 347, 328, 316, 347, 367, 346, 334, 359, 364, 337, 327, 347, 343, 315, 318, 342, 334, 310, 321, 340, 324, 302, 318, 337, 302, 257, 260, 280, 309, 383, 438, 434, 385, 355, 363, 358, 337, 329, 340, 331, 302, 311, 336, 323, 298, 311, 336, 322, 305, 326, 345, 323, 311, 338, 352, 329, 321, 344, 346, 324, 327, 352, 349, 328, 333, 362, 357, 337, 350, 374, 360, 343, 365, 385, 364, 345, 363, 369, 340, 322, 342, 349, 320, 311, 334, 333, 309, 314, 340, 332, 304, 310, 334, 326, 305, 319, 339, 324, 306, 325, 339, 319, 308, 332, 344, 320, 314, 335, 340, 320, 319, 342, 340, 314, 318, 347, 337, 311, 322, 345, 333, 314, 333, 351, 330, 312, 334, 348, 324, 314, 339, 348, 327, 324, 351, 353, 331, 341, 369, 363, 344, 349, 366, 348, 323, 333, 349, 330, 313, 332, 344, 322, 309, 329, 336, 313, 308, 318, 291, 247, 246, 320, 407, 428, 420, 416, 378, 341, 348, 370, 348, 315, 320, 329, 313, 300, 322, 336, 315, 307, 331, 337, 312, 311, 339, 339, 313, 319, 345, 338, 316, 329, 351, 337, 317, 335, 357, 341, 325, 352, 372, 352, 339, 362, 373, 353, 349, 371, 371, 341, 338, 360, 352, 326, 328, 344, 331, 310, 322, 342, 325, 305, 324, 337, 316, 299, 320, 335, 316, 307, 329, 332, 308, 310, 337, 335, 310, 318, 344, 334, 310, 324, 348, 332, 312, 331, 350, 328, 312, 333, 348, 327, 316, 338, 346, 323, 320, 342, 342, 316, 319, 342, 336, 312, 321, 346, 338, 322, 336, 361, 352, 338, 355, 370, 349, 331, 347, 356, 331, 319, 340, 337, 311, 315, 341, 337, 309, 313, 338, 328, 291, 273, 278, 267, 298, 407, 474, 434, 387, 385, 385, 354, 341, 353, 343, 312, 308, 332, 332, 309, 315, 339, 330, 307, 319, 342, 330, 310, 329, 350, 335, 318, 335, 348, 329, 320, 345, 354, 329, 327, 351, 354, 332, 339, 366, 364, 341, 351, 378, 367, 342, 354, 372, 351, 328, 338, 350, 329, 314, 333, 342, 317, 305, 325, 330, 308, 307, 331, 328, 304, 311, 335, 329, 308, 320, 344, 330, 308, 323, 340, 325, 312, 334, 346, 323, 314, 338, 342, 316, 314, 340, 341, 317, 322, 348, 340, 313, 323, 346, 335, 314, 329, 348, 328, 311, 331, 349, 332, 321, 343, 352, 330, 332, 360, 365, 345, 343, 360, 350, 320, 323, 345, 334, 313, 323, 340, 324, 305, 321, 339, 321, 306, 312, 293, 249, 240, 313, 413, 444, 424, 414, 387, 346, 348, 372, 352, 312, 315, 337, 324, 299, 312, 334, 321, 306, 322, 335, 315, 306, 332, 340, 318, 315, 342, 344, 318, 320, 346, 343, 324, 334, 355, 345, 328, 343, 366, 355, 339, 361, 379, 359, 344, 365, 374, 349, 337, 357, 355, 324, 319, 342, 336, 309, 316, 340, 331, 305, 317, 340, 324, 301, 316, 333, 318, 303, 321, 334, 316, 307, 330, 337, 318, 316, 339, 338, 312, 317, 342, 335, 311, 321, 345, 334, 313, 328, 346, 328, 311, 330, 344, 322, 310, 331, 338, 317, 313, 336, 339, 313, 312, 342, 339, 316, 325, 348, 341, 324, 340, 364, 356, 335, 343, 351, 330, 312, 328, 337, 315, 306, 328, 332, 307, 303, 328, 329, 301, 288, 283, 263, 239, 293, 402, 437, 402, 386, 381, 351, 331, 353, 361, 326, 307, 326, 332, 308, 305, 330, 330, 303, 305, 332, 330, 305, 315, 341, 330, 306, 321, 346, 330, 311, 332, 348, 329, 317, 339, 352, 334, 329, 357, 366, 343, 344, 372, 374, 349, 354, 379, 365, 335, 338, 354, 336, 313, 326, 343, 323, 307, 327, 339, 315, 306, 331, 341, 317, 311, 333, 336, 309, 312, 340, 334, 308, 317, 340, 326, 305, 321, 344, 329, 315, 332, 346, 323, 308, 330, 340, 317, 311, 333, 335, 311, 313, 336, 334, 309, 318, 345, 337, 318, 332, 358, 350, 334, 352, 371, 349, 330, 342, 349, 319, 307, 331, 339, 314, 308, 331, 329, 302, 306, 331, 322, 285, 270, 280, 267, 281, 378, 442, 417, 373, 363, 363, 337, 331, 351, 346, 313, 305, 331, 332, 309, 312, 338, 334, 309, 316, 341, 331, 311, 325, 346, 331, 314, 334, 350, 329, 317, 340, 351, 330, 328, 356, 362, 339, 341, 369, 370, 345, 352, 381, 375, 347, 351, 368, 349, 323, 336, 352, 330, 311, 327, 339, 320, 310, 328, 330, 306, 306, 333, 331, 305, 309, 337, 333, 310, 318, 337, 327, 306, 321, 343, 330, 312, 329, 344, 323, 311, 332, 342, 319, 312, 338, 343, 320, 317, 340, 338, 313, 322, 349, 342, 323, 337, 361, 353, 335, 352, 370, 348, 327, 340, 348, 324, 313, 335, 339, 313, 311, 336, 333, 305, 307, 332, 319, 272, 257, 268, 275, 331, 426, 459, 411, 360, 356, 364, 347, 335, 346, 343, 311, 304, 327, 326, 304, 309, 334, 330, 306, 316, 339, 327, 307, 321, 341, 326, 310, 331, 348, 331, 319, 342, 353, 332, 327, 353, 358, 336, 337, 366, 370, 346, 355, 382, 371, 343, 348, 368, 349, 323, 334, 349, 327, 308, 329, 341, 314, 301, 325, 335, 312, 308, 331, 331, 304, 304, 332, 331, 308, 316, 343, 333, 310, 321, 341, 329, 312, 328, 342, 323, 307, 329, 341, 318, 311, 335, 340, 317, 317, 342, 338, 315, 325, 352, 343, 319, 329, 358, 353, 335, 355, 370, 347, 328, 343, 351, 325, 311, 334, 342, 315, 312, 339, 338, 310, 315, 343, 333, 286, 267, 276, 277, 325, 428, 472, 421, 372, 370, 374, 354, 338, 347, 344, 315, 308, 330, 336, 310, 308, 335, 334, 308, 312, 337, 331, 310, 322, 344, 333, 314, 329, 346, 329, 319, 341, 354, 335, 329, 354, 360, 337, 339, 367, 371, 349, 353, 376, 367, 342, 351, 370, 349, 322, 336, 349, 326, 308, 325, 339, 319, 307, 331, 339, 313, 308, 333, 335, 311, 312, 336, 333, 309, 317, 341, 330, 309, 326, 347, 329, 310, 331, 349, 331, 318, 337, 345, 320, 316, 341, 346, 320, 318, 343, 341, 314, 321, 345, 338, 318, 332, 356, 340, 322, 344, 365, 347, 336, 360, 369, 339, 322, 345, 349, 321, 314, 337, 337, 312, 314, 339, 334, 310, 316, 334, 297, 248, 250, 288, 343, 413, 454, 437, 386, 354, 365, 375, 345, 323, 333, 331, 307, 308, 332, 328, 306, 314, 336, 326, 305, 318, 340, 325, 307, 329, 348, 328, 315, 337, 347, 325, 319, 345, 353, 328, 328, 358, 359, 336, 344, 369, 364, 344, 357, 378, 360, 336, 350, 364, 340, 318, 334, 343, 320, 307, 329, 336, 311, 306, 331, 327, 302, 307, 333, 329, 304, 312, 336, 328, 308, 322, 339, 323, 309, 328, 341, 320, 308, 334, 345, 319, 308, 334, 342, 319, 318, 344, 338, 308, 311, 338, 335, 314, 325, 344, 331, 312, 326, 344, 328, 313, 333, 346, 328, 321, 345, 359, 342, 338, 368, 368, 337, 333, 355, 343, 313, 320, 345, 335, 313, 326, 340, 321, 304, 325, 336, 305, 266, 269, 273, 279, 352, 441, 448, 392, 363, 369, 358, 336, 337, 348, 330, 303, 311, 334, 321, 302, 319, 338, 319, 303, 328, 341, 319, 311, 331, 337, 315, 316, 342, 343, 316, 321, 347, 344, 322, 332, 358, 348, 330, 347, 370, 354, 339, 357, 373, 352, 340, 359, 362, 331, 320, 342, 341, 311, 309, 333, 327, 302, 307, 333, 324, 303, 314, 337, 322, 302, 320, 338, 317, 303, 327, 337, 316, 305, 328, 336, 316, 316, 341, 338, 311, 314, 342, 336, 311, 321, 344, 327, 304, 323, 343, 325, 310, 331, 343, 320, 309, 335, 344, 321, 319, 344, 347, 329, 340, 365, 357, 338, 347, 367, 350, 324, 331, 346, 328, 312, 331, 345, 322, 305, 327, 338, 317, 310, 328, 304, 254, 246, 304, 385, 418, 422, 418, 382, 340, 342, 367, 353, 323, 327, 335, 316, 304, 327, 337, 312, 303, 329, 336, 316, 318, 343, 339, 313, 321, 347, 341, 319, 329, 352, 341, 320, 332, 355, 341, 327, 351, 369, 350, 342, 367, 378, 355, 350, 375, 372, 341, 337, 359, 349, 317, 322, 342, 329, 302, 314, 338, 322, 302, 316, 332, 311, 298, 323, 335, 313, 305, 330, 334, 312, 309, 332, 330, 308, 314, 337, 331, 309, 320, 341, 325, 306, 325, 345, 326, 309, 327, 340, 318, 309, 334, 342, 316, 311, 338, 342, 319, 320, 346, 342, 319, 330, 355, 350, 335, 349, 369, 351, 330, 342, 354, 334, 318, 335, 342, 320, 313, 334, 338, 313, 308, 332, 333, 303, 285, 282, 262, 254, 333, 442, 451, 397, 381, 379, 351, 337, 354, 355, 320, 302, 324, 329, 303, 303, 329, 327, 300, 303, 329, 326, 305, 315, 339, 327, 308, 324, 348, 334, 317, 336, 352, 331, 322, 345, 354, 332, 332, 361, 369, 344, 342, 368, 369, 348, 352, 369, 353, 326, 333, 351, 334, 312, 324, 335, 316, 306, 328, 334, 309, 303, 331, 337, 308, 305, 334, 332, 309, 313, 336, 330, 309, 320, 342, 327, 308, 326, 343, 325, 311, 330, 342, 320, 309, 332, 339, 315, 314, 340, 339, 311, 314, 339, 336, 312, 321, 342, 330, 308, 323, 345, 329, 312, 333, 352, 331, 314, 345, 365, 340, 337, 363, 364, 332, 326, 345, 342, 317, 321, 341, 327, 303, 317, 338, 321, 300, 317, 330, 287, 246, 256, 291, 347, 422, 457, 428, 373, 351, 368, 370, 336, 323, 336, 326, 302, 312, 336, 324, 301, 315, 335, 318, 301, 323, 340, 320, 308, 330, 340, 319, 317, 344, 347, 321, 324, 348, 344, 322, 332, 360, 355, 331, 341, 365, 353, 339, 357, 374, 354, 339, 357, 361, 331, 317, 338, 342, 313, 309, 332, 329, 308, 315, 338, 328, 305, 320, 342, 327, 305, 319, 337, 321, 307, 327, 342, 322, 312, 335, 342, 318, 318, 343, 344, 317, 316, 341, 336, 311, 320, 346, 338, 316, 329, 349, 333, 314, 331, 347, 328, 316, 336, 346, 325, 317, 339, 344, 323, 325, 350, 352, 329, 340, 371, 362, 340, 352, 370, 351, 326, 338, 352, 330, 312, 332, 347, 325, 309, 329, 338, 318, 314, 331, 310, 257, 247, 287, 346, 410, 449, 445, 404, 361, 362, 382, 360, 326, 329, 338, 318, 308, 330, 336, 311, 307, 335, 336, 309, 312, 339, 339, 316, 323, 347, 338, 316, 329, 351, 338, 319, 337, 355, 337, 325, 348, 362, 343, 338, 365, 374, 352, 349, 371, 372, 342, 339, 359, 347, 318, 325, 346, 331, 308, 322, 340, 323, 307, 326, 340, 318, 306, 327, 335, 310, 306, 332, 337, 312, 314, 339, 333, 310, 322, 345, 335, 314, 329, 349, 332, 312, 329, 347, 329, 313, 332, 341, 317, 310, 336, 343, 319, 314, 339, 339, 313, 318, 343, 337, 314, 324, 344, 330, 310, 331, 350, 335, 320, 340, 350, 331, 328, 354, 364, 346, 343, 362, 356, 327, 325, 344, 336, 311, 321, 344, 330, 307, 320, 339, 320, 298, 300, 289, 255, 253, 343, 435, 431, 399, 394, 377, 339, 335, 361, 350, 316, 316, 338, 330, 307, 319, 337, 320, 299, 316, 338, 324, 309, 328, 340, 317, 309, 335, 343, 319, 318, 347, 348, 320, 323, 351, 348, 329, 343, 368, 355, 334, 352, 377, 365, 349, 367, 374, 347, 331, 351, 353, 323, 313, 337, 339, 312, 311, 333, 328, 305, 312, 338, 329, 306, 317, 337, 322, 305, 326, 343, 326, 312, 331, 341, 320, 314, 338, 343, 318, 318, 341, 339, 313, 317, 344, 337, 314, 325, 345, 331, 311, 326, 342, 324, 314, 334, 346, 322, 312, 338, 349, 325, 321, 348, 354, 339, 342, 365, 363, 341, 343, 360, 345, 319, 329, 347, 328, 311, 332, 345, 322, 312, 333, 342, 318, 298, 293, 277, 248, 288, 405, 463, 430, 397, 390, 369, 340, 353, 368, 342, 313, 324, 339, 321, 306, 329, 342, 319, 309, 334, 341, 316, 313, 341, 345, 324, 327, 352, 346, 324, 333, 354, 345, 329, 343, 362, 344, 329, 354, 373, 354, 342, 367, 380, 357, 350, 375, 370, 336, 335, 357, 345, 317, 323, 345, 332, 306, 318, 341, 327, 308, 325, 341, 322, 309, 332, 342, 316, 305, 330, 337, 312, 310, 335, 334, 311, 317, 342, 336, 314, 324, 345, 330, 307, 323, 345, 331, 316, 336, 349, 328, 316, 336, 342, 318, 314, 341, 343, 319, 321, 347, 346, 321, 331, 365, 363, 338, 350, 370, 354, 330, 339, 350, 328, 314, 334, 341, 317, 315, 342, 343, 316, 313, 335, 321, 267, 247, 269, 293, 350, 434, 461, 414, 362, 361, 371, 354, 332, 337, 337, 310, 300, 326, 332, 307, 303, 328, 330, 309, 312, 338, 333, 310, 320, 341, 326, 307, 327, 346, 326, 311, 336, 353, 330, 320, 346, 358, 337, 331, 360, 368, 348, 347, 370, 368, 341, 343, 363, 352, 324, 330, 346, 330, 309, 323, 339, 319, 303, 326, 335, 313, 302, 324, 329, 305, 304, 330, 330, 306, 314, 340, 332, 306, 316, 339, 326, 305, 322, 344, 325, 305, 325, 344, 324, 312, 333, 341, 318, 312, 336, 338, 312, 314, 339, 336, 311, 316, 339, 332, 314, 328, 348, 332, 320, 345, 364, 345, 335, 354, 358, 327, 316, 337, 337, 310, 307, 333, 329, 303, 308, 336, 328, 302, 311, 318, 273, 230, 251, 328, 408, 437, 432, 413, 367, 342, 364, 370, 332, 313, 331, 328, 303, 310, 333, 322, 299, 310, 331, 320, 302, 315, 334, 320, 307, 329, 344, 322, 311, 336, 344, 320, 314, 341, 349, 330, 332, 357, 356, 335, 343, 369, 364, 342, 352, 374, 360, 336, 349, 358, 333, 315, 333, 342, 316, 306, 329, 337, 314, 309, 332, 332, 306, 310, 335, 328, 308, 322, 342, 327, 308, 324, 343, 327, 313, 333, 346, 324, 312, 333, 346, 326, 317, 338, 338, 313, 314, 340, 339, 319, 325, 345, 332, 311, 323, 345, 330, 309, 325, 343, 328, 318, 339, 352, 340, 336, 360, 365, 342, 334, 349, 342, 315, 316, 341, 332, 310, 321, 342, 328, 305, 321, 338, 314, 277, 270, 270, 256, 310, 420, 457, 409, 372, 373, 365, 341, 342, 355, 335, 304, 311, 333, 318, 297, 314, 336, 317, 297, 320, 339, 321, 308, 331, 339, 318, 313, 337, 342, 318, 317, 344, 345, 323, 331, 356, 351, 331, 343, 367, 356, 338, 357, 378, 363, 345, 361, 367, 339, 323, 342, 344, 315, 307, 331, 332, 309, 312, 336, 327, 302, 313, 335, 322, 301, 315, 336, 319, 302, 323, 337, 320, 307, 330, 338, 313, 311, 335, 336, 314, 316, 340, 336, 314, 320, 340, 331, 310, 324, 345, 330, 310, 329, 348, 333, 320, 341, 354, 338, 337, 364, 369, 346, 343, 360, 353, 325, 326, 347, 337, 311, 316, 337, 325, 305, 320, 337, 320, 302, 307, 295, 258, 245, 313, 408, 424, 408, 405, 378, 334, 328, 359, 354, 317, 312, 331, 321, 303, 315, 337, 324, 306, 324, 338, 317, 303, 327, 340, 320, 313, 338, 344, 322, 319, 343, 342, 321, 329, 353, 346, 325, 340, 366, 358, 340, 354, 376, 359, 344, 364, 374, 346, 329, 348, 350, 320, 315, 339, 336, 307, 308, 331, 324, 301, 312, 336, 322, 297, 310, 332, 317, 301, 320, 336, 317, 304, 326, 334, 310, 304, 329, 336, 312, 311, 334, 333, 310, 316, 339, 333, 312, 323, 342, 326, 307, 326, 347, 330, 316, 339, 350, 329, 329, 360, 362, 340, 341, 363, 357, 326, 326, 349, 342, 312, 319, 342, 328, 304, 316, 334, 318, 303, 316, 303, 258, 237, 288, 379, 428, 428, 419, 388, 343, 339, 370, 363, 323, 318, 336, 325, 303, 316, 337, 321, 305, 325, 343, 324, 312, 335, 345, 322, 314, 340, 347, 323, 325, 349, 347, 324, 331, 358, 353, 330, 342, 367, 357, 338, 356, 378, 361, 347, 367, 377, 352, 337, 357, 360, 332, 322, 343, 344, 317, 316, 340, 336, 309, 317, 339, 329, 307, 321, 340, 325, 307, 325, 344, 324, 312, 335, 346, 324, 314, 335, 341, 320, 321, 342, 336, 314, 322, 346, 335, 308, 321, 345, 339, 328, 348, 366, 347, 331, 352, 367, 348, 343, 366, 375, 356, 356, 381, 386, 360, 354, 369, 362, 334, 340, 362, 349, 323, 335, 355, 338, 322, 341, 355, 332, 305, 297, 284, 256, 290, 404, 474, 441, 408, 403, 383, 349, 355, 368, 345, 313, 320, 337, 325, 308, 327, 343, 321, 307, 330, 344, 320, 312, 336, 346, 326, 321, 342, 342, 319, 326, 352, 347, 325, 336, 358, 348, 329, 345, 367, 352, 339, 359, 376, 356, 343, 362, 365, 336, 327, 346, 342, 313, 311, 335, 330, 302, 310, 335, 322, 299, 312, 334, 321, 302, 319, 334, 315, 303, 328, 341, 316, 304, 330, 337, 312, 309, 337, 338, 312, 314, 340, 334, 310, 320, 343, 328, 308, 325, 344, 328, 309, 327, 339, 319, 309, 331, 338, 310, 309, 341, 345, 321, 323, 347, 349, 331, 340, 366, 355, 331, 340, 351, 329, 307, 323, 340, 322, 307, 326, 332, 308, 301, 327, 330, 299, 274, 275, 265, 268, 356, 457, 453, 395, 380, 385, 364, 342, 346, 347, 316, 299, 321, 334, 312, 303, 324, 330, 310, 307, 333, 333, 311, 314, 339, 335, 315, 326, 348, 335, 314, 332, 351, 333, 317, 337, 353, 337, 332, 356, 366, 345, 341, 367, 372, 346, 343, 368, 360, 330, 331, 348, 335, 311, 322, 340, 322, 301, 318, 336, 317, 305, 325, 333, 307, 302, 330, 334, 309, 308, 335, 335, 310, 312, 336, 332, 311, 323, 344, 328, 310, 326, 346, 330, 314, 332, 343, 322, 313, 336, 340, 318, 316, 342, 341, 317, 320, 344, 338, 315, 323, 347, 336, 317, 334, 355, 342, 332, 352, 369, 351, 336, 353, 354, 326, 316, 340, 339, 312, 313, 338, 333, 308, 313, 334, 324, 300, 294, 291, 266, 252, 325, 426, 437, 406, 395, 378, 343, 332, 358, 358, 319, 305, 327, 329, 306, 309, 333, 328, 305, 315, 340, 328, 310, 326, 344, 325, 307, 326, 344, 324, 315, 340, 348, 322, 320, 349, 354, 331, 335, 363, 360, 338, 346, 371, 362, 343, 360, 380, 358, 330, 339, 356, 333, 314, 333, 343, 317, 306, 330, 336, 311, 310, 334, 334, 310, 316, 340, 330, 303, 317, 342, 330, 312, 328, 346, 325, 306, 325, 340, 322, 313, 337, 342, 316, 311, 339, 341, 315, 318, 342, 336, 312, 321, 342, 331, 308, 322, 340, 325, 311, 331, 342, 319, 309, 336, 348, 323, 319, 352, 361, 336, 340, 366, 357, 320, 321, 344, 335, 311, 322, 341, 325, 304, 318, 335, 315, 304, 323, 319, 264, 234, 261, 316, 387, 436, 444, 407, 351, 340, 365, 359, 326, 320, 335, 319, 297, 314, 336, 319, 300, 312, 325, 310, 307, 330, 336, 314, 311, 338, 343, 318, 319, 344, 340, 316, 323, 345, 340, 323, 337, 358, 346, 332, 353, 372, 352, 343, 367, 377, 350, 335, 353, 355, 328, 324, 344, 336, 309, 314, 339, 332, 307, 314, 335, 325, 306, 319, 335, 321, 306, 325, 339, 319, 310, 336, 342, 316, 310, 335, 337, 313, 316, 342, 336, 311, 318, 340, 328, 308, 323, 342, 325, 307, 328, 346, 324, 312, 333, 340, 319, 315, 339, 341, 315, 317, 345, 344, 319, 327, 352, 348, 334, 347, 367, 355, 335, 347, 356, 332, 313, 330, 341, 320, 312, 335, 337, 312, 309, 335, 335, 305, 291, 285, 260, 241, 309, 425, 456, 411, 396, 391, 359, 342, 360, 362, 326, 303, 320, 330, 310, 306, 326, 329, 308, 311, 336, 333, 308, 315, 340, 333, 315, 330, 346, 328, 310, 332, 351, 329, 316, 341, 355, 335, 331, 356, 361, 340, 342, 367, 368, 344, 347, 366, 355, 329, 335, 351, 336, 314, 327, 341, 322, 306, 324, 339, 318, 307, 332, 338, 311, 307, 333, 334, 309, 314, 341, 336, 313, 320, 345, 335, 315, 329, 349, 333, 314, 331, 347, 327, 315, 337, 347, 324, 320, 345, 345, 320, 322, 348, 350, 325, 325, 346, 339, 315, 326, 350, 337, 320, 340, 358, 339, 328, 357, 373, 351, 345, 368, 370, 337, 331, 351, 345, 320, 325, 347, 338, 313, 321, 342, 333, 315, 322, 323, 277, 245, 266, 329, 398, 438, 441, 417, 370, 354, 377, 372, 330, 317, 335, 326, 301, 309, 335, 327, 304, 316, 338, 323, 307, 325, 340, 322, 311, 335, 343, 320, 314, 339, 346, 324, 326, 351, 349, 324, 330, 356, 352, 334, 350, 373, 363, 345, 360, 377, 357, 337, 350, 358, 331, 316, 332, 337, 311, 306, 329, 327, 301, 305, 331, 328, 304, 309, 331, 323, 304, 315, 331, 317, 304, 324, 338, 318, 306, 327, 336, 315, 311, 335, 339, 318, 318, 338, 332, 310, 321, 345, 334, 308, 320, 344, 332, 314, 332, 346, 325, 312, 336, 352, 331, 326, 357, 363, 339, 342, 368, 358, 324, 326, 349, 337, 311, 321, 345, 331, 309, 323, 341, 321, 305, 322, 318, 268, 243, 275, 341, 399, 421, 429, 401, 348, 335, 357, 357, 328, 323, 340, 328, 305, 316, 336, 318, 300, 325, 343, 322, 309, 330, 344, 324, 317, 339, 343, 322, 324, 347, 345, 320, 323, 350, 345, 325, 339, 364, 352, 335, 354, 375, 359, 345, 366, 375, 349, 335, 353, 356, 328, 316, 338, 338, 309, 310, 336, 330, 303, 311, 335, 324, 302, 315, 333, 317, 303, 325, 340, 318, 306, 330, 338, 315, 309, 335, 337, 313, 313, 339, 337, 308, 314, 339, 331, 308, 320, 342, 328, 310, 329, 343, 327, 316, 337, 348, 327, 324, 355, 363, 339, 340, 365, 359, 326, 328, 348, 335, 308, 315, 337, 328, 305, 317, 331, 312, 296, 313, 304, 256, 232, 271, 347, 406, 423, 419, 385, 336, 329, 361, 360, 323, 315, 330, 317, 295, 310, 331, 315, 297, 317, 336, 318, 303, 325, 338, 315, 307, 333, 340, 319, 320, 344, 341, 319, 329, 355, 346, 325, 337, 362, 352, 337, 354, 371, 356, 345, 368, 377, 351, 338, 352, 352, 327, 322, 344, 341, 312, 314, 340, 334, 312, 318, 337, 325, 304, 319, 339, 322, 305, 326, 341, 320, 309, 331, 338, 316, 315, 343, 343, 312, 312, 339, 336, 312, 319, 341, 331, 310, 322, 340, 327, 310, 326, 344, 328, 315, 339, 350, 334, 336, 358, 360, 338, 340, 358, 347, 318, 320, 343, 333, 307, 319, 340, 322, 300, 317, 338, 322, 298, 292, 278, 241, 249, 349, 444, 439, 400, 391, 371, 339, 343, 361, 341, 306, 308, 327, 314, 293, 308, 332, 318, 300, 316, 333, 315, 304, 327, 338, 317, 310, 335, 338, 313, 313, 341, 343, 320, 325, 351, 346, 324, 338, 363, 354, 335, 353, 374, 357, 340, 357, 367, 340, 325, 343, 343, 314, 310, 333, 331, 305, 310, 335, 328, 305, 312, 332, 320, 301, 316, 337, 322, 307, 326, 337, 314, 304, 332, 342, 318, 311, 331, 334, 314, 317, 339, 332, 307, 318, 344, 333, 312, 326, 345, 330, 311, 331, 348, 327, 315, 340, 353, 328, 322, 352, 362, 340, 343, 366, 357, 326, 327, 349, 340, 315, 324, 346, 331, 307, 321, 340, 323, 310, 330, 332, 282, 249, 264, 298, 355, 426, 456, 427, 375, 357, 373, 368, 337, 330, 342, 328, 305, 318, 336, 318, 304, 325, 340, 320, 311, 332, 340, 318, 315, 341, 343, 320, 323, 349, 344, 321, 328, 352, 346, 326, 339, 360, 348, 332, 353, 372, 356, 341, 360, 374, 351, 339, 356, 357, 329, 321, 341, 340, 316, 320, 340, 329, 304, 312, 336, 327, 309, 323, 339, 322, 309, 328, 343, 324, 312, 332, 340, 315, 311, 339, 341, 315, 317, 341, 338, 312, 319, 342, 331, 310, 326, 347, 328, 306, 328, 347, 327, 311, 334, 346, 324, 313, 337, 345, 323, 326, 349, 351, 336, 347, 367, 358, 337, 344, 356, 338, 315, 329, 344, 324, 310, 330, 339, 315, 307, 331, 335, 307, 290, 288, 268, 240, 294, 408, 450, 414, 391, 386, 354, 332, 349, 359, 328, 304, 317, 330, 310, 301, 325, 335, 311, 304, 327, 333, 309, 311, 335, 333, 311, 321, 347, 338, 316, 329, 350, 336, 320, 343, 361, 341, 329, 354, 368, 347, 343, 369, 375, 353, 355, 374, 364, 334, 335, 353, 338, 311, 321, 343, 325, 302, 318, 336, 318, 302, 321, 334, 312, 303, 328, 338, 313, 308, 332, 334, 313, 316, 338, 331, 312, 322, 342, 331, 311, 326, 346, 326, 307, 326, 344, 327, 315, 335, 340, 316, 314, 341, 342, 316, 319, 351, 354, 326, 328, 358, 356, 337, 353, 375, 353, 326, 338, 350, 326, 309, 330, 345, 324, 312, 329, 335, 313, 313, 335, 319, 265, 250, 274, 299, 360, 426, 445, 403, 350, 350, 361, 349, 331, 338, 341, 313, 302, 325, 334, 309, 304, 330, 332, 312, 316, 339, 332, 309, 320, 346, 337, 313, 328, 348, 332, 316, 336, 355, 335, 322, 346, 359, 343, 338, 363, 368, 343, 346, 374, 371, 343, 344, 364, 348, 320, 329, 347, 328, 305, 322, 339, 320, 304, 320, 330, 308, 302, 325, 331, 308, 306, 333, 333, 306, 307, 333, 329, 307, 318, 341, 327, 303, 319, 338, 323, 310, 328, 341, 320, 310, 332, 339, 316, 313, 337, 338, 312, 317, 345, 344, 323, 330, 359, 357, 335, 348, 372, 352, 327, 338, 350, 327, 311, 332, 341, 317, 311, 336, 338, 311, 312, 337, 327, 277, 254, 268, 281, 333, 424, 463, 419, 366, 360, 367, 351, 334, 343, 342, 312, 303, 327, 330, 304, 304, 329, 329, 304, 308, 335, 329, 305, 316, 342, 333, 312, 326, 345, 332, 315, 334, 352, 333, 324, 349, 362, 340, 336, 364, 369, 346, 347, 373, 370, 345, 347, 366, 351, 322, 332, 351, 331, 309, 326, 339, 317, 302, 325, 334, 312, 304, 329, 336, 314, 311, 334, 333, 308, 311, 335, 329, 307, 320, 341, 324, 303, 320, 340, 323, 308, 328, 340, 319, 312, 336, 342, 316, 313, 339, 339, 314, 318, 344, 340, 320, 332, 355, 350, 336, 349, 369, 358, 339, 350, 355, 330, 318, 339, 346, 320, 314, 337, 340, 316, 314, 337, 336, 307, 294, 291, 264, 250, 333, 449, 463, 408, 390, 388, 358, 344, 360, 355, 319, 304, 324, 329, 307, 307, 330, 329, 303, 308, 334, 326, 302, 318, 344, 332, 311, 327, 347, 331, 318, 338, 352, 333, 323, 347, 356, 334, 332, 361, 368, 345, 348, 372, 367, 347, 354, 372, 358, 333, 342, 355, 335, 312, 328, 343, 322, 306, 327, 335, 312, 308, 335, 340, 313, 310, 334, 334, 310, 315, 339, 331, 308, 318, 339, 327, 309, 326, 343, 325, 311, 333, 344, 321, 314, 340, 345, 319, 316, 343, 341, 314, 317, 340, 334, 311, 322, 347, 339, 318, 331, 353, 346, 335, 352, 367, 347, 334, 348, 351, 325, 314, 334, 334, 308, 309, 334, 330, 305, 313, 336, 323, 290, 274, 274, 253, 273, 381, 463, 436, 389, 384, 380, 352, 343, 353, 341, 308, 303, 329, 325, 298, 307, 335, 326, 304, 316, 340, 325, 305, 323, 343, 324, 308, 331, 345, 326, 317, 340, 346, 325, 326, 353, 356, 331, 337, 364, 361, 337, 347, 373, 366, 341, 352, 370, 351, 326, 341, 351, 325, 307, 328, 337, 311, 304, 328, 328, 302, 304, 330, 325, 305, 313, 335, 326, 305, 318, 338, 322, 306, 325, 341, 323, 309, 328, 343, 324, 314, 336, 337, 313, 313, 340, 337, 310, 314, 340, 335, 311, 322, 345, 328, 308, 328, 347, 329, 311, 330, 346, 326, 315, 346, 366, 341, 336, 364, 362, 331, 326, 347, 338, 311, 321, 340, 324, 305, 318, 337, 320, 302, 319, 332, 294, 254, 261, 276, 311, 388, 445, 430, 373, 344, 358, 359, 339, 333, 342, 326, 300, 312, 334, 318, 301, 321, 339, 320, 305, 328, 341, 318, 309, 335, 342, 317, 316, 343, 344, 321, 326, 351, 347, 326, 337, 365, 356, 336, 351, 370, 356, 344, 365, 380, 357, 341, 361, 366, 336, 326, 344, 340, 313, 314, 338, 331, 308, 319, 340, 326, 302, 313, 336, 324, 307, 325, 339, 321, 308, 329, 338, 318, 312, 334, 339, 314, 315, 341, 336, 308, 314, 341, 332, 310, 322, 342, 328, 309, 327, 348, 330, 313, 333, 343, 318, 313, 344, 354, 334, 338, 363, 364, 341, 342, 363, 352, 323, 329, 349, 333, 308, 319, 334, 320, 305, 327, 339, 317, 303, 311, 294, 254, 250, 327, 415, 420, 404, 405, 377, 335, 335, 359, 342, 310, 316, 335, 322, 306, 322, 334, 313, 300, 322, 334, 312, 307, 336, 341, 313, 314, 339, 338, 315, 323, 350, 342, 319, 331, 351, 341, 327, 345, 365, 349, 335, 357, 371, 353, 347, 369, 370, 340, 330, 351, 349, 318, 316, 339, 331, 303, 311, 332, 321, 302, 316, 332, 314, 299, 320, 336, 317, 307, 329, 334, 310, 307, 332, 334, 309, 311, 336, 334, 314, 320, 341, 330, 308, 320, 340, 324, 306, 324, 342, 325, 313, 332, 337, 318, 315, 341, 349, 323, 325, 359, 359, 336, 345, 367, 352, 321, 328, 349, 333, 309, 325, 345, 325, 304, 322, 337, 313, 303, 322, 309, 257, 238, 274, 342, 404, 434, 435, 401, 350, 344, 370, 360, 325, 325, 336, 318, 302, 324, 338, 314, 302, 326, 338, 317, 312, 341, 346, 321, 322, 345, 342, 320, 326, 349, 340, 320, 336, 358, 346, 328, 349, 367, 350, 338, 362, 377, 357, 350, 378, 383, 349, 337, 358, 355, 325, 322, 342, 334, 311, 318, 339, 328, 311, 326, 342, 322, 306, 325, 339, 318, 310, 336, 345, 321, 316, 339, 339, 317, 320, 344, 337, 314, 323, 347, 337, 314, 329, 349, 333, 316, 336, 349, 328, 315, 338, 349, 324, 317, 342, 347, 324, 328, 354, 354, 336, 346, 374, 367, 340, 341, 355, 339, 320, 334, 346, 325, 311, 331, 341, 318, 310, 334, 338, 310, 289, 284, 267, 251, 316, 435, 468, 415, 388, 387, 364, 348, 358, 361, 327, 302, 320, 336, 314, 301, 322, 332, 312, 308, 330, 333, 307, 310, 340, 337, 314, 322, 344, 334, 312, 326, 351, 337, 317, 334, 354, 339, 329, 352, 364, 344, 339, 364, 370, 346, 346, 366, 358, 333, 333, 352, 340, 313, 320, 340, 327, 305, 320, 336, 316, 302, 323, 333, 313, 308, 334, 341, 317, 312, 338, 341, 316, 316, 341, 335, 312, 323, 345, 335, 317, 336, 356, 338, 318, 335, 347, 327, 316, 337, 346, 322, 318, 344, 344, 319, 323, 351, 351, 331, 337, 361, 356, 335, 351, 375, 356, 328, 339, 353, 332, 319, 339, 346, 321, 308, 326, 336, 317, 316, 337, 329, 282, 262, 274, 283, 331, 423, 466, 423, 368, 361, 371, 357, 336, 338, 339, 313, 304, 329, 336, 310, 305, 329, 331, 307, 310, 338, 329, 301, 315, 343, 333, 312, 324, 348, 338, 320, 334, 349, 332, 324, 347, 360, 340, 337, 366, 371, 349, 350, 378, 377, 348, 349, 366, 350, 320, 329, 350, 332, 308, 322, 336, 315, 299, 320, 333, 313, 306, 328, 331, 311, 310, 333, 333, 308, 312, 337, 329, 306, 315, 335, 327, 309, 322, 339, 323, 310, 328, 339, 319, 311, 337, 339, 311, 308, 333, 336, 312, 317, 344, 344, 322, 331, 353, 347, 331, 346, 369, 356, 333, 344, 353, 329, 312, 331, 340, 314, 308, 337, 342, 312, 306, 330, 330, 299, 279, 276, 257, 258, 354, 442, 430, 384, 371, 366, 338, 327, 348, 350, 320, 304, 329, 335, 307, 304, 330, 331, 309, 313, 336, 333, 314, 323, 344, 335, 315, 331, 350, 332, 314, 336, 353, 333, 320, 345, 359, 337, 334, 362, 367, 346, 349, 375, 370, 346, 350, 369, 357, 330, 336, 353, 332, 309, 324, 339, 318, 305, 327, 337, 313, 304, 325, 329, 308, 311, 338, 335, 305, 309, 335, 331, 310, 320, 340, 330, 309, 325, 342, 323, 306, 326, 345, 327, 314, 333, 335, 309, 309, 338, 340, 318, 321, 350, 347, 325, 337, 365, 357, 337, 352, 370, 350, 327, 341, 349, 321, 310, 334, 342, 317, 312, 339, 339, 311, 306, 328, 308, 258, 250, 282, 335, 399, 437, 432, 383, 339, 347, 368, 351, 328, 332, 333, 309, 301, 326, 331, 305, 303, 330, 330, 305, 311, 338, 331, 310, 324, 346, 330, 311, 331, 350, 336, 322, 341, 352, 333, 329, 357, 368, 345, 344, 367, 369, 346, 352, 377, 371, 343, 345, 364, 349, 320, 328, 342, 323, 304, 321, 334, 313, 303, 326, 334, 308, 300, 326, 330, 306, 308, 334, 330, 304, 312, 335, 325, 305, 319, 341, 328, 311, 329, 342, 320, 306, 326, 341, 320, 309, 335, 340, 317, 313, 337, 338, 319, 327, 352, 347, 329, 341, 366, 355, 329, 337, 349, 327, 307, 324, 340, 317, 306, 329, 336, 310, 301, 325, 329, 295, 270, 273, 258, 265, 360, 457, 443, 382, 368, 376, 356, 338, 344, 343, 313, 296, 319, 328, 304, 298, 326, 331, 306, 305, 331, 328, 303, 307, 333, 328, 311, 325, 347, 332, 310, 330, 349, 332, 321, 345, 355, 331, 323, 349, 361, 340, 338, 366, 369, 342, 341, 360, 353, 327, 329, 342, 324, 304, 320, 338, 319, 303, 321, 334, 311, 304, 329, 336, 312, 308, 335, 336, 309, 313, 342, 335, 310, 320, 345, 335, 310, 321, 344, 330, 313, 331, 345, 324, 313, 337, 344, 319, 314, 343, 348, 323, 326, 354, 355, 337, 342, 363, 360, 339, 342, 353, 336, 314, 331, 345, 321, 308, 331, 342, 319, 306, 328, 334, 307, 287, 288, 272, 245, 297, 415, 453, 404, 379, 379, 356, 335, 348, 357, 328, 301, 314, 330, 311, 300, 323, 332, 307, 299, 325, 331, 306, 303, 331, 334, 313, 322, 346, 333, 310, 326, 348, 334, 316, 338, 359, 344, 335, 361, 373, 351, 345, 372, 379, 352, 349, 374, 365, 330, 327, 346, 335, 308, 316, 335, 319, 302, 317, 333, 314, 298, 318, 331, 307, 297, 322, 331, 306, 303, 332, 335, 310, 315, 341, 333, 309, 319, 342, 331, 309, 323, 343, 327, 311, 329, 342, 322, 313, 337, 345, 321, 319, 348, 354, 331, 333, 363, 363, 333, 333, 351, 337, 308, 322, 342, 323, 304, 321, 338, 318, 303, 323, 337, 311, 276, 275, 273, 274, 350, 438, 439, 388, 364, 363, 345, 324, 339, 358, 333, 304, 317, 336, 321, 306, 323, 332, 310, 303, 331, 339, 314, 314, 343, 347, 320, 319, 344, 342, 318, 329, 354, 344, 323, 343, 369, 352, 337, 358, 379, 362, 350, 372, 383, 358, 347, 365, 365, 334, 328, 350, 343, 315, 320, 345, 336, 313, 322, 345, 331, 309, 326, 344, 325, 307, 328, 344, 322, 314, 336, 341, 317, 317, 343, 342, 315, 318, 343, 336, 313, 324, 348, 333, 311, 326, 342, 327, 321, 338, 345, 334, 335, 360, 367, 346, 344, 359, 357, 334, 330, 341, 334, 315, 321, 336, 328, 313, 321, 332, 324, 314, 321, 309, 269, 251, 277, 344, 410, 426, 414, 384, 343, 336, 354, 357, 335, 328, 334, 324, 309, 320, 335, 321, 307, 319, 332, 322, 315, 333, 342, 327, 321, 338, 346, 330, 327, 342, 343, 334, 341, 355, 347, 336, 348, 365, 359, 349, 363, 372, 363, 357, 370, 371, 352, 343, 352, 350, 329, 320, 329, 329, 317, 319, 332, 326, 312, 317, 329, 322, 309, 318, 330, 320, 310, 323, 333, 319, 311, 326, 332, 318, 313, 328, 330, 316, 316, 332, 332, 318, 324, 339, 336, 322, 327, 342, 336, 330, 348, 359, 349, 344, 354, 352, 332, 320, 329, 331, 318, 316, 331, 328, 309, 311, 326, 319, 305, 304, 293, 264, 243, 279, 373, 437, 427, 400, 380, 351, 340, 355, 350, 323, 312, 322, 323, 310, 313, 329, 328, 310, 313, 328, 320, 308, 317, 330, 323, 315, 326, 334, 325, 321, 335, 341, 329, 330, 345, 350, 338, 340, 359, 360, 349, 355, 370, 366, 354, 361, 367, 355, 338, 340, 346, 331, 319, 329, 335, 321, 317, 330, 330, 318, 323, 339, 334, 316, 318, 332, 326, 314, 322, 334, 325, 315, 324, 334, 326, 320, 330, 336, 326, 322, 337, 343, 327, 323, 338, 340, 330, 336, 351, 348, 342, 353, 365, 361, 348, 354, 360, 341, 326, 332, 338, 325, 318, 332, 337, 321, 316, 332, 333, 315, 303, 292, 273, 258, 303, 407, 461, 429, 396, 382, 361, 351, 363, 359, 329, 313, 323, 329, 318, 313, 326, 327, 312, 313, 331, 332, 322, 325, 337, 333, 323, 329, 343, 335, 325, 339, 353, 343, 332, 344, 354, 346, 343, 357, 365, 356, 357, 372, 372, 360, 360, 369, 362, 341, 340, 349, 339, 325, 330, 337, 327, 314, 325, 335, 321, 315, 328, 331, 320, 318, 331, 332, 318, 320, 335, 331, 319, 325, 339, 333, 320, 325, 338, 331, 318, 328, 338, 328, 320, 328, 329, 316, 314, 328, 338, 333, 336, 352, 356, 346, 351, 365, 354, 333, 333, 341, 332, 321, 327, 334, 324, 316, 326, 330, 317, 311, 323, 319, 284, 258, 261, 289, 357, 431, 446, 411, 373, 361, 365, 358, 339, 331, 329, 319, 310, 317, 325, 316, 311, 323, 330, 318, 316, 333, 335, 320, 321, 334, 332, 319, 324, 337, 332, 324, 331, 345, 343, 332, 341, 350, 347, 345, 356, 363, 355, 353, 365, 367, 353, 348, 351, 339, 322, 324, 334, 328, 313, 317, 327, 320, 313, 320, 324, 313, 306, 316, 323, 313, 310, 323, 328, 316, 313, 322, 323, 314, 319, 331, 327, 315, 319, 333, 331, 317, 322, 330, 324, 319, 330, 335, 321, 318, 335, 341, 330, 325, 338, 348, 342, 342, 358, 359, 343, 336, 341, 335, 320, 323, 332, 323, 314, 320, 326, 316, 311, 323, 324, 294, 259, 256, 260, 300, 395, 460, 432, 379, 359, 361, 361, 349, 338, 332, 321, 309, 320, 328, 317, 309, 323, 333, 322, 314, 327, 335, 325, 323, 337, 338, 323, 327, 342, 339, 329, 337, 349, 344, 330, 340, 357, 352, 343, 354, 365, 357, 354, 368, 374, 361, 355, 361, 354, 337, 336, 344, 339, 324, 324, 335, 329, 313, 319, 333, 327, 317, 324, 333, 324, 319, 334, 339, 325, 321, 333, 337, 326, 325, 340, 341, 325, 324, 340, 337, 325, 331, 345, 336, 323, 330, 342, 336, 328, 340, 350, 344, 342, 357, 365, 353, 344, 349, 346, 330, 329, 339, 332, 318, 323, 334, 326, 314, 322, 332, 321, 294, 272, 265, 258, 302, 406, 457, 421, 385, 377, 368, 356, 351, 347, 331, 310, 311, 323, 318, 308, 313, 325, 319, 308, 318, 330, 321, 309, 320, 331, 322, 319, 333, 337, 325, 324, 339, 339, 327, 332, 345, 341, 329, 339, 357, 352, 342, 351, 364, 355, 342, 348, 351, 335, 323, 331, 334, 320, 316, 329, 329, 313, 311, 324, 321, 308, 310, 324, 321, 308, 315, 328, 320, 313, 326, 333, 322, 315, 326, 335, 325, 319, 328, 330, 318, 320, 332, 330, 318, 322, 337, 335, 327, 333, 347, 345, 336, 346, 359, 350, 337, 337, 334, 320, 313, 326, 331, 316, 313, 323, 319, 304, 310, 324, 314, 282, 259, 257, 261, 314, 410, 448, 412, 372, 361, 358, 351, 339, 335, 328, 310, 303, 315, 314, 301, 307, 321, 318, 305, 312, 325, 320, 312, 322, 331, 321, 314, 328, 340, 330, 318, 332, 341, 331, 329, 344, 348, 337, 335, 350, 353, 343, 348, 361, 357, 343, 346, 353, 339, 324, 330, 334, 323, 314, 320, 325, 315, 308, 322, 327, 314, 311, 327, 332, 319, 320, 332, 330, 318, 323, 337, 329, 315, 323, 335, 326, 318, 327, 335, 322, 316, 330, 337, 321, 316, 331, 336, 325, 328, 345, 349, 340, 344, 362, 355, 337, 338, 345, 332, 320, 330, 338, 324, 314, 323, 328, 318, 316, 329, 325, 293, 265, 264, 266, 303, 397, 456, 432, 382, 357, 356, 352, 339, 336, 333, 315, 305, 316, 322, 309, 303, 317, 321, 306, 307, 326, 327, 313, 315, 331, 328, 317, 326, 342, 330, 313, 331, 352, 334, 317, 337, 353, 337, 326, 350, 363, 342, 337, 361, 367, 341, 337, 360, 351, 318, 321, 344, 333, 304, 314, 334, 320, 303, 320, 339, 318, 300, 321, 332, 310, 304, 333, 338, 309, 305, 332, 335, 309, 313, 342, 339, 314, 321, 343, 330, 307, 321, 344, 328, 312, 333, 349, 331, 322, 353, 364, 340, 335, 362, 364, 335, 328, 346, 340, 316, 322, 344, 329, 305, 320, 340, 322, 303, 319, 328, 282, 252, 270, 307, 359, 412, 441, 423, 360, 333, 352, 357, 336, 332, 346, 329, 301, 313, 337, 326, 305, 321, 340, 323, 308, 329, 343, 323, 312, 337, 346, 325, 322, 343, 342, 323, 331, 354, 348, 326, 340, 366, 353, 334, 355, 380, 362, 344, 365, 379, 352, 336, 358, 361, 332, 322, 343, 344, 315, 314, 341, 337, 311, 318, 342, 330, 309, 325, 348, 331, 304, 322, 344, 325, 312, 335, 347, 323, 313, 339, 345, 320, 316, 338, 338, 314, 318, 342, 339, 319, 334, 357, 340, 321, 347, 372, 356, 343, 361, 367, 335, 321, 344, 344, 314, 310, 336, 336, 307, 312, 341, 334, 305, 304, 301, 264, 234, 272, 373, 430, 420, 408, 388, 345, 325, 354, 365, 332, 313, 328, 328, 303, 306, 331, 327, 302, 310, 335, 323, 303, 319, 344, 331, 312, 331, 346, 325, 316, 343, 357, 333, 324, 349, 355, 332, 332, 359, 361, 336, 343, 371, 365, 341, 351, 372, 356, 332, 343, 356, 333, 316, 332, 344, 323, 311, 333, 339, 311, 309, 337, 338, 312, 316, 342, 335, 309, 319, 340, 328, 307, 323, 344, 328, 312, 332, 345, 325, 315, 339, 344, 317, 311, 340, 348, 326, 327, 354, 358, 339, 349, 374, 362, 333, 340, 355, 337, 314, 329, 346, 322, 307, 329, 340, 319, 312, 332, 329, 282, 256, 269, 282, 326, 418, 469, 432, 369, 357, 372, 362, 338, 343, 346, 320, 304, 325, 338, 318, 309, 332, 339, 314, 310, 333, 334, 312, 319, 344, 334, 310, 323, 350, 339, 317, 334, 355, 340, 322, 342, 357, 341, 331, 355, 366, 344, 342, 370, 370, 344, 341, 361, 352, 324, 326, 345, 333, 307, 318, 339, 322, 303, 319, 336, 319, 307, 327, 335, 314, 309, 336, 341, 316, 315, 338, 335, 307, 314, 342, 335, 313, 325, 348, 334, 311, 327, 344, 328, 316, 340, 356, 335, 328, 357, 365, 343, 343, 365, 357, 326, 323, 343, 336, 313, 325, 350, 334, 305, 319, 340, 324, 306, 314, 300, 254, 236, 301, 403, 441, 423, 412, 384, 344, 342, 369, 356, 316, 312, 336, 327, 305, 313, 336, 322, 302, 321, 336, 317, 306, 333, 347, 323, 315, 339, 345, 321, 323, 353, 349, 322, 329, 354, 347, 325, 340, 366, 356, 337, 353, 372, 355, 341, 362, 370, 343, 329, 348, 352, 325, 322, 343, 336, 309, 315, 338, 328, 308, 323, 342, 326, 302, 319, 343, 329, 309, 325, 342, 326, 313, 335, 341, 319, 317, 342, 344, 319, 321, 346, 342, 317, 324, 345, 337, 322, 339, 356, 339, 325, 352, 370, 352, 339, 360, 361, 329, 317, 338, 343, 319, 316, 340, 338, 312, 315, 340, 330, 298, 292, 291, 261, 240, 309, 423, 450, 411, 396, 384, 352, 345, 366, 356, 314, 304, 329, 331, 307, 308, 334, 330, 306, 315, 340, 329, 304, 316, 338, 324, 309, 333, 345, 320, 312, 344, 352, 323, 317, 350, 356, 330, 332, 359, 360, 338, 350, 375, 363, 339, 349, 365, 342, 319, 335, 348, 320, 303, 326, 334, 307, 299, 324, 330, 307, 311, 334, 328, 300, 303, 330, 323, 302, 317, 338, 322, 303, 322, 342, 323, 309, 333, 343, 316, 304, 329, 337, 316, 312, 336, 341, 322, 324, 351, 350, 334, 344, 362, 353, 331, 340, 356, 337, 313, 326, 339, 319, 310, 333, 340, 318, 309, 330, 328, 295, 268, 272, 265, 276, 371, 462, 452, 392, 368, 370, 349, 336, 347, 348, 319, 303, 324, 338, 312, 305, 331, 336, 310, 307, 334, 332, 307, 314, 340, 334, 312, 326, 349, 338, 318, 336, 355, 337, 324, 348, 363, 338, 325, 351, 362, 338, 336, 362, 365, 341, 341, 364, 354, 326, 334, 352, 333, 309, 324, 341, 321, 305, 327, 335, 311, 306, 335, 345, 316, 308, 334, 338, 315, 318, 345, 343, 314, 320, 345, 336, 314, 327, 345, 331, 311, 329, 347, 325, 307, 331, 348, 327, 322, 346, 352, 332, 332, 355, 354, 331, 334, 352, 337, 312, 324, 349, 336, 313, 324, 340, 322, 304, 325, 341, 301, 256, 259, 269, 295, 384, 462, 448, 392, 371, 383, 378, 348, 336, 343, 326, 304, 315, 333, 318, 304, 325, 341, 318, 304, 332, 343, 318, 316, 338, 342, 319, 323, 351, 347, 321, 331, 360, 351, 326, 340, 359, 343, 326, 345, 365, 349, 340, 361, 377, 352, 337, 357, 356, 322, 314, 336, 333, 304, 306, 330, 325, 302, 314, 334, 319, 299, 316, 336, 319, 301, 319, 335, 316, 305, 328, 331, 305, 302, 329, 332, 312, 316, 338, 332, 305, 310, 338, 330, 308, 326, 346, 330, 315, 336, 355, 339, 328, 352, 366, 340, 327, 347, 347, 319, 316, 338, 334, 311, 317, 340, 326, 304, 318, 337, 300, 251, 257, 283, 321, 397, 454, 439, 380, 354, 376, 383, 349, 328, 340, 333, 304, 308, 336, 326, 306, 315, 335, 323, 306, 323, 338, 322, 314, 338, 349, 326, 316, 340, 345, 323, 326, 350, 348, 326, 330, 353, 353, 337, 343, 362, 352, 335, 351, 369, 353, 337, 354, 355, 329, 315, 340, 346, 319, 314, 334, 336, 317, 322, 342, 330, 304, 316, 346, 338, 317, 328, 344, 326, 309, 326, 340, 323, 318, 341, 348, 320, 309, 334, 339, 316, 316, 340, 336, 316, 324, 351, 341, 316, 331, 359, 352, 342, 365, 383, 354, 322, 335, 345, 324, 320, 347, 353, 330, 324, 341, 335, 304, 295, 292, 262, 230, 260, 359, 412, 408, 409, 404, 367, 343, 364, 374, 336, 312, 334, 344, 319, 314, 336, 337, 315, 316, 336, 330, 312, 325, 350, 341, 326, 343, 354, 331, 325, 349, 353, 332, 328, 350, 359, 336, 328, 355, 366, 355, 358, 379, 373, 347, 354, 379, 373, 348, 360, 376, 348, 323, 331, 338, 319, 305, 325, 338, 314, 302, 320, 322, 302, 301, 327, 331, 306, 308, 326, 321, 307, 316, 334, 326, 311, 324, 337, 317, 312, 342, 348, 322, 318, 349, 361, 332, 322, 352, 361, 342, 349, 377, 370, 339, 342, 367, 355, 325, 326, 342, 333, 315, 321, 328, 311, 305, 324, 309, 259, 241, 280, 350, 413, 442, 436, 400, 358, 355, 377, 363, 323, 315, 329, 316, 301, 318, 332, 314, 301, 327, 340, 317, 309, 333, 337, 312, 313, 338, 338, 312, 313, 339, 333, 308, 315, 334, 324, 310, 330, 351, 331, 312, 338, 357, 336, 327, 352, 359, 334, 325, 345, 343, 322, 321, 343, 336, 312, 320, 345, 332, 309, 331, 354, 337, 320, 339, 358, 343, 338, 360, 370, 355, 349, 373, 379, 365, 354, 365, 359, 337, 347, 372, 361, 340, 350, 370, 355, 331, 346, 362, 344, 326, 343, 352, 328, 317, 339, 336, 304, 297, 328, 331, 292, 284, 309, 310, 290, 295, 308, 269, 226, 238, 300, 370, 412, 429, 415, 368, 341, 359, 367, 332, 312, 327, 323, 292, 288, 312, 313, 291, 300, 322, 308, 291, 309, 328, 311, 296, 319, 339, 322, 311, 333, 342, 321, 320, 348, 357, 331, 325, 353, 362, 341, 347, 365, 355, 335, 351, 372, 355, 331, 344, 360, 336, 315, 331, 336, 315, 312, 328, 326, 306, 313, 342, 337, 313, 319, 348, 346, 318, 324, 340, 326, 309, 327, 347, 333, 320, 341, 350, 325, 317, 346, 353, 329, 325, 351, 364, 348, 356, 383, 384, 369, 385, 414, 403, 380, 384, 390, 364, 335, 339, 344, 321, 306, 325, 334, 315, 312, 335, 328, 277, 259, 282, 306, 352, 416, 443, 407, 350, 338, 346, 324, 282, 280, 291, 266, 251, 276, 290, 268, 261, 293, 306, 282, 280, 302, 301, 283, 297, 322, 311, 289, 297, 318, 307, 289, 318, 350, 344, 335, 359, 371, 351, 345, 370, 379, 355, 356, 379, 377, 342, 336, 361, 352, 319, 318, 338, 325, 299, 311, 331, 310, 291, 314, 329, 309, 296, 313, 317, 293, 287, 313, 309, 279, 280, 312, 317, 295, 298, 324, 321, 303, 322, 342, 327, 311, 336, 351, 331, 325, 350, 365, 356, 352, 370, 371, 343, 331, 346, 338, 306, 310, 339, 333, 311, 331, 363, 354, 338, 353, 364, 328, 273, 261, 265, 285, 363, 437, 441, 396, 383, 425, 474, 424, 373, 367, 351, 329, 339, 369, 363, 335, 334, 316, 277, 260, 287, 310, 299, 295, 326, 335, 306, 306, 336, 335, 315, 323, 349, 341, 309, 317, 344, 327, 306, 321, 350, 356, 355, 368, 375, 356, 340, 348, 352, 323, 309, 324, 327, 304, 298, 317, 316, 295, 308, 338, 328, 300, 316, 338, 315, 295, 315, 342, 326, 315, 333, 342, 323, 310, 337, 350, 321, 314, 338, 339, 313, 324, 348, 336, 312, 320, 350, 349, 329, 341, 371, 357, 331, 341, 349, 327, 312, 332, 348, 325, 310, 337, 337, 307, 310, 341, 329, 274, 264, 281, 290, 352, 430, 465, 435, 385, 369, 376, 362, 330, 328, 331, 314, 311, 327, 326, 310, 315, 342, 338, 308, 312, 339, 334, 317, 323, 345, 337, 323, 338, 349, 333, 331, 354, 357, 336, 333, 363, 376, 350, 348, 375, 376, 354, 356, 384, 380, 350, 349, 365, 355, 329, 328, 341, 326, 311, 330, 346, 322, 308, 337, 347, 316, 306, 333, 336, 313, 326, 353, 340, 320, 331, 352, 345, 321, 329, 352, 337, 318, 337, 350, 328, 323, 360, 372, 339, 328, 361, 375, 359, 357, 370, 360, 336, 341, 357, 337, 314, 325, 346, 337, 322, 338, 348, 323, 298, 292, 279, 251, 292, 413, 472, 442, 400, 395, 374, 339, 346, 362, 339, 306, 315, 333, 319, 304, 318, 335, 318, 303, 328, 343, 321, 310, 336, 346, 324, 323, 351, 348, 317, 323, 350, 342, 321, 331, 355, 347, 329, 344, 365, 352, 339, 361, 378, 355, 340, 365, 370, 337, 320, 338, 337, 311, 310, 337, 334, 308, 312, 339, 330, 305, 316, 335, 324, 309, 331, 345, 320, 305, 329, 343, 319, 309, 334, 340, 317, 317, 343, 340, 315, 323, 353, 348, 323, 340, 369, 356, 332, 348, 363, 333, 312, 332, 340, 311, 297, 322, 332, 307, 303, 327, 326, 290, 264, 264, 251, 249, 336, 445, 448, 388, 368, 370, 346, 332, 347, 346, 312, 297, 318, 323, 300, 297, 322, 324, 298, 299, 326, 323, 298, 307, 332, 322, 299, 317, 343, 326, 310, 333, 349, 326, 310, 336, 348, 326, 317, 345, 354, 331, 331, 360, 359, 331, 332, 355, 345, 315, 321, 339, 325, 304, 320, 338, 317, 300, 323, 336, 312, 302, 327, 335, 313, 310, 334, 334, 309, 315, 343, 333, 304, 316, 342, 331, 307, 325, 347, 329, 313, 336, 355, 339, 333, 358, 369, 346, 335, 355, 354, 323, 321, 344, 341, 312, 318, 341, 329, 306, 320, 339, 312, 264, 258, 269, 281, 355, 453, 466, 399, 355, 368, 368, 345, 338, 346, 329, 301, 313, 340, 327, 300, 311, 333, 316, 297, 314, 334, 318, 304, 326, 340, 318, 313, 340, 348, 326, 325, 348, 343, 318, 327, 359, 353, 326, 336, 364, 351, 329, 346, 366, 342, 318, 339, 352, 328, 311, 333, 341, 314, 306, 333, 338, 313, 313, 339, 332, 307, 318, 345, 336, 311, 326, 350, 333, 315, 338, 357, 335, 316, 336, 346, 326, 327, 358, 363, 333, 336, 371, 373, 347, 354, 376, 360, 326, 335, 353, 329, 302, 319, 340, 322, 306, 326, 334, 310, 296, 295, 274, 239, 259, 357, 438, 432, 407, 395, 362, 331, 342, 368, 344, 313, 325, 341, 321, 306, 329, 346, 323, 308, 330, 343, 321, 320, 345, 348, 324, 326, 350, 344, 318, 329, 357, 346, 325, 339, 356, 339, 323, 346, 368, 348, 336, 360, 371, 349, 343, 363, 362, 331, 325, 347, 344, 318, 327, 348, 332, 306, 323, 349, 339, 317, 328, 343, 324, 313, 337, 349, 324, 316, 344, 350, 323, 323, 355, 355, 331, 338, 361, 351, 338, 365, 386, 359, 331, 349, 368, 345, 317, 331, 342, 317, 310, 337, 342, 313, 308, 337, 339, 311, 295, 290, 274, 261, 330, 443, 464, 419, 400, 395, 363, 338, 363, 378, 342, 318, 336, 343, 321, 316, 341, 342, 315, 319, 344, 338, 311, 320, 350, 341, 315, 329, 352, 338, 320, 339, 358, 339, 331, 356, 361, 328, 318, 353, 369, 343, 338, 362, 360, 334, 338, 359, 341, 308, 318, 336, 319, 296, 316, 334, 314, 300, 327, 345, 318, 299, 321, 330, 311, 313, 339, 340, 312, 316, 343, 336, 310, 321, 350, 342, 322, 335, 353, 339, 329, 353, 369, 349, 337, 354, 353, 322, 312, 335, 336, 308, 307, 330, 324, 302, 310, 331, 307, 256, 250, 270, 289, 354, 452, 472, 408, 356, 357, 358, 337, 329, 342, 335, 306, 306, 332, 331, 307, 314, 339, 329, 307, 323, 345, 331, 310, 328, 347, 330, 316, 339, 351, 330, 324, 348, 352, 328, 325, 355, 356, 330, 334, 360, 356, 333, 343, 365, 352, 326, 336, 351, 328, 306, 325, 335, 312, 301, 324, 332, 308, 303, 330, 331, 305, 306, 334, 329, 303, 311, 334, 324, 302, 315, 336, 318, 300, 323, 343, 323, 312, 338, 356, 339, 328, 353, 361, 333, 319, 336, 331, 299, 302, 330, 319, 292, 304, 326, 310, 289, 310, 309, 254, 219, 249, 321, 408, 469, 469, 426, 365, 349, 378, 367, 320, 310, 328, 320, 296, 309, 330, 311, 288, 308, 328, 305, 286, 311, 326, 308, 307, 343, 359, 337, 336, 371, 373, 348, 352, 379, 372, 343, 347, 373, 363, 339, 349, 367, 347, 326, 345, 362, 330, 307, 326, 330, 300, 288, 309, 306, 272, 274, 304, 298, 265, 275, 311, 300, 278, 295, 323, 315, 299, 319, 336, 327, 322, 338, 354, 340, 331, 343, 352, 341, 340, 357, 366, 360, 357, 369, 364, 340, 334, 338, 329, 309, 307, 316, 303, 288, 298, 305, 291, 285, 300, 294, 242, 213, 230, 278, 360, 436, 455, 430, 393, 385, 397, 400, 387, 384, 392, 383, 370, 381, 399, 381, 363, 368, 371, 354, 340, 350, 352, 328, 315, 324, 317, 291, 288, 307, 305, 281, 283, 303, 297, 282, 297, 311, 301, 292, 306, 322, 311, 305, 320, 326, 308, 299, 316, 320, 306, 308, 323, 322, 313, 312, 329, 328, 316, 322, 333, 326, 317, 340, 358, 339, 329, 343, 348, 333, 331, 352, 353, 331, 329, 348, 354, 338, 345, 364, 354, 336, 354, 379, 361, 345, 363, 376, 348, 330, 349, 353, 326, 322, 349, 354, 325, 317, 338, 333, 304, 294, 295, 270, 257, 337, 431, 434, 397, 391, 385, 356, 349, 372, 370, 330, 314, 341, 350, 323, 318, 342, 343, 323, 325, 346, 336, 314, 324, 346, 334, 314, 330, 343, 313, 292, 315, 332, 309, 304, 335, 348, 328, 332, 365, 370, 345, 347, 372, 361, 331, 338, 356, 339, 315, 326, 341, 318, 299, 320, 337, 313, 303, 328, 337, 310, 301, 328, 330, 302, 303, 335, 334, 309, 315, 337, 326, 307, 323, 344, 323, 305, 327, 344, 326, 321, 344, 356, 337, 333, 357, 361, 331, 327, 353, 348, 319, 322, 344, 333, 311, 322, 343, 327, 304, 318, 320, 273, 247, 274, 328, 385, 416, 431, 411, 355, 335, 355, 354, 320, 316, 336, 325, 295, 302, 329, 319, 297, 314, 334, 318, 303, 324, 341, 324, 314, 336, 345, 320, 319, 345, 345, 322, 329, 358, 355, 328, 338, 365, 356, 339, 361, 385, 361, 337, 358, 372, 342, 324, 342, 347, 320, 313, 336, 337, 309, 306, 328, 324, 300, 312, 339, 325, 302, 318, 342, 325, 303, 322, 341, 323, 311, 336, 348, 323, 312, 338, 346, 324, 329, 358, 357, 334, 349, 380, 374, 348, 357, 376, 354, 330, 346, 362, 337, 318, 341, 354, 332, 324, 348, 346, 314, 295, 297, 283, 261, 321, 439, 467, 409, 384, 385, 366, 343, 359, 370, 339, 313, 331, 344, 324, 313, 331, 336, 312, 313, 344, 345, 316, 319, 352, 350, 321, 329, 354, 342, 316, 329, 357, 345, 327, 346, 364, 349, 336, 357, 370, 347, 342, 370, 374, 345, 340, 363, 352, 321, 326, 347, 336, 311, 322, 342, 324, 300, 316, 333, 312, 299, 324, 338, 315, 307, 335, 341, 315, 312, 337, 334, 309, 312, 339, 334, 310, 324, 350, 341, 324, 342, 362, 344, 328, 352, 370, 346, 328, 344, 346, 317, 312, 337, 336, 308, 310, 335, 326, 299, 308, 326, 291, 243, 250, 295, 355, 417, 441, 416, 363, 336, 353, 366, 337, 318, 334, 329, 300, 302, 326, 322, 302, 314, 337, 322, 298, 314, 337, 322, 307, 329, 345, 318, 304, 334, 353, 331, 313, 338, 349, 329, 330, 358, 359, 334, 342, 370, 363, 339, 350, 371, 351, 323, 337, 354, 331, 312, 333, 341, 315, 305, 329, 334, 309, 306, 331, 330, 302, 311, 341, 334, 308, 321, 346, 332, 310, 326, 349, 335, 312, 330, 344, 323, 313, 340, 351, 328, 324, 356, 368, 341, 340, 367, 364, 333, 335, 353, 337, 308, 319, 342, 324, 306, 326, 342, 317, 297, 316, 312, 259, 235, 274, 347, 416, 442, 437, 402, 355, 353, 380, 361, 318, 318, 335, 319, 299, 315, 339, 320, 301, 323, 339, 317, 307, 334, 339, 314, 312, 339, 337, 309, 313, 344, 338, 314, 328, 354, 343, 322, 340, 362, 346, 330, 354, 369, 344, 331, 355, 361, 328, 316, 339, 338, 308, 306, 330, 323, 295, 304, 330, 316, 295, 311, 334, 317, 298, 317, 333, 311, 299, 323, 333, 308, 304, 331, 331, 305, 307, 335, 334, 308, 313, 339, 330, 307, 320, 342, 324, 306, 327, 348, 326, 310, 332, 346, 325, 316, 341, 349, 324, 323, 348, 354, 338, 344, 371, 366, 338, 342, 358, 338, 311, 328, 347, 329, 314, 334, 345, 320, 308, 331, 327, 272, 245, 270, 317, 385, 445, 455, 418, 368, 362, 381, 367, 330, 331, 343, 324, 303, 322, 340, 320, 308, 332, 341, 316, 315, 342, 342, 315, 318, 343, 338, 315, 326, 350, 339, 321, 338, 361, 343, 323, 346, 367, 349, 337, 361, 374, 350, 345, 372, 375, 344, 336, 356, 351, 321, 322, 345, 335, 308, 318, 341, 328, 308, 322, 337, 318, 305, 330, 345, 322, 308, 330, 338, 313, 311, 338, 337, 312, 316, 342, 337, 313, 325, 348, 328, 306, 322, 346, 330, 313, 333, 348, 324, 311, 336, 344, 317, 312, 338, 342, 317, 319, 343, 339, 315, 320, 346, 337, 311, 326, 350, 335, 315, 332, 349, 327, 313, 337, 347, 324, 320, 349, 354, 328, 334, 367, 367, 340, 344, 366, 350, 316, 325, 348, 333, 313, 325, 341, 322, 307, 329, 343, 311, 276, 273, 269, 263, 334, 445, 458, 397, 376, 386, 370, 345, 347, 353, 328, 300, 315, 337, 321, 300, 318, 332, 311, 298, 323, 332, 309, 306, 334, 340, 319, 323, 345, 338, 313, 319, 347, 341, 317, 331, 357, 344, 328, 349, 367, 347, 337, 360, 367, 345, 339, 361, 358, 328, 326, 348, 340, 311, 315, 337, 327, 304, 316, 335, 318, 300, 321, 339, 318, 302, 325, 339, 317, 308, 334, 340, 314, 312, 338, 339, 316, 322, 345, 334, 307, 318, 341, 330, 312, 329, 346, 327, 311, 333, 347, 324, 315, 337, 341, 314, 313, 341, 342, 317, 320, 343, 335, 310, 319, 345, 335, 314, 328, 347, 329, 313, 333, 349, 324, 309, 335, 350, 327, 323, 350, 355, 331, 338, 366, 364, 341, 346, 365, 348, 319, 329, 351, 336, 314, 329, 343, 320, 307, 332, 342, 309, 275, 276, 271, 261, 332, 435, 444, 393, 373, 383, 364, 337, 347, 364, 336, 304, 313, 333, 315, 299, 324, 339, 315, 300, 325, 336, 311, 308, 338, 339, 316, 317, 339, 336, 314, 326, 351, 336, 313, 333, 361, 348, 328, 349, 366, 345, 337, 365, 375, 348, 339, 363, 365, 333, 326, 348, 339, 308, 313, 335, 320, 297, 314, 336, 318, 299, 317, 332, 313, 300, 322, 332, 310, 304, 331, 338, 312, 310, 335, 333, 311, 319, 343, 331, 309, 323, 346, 329, 307, 325, 346, 326, 314, 336, 347, 322, 315, 343, 348, 321, 320, 345, 342, 315, 322, 348, 340, 314, 323, 348, 336, 317, 334, 351, 330, 311, 331, 347, 328, 320, 344, 354, 337, 338, 367, 369, 337, 332, 351, 343, 314, 320, 343, 329, 304, 319, 341, 323, 304, 321, 323, 274, 239, 256, 312, 380, 430, 444, 418, 368, 351, 366, 364, 332, 327, 341, 326, 302, 316, 340, 323, 302, 322, 340, 319, 306, 330, 341, 319, 315, 340, 346, 321, 322, 349, 347, 323, 330, 358, 350, 329, 341, 364, 353, 334, 353, 376, 360, 343, 364, 380, 355, 339, 359, 361, 331, 323, 346, 345, 319, 320, 342, 332, 306, 316, 343, 330, 307, 319, 339, 322, 307, 329, 344, 324, 312, 336, 344, 319, 313, 340, 344, 318, 320, 345, 338, 312, 323, 348, 335, 313, 327, 350, 331, 312, 332, 349, 326, 315, 339, 350, 325, 314, 341, 349, 323, 321, 345, 344, 317, 324, 352, 344, 314, 324, 347, 336, 321, 338, 357, 338, 323, 352, 372, 348, 342, 369, 371, 335, 328, 354, 347, 320, 325, 349, 341, 315, 323, 348, 333, 308, 304, 291, 254, 243, 324, 437, 459, 415, 407, 397, 358, 350, 374, 361, 318, 313, 338, 332, 304, 314, 340, 326, 301, 316, 341, 323, 303, 323, 341, 321, 308, 331, 343, 322, 315, 341, 346, 322, 322, 350, 350, 328, 336, 359, 351, 333, 348, 370, 355, 337, 355, 371, 349, 330, 344, 351, 322, 312, 334, 337, 306, 303, 329, 328, 299, 306, 333, 324, 298, 311, 339, 323, 303, 318, 337, 322, 305, 324, 340, 317, 307, 332, 339, 313, 309, 335, 336, 312, 313, 338, 334, 310, 319, 344, 331, 308, 325, 347, 328, 306, 328, 345, 324, 312, 333, 340, 316, 312, 339, 343, 317, 316, 342, 335, 309, 318, 346, 339, 318, 333, 353, 343, 330, 351, 369, 351, 336, 355, 359, 329, 318, 339, 337, 309, 310, 337, 332, 305, 314, 337, 313, 254, 246, 268, 296, 365, 445, 453, 403, 362, 369, 378, 352, 330, 339, 339, 310, 302, 328, 330, 303, 304, 332, 330, 306, 316, 338, 323, 301, 320, 341, 322, 307, 331, 348, 327, 316, 343, 351, 324, 318, 345, 352, 330, 330, 360, 361, 334, 341, 368, 359, 335, 348, 364, 340, 318, 335, 348, 321, 304, 327, 336, 311, 303, 327, 330, 304, 304, 332, 330, 304, 310, 338, 329, 303, 315, 338, 327, 308, 323, 338, 318, 305, 330, 344, 323, 311, 333, 338, 312, 309, 339, 340, 310, 315, 345, 337, 312, 323, 346, 330, 310, 327, 346, 326, 309, 331, 348, 327, 314, 334, 343, 321, 318, 348, 350, 324, 330, 365, 366, 342, 346, 366, 350, 319, 331, 351, 328, 306, 327, 344, 322, 306, 327, 337, 311, 290, 281, 262, 233, 280, 399, 460, 420, 395, 399, 374, 340, 350, 365, 340, 307, 316, 333, 317, 304, 322, 332, 308, 300, 328, 335, 307, 308, 337, 335, 308, 315, 343, 337, 312, 324, 349, 337, 316, 333, 355, 341, 325, 344, 359, 340, 332, 359, 367, 344, 342, 368, 366, 335, 331, 354, 344, 312, 318, 343, 330, 304, 318, 339, 325, 304, 321, 339, 319, 309, 330, 336, 314, 312, 338, 342, 316, 318, 343, 337, 313, 323, 349, 340, 313, 324, 349, 333, 314, 333, 349, 330, 314, 334, 347, 323, 313, 338, 341, 318, 317, 343, 342, 317, 321, 345, 338, 316, 329, 355, 344, 324, 344, 369, 350, 335, 358, 370, 338, 320, 343, 347, 319, 315, 341, 337, 306, 312, 342, 335, 307, 301, 293, 262, 238, 301, 414, 453, 418, 408, 400, 364, 346, 367, 365, 323, 310, 333, 337, 308, 307, 334, 331, 306, 315, 345, 332, 305, 319, 341, 326, 311, 333, 347, 325, 315, 342, 357, 333, 320, 343, 353, 332, 335, 362, 361, 334, 343, 376, 367, 339, 350, 374, 356, 331, 339, 350, 327, 309, 328, 341, 316, 303, 329, 336, 304, 298, 328, 332, 308, 308, 334, 327, 302, 315, 337, 326, 305, 318, 337, 326, 314, 331, 341, 317, 305, 328, 337, 313, 309, 338, 341, 316, 316, 336, 332, 307, 317, 342, 328, 305, 320, 341, 329, 317, 341, 359, 341, 332, 356, 369, 347, 338, 357, 352, 321, 317, 339, 337, 311, 320, 345, 332, 306, 318, 339, 313, 266, 260, 271, 276, 342, 431, 446, 394, 354, 361, 358, 330, 330, 350, 340, 306, 304, 329, 324, 303, 315, 338, 324, 303, 321, 339, 324, 313, 334, 346, 324, 313, 334, 343, 323, 317, 342, 344, 320, 324, 353, 350, 325, 332, 361, 356, 334, 348, 367, 350, 335, 352, 362, 335, 315, 333, 338, 312, 310, 333, 327, 296, 297, 324, 326, 304, 310, 332, 325, 302, 317, 335, 319, 302, 325, 341, 319, 305, 328, 339, 315, 308, 335, 341, 315, 312, 337, 339, 315, 317, 339, 333, 310, 322, 343, 330, 310, 327, 347, 334, 320, 344, 365, 347, 339, 364, 369, 337, 327, 347, 343, 316, 316, 339, 333, 308, 317, 341, 329, 306, 316, 314, 269, 234, 257, 323, 394, 434, 440, 417, 363, 342, 367, 373, 337, 320, 337, 333, 308, 317, 339, 328, 304, 318, 342, 328, 309, 327, 346, 328, 315, 338, 352, 330, 322, 349, 353, 327, 328, 357, 361, 337, 345, 371, 367, 343, 353, 377, 365, 344, 361, 382, 363, 342, 356, 364, 335, 320, 341, 348, 324, 317, 335, 333, 311, 316, 340, 334, 308, 315, 337, 327, 306, 324, 344, 327, 311, 329, 341, 321, 310, 332, 341, 317, 311, 337, 343, 318, 318, 344, 339, 315, 324, 349, 340, 314, 322, 342, 327, 311, 332, 351, 334, 327, 355, 366, 345, 340, 359, 355, 323, 318, 341, 337, 308, 312, 334, 326, 303, 314, 335, 320, 290, 282, 278, 247, 261, 372, 457, 429, 386, 386, 378, 347, 343, 356, 339, 305, 307, 327, 318, 293, 307, 334, 320, 295, 310, 329, 314, 300, 318, 335, 317, 310, 334, 340, 317, 315, 338, 341, 315, 320, 350, 346, 321, 332, 358, 349, 331, 344, 364, 347, 330, 351, 362, 335, 318, 336, 343, 316, 305, 328, 332, 305, 302, 326, 323, 300, 307, 333, 324, 300, 312, 336, 321, 302, 319, 335, 315, 301, 324, 337, 316, 309, 335, 338, 312, 308, 331, 330, 307, 314, 342, 334, 308, 321, 348, 332, 307, 325, 349, 333, 321, 350, 368, 347, 340, 363, 366, 332, 319, 341, 339, 313, 315, 341, 334, 308, 318, 346, 332, 300, 296, 293, 258, 241, 321, 434, 448, 405, 402, 397, 360, 353, 374, 356, 312, 303, 331, 335, 309, 312, 337, 333, 310, 323, 343, 326, 309, 332, 347, 323, 311, 337, 353, 329, 316, 344, 355, 333, 329, 358, 362, 334, 336, 360, 359, 342, 355, 379, 367, 348, 361, 373, 348, 324, 343, 357, 331, 315, 338, 344, 316, 309, 335, 336, 311, 309, 333, 332, 309, 315, 341, 334, 309, 320, 343, 332, 315, 330, 346, 329, 318, 340, 349, 326, 319, 344, 345, 320, 322, 350, 347, 324, 329, 351, 339, 317, 333, 355, 340, 320, 338, 358, 336, 319, 345, 364, 344, 329, 353, 365, 345, 345, 372, 369, 338, 336, 357, 346, 319, 329, 354, 336, 309, 323, 346, 328, 307, 325, 324, 276, 246, 268, 332, 400, 434, 438, 403, 352, 345, 367, 364, 331, 326, 342, 325, 304, 320, 337, 317, 298, 316, 335, 317, 304, 329, 341, 318, 313, 341, 341, 315, 316, 344, 344, 322, 331, 354, 345, 324, 337, 359, 347, 334, 356, 371, 346, 339, 367, 378, 349, 331, 352, 355, 326, 320, 342, 337, 306, 309, 331, 325, 302, 314, 336, 317, 299, 320, 340, 319, 302, 325, 336, 316, 308, 331, 340, 315, 310, 333, 334, 309, 309, 336, 334, 308, 314, 336, 331, 314, 322, 341, 328, 311, 326, 338, 319, 310, 329, 336, 316, 315, 338, 335, 310, 319, 352, 348, 319, 329, 361, 355, 335, 349, 371, 351, 322, 336, 348, 323, 310, 332, 342, 318, 309, 334, 339, 311, 307, 322, 292, 247, 248, 309, 385, 421, 431, 431, 388, 341, 347, 368, 353, 329, 333, 335, 310, 301, 329, 335, 310, 309, 336, 337, 312, 316, 344, 338, 309, 320, 348, 342, 321, 334, 357, 344, 325, 345, 362, 340, 329, 358, 374, 351, 343, 370, 376, 351, 350, 373, 367, 341, 346, 367, 353, 322, 327, 343, 328, 308, 323, 339, 320, 306, 329, 341, 315, 305, 329, 337, 316, 315, 337, 331, 305, 314, 345, 341, 313, 322, 346, 337, 318, 331, 346, 329, 314, 330, 343, 328, 322, 339, 342, 318, 319, 349, 349, 318, 315, 343, 338, 314, 325, 347, 335, 316, 331, 349, 333, 320, 342, 352, 333, 320, 346, 369, 353, 345, 367, 370, 344, 341, 362, 353, 320, 321, 342, 332, 311, 323, 341, 324, 310, 328, 329, 280, 247, 262, 303, 363, 429, 455, 431, 379, 356, 375, 376, 338, 319, 333, 324, 303, 312, 332, 319, 297, 312, 334, 323, 308, 325, 336, 314, 305, 334, 347, 325, 318, 343, 346, 319, 319, 344, 343, 322, 335, 361, 351, 333, 350, 370, 355, 339, 360, 374, 345, 331, 354, 358, 328, 318, 341, 341, 312, 311, 336, 330, 305, 310, 332, 324, 303, 316, 337, 321, 302, 321, 341, 325, 309, 325, 335, 316, 312, 336, 340, 314, 316, 337, 333, 314, 320, 346, 339, 310, 322, 347, 336, 316, 329, 345, 324, 308, 331, 346, 323, 314, 337, 344, 319, 312, 339, 344, 321, 323, 343, 337, 315, 322, 343, 337, 323, 331, 345, 329, 320, 346, 358, 335, 331, 357, 365, 343, 343, 363, 356, 325, 325, 349, 343, 318, 320, 338, 327, 308, 320, 338, 321, 285, 270, 268, 253, 285, 399, 464, 424, 388, 395, 386, 353, 343, 354, 340, 307, 307, 330, 326, 303, 313, 338, 326, 305, 320, 340, 323, 308, 330, 342, 320, 313, 341, 349, 327, 322, 346, 349, 327, 332, 357, 352, 328, 336, 364, 365, 345, 356, 377, 359, 338, 359, 378, 350, 327, 346, 354, 329, 317, 335, 338, 314, 310, 333, 333, 310, 313, 333, 323, 304, 322, 347, 328, 305, 321, 339, 326, 317, 330, 337, 324, 317, 336, 339, 321, 324, 346, 343, 320, 326, 351, 338, 309, 321, 349, 340, 320, 332, 348, 327, 309, 330, 347, 325, 314, 339, 346, 320, 317, 347, 347, 318, 318, 343, 341, 316, 322, 346, 338, 314, 326, 347, 332, 316, 335, 351, 334, 325, 347, 357, 342, 338, 364, 371, 345, 341, 360, 353, 320, 321, 344, 334, 313, 326, 350, 329, 306, 326, 346, 305, 256, 261, 277, 306, 384, 448, 443, 387, 355, 364, 359, 341, 344, 353, 329, 301, 311, 332, 323, 304, 320, 335, 318, 307, 328, 337, 315, 307, 331, 336, 314, 319, 342, 337, 318, 328, 354, 343, 315, 328, 353, 342, 327, 345, 364, 353, 341, 358, 374, 362, 346, 353, 356, 334, 326, 341, 337, 310, 309, 332, 325, 304, 314, 334, 319, 290, 301, 329, 321, 299, 315, 330, 310, 299, 323, 334, 311, 304, 331, 333, 307, 308, 330, 329, 307, 315, 340, 330, 306, 321, 339, 324, 306, 323, 340, 324, 313, 334, 340, 318, 311, 337, 346, 323, 316, 333, 335, 316, 321, 346, 340, 314, 316, 345, 339, 325, 340, 358, 346, 329, 349, 368, 354, 343, 354, 350, 322, 316, 342, 344, 318, 320, 341, 333, 308, 315, 342, 324, 271, 256, 269, 280, 336, 432, 463, 416, 371, 367, 363, 341, 332, 347, 340, 309, 306, 333, 331, 301, 305, 336, 332, 309, 314, 334, 323, 306, 323, 345, 329, 312, 334, 349, 328, 319, 345, 353, 328, 326, 354, 354, 328, 335, 368, 366, 339, 349, 372, 360, 340, 354, 373, 349, 321, 332, 343, 319, 306, 324, 331, 309, 309, 333, 332, 304, 303, 333, 330, 303, 307, 332, 326, 307, 319, 338, 324, 309, 328, 339, 318, 308, 330, 333, 307, 302, 331, 341, 315, 312, 337, 334, 306, 311, 340, 333, 309, 320, 345, 335, 312, 322, 337, 321, 310, 331, 343, 322, 310, 336, 346, 324, 318, 342, 346, 327, 337, 366, 361, 338, 347, 365, 349, 324, 333, 344, 330, 314, 326, 334, 320, 317, 334, 334, 313, 299, 284, 265, 251, 299, 404, 462, 434, 407, 397, 374, 356, 358, 354, 332, 317, 321, 324, 312, 308, 320, 325, 312, 308, 323, 329, 317, 319, 331, 325, 313, 324, 343, 334, 319, 325, 340, 335, 327, 340, 349, 338, 332, 344, 353, 345, 343, 358, 363, 351, 351, 362, 361, 344, 337, 342, 333, 319, 326, 334, 320, 309, 319, 328, 317, 310, 323, 326, 313, 313, 324, 325, 313, 313, 326, 326, 314, 317, 330, 325, 311, 318, 336, 329, 310, 318, 334, 326, 316, 323, 329, 321, 316, 330, 335, 322, 319, 336, 339, 322, 318, 331, 334, 322, 325, 337, 331, 315, 321, 335, 331, 324, 333, 342, 335, 330, 346, 360, 347, 341, 355, 355, 336, 330, 341, 338, 322, 320, 329, 324, 316, 324, 335, 328, 313, 311, 299, 264, 238, 258, 331, 414, 445, 428, 401, 366, 353, 366, 359, 329, 321, 331, 325, 307, 311, 325, 325, 316, 322, 328, 319, 309, 320, 329, 318, 317, 336, 342, 326, 322, 334, 336, 327, 332, 344, 340, 330, 337, 349, 348, 345, 355, 368, 364, 353, 359, 369, 359, 347, 352, 352, 337, 330, 335, 332, 319, 319, 328, 325, 316, 317, 327, 321, 304, 308, 323, 318, 308, 316, 330, 324, 316, 322, 325, 314, 312, 328, 332, 319, 318, 331, 330, 318, 321, 332, 328, 314, 317, 331, 324, 311, 320, 334, 327, 318, 328, 335, 325, 315, 327, 336, 326, 323, 336, 340, 325, 325, 350, 356, 342, 348, 365, 355, 333, 334, 341, 332, 325, 331, 336, 324, 315, 323, 328, 317, 315, 320, 297, 257, 248, 282, 353, 414, 428, 416, 386, 352, 344, 360, 358, 335, 326, 324, 315, 316, 327, 328, 314, 313, 333, 335, 320, 321, 336, 338, 321, 320, 335, 337, 327, 328, 342, 345, 337, 338, 345, 341, 337, 352, 362, 351, 349, 366, 371, 355, 352, 370, 373, 354, 348, 357, 352, 334, 331, 336, 329, 317, 325, 336, 322, 307, 314, 322, 318, 313, 326, 331, 316, 311, 325, 331, 313, 311, 328, 330, 320, 321, 331, 326, 313, 324, 343, 338, 320, 323, 334, 328, 321, 330, 337, 329, 323, 332, 336, 325, 324, 343, 349, 337, 338, 357, 358, 348, 356, 367, 348, 335, 342, 346, 326, 321, 339, 343, 325, 315, 321, 328, 322, 316, 315, 283, 247, 248, 319, 411, 444, 432, 415, 371, 334, 340, 366, 350, 321, 329, 338, 320, 306, 325, 339, 317, 307, 331, 339, 320, 314, 330, 332, 313, 323, 344, 334, 318, 333, 350, 337, 318, 336, 355, 339, 328, 348, 360, 346, 342, 363, 372, 352, 347, 367, 371, 348, 340, 353, 342, 321, 323, 336, 326, 305, 319, 337, 323, 308, 324, 339, 318, 305, 322, 331, 314, 310, 330, 335, 320, 324, 341, 338, 317, 322, 341, 330, 311, 327, 348, 338, 320, 331, 349, 336, 328, 346, 346, 322, 318, 344, 354, 336, 335, 365, 368, 346, 359, 380, 363, 334, 346, 362, 333, 310, 330, 348, 329, 309, 325, 345, 329, 320, 334, 314, 259, 233, 272, 353, 432, 458, 444, 405, 364, 364, 386, 371, 338, 330, 335, 316, 308, 328, 342, 322, 308, 328, 341, 327, 324, 339, 338, 322, 328, 349, 343, 321, 324, 349, 349, 335, 342, 355, 343, 331, 351, 371, 354, 339, 360, 374, 354, 348, 370, 374, 352, 349, 360, 349, 330, 335, 347, 330, 312, 322, 338, 328, 313, 323, 341, 331, 318, 331, 336, 318, 314, 337, 342, 318, 315, 339, 342, 321, 324, 341, 333, 313, 324, 342, 331, 317, 331, 343, 331, 318, 331, 343, 328, 318, 342, 346, 322, 318, 341, 353, 344, 345, 367, 368, 340, 335, 346, 336, 314, 324, 346, 334, 315, 325, 331, 313, 304, 322, 327, 294, 256, 253, 266, 310, 407, 467, 439, 385, 367, 376, 366, 336, 327, 332, 317, 306, 316, 326, 310, 300, 319, 333, 313, 305, 322, 329, 315, 311, 327, 328, 314, 319, 338, 334, 316, 327, 345, 335, 320, 329, 345, 336, 330, 349, 359, 343, 334, 356, 364, 341, 332, 348, 346, 328, 326, 337, 326, 308, 313, 326, 320, 307, 315, 326, 312, 301, 318, 328, 311, 302, 325, 335, 313, 306, 325, 332, 320, 315, 328, 330, 314, 321, 337, 327, 308, 320, 337, 326, 308, 322, 339, 331, 317, 332, 343, 326, 316, 333, 340, 323, 320, 343, 356, 338, 337, 356, 356, 334, 332, 348, 337, 316, 330, 343, 329, 313, 320, 331, 315, 304, 319, 319, 278, 249, 256, 280, 338, 418, 453, 424, 372, 352, 360, 355, 332, 327, 330, 315, 301, 314, 326, 311, 301, 320, 330, 311, 303, 322, 333, 311, 308, 330, 332, 316, 320, 339, 335, 316, 322, 344, 337, 323, 338, 358, 347, 332, 351, 365, 348, 345, 363, 368, 345, 338, 351, 350, 329, 325, 335, 329, 311, 317, 336, 323, 306, 317, 330, 319, 305, 319, 329, 312, 306, 327, 336, 317, 313, 333, 338, 323, 317, 333, 335, 320, 324, 339, 335, 316, 324, 344, 337, 315, 323, 339, 327, 313, 328, 341, 328, 319, 335, 340, 326, 329, 348, 348, 331, 341, 366, 359, 340, 346, 363, 349, 328, 333, 347, 336, 318, 330, 339, 325, 318, 329, 333, 314, 309, 307, 283, 251, 262, 346, 425, 425, 412, 397, 366, 340, 352, 368, 345, 321, 328, 335, 318, 309, 326, 336, 318, 312, 333, 340, 322, 320, 339, 344, 329, 328, 344, 339, 324, 337, 350, 336, 325, 344, 359, 343, 336, 356, 367, 352, 345, 366, 375, 355, 352, 369, 365, 340, 344, 361, 347, 320, 326, 342, 331, 310, 320, 335, 323, 313, 327, 334, 320, 316, 332, 335, 317, 319, 338, 334, 313, 318, 336, 328, 311, 323, 344, 334, 315, 324, 342, 332, 319, 327, 335, 323, 314, 330, 341, 324, 318, 338, 342, 321, 323, 345, 351, 336, 331, 346, 348, 337, 347, 363, 352, 332, 339, 349, 335, 326, 338, 337, 318, 316, 335, 335, 316, 314, 331, 318, 266, 245, 258, 293, 370, 446, 457, 405, 355, 356, 373, 359, 331, 334, 338, 314, 299, 316, 324, 311, 312, 330, 333, 312, 312, 331, 329, 312, 321, 337, 329, 317, 332, 350, 338, 326, 342, 354, 337, 330, 351, 363, 348, 343, 362, 368, 346, 344, 366, 368, 344, 342, 357, 350, 330, 334, 342, 324, 309, 324, 335, 317, 304, 318, 328, 314, 309, 328, 331, 317, 319, 333, 328, 308, 315, 337, 330, 308, 318, 340, 326, 309, 327, 344, 328, 314, 330, 340, 325, 318, 332, 334, 314, 313, 334, 335, 319, 320, 339, 338, 319, 323, 342, 338, 325, 334, 343, 333, 331, 350, 364, 351, 337, 346, 349, 334, 330, 337, 334, 314, 312, 328, 325, 310, 318, 336, 325, 298, 283, 274, 253, 270, 369, 459, 449, 403, 383, 376, 357, 351, 358, 344, 316, 310, 329, 329, 307, 307, 329, 323, 303, 313, 334, 326, 313, 325, 340, 326, 313, 331, 346, 330, 317, 334, 344, 334, 337, 352, 350, 329, 335, 361, 363, 346, 351, 367, 359, 342, 351, 365, 350, 334, 342, 348, 330, 316, 330, 338, 322, 316, 330, 328, 311, 314, 333, 333, 315, 321, 342, 338, 319, 324, 335, 321, 306, 321, 338, 328, 320, 339, 350, 330, 318, 335, 340, 323, 323, 341, 339, 319, 326, 345, 336, 314, 327, 352, 341, 320, 327, 340, 329, 316, 330, 342, 329, 323, 341, 349, 333, 329, 347, 357, 345, 346, 363, 361, 339, 336, 352, 345, 327, 334, 347, 329, 315, 334, 347, 324, 308, 324, 332, 300, 267, 266, 268, 301, 397, 465, 444, 391, 372, 383, 373, 346, 338, 342, 324, 306, 318, 333, 319, 309, 326, 334, 315, 304, 320, 326, 315, 315, 332, 334, 320, 322, 340, 335, 322, 331, 346, 336, 320, 328, 340, 335, 331, 345, 355, 341, 337, 361, 366, 343, 337, 358, 357, 326, 317, 334, 333, 310, 316, 334, 321, 299, 308, 320, 307, 299, 315, 324, 307, 302, 321, 329, 312, 304, 320, 323, 314, 310, 325, 323, 310, 319, 331, 318, 307, 322, 338, 326, 314, 323, 331, 319, 311, 322, 330, 317, 312, 330, 337, 320, 316, 333, 334, 317, 318, 333, 327, 307, 320, 340, 332, 314, 324, 343, 332, 321, 342, 358, 345, 340, 358, 356, 331, 323, 340, 342, 321, 318, 333, 326, 307, 313, 331, 324, 309, 316, 324, 292, 254, 256, 268, 302, 381, 435, 426, 376, 351, 356, 354, 339, 336, 338, 322, 303, 311, 331, 330, 311, 315, 330, 322, 309, 318, 328, 321, 313, 325, 334, 320, 315, 334, 340, 326, 325, 344, 347, 332, 334, 346, 342, 336, 345, 357, 355, 348, 356, 368, 359, 347, 355, 367, 353, 330, 327, 326, 309, 309, 329, 329, 309, 307, 323, 322, 310, 314, 328, 320, 308, 315, 325, 317, 307, 319, 328, 319, 314, 330, 337, 319, 311, 325, 329, 316, 313, 325, 328, 315, 317, 330, 326, 315, 316, 328, 322, 309, 321, 336, 329, 322, 330, 328, 322, 324, 337, 342, 336, 336, 350, 355, 339, 336, 346, 336, 318, 324, 332, 321, 311, 317, 328, 316, 301, 313, 326, 308, 277, 272, 268, 271, 340, 422, 426, 384, 354, 357, 352, 337, 343, 347, 326, 308, 313, 325, 321, 305, 309, 319, 313, 310, 324, 324, 306, 312, 333, 332, 313, 319, 340, 338, 316, 318, 340, 341, 326, 336, 351, 345, 337, 350, 362, 354, 349, 354, 359, 351, 352, 362, 352, 329, 325, 340, 339, 321, 320, 331, 330, 318, 317, 323, 318, 306, 317, 336, 327, 317, 328, 335, 324, 320, 331, 334, 324, 328, 345, 339, 320, 317, 331, 333, 318, 322, 338, 335, 319, 323, 338, 331, 321, 329, 339, 332, 323, 328, 338, 335, 336, 353, 360, 352, 353, 371, 367, 344, 341, 356, 345, 320, 325, 342, 334, 324, 338, 344, 332, 331, 348, 341, 297, 267, 277, 308, 360, 431, 456, 430, 396, 377, 379, 377, 366, 360, 355, 339, 327, 337, 345, 332, 329, 345, 352, 340, 334, 350, 358, 342, 335, 350, 356, 346, 345, 360, 360, 340, 344, 359, 354, 343, 354, 370, 362, 351, 361, 371, 363, 359, 376, 381, 359, 353, 369, 371, 353, 345, 348, 341, 326, 331, 343, 334, 319, 323, 338, 332, 321, 327, 336, 328, 325, 341, 345, 329, 320, 334, 340, 325, 322, 333, 334, 323, 325, 339, 337, 323, 328, 338, 329, 319, 327, 336, 325, 318, 334, 343, 330, 331, 349, 355, 341, 338, 356, 359, 349, 354, 362, 347, 328, 332, 342, 332, 317, 323, 330, 323, 311, 320, 329, 318, 305, 298, 273, 246, 254, 328, 443, 494, 455, 417, 386, 363, 364, 365, 339, 307, 307, 322, 312, 299, 307, 321, 314, 306, 317, 320, 304, 300, 319, 324, 313, 318, 334, 329, 306, 310, 327, 327, 319, 320, 332, 330, 318, 326, 337, 330, 326, 342, 346, 331, 330, 349, 355, 341, 332, 339, 338, 326, 324, 333, 325, 310, 317, 331, 326, 318, 324, 330, 318, 308, 321, 332, 319, 311, 324, 333, 321, 319, 331, 328, 317, 321, 334, 330, 315, 319, 333, 329, 316, 324, 338, 330, 315, 328, 341, 327, 313, 329, 338, 327, 326, 342, 345, 334, 334, 347, 356, 349, 351, 364, 358, 336, 337, 344, 333, 319, 326, 334, 323, 312, 320, 322, 310, 306, 319, 316, 273, 240, 245, 277, 346, 456, 492, 441, 375, 359, 376, 371, 338, 320, 318, 308, 299, 308, 316, 309, 303, 318, 324, 306, 301, 318, 319, 304, 310, 330, 323, 301, 307, 329, 326, 314, 320, 333, 325, 310, 318, 328, 323, 320, 338, 342, 325, 321, 336, 342, 333, 338, 349, 343, 326, 326, 332, 324, 309, 312, 324, 318, 304, 311, 325, 316, 308, 326, 332, 318, 313, 320, 326, 317, 313, 328, 326, 313, 314, 331, 330, 316, 316, 328, 324, 314, 319, 332, 331, 325, 330, 334, 322, 318, 330, 333, 326, 329, 337, 331, 318, 325, 344, 344, 333, 346, 369, 364, 346, 351, 361, 346, 330, 339, 342, 324, 314, 327, 329, 311, 310, 329, 333, 313, 301, 296, 271, 245, 271, 360, 446, 454, 429, 406, 378, 359, 367, 366, 336, 313, 318, 323, 314, 309, 323, 324, 308, 308, 321, 322, 311, 316, 331, 332, 324, 330, 340, 331, 317, 322, 333, 327, 324, 339, 344, 331, 325, 343, 351, 341, 335, 348, 353, 342, 344, 358, 356, 343, 344, 350, 339, 323, 331, 342, 334, 321, 326, 332, 321, 319, 333, 338, 321, 317, 329, 332, 321, 321, 336, 334, 322, 326, 337, 332, 321, 326, 337, 335, 326, 332, 341, 332, 322, 334, 342, 332, 326, 337, 337, 323, 324, 340, 343, 329, 328, 339, 337, 326, 334, 347, 343, 336, 349, 362, 351, 347, 360, 366, 343, 333, 340, 340, 323, 321, 334, 335, 320, 320, 334, 335, 320, 309, 287, 258, 244, 287, 388, 453, 441, 412, 389, 367, 357, 369, 361, 328, 314, 325, 324, 311, 311, 325, 325, 314, 315, 328, 325, 313, 320, 333, 326, 315, 328, 340, 330, 321, 331, 337, 327, 323, 337, 342, 330, 332, 345, 341, 330, 338, 357, 358, 346, 348, 359, 351, 336, 343, 351, 337, 322, 327, 332, 322, 318, 329, 332, 320, 316, 331, 331, 317, 319, 333, 330, 318, 325, 337, 331, 318, 326, 336, 325, 317, 331, 338, 326, 318, 328, 331, 317, 317, 333, 335, 317, 315, 331, 330, 316, 318, 333, 331, 319, 325, 338, 329, 316, 328, 342, 334, 325, 341, 355, 346, 339, 353, 356, 340, 335, 343, 337, 318, 315, 330, 327, 311, 316, 328, 321, 309, 315, 311, 271, 240, 252, 309, 391, 438, 436, 407, 370, 354, 366, 366, 339, 327, 333, 326, 310, 314, 326, 321, 310, 316, 328, 321, 316, 329, 336, 323, 318, 333, 339, 325, 324, 337, 337, 324, 329, 339, 335, 327, 337, 354, 350, 339, 344, 354, 347, 342, 358, 362, 344, 334, 346, 350, 332, 323, 331, 330, 314, 312, 327, 329, 314, 315, 328, 323, 311, 317, 331, 323, 312, 320, 330, 324, 316, 325, 330, 318, 314, 326, 329, 319, 320, 335, 336, 318, 318, 333, 330, 320, 326, 338, 328, 312, 319, 333, 328, 320, 329, 337, 327, 320, 334, 341, 328, 328, 352, 366, 351, 341, 352, 350, 337, 340, 351, 340, 322, 322, 332, 325, 316, 325, 335, 325, 316, 322, 316, 276, 251, 260, 294, 363, 424, 442, 417, 375, 360, 366, 361, 341, 334, 332, 317, 304, 311, 323, 320, 313, 323, 328, 316, 311, 327, 334, 318, 316, 334, 335, 325, 330, 344, 341, 332, 339, 353, 347, 334, 340, 351, 344, 340, 354, 365, 355, 352, 364, 368, 358, 352, 354, 348, 335, 331, 335, 327, 313, 318, 333, 327, 314, 322, 334, 326, 316, 322, 329, 321, 317, 331, 331, 318, 315, 331, 335, 322, 321, 336, 335, 319, 321, 334, 331, 322, 330, 342, 334, 322, 331, 343, 337, 330, 338, 340, 326, 324, 337, 340, 329, 335, 349, 347, 340, 351, 365, 363, 345, 345, 352, 343, 330, 333, 339, 325, 316, 329, 338, 324, 318, 326, 328, 312, 287, 272, 260, 261, 337, 432, 450, 409, 383, 375, 363, 354, 355, 350, 332, 316, 323, 328, 314, 307, 322, 329, 320, 320, 332, 329, 314, 317, 334, 334, 323, 327, 342, 339, 327, 333, 346, 341, 334, 345, 351, 344, 341, 352, 357, 345, 345, 364, 374, 359, 353, 362, 357, 336, 334, 343, 334, 313, 316, 329, 323, 315, 322, 328, 319, 314, 325, 331, 319, 310, 324, 329, 320, 316, 324, 327, 317, 321, 333, 332, 319, 323, 339, 333, 321, 325, 334, 325, 317, 327, 336, 326, 321, 334, 340, 330, 329, 343, 346, 333, 333, 351, 358, 345, 349, 362, 355, 336, 336, 349, 340, 326, 331, 338, 328, 316, 325, 330, 319, 317, 327, 314, 273, 254, 270, 315, 387, 434, 437, 402, 366, 363, 368, 357, 333, 331, 336, 324, 314, 326, 335, 322, 316, 328, 333, 320, 319, 337, 339, 322, 323, 336, 334, 324, 333, 347, 340, 329, 338, 349, 343, 335, 344, 354, 347, 345, 361, 364, 350, 349, 365, 370, 354, 349, 354, 345, 330, 330, 339, 331, 314, 321, 337, 329, 314, 318, 328, 322, 320, 326, 327, 318, 314, 326, 327, 314, 316, 333, 332, 320, 325, 337, 330, 317, 322, 333, 324, 315, 327, 335, 323, 319, 331, 338, 327, 322, 334, 335, 320, 319, 340, 345, 333, 330, 348, 354, 347, 352, 364, 356, 339, 338, 340, 326, 314, 325, 331, 319, 315, 328, 332, 318, 316, 329, 316, 271, 248, 258, 301, 379, 440, 438, 400, 364, 361, 370, 362, 338, 332, 336, 323, 314, 322, 320, 307, 310, 326, 326, 313, 316, 333, 332, 320, 325, 340, 338, 327, 332, 343, 336, 326, 337, 348, 338, 335, 350, 359, 348, 345, 363, 370, 358, 356, 366, 363, 349, 351, 360, 350, 333, 336, 345, 336, 322, 329, 339, 328, 316, 327, 335, 324, 318, 329, 333, 322, 322, 337, 340, 328, 327, 340, 337, 322, 327, 341, 336, 324, 330, 342, 334, 326, 338, 345, 334, 327, 338, 343, 330, 325, 338, 340, 326, 329, 342, 340, 331, 338, 348, 342, 336, 349, 363, 357, 345, 351, 352, 338, 328, 337, 340, 325, 318, 331, 332, 320, 322, 336, 334, 316, 297, 277, 257, 256, 327, 431, 454, 410, 387, 380, 363, 358, 363, 348, 321, 313, 326, 329, 318, 319, 330, 328, 312, 313, 331, 327, 313, 323, 339, 332, 320, 331, 342, 332, 322, 336, 345, 336, 334, 347, 349, 336, 336, 354, 358, 346, 350, 365, 364, 351, 351, 361, 351, 335, 336, 340, 329, 316, 325, 333, 321, 312, 324, 329, 315, 312, 325, 326, 311, 312, 327, 327, 316, 319, 332, 329, 318, 326, 338, 330, 317, 326, 337, 329, 319, 332, 339, 325, 320, 334, 340, 327, 327, 343, 338, 322, 324, 338, 335, 326, 334, 342, 331, 321, 332, 341, 329, 323, 338, 350, 336, 332, 348, 357, 350, 345, 357, 359, 344, 341, 352, 344, 327, 329, 338, 332, 318, 323, 336, 328, 316, 326, 328, 300, 265, 254, 265, 318, 405, 453, 433, 389, 368, 370, 369, 347, 333, 336, 324, 307, 315, 328, 321, 311, 321, 332, 324, 315, 324, 329, 318, 318, 334, 338, 326, 324, 338, 339, 331, 337, 350, 348, 337, 341, 351, 350, 345, 355, 369, 357, 348, 361, 370, 358, 349, 357, 356, 337, 325, 333, 333, 321, 324, 333, 325, 309, 315, 330, 324, 313, 321, 329, 314, 304, 319, 331, 320, 314, 326, 331, 320, 318, 330, 332, 320, 322, 336, 331, 313, 316, 334, 333, 323, 325, 333, 324, 319, 331, 339, 323, 316, 327, 332, 324, 322, 334, 334, 320, 323, 337, 334, 323, 331, 350, 346, 332, 341, 360, 351, 342, 354, 359, 339, 327, 335, 340, 326, 322, 335, 338, 324, 318, 330, 329, 313, 311, 306, 276, 248, 264, 346, 418, 424, 413, 394, 363, 347, 357, 364, 342, 324, 330, 328, 316, 315, 330, 332, 319, 321, 335, 333, 324, 329, 338, 332, 324, 333, 342, 334, 327, 338, 346, 335, 332, 346, 351, 342, 343, 358, 364, 352, 353, 364, 363, 350, 355, 370, 359, 338, 338, 345, 332, 319, 326, 332, 320, 315, 322, 324, 310, 307, 322, 326, 313, 311, 326, 324, 312, 318, 329, 323, 313, 323, 337, 327, 314, 324, 332, 322, 317, 329, 336, 324, 319, 329, 335, 321, 316, 332, 334, 320, 320, 332, 329, 320, 327, 338, 331, 321, 333, 340, 325, 314, 329, 339, 329, 325, 341, 347, 336, 339, 358, 358, 346, 345, 356, 348, 329, 332, 341, 332, 320, 324, 334, 324, 313, 322, 329, 315, 304, 296, 274, 245, 262, 353, 438, 438, 407, 390, 367, 342, 347, 357, 338, 313, 317, 329, 321, 313, 324, 330, 316, 307, 320, 329, 319, 314, 331, 336, 324, 321, 333, 334, 325, 330, 343, 339, 329, 335, 347, 340, 335, 350, 361, 351, 344, 355, 364, 354, 348, 361, 367, 347, 333, 336, 336, 323, 325, 334, 328, 313, 317, 329, 324, 315, 321, 329, 320, 312, 321, 330, 321, 316, 328, 333, 320, 314, 330, 331, 318, 323, 337, 335, 321, 325, 339, 335, 326, 332, 340, 333, 325, 338, 347, 331, 322, 340, 344, 326, 319, 335, 340, 329, 332, 344, 340, 325, 329, 340, 337, 327, 336, 348, 337, 324, 334, 344, 337, 333, 350, 357, 340, 340, 361, 366, 357, 357, 366, 361, 340, 335, 344, 342, 328, 329, 337, 328, 318, 328, 339, 328, 312, 304, 284, 258, 267, 344, 435, 447, 413, 403, 391, 367, 366, 374, 357, 329, 325, 334, 326, 316, 330, 343, 329, 315, 326, 337, 329, 325, 336, 339, 327, 328, 346, 347, 332, 333, 349, 348, 336, 339, 353, 355, 342, 347, 361, 359, 353, 362, 372, 363, 358, 372, 376, 354, 340, 348, 346, 330, 328, 339, 335, 318, 321, 333, 328, 317, 324, 336, 329, 316, 325, 334, 324, 317, 328, 339, 330, 322, 335, 339, 325, 324, 338, 340, 329, 332, 343, 338, 328, 334, 346, 342, 331, 334, 338, 331, 325, 333, 341, 329, 323, 339, 345, 331, 329, 343, 345, 331, 329, 341, 338, 324, 327, 338, 333, 322, 330, 343, 336, 328, 334, 340, 331, 327, 338, 341, 328, 322, 336, 342, 332, 334, 350, 348, 331, 340, 363, 361, 347, 354, 362, 345, 330, 337, 341, 328, 321, 333, 341, 329, 318, 327, 330, 317, 312, 310, 283, 254, 257, 321, 407, 441, 426, 410, 378, 352, 361, 371, 346, 321, 326, 332, 320, 313, 326, 330, 318, 315, 325, 325, 314, 321, 334, 331, 320, 326, 339, 333, 322, 332, 341, 332, 325, 340, 351, 341, 335, 349, 356, 345, 345, 361, 367, 353, 351, 365, 363, 346, 346, 355, 343, 325, 328, 339, 331, 315, 321, 331, 320, 312, 324, 327, 314, 311, 324, 326, 314, 315, 331, 329, 313, 317, 333, 326, 313, 321, 332, 327, 318, 327, 339, 333, 320, 327, 332, 321, 320, 336, 337, 318, 315, 328, 329, 320, 323, 336, 334, 324, 328, 338, 328, 314, 322, 334, 325, 315, 327, 337, 327, 319, 329, 334, 323, 322, 336, 334, 318, 319, 334, 335, 323, 328, 340, 332, 317, 323, 335, 323, 316, 331, 341, 332, 326, 338, 349, 346, 345, 356, 360, 345, 337, 339, 330, 316, 322, 338, 329, 313, 318, 331, 325, 317, 327, 329, 296, 256, 251, 266, 311, 398, 451, 430, 390, 368, 365, 361, 347, 335, 331, 319, 303, 308, 322, 315, 305, 312, 322, 316, 310, 320, 328, 319, 313, 321, 326, 319, 318, 331, 330, 318, 325, 345, 341, 324, 332, 345, 339, 332, 342, 353, 348, 343, 356, 366, 352, 343, 351, 350, 333, 326, 338, 336, 318, 312, 323, 323, 311, 316, 333, 323, 304, 308, 322, 314, 307, 322, 331, 318, 311, 325, 331, 321, 320, 330, 329, 313, 313, 328, 332, 319, 319, 330, 326, 316, 326, 336, 324, 312, 324, 336, 329, 322, 328, 332, 323, 322, 334, 334, 318, 318, 334, 335, 324, 326, 336, 331, 319, 322, 334, 331, 321, 324, 332, 324, 319, 329, 335, 321, 317, 334, 339, 328, 330, 343, 341, 333, 336, 352, 353, 341, 339, 345, 333, 317, 326, 339, 329, 314, 320, 328, 317, 312, 323, 320, 287, 257, 258, 274, 326, 409, 446, 421, 376, 354, 355, 355, 345, 339, 337, 322, 307, 315, 327, 317, 308, 320, 325, 310, 310, 327, 328, 313, 315, 329, 329, 319, 323, 335, 330, 318, 325, 341, 337, 331, 340, 352, 344, 339, 353, 361, 352, 348, 361, 366, 354, 352, 364, 358, 339, 335, 340, 334, 321, 325, 331, 317, 305, 316, 330, 321, 310, 317, 322, 314, 310, 323, 331, 317, 306, 317, 325, 314, 317, 331, 328, 314, 319, 333, 330, 319, 324, 334, 326, 315, 325, 336, 328, 324, 334, 334, 322, 320, 332, 334, 323, 322, 334, 330, 320, 326, 339, 333, 321, 325, 338, 333, 321, 332, 347, 346, 344, 353, 354, 346, 344, 353, 350, 331, 327, 337, 332, 315, 319, 332, 328, 317, 322, 329, 318, 294, 273, 265, 253, 280, 377, 442, 427, 392, 374, 358, 340, 343, 356, 345, 319, 313, 323, 319, 304, 310, 324, 320, 309, 315, 325, 321, 316, 327, 333, 326, 323, 331, 332, 323, 325, 340, 341, 331, 331, 347, 350, 339, 346, 361, 355, 345, 356, 367, 361, 354, 363, 369, 353, 337, 342, 346, 334, 327, 333, 324, 308, 311, 326, 325, 314, 319, 332, 327, 315, 320, 331, 323, 311, 320, 331, 319, 312, 325, 330, 320, 320, 331, 332, 320, 324, 338, 333, 318, 322, 334, 327, 314, 322, 337, 331, 320, 329, 336, 330, 325, 338, 344, 325, 318, 332, 339, 329, 329, 350, 361, 347, 348, 364, 358, 336, 336, 349, 340, 321, 325, 337, 328, 318, 327, 333, 319, 311, 322, 310, 269, 248, 270, 330, 406, 444, 436, 401, 361, 355, 368, 363, 337, 327, 331, 321, 315, 325, 332, 320, 310, 321, 327, 317, 316, 330, 332, 319, 319, 333, 335, 324, 328, 345, 342, 330, 334, 343, 338, 332, 343, 354, 351, 346, 356, 365, 356, 354, 366, 367, 351, 344, 352, 347, 328, 325, 333, 327, 312, 315, 326, 321, 310, 317, 325, 316, 308, 317, 325, 316, 314, 329, 332, 317, 317, 332, 332, 322, 324, 334, 333, 321, 322, 332, 326, 319, 331, 342, 329, 319, 331, 338, 326, 321, 334, 338, 322, 320, 333, 333, 322, 327, 346, 343, 330, 342, 364, 357, 343, 350, 355, 336, 324, 333, 336, 322, 317, 330, 334, 321, 315, 327, 329, 318, 308, 293, 266, 244, 278, 376, 452, 440, 407, 388, 366, 350, 362, 360, 332, 312, 321, 325, 313, 310, 323, 328, 314, 315, 328, 327, 315, 318, 334, 331, 316, 320, 334, 333, 327, 337, 344, 334, 326, 339, 346, 339, 337, 351, 357, 346, 347, 361, 364, 354, 356, 366, 359, 338, 335, 345, 337, 322, 325, 334, 324, 313, 319, 328, 322, 315, 325, 330, 317, 312, 325, 326, 313, 314, 327, 327, 319, 324, 336, 328, 316, 326, 337, 332, 322, 331, 338, 326, 319, 330, 338, 325, 320, 330, 332, 322, 323, 336, 336, 323, 327, 341, 342, 333, 340, 356, 348, 336, 350, 365, 354, 342, 344, 346, 329, 323, 333, 336, 326, 323, 336, 334, 314, 314, 325, 303, 266, 253, 272, 332, 417, 457, 435, 397, 366, 366, 375, 359, 337, 330, 330, 318, 316, 329, 330, 318, 319, 331, 330, 318, 327, 337, 324, 310, 321, 336, 332, 325, 333, 338, 325, 321, 337, 345, 333, 332, 350, 352, 336, 339, 356, 359, 348, 354, 366, 357, 346, 350, 354, 343, 329, 334, 335, 319, 311, 324, 328, 314, 310, 323, 326, 312, 313, 328, 321, 306, 314, 328, 323, 309, 317, 330, 325, 316, 322, 330, 324, 317, 323, 328, 319, 315, 324, 326, 315, 316, 328, 326, 312, 315, 329, 330, 317, 321, 331, 324, 315, 330, 342, 338, 338, 351, 353, 344, 340, 348, 344, 324, 317, 326, 324, 313, 315, 329, 324, 306, 313, 332, 329, 301, 274, 260, 249, 273, 368, 435, 422, 385, 370, 359, 342, 342, 353, 342, 317, 311, 320, 319, 309, 313, 325, 319, 306, 315, 329, 325, 317, 328, 339, 330, 319, 329, 336, 329, 327, 338, 340, 332, 336, 352, 356, 344, 349, 363, 358, 344, 354, 371, 366, 355, 362, 366, 351, 339, 347, 350, 332, 318, 325, 326, 314, 312, 325, 324, 310, 315, 330, 322, 307, 315, 330, 326, 313, 320, 331, 325, 312, 324, 334, 323, 318, 331, 335, 319, 318, 332, 333, 321, 326, 339, 332, 319, 322, 335, 331, 322, 332, 346, 338, 331, 343, 356, 347, 342, 356, 362, 344, 329, 336, 335, 322, 321, 333, 333, 317, 315, 327, 321, 311, 315, 310, 274, 247, 260, 321, 397, 426, 419, 394, 354, 337, 352, 358, 334, 318, 324, 322, 310, 316, 327, 320, 312, 320, 331, 322, 314, 326, 337, 328, 320, 331, 337, 326, 322, 339, 348, 338, 333, 343, 343, 332, 339, 355, 356, 350, 357, 367, 361, 352, 362, 371, 359, 344, 347, 350, 334, 321, 330, 334, 322, 320, 330, 325, 311, 312, 326, 324, 314, 316, 326, 320, 309, 316, 330, 322, 315, 327, 338, 326, 317, 328, 331, 321, 322, 334, 334, 320, 319, 336, 335, 321, 324, 336, 328, 317, 326, 339, 332, 323, 334, 343, 339, 333, 342, 352, 346, 343, 360, 362, 344, 336, 344, 339, 322, 323, 336, 333, 318, 322, 335, 327, 312, 318, 322, 291, 256, 254, 280, 344, 429, 461, 429, 380, 360, 367, 367, 347, 334, 333, 324, 307, 312, 323, 318, 310, 320, 331, 321, 311, 320, 329, 317, 314, 326, 333, 322, 321, 335, 336, 323, 328, 347, 343, 329, 336, 354, 349, 340, 352, 365, 357, 350, 359, 365, 355, 348, 356, 355, 337, 328, 335, 335, 319, 317, 327, 325, 309, 310, 325, 324, 314, 320, 330, 325, 312, 321, 332, 324, 315, 325, 333, 322, 320, 332, 333, 321, 322, 336, 332, 319, 325, 337, 330, 316, 320, 333, 328, 315, 322, 333, 323, 317, 328, 332, 322, 319, 329, 334, 325, 328, 344, 346, 337, 343, 354, 351, 344, 350, 359, 347, 331, 333, 339, 327, 320, 331, 336, 322, 314, 324, 327, 314, 302, 289, 269, 250, 291, 393, 457, 435, 404, 390, 370, 355, 357, 355, 332, 314, 321, 328, 315, 309, 325, 330, 310, 307, 325, 329, 314, 313, 329, 331, 320, 323, 336, 333, 321, 328, 342, 337, 328, 337, 345, 340, 338, 348, 352, 343, 343, 360, 365, 351, 349, 362, 358, 334, 330, 341, 335, 318, 321, 331, 321, 310, 318, 328, 318, 307, 320, 328, 317, 314, 328, 328, 312, 314, 329, 333, 318, 320, 332, 329, 319, 326, 337, 329, 317, 323, 332, 324, 316, 327, 332, 321, 319, 334, 337, 323, 320, 335, 336, 321, 320, 332, 330, 319, 324, 335, 331, 322, 330, 339, 327, 316, 333, 349, 337, 324, 337, 351, 345, 343, 359, 360, 341, 333, 344, 340, 324, 326, 337, 333, 319, 326, 338, 328, 318, 330, 327, 290, 262, 265, 306, 381, 443, 449, 417, 379, 364, 371, 371, 350, 337, 337, 329, 320, 323, 328, 321, 315, 324, 329, 317, 312, 327, 334, 323, 321, 336, 342, 334, 335, 344, 342, 328, 332, 348, 346, 340, 347, 359, 355, 348, 362, 374, 365, 358, 369, 371, 354, 350, 363, 361, 341, 334, 342, 338, 321, 321, 336, 332, 313, 316, 326, 320, 310, 322, 335, 324, 314, 322, 330, 324, 323, 332, 336, 326, 321, 329, 330, 320, 320, 333, 331, 318, 322, 334, 333, 324, 329, 335, 328, 321, 331, 334, 321, 316, 329, 335, 323, 321, 334, 336, 323, 322, 338, 336, 316, 317, 332, 329, 320, 328, 343, 336, 329, 343, 359, 350, 344, 358, 362, 340, 329, 339, 336, 321, 321, 330, 327, 313, 315, 328, 326, 314, 314, 308, 274, 248, 261, 320, 396, 423, 416, 391, 357, 341, 349, 357, 341, 328, 332, 325, 308, 312, 328, 325, 308, 310, 326, 323, 315, 323, 337, 327, 316, 327, 338, 330, 322, 336, 343, 330, 329, 349, 348, 329, 332, 351, 353, 342, 346, 362, 361, 352, 360, 370, 356, 344, 347, 349, 335, 325, 329, 329, 315, 310, 322, 321, 310, 310, 324, 321, 306, 313, 326, 318, 305, 315, 329, 320, 309, 319, 330, 321, 313, 325, 333, 320, 312, 328, 335, 320, 316, 330, 331, 318, 321, 333, 329, 317, 323, 337, 330, 318, 326, 339, 327, 316, 329, 335, 324, 318, 332, 338, 326, 324, 340, 343, 331, 329, 346, 353, 348, 352, 365, 361, 345, 342, 346, 334, 325, 335, 341, 330, 323, 331, 333, 322, 319, 334, 328, 292, 266, 264, 279, 335, 426, 461, 423, 377, 363, 367, 363, 351, 344, 340, 323, 312, 323, 329, 319, 316, 327, 326, 312, 315, 334, 335, 321, 324, 339, 336, 328, 336, 344, 336, 328, 338, 350, 340, 333, 347, 356, 346, 344, 358, 365, 359, 361, 372, 371, 358, 354, 359, 351, 334, 336, 345, 334, 320, 326, 334, 323, 316, 331, 338, 320, 313, 322, 324, 313, 314, 327, 328, 314, 317, 332, 331, 321, 327, 339, 330, 318, 325, 333, 324, 317, 331, 340, 328, 318, 330, 335, 323, 323, 337, 342, 331, 328, 338, 334, 322, 328, 338, 331, 319, 327, 340, 331, 326, 336, 343, 331, 323, 334, 341, 329, 329, 346, 352, 344, 351, 365, 364, 351, 354, 362, 350, 331, 330, 336, 328, 320, 326, 334, 323, 315, 328, 333, 317, 292, 273, 254, 254, 322, 431, 459, 416, 386, 378, 362, 351, 352, 346, 325, 310, 318, 326, 317, 312, 320, 323, 312, 311, 327, 330, 317, 317, 332, 338, 325, 323, 336, 340, 330, 327, 337, 333, 328, 338, 351, 342, 335, 349, 360, 350, 345, 361, 368, 356, 353, 364, 356, 337, 336, 346, 337, 318, 324, 337, 325, 309, 318, 331, 323, 309, 317, 327, 316, 307, 320, 328, 316, 312, 328, 334, 319, 320, 332, 329, 317, 322, 334, 331, 320, 326, 335, 328, 320, 334, 344, 332, 321, 329, 331, 320, 319, 331, 335, 323, 326, 340, 339, 328, 333, 345, 333, 318, 329, 345, 335, 320, 330, 343, 333, 325, 340, 349, 341, 337, 348, 350, 341, 346, 361, 363, 351, 347, 356, 348, 334, 334, 344, 336, 322, 330, 338, 325, 319, 331, 335, 317, 296, 282, 269, 254, 301, 408, 458, 428, 395, 375, 360, 347, 352, 351, 330, 309, 314, 326, 315, 304, 314, 323, 317, 314, 327, 330, 315, 310, 322, 326, 318, 324, 336, 331, 319, 328, 341, 335, 325, 336, 352, 344, 333, 344, 358, 352, 346, 361, 368, 356, 355, 371, 369, 346, 337, 345, 343, 324, 319, 328, 323, 312, 317, 326, 319, 307, 313, 322, 312, 304, 318, 328, 315, 308, 325, 332, 318, 316, 331, 330, 320, 323, 337, 333, 318, 323, 336, 333, 322, 326, 335, 329, 321, 332, 338, 323, 320, 330, 333, 324, 322, 335, 335, 323, 324, 338, 336, 322, 325, 341, 336, 325, 329, 339, 333, 331, 339, 348, 345, 344, 357, 362, 349, 343, 351, 347, 332, 327, 332, 329, 320, 324, 336, 332, 319, 326, 340, 328, 295, 270, 267, 271, 322, 414, 448, 416, 378, 365, 357, 342, 348, 356, 345, 321, 316, 328, 327, 317, 317, 328, 323, 314, 326, 337, 329, 321, 332, 342, 333, 330, 340, 340, 327, 326, 342, 344, 334, 336, 350, 352, 341, 348, 365, 358, 345, 354, 369, 363, 354, 361, 364, 349, 335, 338, 341, 329, 321, 327, 325, 310, 310, 321, 318, 307, 313, 325, 317, 306, 314, 326, 322, 316, 325, 326, 309, 304, 320, 328, 316, 313, 326, 328, 317, 320, 333, 327, 314, 320, 332, 323, 308, 317, 334, 330, 318, 327, 336, 327, 321, 332, 336, 324, 323, 342, 345, 329, 324, 337, 339, 327, 330, 350, 354, 343, 347, 359, 350, 335, 333, 339, 331, 321, 326, 333, 321, 315, 325, 328, 317, 314, 322, 308, 268, 249, 257, 300, 380, 450, 447, 405, 364, 352, 361, 355, 335, 327, 323, 312, 304, 312, 319, 309, 308, 325, 330, 318, 315, 327, 326, 314, 324, 341, 331, 317, 327, 341, 334, 325, 335, 347, 338, 330, 338, 346, 340, 336, 346, 350, 345, 346, 358, 356, 342, 341, 350, 343, 327, 328, 335, 325, 315, 323, 325, 312, 306, 319, 323, 309, 305, 321, 324, 310, 311, 327, 331, 322, 320, 327, 322, 311, 319, 329, 323, 311, 321, 331, 323, 315, 328, 336, 327, 322, 333, 333, 317, 320, 338, 337, 320, 324, 339, 335, 321, 327, 340, 335, 322, 327, 338, 332, 325, 340, 350, 343, 341, 356, 361, 347, 347, 357, 352, 336, 332, 339, 336, 322, 325, 337, 330, 313, 320, 335, 330, 315, 305, 281, 254, 253, 312, 408, 450, 428, 408, 380, 351, 349, 361, 351, 324, 314, 321, 318, 308, 314, 327, 322, 313, 320, 331, 322, 310, 320, 331, 323, 316, 327, 332, 322, 324, 340, 343, 331, 330, 345, 349, 340, 346, 357, 350, 338, 352, 369, 361, 350, 359, 366, 353, 345, 352, 350, 333, 330, 338, 332, 315, 317, 330, 327, 313, 316, 329, 325, 315, 322, 333, 325, 319, 327, 332, 323, 318, 331, 336, 322, 323, 335, 335, 321, 325, 341, 337, 324, 328, 341, 337, 324, 328, 339, 329, 319, 331, 343, 331, 324, 335, 338, 328, 328, 337, 339, 330, 330, 342, 344, 332, 338, 355, 355, 346, 355, 367, 355, 336, 340, 347, 335, 326, 333, 333, 324, 320, 329, 331, 320, 316, 323, 303, 264, 250, 273, 332, 411, 448, 431, 392, 358, 354, 369, 358, 334, 330, 332, 319, 312, 324, 326, 314, 310, 324, 324, 308, 311, 332, 333, 317, 323, 339, 331, 320, 328, 338, 327, 321, 336, 347, 338, 331, 346, 352, 340, 337, 354, 363, 353, 353, 364, 358, 345, 348, 354, 339, 324, 330, 336, 323, 313, 322, 325, 316, 310, 319, 320, 305, 304, 320, 322, 306, 306, 324, 326, 313, 318, 329, 321, 306, 313, 327, 325, 316, 325, 332, 323, 319, 329, 336, 319, 313, 327, 333, 323, 318, 327, 329, 320, 323, 336, 335, 323, 325, 338, 335, 326, 338, 355, 355, 349, 360, 368, 351, 338, 345, 345, 325, 320, 331, 330, 317, 321, 334, 329, 311, 316, 330, 313, 274, 260, 267, 288, 358, 424, 430, 393, 360, 352, 346, 336, 338, 344, 335, 315, 311, 322, 320, 309, 313, 327, 326, 317, 323, 335, 325, 315, 327, 333, 325, 325, 340, 347, 334, 328, 341, 345, 330, 332, 352, 355, 343, 348, 367, 366, 356, 362, 371, 361, 352, 361, 365, 345, 329, 336, 344, 331, 321, 330, 330, 318, 317, 327, 324, 312, 316, 326, 321, 309, 318, 332, 324, 312, 320, 334, 328, 317, 323, 335, 327, 318, 329, 333, 324, 322, 336, 334, 321, 322, 336, 335, 322, 325, 334, 329, 322, 333, 345, 337, 325, 340, 361, 357, 350, 360, 362, 346, 337, 345, 345, 330, 326, 334, 332, 320, 321, 333, 327, 317, 326, 336, 309, 267, 257, 270, 309, 392, 449, 438, 389, 351, 351, 356, 350, 341, 338, 329, 314, 315, 328, 322, 311, 320, 334, 325, 310, 322, 338, 331, 321, 335, 344, 331, 325, 338, 345, 334, 335, 349, 350, 332, 336, 355, 354, 345, 354, 368, 362, 350, 362, 373, 358, 346, 354, 359, 343, 334, 339, 335, 318, 316, 330, 332, 316, 316, 330, 328, 313, 315, 327, 323, 311, 318, 329, 323, 319, 331, 331, 318, 310, 325, 333, 322, 323, 337, 336, 322, 321, 335, 336, 324, 324, 336, 333, 320, 324, 334, 330, 321, 332, 344, 338, 332, 346, 357, 351, 343, 353, 359, 346, 336, 344, 337, 318, 320, 332, 331, 320, 323, 332, 323, 312, 321, 326, 298, 262, 256, 269, 317, 405, 457, 432, 385, 358, 362, 368, 349, 334, 334, 323, 306, 312, 326, 320, 307, 315, 329, 324, 320, 328, 334, 326, 320, 333, 340, 328, 325, 339, 342, 329, 330, 344, 345, 334, 339, 357, 353, 340, 349, 364, 357, 348, 359, 369, 358, 349, 355, 353, 337, 331, 338, 334, 316, 316, 330, 330, 316, 319, 327, 323, 311, 318, 329, 320, 309, 318, 330, 321, 316, 331, 336, 325, 320, 333, 338, 326, 325, 334, 332, 318, 319, 333, 332, 318, 324, 338, 330, 319, 329, 339, 328, 319, 329, 337, 326, 324, 339, 348, 342, 345, 358, 362, 348, 346, 352, 344, 326, 324, 335, 329, 316, 324, 332, 322, 316, 329, 337, 321, 297, 275, 264, 253, 301, 409, 464, 426, 390, 378, 365, 352, 353, 351, 328, 308, 317, 328, 316, 306, 314, 321, 311, 307, 321, 326, 312, 309, 325, 331, 318, 321, 336, 334, 322, 328, 343, 338, 328, 335, 348, 343, 335, 347, 359, 351, 346, 358, 365, 356, 350, 361, 363, 346, 338, 341, 336, 322, 323, 334, 329, 311, 315, 325, 318, 308, 318, 331, 323, 314, 321, 326, 320, 318, 328, 329, 316, 319, 331, 334, 320, 321, 337, 337, 321, 324, 338, 331, 319, 326, 337, 332, 325, 332, 340, 332, 330, 341, 341, 328, 329, 347, 351, 336, 340, 360, 357, 347, 354, 362, 348, 334, 336, 338, 325, 313, 322, 333, 325, 316, 321, 324, 317, 313, 313, 289, 254, 247, 302, 388, 429, 419, 401, 368, 343, 348, 359, 344
]

In [14]:
print(len(sample_data_new))

57526


In [15]:
signal_t = []
#data = pd.read_csv(path, encoding='encoding')
#print(data)
# f = open(path, "r")
for i in sample_data_new:
  # if len(i)>1:
  signal_t.append(int(i)*3.3/1023)


In [17]:
temp = signal_t
temp

[0.996774193548387,
 0.9903225806451612,
 1.032258064516129,
 1.0354838709677419,
 0.9903225806451612,
 0.996774193548387,
 1.0483870967741935,
 1.0451612903225806,
 1.0064516129032257,
 1.025806451612903,
 1.058064516129032,
 1.0354838709677419,
 1.0064516129032257,
 1.0290322580645161,
 1.0741935483870966,
 1.0548387096774192,
 1.0193548387096774,
 1.0548387096774192,
 1.090322580645161,
 1.064516129032258,
 1.0451612903225806,
 1.0870967741935482,
 1.1161290322580644,
 1.0806451612903225,
 1.0741935483870966,
 1.12258064516129,
 1.135483870967742,
 1.090322580645161,
 1.1032258064516127,
 1.1548387096774193,
 1.1516129032258065,
 1.1129032258064515,
 1.12258064516129,
 1.167741935483871,
 1.1483870967741936,
 1.096774193548387,
 1.1032258064516127,
 1.1290322580645162,
 1.0806451612903225,
 1.025806451612903,
 1.0483870967741935,
 1.064516129032258,
 1.0290322580645161,
 1.0032258064516129,
 1.0354838709677419,
 1.0677419354838709,
 1.025806451612903,
 0.9935483870967742,
 1.0354838

In [19]:
image = np.array(temp)
temp = np.column_stack((image,image))
sample = [temp, {
    "fs": 192,
    "sig_len":len(image),
    "sig_name": ["ECG", "ECG"],
    "n_sig":2
}]
sample

[array([[0.99677419, 0.99677419, 0.99677419, 0.99677419],
        [0.99032258, 0.99032258, 0.99032258, 0.99032258],
        [1.03225806, 1.03225806, 1.03225806, 1.03225806],
        ...,
        [1.12258065, 1.12258065, 1.12258065, 1.12258065],
        [1.15806452, 1.15806452, 1.15806452, 1.15806452],
        [1.10967742, 1.10967742, 1.10967742, 1.10967742]]),
 {'fs': 192, 'sig_len': 57526, 'sig_name': ['ECG', 'ECG'], 'n_sig': 2}]

In [23]:
ans = DataShaper._DataShaper__split_signal_and_add_label(sample, stamp_length=10)

In [24]:
len(ans)

29

In [25]:
import torch

In [44]:
import cv2

# Read the PNG Image in BGR format
images = []
for i in range(len(ans)):
  images.append(cv2.imread(f"/content/afdb_{i}.png"))

In [45]:
!mkdir /content/data

mkdir: cannot create directory ‘/content/data’: File exists


In [46]:
!mkdir /content/data/class1

mkdir: cannot create directory ‘/content/data/class1’: File exists


In [47]:
!mv /content/data/*.png /content/data/class1

In [48]:
images

[array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
  

In [49]:
torch.from_numpy(np.array(images)).shape

torch.Size([29, 1800, 1800, 3])

In [50]:
input = torch.permute(torch.from_numpy(np.array(images)),(0,3,1,2))/255

In [51]:
input.shape

torch.Size([29, 3, 1800, 1800])

In [52]:
model.train(False)
pass

NameError: ignored

In [ ]:
import gc
gc.collect()

83453

In [ ]:
model.train(False)
m = torch.nn.Linear(512, 5)
model.fc = m

In [ ]:
len(input)

27

In [ ]:
for i in range(len(input)-1):
  gc.collect()
  print(model(input[i:i+1]))


In [ ]:
res = model(input)

In [ ]:
res

In [ ]:
import torchvision,torch
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import Counter

def get_data(data_dir, batch_size=2, train = True):

    # define how we augment the data for composing the batch-dataset in train and test step
    transform = {
        'train': transforms.Compose([
            transforms.Resize([224,224]), # Resizing the image as the VGG only take 224 x 244 as input size
            #TODO if it is needed, add the random crop
            transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
        'test': transforms.Compose([
            transforms.Resize([224,224]),
            transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    }

    # ImageFloder with root directory and defined transformation methods for batch as well as data augmentation
    data = torchvision.datasets.ImageFolder(root=data_dir, transform=transform['train'] if train else transform['test'])

    return data

data = get_data("/content/data",batch_size=2, train=False)

In [ ]:
data

Dataset ImageFolder
    Number of datapoints: 27
    Root location: /content/data
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
for i, (image, label) in enumerate(data):
  res = model(torch.unsqueeze(image,0))
  print(torch.argmax(res), label)

tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
tensor(0) 0
